In [1]:
import gc
gc.collect()

31

# <font color='#3A59D1' size=10 >This notebook is the analyse the Images from HSC SSP</font>
The notebook uses data from the HSC SSP Images to:
- Remove "bad" Images (aka Images that are blank, over saturated, sources that are too close together, and other imperfections.
- Find the center of each of the remaining images
- Shift the centers of all the images so that they are centered over each other
- Find the average value of each individual pixel for all of the images
- With the final images, I take consentric rings out from the center to the edge of the image.
- This is used to make radial profiles of the counts/second vs. radius from center in arcseconds
- With these radial profiles I then make multiple fits
- The first set of fits uses two gaussians. One is the central narrow gaussian for the AGN component and the broad component is the galaxy component. I mirror the data around the zero x axis to make these fits.
- The second set of fits use a gaussian and a sersic profile. The gaussian is for tha AGN and the sersic is for the galaxy component.
- The third set of fits use the same gaussian and sersic profile. However, the sigma for the gaussian is fixed at the PSF/2.354 (aka the FWHM) and the N for the sersic is fixed at 1.
- The fourth fit uses the gaussian and sersic profile. This time the sigma is fixed for the gaussian at the same FWHM and the N is fixed so that it is at or above 1.
- I also find the average value of the total magnitude for the final images. I do this by turning the magnitudes into flux, find the average flux, and then turn the average flux back into magnitude.
- Using the fits, for each of them, I get the imtegrated counts/second/arcsecond for the AGN and Galaxy component. With this I take the ratio of the two values and get the magnitude of just the galaxy component


## <font color='#FF0B55' size=6 >Imports</font>

In [2]:
from dustmaps.config import config
config.reset()

In [3]:
from hetdex_api.detections import Detections
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.convolution import convolve, Box1DKernel
import tables
import numpy as np
import os.path as op
import math
import os
import h5py

from elixer import global_config as G
G.GLOBAL_LOGGING = True
from elixer import spectrum_utilities as ESU
from elixer import catalogs


from hetdex_api.elixer_widget_cls import ElixerWidget
from hetdex_api.config import HDRconfig
from hetdex_tools.get_spec import get_spectra
from hetdex_api.shot import *
from hetdex_api.survey import FiberIndex
from hetdex_api.detections import Detections

from multiprocessing import Pool
from astropy.table import QTable, Table, Column
from scipy.interpolate import interp1d
from scipy import interpolate

import inspect
import scipy.stats as stats
from astropy.table import Table
from astropy.table import QTable, Table, Column
import warnings
from astropy.modeling import models
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum
from specutils.fitting.continuum import fit_continuum
from astropy.stats import biweight_location
from scipy.integrate import quad as quad
from numpy.polynomial.polynomial import polyfit
from numpy.polynomial.polynomial import polyval
from photutils.centroids import centroid_2dg, centroid_sources
from numpy import inf
from photutils.centroids import centroid_quadratic
from matplotlib.patches import Circle
import statistics
from scipy.interpolate import CubicSpline
import emcee
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import fftconvolve

from photutils.aperture import CircularAperture, aperture_photometry, ApertureStats
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from matplotlib.colors import LinearSegmentedColormap

from matplotlib.backends.backend_pdf import PdfPages
from photutils.aperture import ApertureMask

import extinction
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord

Populating dustmaps config with /home/jovyan/Hobby-Eberly-Telesco/hdr3/calib/dustmaps


## <font color='#FF0B55' size=6 >Color Palette</font>

In [4]:
'''
For plotting clear distinctions 

Teal - #309898 - Original teal base
Orange - #FF9F00 - Original orange base
Red - #CB0404 - Original red base
Black - #000000 - Original black base
Navy Blue - #1D5799 - Deep blue with good contrast against teal
Yellow - #ECC700 - Bright yellow distinguishable from orange
Purple - #8B3D88 - Distinct from blues and reds
Lime Green - #74B741 - Bright green distinguishable from teal
Magenta - #DB3EB1 - Distinct pink/purple
Brown - #8D6E42 - Earth tone distinct from oranges and reds




Sequential Teal Variants (for heatmaps or gradients)

Teal 100% - #309898 - Base teal
Teal 80% - #5CACA6 - Lighter teal
Teal 60% - #87BFB5 - Even lighter teal
Teal 40% - #B2D2CC - Very light teal
Teal 20% - #D9E6E3 - Almost white teal




Sequential Orange Variants (for heatmaps or gradients)

Orange 100% - #FF9F00 - Base orange
Orange 80% - #FFB440 - Lighter orange
Orange 60% - #FFC977 - Even lighter orange
Orange 40% - #FFDEA6 - Very light orange
Orange 20% - #FFF0D9 - Almost white orange




High Contrast Pairs
These pairs are specifically designed to be highly distinguishable from each other:

Vivid Blue - #0072B2 - ColorBrewer-inspired blue
Vivid Orange - #E69F00 - ColorBrewer-inspired orange
Vivid Green - #009E73 - ColorBrewer-inspired green
Vivid Red - #D55E00 - ColorBrewer-inspired red
Vivid Purple - #9467BD - ColorBrewer-inspired purple




Special Purpose

Background Gray - #F0F0F0 - Light gray for plot backgrounds
Grid Line Gray - #CCCCCC - Medium gray for grid lines
Highlight Yellow - #FFFB54 - Attention-grabbing highlight
Reference Line - #505050 - Dark gray for reference lines
Annotation Red - #E41A1C - Bright red for important annotations
'''


'\nFor plotting clear distinctions \n\nTeal - #309898 - Original teal base\nOrange - #FF9F00 - Original orange base\nRed - #CB0404 - Original red base\nBlack - #000000 - Original black base\nNavy Blue - #1D5799 - Deep blue with good contrast against teal\nYellow - #ECC700 - Bright yellow distinguishable from orange\nPurple - #8B3D88 - Distinct from blues and reds\nLime Green - #74B741 - Bright green distinguishable from teal\nMagenta - #DB3EB1 - Distinct pink/purple\nBrown - #8D6E42 - Earth tone distinct from oranges and reds\n\n\n\n\nSequential Teal Variants (for heatmaps or gradients)\n\nTeal 100% - #309898 - Base teal\nTeal 80% - #5CACA6 - Lighter teal\nTeal 60% - #87BFB5 - Even lighter teal\nTeal 40% - #B2D2CC - Very light teal\nTeal 20% - #D9E6E3 - Almost white teal\n\n\n\n\nSequential Orange Variants (for heatmaps or gradients)\n\nOrange 100% - #FF9F00 - Base orange\nOrange 80% - #FFB440 - Lighter orange\nOrange 60% - #FFC977 - Even lighter orange\nOrange 40% - #FFDEA6 - Very lig

## <font color='#FF0B55' size=6 >Plotting imports and settings</font>

In [5]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
import matplotlib.ticker
import matplotlib.patches as patches

%matplotlib ipympl
plt.style.use('default')
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams.update({'font.size': 14})
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['mathtext.default'] = 'regular'
plt.rcParams['xtick.direction']= 'in'
plt.rcParams['ytick.direction']= 'in'
plt.rcParams['xtick.labelsize']= 14.0
plt.rcParams['ytick.labelsize']= 14.0

## <font color='#FF0B55' size=6 >Values</font>

In [6]:
c = 299792.5 #km s^-1
H_0 = 70 #km s^-1 MpC^-1
R = 800
PSF = 0.75/2.354 # Divid the FWHM by 2.354 to get the sigma
Mag_Zero_Point = 27 #mag
Mag_Zeropoint_Err = Mag_Zero_Point * 0.005
Pixel_Scale = 0.168

# Bands wavelength
G_Band_Wave = 4740.0 #AA
I_Band_Wave = 6170.0
R_Band_Wave = 7650.0
Y_Band_Wave = 8890.0
Z_Band_Wave = 9760.0

V = c/R
print(V)

374.740625


## <font color='#FF0B55' size=6 >Definitions</font>

In [7]:
# Define colors inspired by the trans flag
# Starting with dark versions of the colors
black = '#000000'
dark_blue = '#001B79'  # Dark version of trans flag blue
light_blue = '#4CB9E7'
mid_blue = '#1640D6'   # Light blue from trans flag
dark_pink = '#ED5AB3'  # Dark version of trans flag pink
mid_pink = '#FF90C2'   # Pink from trans flag
light_pink = '#F2AFEF'
white = '#FFFFFF'      # White from trans flag

# Create custom colormap
colors = [black, dark_blue, mid_blue,light_blue, dark_pink, mid_pink,light_pink, white]
trans_cmap = LinearSegmentedColormap.from_list('New_Trans', colors, N=256)

plt.colormaps.register(trans_cmap)




def Gaussian(x, amp, wid):
    """
    1D Gaussian function centered at zero.
    
    This function calculates the value of a Gaussian/normal distribution 
    at position x with amplitude amp and width parameter wid.
    The Gaussian is centered at x=0.
    
    Parameters:
    -----------
    x : float or numpy.ndarray
        Position(s) at which to evaluate the Gaussian function.
        Can be a single value or an array of values.
    amp : float
        Amplitude (maximum height) of the Gaussian peak.
    wid : float
        Width parameter controlling the spread of the Gaussian.
        This represents the standard deviation of the distribution.
        
    Returns:
    --------
    float or numpy.ndarray
        The Gaussian function value(s) at position(s) x.
        
    Notes:
    ------
    The formula used is: f(x) = amp * exp(-0.5 * ((x - 0) / wid)²)
    The factor 0.5 in the exponent comes from the standard form of 
    the Gaussian/normal distribution.
    """
    return amp * np.exp(-0.5 * ((x - 0) / wid)**2)

def two_Gaussian(x, amp1, wid1, amp2, wid2):
    """Sum of two 1D Gaussians."""
    return Gaussian(x, amp1, wid1) + Gaussian(x, amp2, wid2)

def Sersic_Fit(x, I_e, r_e, n):
    """
    Computes a single Sérsic profile for galaxy light distribution.
    
    The Sérsic profile is a mathematical function that describes how the intensity
    of a galaxy varies with distance from its center. It's commonly used in
    astronomical image fitting to model the radial surface brightness profiles
    of galaxies.
    
    Parameters:
    -----------
    x : float or numpy.ndarray
        Radial distance(s) from the center of the galaxy, typically in arcseconds
        or kiloparsecs. Can be a single value or an array of values.
    I_e : float
        Intensity at the effective radius r_e. This is the surface brightness
        at the half-light radius of the galaxy.
    r_e : float
        Effective radius (half-light radius) of the galaxy. This is the radius
        within which half of the total light of the galaxy is contained.
    n : float
        Sérsic index that describes the shape of the profile. Common values:
        n=1: exponential disk (spiral galaxies)
        n=4: de Vaucouleurs profile (elliptical galaxies)
        Higher values of n create more centrally concentrated profiles with
        extended wings.
        
    Returns:
    --------
    float or numpy.ndarray
        The intensity value(s) of the Sérsic profile at the given radius/radii.
        
    Notes:
    ------
    The formula used is: I(r) = I_e * exp(-b_n * ((r/r_e)^(1/n) - 1))
    
    b_n is approximated as (2*n - 1/3), which is accurate enough for most
    applications when n > 0.5. More precise values of b_n can be computed
    by solving the equation Γ(2n) = 2γ(2n, b_n), where Γ is the complete
    gamma function and γ is the incomplete gamma function.
    """
    b_n = 2 * n - 1 / 3  # Approximate formula for b_n
    return I_e * np.exp(-b_n * ((x / r_e) ** (1 / n) - 1))


def getting_the_centers(data):
    """
    Calculates the centroids for a series of astronomical images or frames.
    
    This function processes each frame in the input data array to find the precise
    centroid position of an object (like a star or galaxy) using quadratic centroiding.
    It starts from an initial guess position and returns arrays of the refined X and Y
    centroid coordinates across all frames.
    
    Parameters:
    -----------
    data : numpy.ndarray
        3D array of image data with shape (n_frames, height, width) where:
        - n_frames is the number of frames/images to process
        - height and width are the dimensions of each image
        
    Returns:
    --------
    tuple of two lists
        - Centroid_X_Array : list of floats
            X-coordinates of the centroids for each frame
        - Centroid_Y_Array : list of floats
            Y-coordinates of the centroids for each frame
            
    Notes:
    ------
    The function uses the external centroid_quadratic function with fixed parameters:
    - Initial peak position guess at (42, 43)
    - Search box size of 21 pixels
    
    This assumes the object of interest is near position (42, 43) in each frame.
    For objects at different positions, these parameters should be adjusted.
    
    The centroid_quadratic function is expected to perform sub-pixel centroid
    determination by fitting a quadratic surface to the intensity distribution
    around the peak.
    
    Example:
    --------
    >>> frames = load_image_sequence('star_data.fits')
    >>> x_centers, y_centers = getting_the_centers(frames)
    >>> plt.plot(x_centers, y_centers, 'o-')  # Plot the centroid path
    """
    
    # Initialize empty lists to store centroid coordinates
    Centroid_X_Array = []
    Centroid_Y_Array = []
    
    # Process each frame in the data array
    for i in np.arange(len(data)):
        # Calculate the centroid position using quadratic fitting
        # Starting from position (42, 43) with a 21x21 pixel search box
        X, Y = centroid_quadratic(data[i], xpeak=42, ypeak=43, search_boxsize=21)
        
        # Store the calculated centroid coordinates
        Centroid_X_Array.append(X)
        Centroid_Y_Array.append(Y)
        
    # Return the arrays of centroid positions
    return Centroid_X_Array, Centroid_Y_Array


def shifting_in_the_y_direction(data, y_center):
    """
    Shifts a series of images to center objects in the Y-direction.
    
    This function takes a stack of images and corresponding Y-center positions,
    then shifts each image vertically to place the object at the center of the frame.
    The function maintains the original image dimensions by adding zeros (black pixels)
    to the appropriate edge after shifting.
    
    Parameters:
    -----------
    data : numpy.ndarray
        3D array of image data with shape (n_frames, height, width) where:
        - n_frames is the number of frames/images to process
        - height and width are the dimensions of each image
    
    y_center : list or numpy.ndarray
        List of Y-coordinates of the object centers in each frame.
        Should have the same length as the first dimension of data.
    
    Returns:
    --------
    list of numpy.ndarray
        List of shifted images with the same dimensions as the input images.
        Only images where the center differs from the middle of the frame
        are included in the returned list.
    
    Notes:
    ------
    - The function calculates how far each center is from the middle of the frame,
      then shifts the image by that amount.
    - If the center is above the middle (positive difference), the image is shifted down.
    - If the center is below the middle (negative difference), the image is shifted up.
    - Images where the object is already in the center (difference = 0) are skipped.
    - The function maintains the original image dimensions by adding zeros to fill
      the space created by the shift.
    
    Warning:
    --------
    There's a potential bug in the current implementation: when dif_y == 0,
    the image is skipped due to the else clause. This means images that are
    already centered won't be included in the output array.
    
    Example:
    --------
    >>> frames = load_image_sequence('star_data.fits')
    >>> centers_x, centers_y = getting_the_centers(frames)
    >>> centered_frames = shifting_in_the_y_direction(frames, centers_y)
    """
    
    # Initialize empty list to store shifted images
    shifted_y_array = []
    
    # Process each frame in the data array
    for i in np.arange(len(data)):
        # Calculate how far the object's center is from the middle of the frame
        # Positive dif_y means the object is below the middle
        # Negative dif_y means the object is above the middle
        dif_y = y_center[i] - len(data[i])/2
        
        # Case 1: Object is below the middle of the frame (dif_y > 0)
        # Need to shift image down by removing rows from top and adding zeros at bottom
        if dif_y > 0:
            # Remove dif_y rows from the top of the image
            # This effectively shifts the image content downward
            shift_y_pos = np.delete(data[i], slice(0, int(dif_y)), 0)  # 0 specifies rows
            
            # Create a block of zeros (black pixels) with the same width as the image
            # and height equal to the amount we need to shift
            zero_col = np.zeros((int(dif_y), shift_y_pos.shape[1]))
            
            # Stack the shifted image on top of the zeros block
            # This places the zeros at the bottom of the image
            shifted = np.vstack((shift_y_pos, zero_col))
            
            # Add the shifted image to our result list
            shifted_y_array.append(shifted)
            
        # Case 2: Object is above the middle of the frame (dif_y < 0)
        # Need to shift image up by removing rows from bottom and adding zeros at top
        if dif_y < 0:
            # Remove |dif_y| rows from the bottom of the image
            # This effectively shifts the image content upward
            shift_y_pos = np.delete(data[i], slice(len(data[i]) + int(dif_y), len(data[i])), 0)
            
            # Create a block of zeros (black pixels) with the same width as the image
            # and height equal to the amount we need to shift
            zero_col = np.zeros((abs(int(dif_y)), shift_y_pos.shape[1]))
            
            # Stack the zeros block on top of the shifted image
            # This places the zeros at the top of the image
            shifted = np.vstack((zero_col, shift_y_pos))
            
            # Add the shifted image to our result list
            shifted_y_array.append(shifted)
            
        # Case 3: Object is already centered (dif_y == 0)
        # Note: This else clause will skip images where dif_y == 0,
        # which means perfectly centered images won't be included in the result
        else:
            continue
            
    # Return the list of shifted images
    return shifted_y_array

def shifting_in_the_x_direction(data, x_center):
    """
    Shifts a series of images to center objects in the X-direction.
    
    This function takes a stack of images and corresponding X-center positions,
    then shifts each image horizontally to place the object at the center of the frame.
    The function maintains the original image dimensions by adding zeros (black pixels)
    to the appropriate edge after shifting.
    
    Parameters:
    -----------
    data : numpy.ndarray
        3D array of image data with shape (n_frames, height, width) where:
        - n_frames is the number of frames/images to process
        - height and width are the dimensions of each image
    
    x_center : list or numpy.ndarray
        List of X-coordinates of the object centers in each frame.
        Should have the same length as the first dimension of data.
    
    Returns:
    --------
    list of numpy.ndarray
        List of shifted images with the same dimensions as the input images.
        Only images where the center differs from the middle of the frame
        are included in the returned list.
    
    Notes:
    ------
    - The function calculates how far each center is from the middle of the frame,
      then shifts the image by that amount.
    - If the center is to the right of the middle (positive difference), 
      the image is shifted left.
    - If the center is to the left of the middle (negative difference), 
      the image is shifted right.
    - Images where the object is already centered horizontally (difference = 0) 
      are skipped.
    - The function maintains the original image dimensions by adding zeros to fill
      the space created by the shift.
    
    Warning:
    --------
    There's a potential bug in the current implementation: when dif_x == 0,
    the image is skipped due to the else clause. This means images that are
    already horizontally centered won't be included in the output array.
    
    Example:
    --------
    >>> frames = load_image_sequence('star_data.fits')
    >>> centers_x, centers_y = getting_the_centers(frames)
    >>> y_centered_frames = shifting_in_the_y_direction(frames, centers_y)
    >>> fully_centered_frames = shifting_in_the_x_direction(y_centered_frames, centers_x)
    """
    
    # Initialize empty list to store shifted images
    shifted_x_array = []
    
    # Process each frame in the data array
    for i in np.arange(len(data)):
        # Calculate how far the object's center is from the middle of the frame
        # Positive dif_x means the object is to the right of the middle
        # Negative dif_x means the object is to the left of the middle
        dif_x = x_center[i] - len(data[i])/2
        
        # Case 1: Object is to the right of the middle (dif_x > 0)
        # Need to shift image left by removing columns from left and adding zeros at right
        if dif_x > 0:
            # Remove dif_x columns from the left of the image
            # This effectively shifts the image content leftward
            # The '1' parameter specifies we're operating on columns, not rows
            shift_x_pos = np.delete(data[i], slice(0, int(dif_x)), 1)
            
            # Create a block of zeros (black pixels) with the same height as the image
            # and width equal to the amount we need to shift
            zero_col = np.zeros((shift_x_pos.shape[0], int(dif_x)))
            
            # Stack the shifted image side by side with the zeros block
            # This places the zeros at the right of the image
            shifted = np.hstack((shift_x_pos, zero_col))
            
            # Add the shifted image to our result list
            shifted_x_array.append(shifted)
            
        # Case 2: Object is to the left of the middle (dif_x < 0)
        # Need to shift image right by removing columns from right and adding zeros at left
        if dif_x < 0:
            # Remove |dif_x| columns from the right of the image
            # This effectively shifts the image content rightward
            shift_x_pos = np.delete(data[i], slice(len(data[i]) + int(dif_x), len(data[i])), 1)
            
            # Create a block of zeros (black pixels) with the same height as the image
            # and width equal to the amount we need to shift
            zero_col = np.zeros((shift_x_pos.shape[0], abs(int(dif_x))))
            
            # Stack the zeros block side by side with the shifted image
            # This places the zeros at the left of the image
            shifted = np.hstack((zero_col, shift_x_pos))
            
            # Add the shifted image to our result list
            shifted_x_array.append(shifted)
            
        # Case 3: Object is already centered horizontally (dif_x == 0)
        # Note: This else clause will skip images where dif_x == 0,
        # which means perfectly centered images won't be included in the result
        else:
            continue
            
    # Return the list of shifted images
    return shifted_x_array

def single_rows(data, num):
    '''
    Gets the rows for each of the shifted images. 
    
    Parameters:
    -----------
    data : numpy.ndarray
        An array of all of the shifted images.
    
    num : interger
        The row. - Imput in the all_the_rows definition
        
    
    Returns:
    --------
    list of numpy.ndarray
        Rows contaning pixels for each given num. 
    '''
    Single_Row_Arrays = []
    
    for i in data:
        single_row_array = i[num]
        Single_Row_Arrays.append(single_row_array)

    return Single_Row_Arrays

def all_the_rows(data):
    '''
    Combine the individual rows from the single_rows definition into a single array of each array.
    
    Parameters:
    -----------
    data : numpy.ndarray
        An array of all of the shifted images.        
    
    Returns:
    --------
    list of numpy.ndarray
        Arrays containing the rows for each shifted image. (each array contains the same row for each image) 
    '''
    All_the_Single_Rows = []

    for i in np.arange(data[0].shape[1]):
        All_the_Single_Rows.append(single_rows(data, i))

    return All_the_Single_Rows

def row_median(all_the_single_rows):
    '''
    Finds the median value of each pixel for all the shifted images.
    
    Parameters:
    -----------
    all_the_single_rows : numpy.ndarray
        Combine the individual rows from the single_rows definition into a single array of each array.
        
    Returns:
    --------
    list of numpy.ndarray
        The median value for each pixel in the shifted images 
    '''
    Row_Median = []

    for i in all_the_single_rows:
        Row_median = np.median(i, axis=0)
        Row_Median.append(Row_median)

    return Row_Median


def Sersic_Fit_Fixed_N(x, I_e, r_e):
    """
    Computes a single Sérsic profile with fixed Sérsic index n = 1 (i.e., exponential profile).
    
    Parameters:
    - x : array_like
        Radial distance from the center.
    - I_e : float
        Intensity at the effective radius r_e.
    - r_e : float
        Effective radius (scale parameter).
    
    Returns:
    - Intensity profile at each x.
    """
    b_n = 2 * 1 - 1 / 3  # Approximate value of b_n for n = 1 (n is fixed here)
    return I_e * np.exp(-b_n * ((x / r_e) ** (1 / 1) - 1))  # Exponential profile

def Gaussian_Fixed_Sigma(x, a):
    """
    Computes a Gaussian function centered at zero with a fixed sigma (standard deviation).
    
    Parameters:
    - x : array_like
        Input values over which the Gaussian is evaluated.
    - a : float
        Amplitude of the Gaussian.
    
    Assumes:
    - A global variable `PSF` is defined, representing the full width at half maximum (FWHM),
      so sigma is set to PSF/2.
    
    Returns:
    - Gaussian function values at each x.
    """
    return a * np.exp(-((x - 0) ** 2) / (2 * (PSF / 2) ** 2))

def Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e, r_e, a):
    """
    Computes the sum of a Sérsic profile (with fixed n = 1) and a Gaussian profile 
    (with fixed sigma).
    
    Parameters:
    - x : array_like
        Radial coordinate.
    - i_e : float
        Sérsic profile intensity at r_e.
    - r_e : float
        Sérsic effective radius.
    - a : float
        Amplitude of the Gaussian profile.
    
    Returns:
    - Combined intensity profile (Sérsic + Gaussian) at each x.
    """
    Ser = Sersic_Fit_Fixed_N(x, i_e, r_e)  # Compute Sérsic profile
    Gaus = Gaussian_Fixed_Sigma(x, a)      # Compute Gaussian profile
    return Ser + Gaus                      # Return the sum of both components


    
def adu_to_flux(adu_counts_per_sec, adu_err=None, zeropoint=27.0, zeropoint_err=0.0):
    """
    Convert ADU counts per second to flux in microJanskys (µJy) using the AB magnitude system.
    
    Parameters:
    -----------
    adu_counts_per_sec : float or array-like
        The ADU counts per second from the instrument.
    adu_err : float or array-like, optional
        The uncertainty in ADU counts per second.
    zeropoint : float, optional
        The photometric zero point in mag/DN (default: 27.0).
    zeropoint_err : float, optional
        The uncertainty in the zero point (default: 0.0).
        
    Returns:
    --------
    tuple: (flux_uJy, flux_err_uJy)
        Flux in microJanskys and its uncertainty.
    """
    # Prevent issues with negative or zero counts
    if np.any(adu_counts_per_sec <= 0):
        raise ValueError("ADU counts must be positive")

    # Convert counts to magnitude
    magnitude = zeropoint - 2.5 * np.log10(adu_counts_per_sec)

    # Convert magnitude to flux using AB system: f_ν [µJy] = 10^((23.9 - m)/2.5)
    flux_uJy = 10 ** ((23.9 - magnitude) / 2.5)

    # If error is provided, propagate it
    if adu_err is not None:
        # Error propagation formula considering both zeropoint and ADU errors
        mag_err = np.sqrt(
            zeropoint_err**2 + (2.5 / (np.log(10) * adu_counts_per_sec))**2 * adu_err**2
        )
    else:
        # If no ADU error is provided, only include zeropoint error
        mag_err = zeropoint_err

    # Derivative of flux with respect to magnitude (for error propagation)
    d_flux_dm = (np.log(10) / 2.5) * flux_uJy
    flux_err_uJy = d_flux_dm * mag_err

    return flux_uJy, flux_err_uJy

def calculate_ring_areas(radii):
    """
    Calculate the areas of concentric rings given an array of radii.
    
    Parameters:
    -----------
    radii : array-like
        Array of ring radii in ascending order.
        These represent the outer radius of each ring.
        The innermost ring starts at radius 0.
    
    Returns:
    --------
    areas : numpy.ndarray
        Array of ring areas
    """
    # Convert to numpy array if it's not already
    radii = np.asarray(radii)
    
    # Ensure radii are sorted in ascending order
    if not np.all(np.diff(radii) >= 0):
        raise ValueError("Radii must be in ascending order")
    
    # Create an array of inner radii (starting with 0)
    inner_radii = np.zeros_like(radii)
    inner_radii[1:] = radii[:-1]
    
    # Calculate the area of each ring: π(R_outer² - R_inner²)
    areas = np.pi * (radii**2 - inner_radii**2)

    equal_area_radii = np.sqrt((inner_radii**2 + radii**2) / 2)

    return areas, equal_area_radii


# Sérsic profile with background
def Sersic_Profile_BG(x, I_e, r_e, n, bg):
    """
    Computes a Sérsic profile with an added constant background.

    Parameters:
    - x : array_like
        Radial positions.
    - I_e : float
        Intensity at the effective radius r_e.
    - r_e : float
        Effective radius (scale radius of the profile).
    - n : float
        Sérsic index controlling the concentration of the profile.
    - bg : float
        Constant background level added to the profile.

    Returns:
    - Intensity profile values at each x including background.
    """
    b_n = 2 * n - 1 / 3  # Approximate value for b_n
    return I_e * np.exp(-b_n * ((np.abs(x - Equal_Area_Radius_035_045_Array[0] * Pixel_Scale) / r_e) ** (1 / n) - 1)) + bg

# Gaussian PSF
def Gaussian_psf_bg(x, fwhm):
    """
    Computes a normalized Gaussian Point Spread Function (PSF).

    Parameters:
    - x : array_like
        Radial coordinates for the PSF.
    - fwhm : float
        Full Width at Half Maximum of the Gaussian.

    Returns:
    - Normalized Gaussian PSF values.
    """
    sigma = fwhm / (2 * np.sqrt(2 * np.log(2)))  # Convert FWHM to sigma
    psf = np.exp(-0.5 * (x / sigma) ** 2)
    return psf / psf.sum()  # Normalize to unit area

# Sérsic convolved with PSF, including background
def Sersic_Convolved_Model_BG(x, I_e, r_e, n, bg, fwhm_psf=PSF * 2.35):
    """
    Computes a Sérsic profile convolved with a Gaussian PSF, then adds a background level.

    Parameters:
    - x : array_like
        Radial positions.
    - I_e : float
        Sérsic intensity at effective radius r_e.
    - r_e : float
        Sérsic effective radius.
    - n : float
        Sérsic index.
    - bg : float
        Background intensity level.
    - fwhm_psf : float, optional
        FWHM of the Gaussian PSF (default is PSF * 2.35).

    Returns:
    - Convolved Sérsic profile values interpolated to input x, with background added.
    """
    # Define extended x range for accurate convolution
    x_range = max(abs(x.min() - Equal_Area_Radius_035_045_Array[0]), abs(x.max() - Equal_Area_Radius_035_045_Array[0]))
    x_pad = np.linspace(Equal_Area_Radius_035_045_Array[0] - 2 * x_range,
                        Equal_Area_Radius_035_045_Array[0] + 2 * x_range,
                        8 * len(x))

    # Compute Sérsic profile on padded x range (without background)
    sersic = Sersic_Profile_BG(x_pad, I_e, r_e, n, bg=0.0)

    # Compute PSF and perform convolution
    psf_x = np.linspace(-x_range, x_range, len(x_pad))
    psf = Gaussian_psf_bg(psf_x, fwhm_psf)
    convolved = fftconvolve(sersic, psf, mode='same')

    # Add background after convolution
    convolved += bg

    # Interpolate result back to original x values
    return np.interp(x, x_pad, convolved)

# Central Gaussian component (unconvolved)
def Gaussian_Component_BG(x, bg, A):
    """
    Computes an additional central Gaussian component with a background.

    Parameters:
    - x : array_like
        Radial positions.
    - bg : float
        Constant background level.
    - A : float
        Amplitude of the Gaussian.

    Returns:
    - Gaussian function values at each x with added background.
    """
    return A * np.exp(-0.5 * ((x - Equal_Area_Radius_035_045_Array[0]) / PSF) ** 2) + bg

# Total model with background
def Total_Model_BG(x, I_e, r_e, n, bg, A):
    """
    Computes the full model: Sérsic profile convolved with PSF plus an additional 
    central Gaussian component. Both components include background.

    Parameters:
    - x : array_like
        Radial positions.
    - I_e : float
        Sérsic intensity at r_e.
    - r_e : float
        Sérsic effective radius.
    - n : float
        Sérsic index.
    - bg : float
        Background level.
    - A : float
        Amplitude of the central Gaussian.

    Returns:
    - Combined intensity profile at each x.
    """
    sersic = Sersic_Convolved_Model_BG(x, I_e, r_e, n, bg)
    gauss = Gaussian_Component_BG(x, bg, A)
    return sersic + gauss



def row_std_dev(all_the_single_rows):
    """
    Calculate the standard deviation for each median value.
    
    Args:
        all_the_single_rows: List of lists, where each inner list contains the same row from each 2D array
        
    Returns:
        List of standard deviations for each row
    """
    Row_StdDev = []
    for i in all_the_single_rows:
        Row_stddev = np.std(i, axis=0)
        Row_StdDev.append(Row_stddev)
    return Row_StdDev


def bootstrap_median_std_with_errors(magnitudes, magnitude_errors, n_bootstrap=1000):
    """
    Calculate the median and its standard deviation using bootstrap resampling
    for a 1D array of magnitudes, incorporating magnitude errors.
    
    Args:
        magnitudes: 1D array of magnitude measurements
        magnitude_errors: 1D array of magnitude errors (max - min possible values)
        n_bootstrap: Number of bootstrap samples (default: 1000)
    
    Returns:
        tuple: (median_magnitude, median_std_dev)
            - median_magnitude: the median value of the input magnitudes
            - median_std_dev: standard deviation of the median estimated via bootstrapping
            - error_propagated_std: standard deviation with error propagation
    """
    # Calculate the median
    median_magnitude = np.median(magnitudes)
    
    # Number of measurements
    n_measurements = len(magnitudes)
    
    # Basic bootstrap resampling (ignoring errors for now)
    bootstrap_medians = []
    
    for i in range(n_bootstrap):  # Fixed: use 'range(n_bootstrap)' not 'n_bootstrap'
        # For each bootstrap iteration, randomly sample with replacement
        idx = np.random.randint(0, n_measurements, size=n_measurements)
        bootstrap_sample = magnitudes[idx]
        bootstrap_medians.append(np.median(bootstrap_sample))
    
    # Calculate standard deviation of bootstrapped medians
    bootstrap_medians = np.array(bootstrap_medians)
    median_std_dev = np.std(bootstrap_medians)
    
    # Enhanced bootstrap with error propagation
    # For each bootstrap iteration, we'll also add random noise based on the magnitude errors
    bootstrap_medians_with_errors = []
    
    for i in range(n_bootstrap):  # Fixed: use 'range(n_bootstrap)' not 'n_bootstrap'
        # Sample indices with replacement
        idx = np.random.randint(0, n_measurements, size=n_measurements)
        bootstrap_sample = magnitudes[idx]
        bootstrap_errors = magnitude_errors[idx]
        
        # Add random noise based on the errors
        # Assuming errors represent max-min range, we'll use uniform distribution
        # centered at the magnitude with width equal to the error
        noise = np.random.uniform(-0.5, 0.5, size=n_measurements) * bootstrap_errors
        noisy_sample = bootstrap_sample + noise
        
        bootstrap_medians_with_errors.append(np.median(noisy_sample))
    
    bootstrap_medians_with_errors = np.array(bootstrap_medians_with_errors)
    error_propagated_std = np.std(bootstrap_medians_with_errors)
    
    return median_magnitude, median_std_dev, error_propagated_std

def monte_carlo_gaussian_area(x_vals, amp, wid, amp_std, wid_std, N=5000):
    """
    Estimate the uncertainty in the area under a Gaussian using Monte Carlo sampling.

    Parameters:
        x_vals (np.array): Array of x values to integrate over.
        amp (float): Best-fit amplitude.
        wid (float): Best-fit width (sigma).
        amp_std (float): Standard deviation of amplitude.
        wid_std (float): Standard deviation of width.
        N (int): Number of Monte Carlo samples.

    Returns:
        area_mean (float): Mean of integrated areas.
        area_std (float): Standard deviation (uncertainty) of areas.
    """
    amp_samples = np.random.normal(loc=amp, scale=amp_std, size=N)
    wid_samples = np.random.normal(loc=wid, scale=wid_std, size=N)

    area_samples = [
        np.trapz(Gaussian(x_vals, a, w), x=x_vals)
        for a, w in zip(amp_samples, wid_samples)
    ]

    area_mean = np.mean(area_samples)
    area_std = np.std(area_samples)

    return area_mean, area_std

def fractional_area_uncertainty(A_broad, A_narrow, sigma_broad, sigma_narrow):
    """
    Computes the fractional area f = A_broad / (A_broad + A_narrow) and its uncertainty 
    using standard error propagation.

    Parameters:
    - A_broad : float
        Area of the broad component.
    - A_narrow : float
        Area of the narrow component.
    - sigma_broad : float
        Uncertainty (standard deviation) of the broad area.
    - sigma_narrow : float
        Uncertainty (standard deviation) of the narrow area.

    Returns:
    - f : float
        Fractional area of the broad component.
    - sigma_frac : float
        Uncertainty in the fractional area (standard deviation).
    """

    denom = (A_broad + A_narrow) ** 2  # Denominator squared for derivatives

    # Partial derivatives of f with respect to A_broad and A_narrow
    df_dA_broad = A_narrow / denom
    df_dA_narrow = -A_broad / denom

    # Error propagation formula for independent variables
    sigma_frac = np.sqrt(
        (df_dA_broad * sigma_broad) ** 2 +
        (df_dA_narrow * sigma_narrow) ** 2
    )

    # Compute fractional area
    f = A_broad / (A_broad + A_narrow)

    return f, sigma_frac


def mag_to_flux(mag_array, mag_err_array, mag_zero_point, wavelength, RA, DEC):
    """
    Convert magnitudes and their uncertainties to fluxes and flux uncertainties.
    Parameters:
        mag_array (array-like): Array of magnitudes.
        mag_err_array (array-like): Array of magnitude uncertainties.
        mag_zero_point (float): Photometric zero point.
        wavelength (float): Filter wavelength in Angstroms.
        RA (array-like): Array of Right Ascension coordinates in degrees.
        DEC (array-like): Array of Declination coordinates in degrees.
    Returns:
        flux_array (np.ndarray): Fluxes corresponding to input magnitudes.
        flux_err_array (np.ndarray): Flux uncertainties.
    """
    mag_array = np.asarray(mag_array)
    mag_err_array = np.asarray(mag_err_array)
    RA = np.asarray(RA)
    DEC = np.asarray(DEC)
    
    # Initialize the query object
    sfd = SFDQuery()
    
    # Your coordinates
    coords = SkyCoord(ra=RA, dec=DEC, unit='deg', frame='icrs')
    
    # Get E(B-V) for all coordinates
    ebv_array = sfd(coords)
    
    # Prepare wavelength array for extinction calculation
    wave_array = np.array([wavelength], dtype=np.float64)
    
    # Calculate extinction correction for each source
    mag_corrected = np.zeros_like(mag_array)
    for i in range(len(ebv_array)):
        ebv_value = float(ebv_array[i]) if hasattr(ebv_array[i], 'value') else float(ebv_array[i])
        a_v = 3.1 * ebv_value
        a_lambda = extinction.ccm89(wave_array, a_v, 3.1)[0]
        mag_corrected[i] = mag_array[i] - a_lambda
    
    # Convert corrected magnitudes to flux
    flux_array = 10 ** (-0.4 * (mag_corrected - mag_zero_point))
    flux_err_array = 0.921034 * flux_array * mag_err_array
    
    return flux_array, flux_err_array

def median_with_uncertainty(flux_array, flux_err_array, N=5000):
    """
    Estimate the uncertainty on the median of flux values using Monte Carlo sampling.

    Parameters:
        flux_array (array-like): Flux values.
        flux_err_array (array-like): Corresponding 1σ uncertainties.
        N (int): Number of Monte Carlo samples.

    Returns:
        median_flux (float): Median of flux values.
        median_uncertainty (float): Standard deviation of sampled medians.
    """
    flux_array = np.asarray(flux_array)
    flux_err_array = np.asarray(flux_err_array)

    # Generate N synthetic datasets
    samples = np.random.normal(loc=flux_array[:, None],
                               scale=flux_err_array[:, None],
                               size=(len(flux_array), N))

    # Median of each realization
    medians = np.median(samples, axis=0)

    median_flux = np.median(flux_array)
    median_uncertainty = np.std(medians)

    return median_flux, median_uncertainty

def flux_to_mag_and_uncertainty(flux, flux_err, mag_zero_point):
    """
    Convert flux and flux uncertainty to magnitude and magnitude uncertainty.

    Parameters:
        flux (float): Flux value (e.g. median flux).
        flux_err (float): Uncertainty on the flux.
        mag_zero_point (float): Photometric zero point.

    Returns:
        mag (float): Converted magnitude.
        mag_err (float): Uncertainty on the magnitude.
    """
    mag = -2.5 * np.log10(flux) + mag_zero_point
    mag_err = 1.0857 * (flux_err / flux)
    return mag, mag_err

def gal_mag_uncertainty(mag_total, mag_total_err, frac_area, frac_area_err):
    """
    Propagate uncertainties to get the uncertainty on galaxy magnitude derived
    from total magnitude and fractional area.

    Parameters:
        mag_total (float): Total AB magnitude.
        mag_total_err (float): Uncertainty on total magnitude.
        frac_area (float): Fractional area (e.g. from two-Gaussian model).
        frac_area_err (float): Uncertainty on fractional area.

    Returns:
        gal_mag (float): Derived galaxy magnitude.
        gal_mag_err (float): Uncertainty on galaxy magnitude.
    """
    gal_mag = mag_total - 2.5 * np.log10(frac_area)
    gal_mag_err = np.sqrt(
        mag_total_err**2 +
        (1.0857 * frac_area_err / frac_area)**2
    )
    return gal_mag, gal_mag_err


def Monte_Carlo_Sersic_Area(r_vals, I_e, r_e, n, I_e_std, r_e_std, n_std=None, N=5000):
    """
    Estimate the uncertainty in the area under a Sérsic profile using Monte Carlo sampling.
    """
    I_e_samples = np.random.normal(loc=I_e, scale=I_e_std, size=N)
    r_e_samples = np.random.normal(loc=r_e, scale=r_e_std, size=N)
    
    if n_std is not None:
        n_samples = np.random.normal(loc=n, scale=n_std, size=N)
    else:
        n_samples = np.full(N, n)
    
    # Using numerical integration like in the Gaussian example
    area_samples = [
        np.trapz(2*np.pi*r_vals * Sersic_Fit(r_vals, I, r, ns), x=r_vals)
        for I, r, ns in zip(I_e_samples, r_e_samples, n_samples)
    ]
    
    area_mean = np.mean(area_samples)
    area_std = np.std(area_samples)
    return area_mean, area_std
    

def Sersic_Gauss(x, I_e, r_e, n, a, sigma):
    """
    Combines a Sérsic profile and a Gaussian function.
    
    Parameters:
        x (array): Radius/position values
        I_e (float): Sérsic intensity at effective radius
        r_e (float): Sérsic effective radius
        n (float): Sérsic index
        a (float): Gaussian amplitude
        x0 (float): Gaussian center position
        sigma (float): Gaussian standard deviation
        
    Returns:
        array: Combined profile at each position
    """
    Ser = Sersic_Fit(x, I_e, r_e, n)
    Gaus = Gaussian(x, a, sigma)
    return Ser + Gaus


def Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_vals, amp, amp_std, N=5000):
    """
    Estimate the uncertainty in the area under a Gaussian using Monte Carlo sampling.

    Parameters:
        x_vals (np.array): Array of x values to integrate over.
        amp (float): Best-fit amplitude.
        amp_std (float): Standard deviation of amplitude.
        N (int): Number of Monte Carlo samples.

    Returns:
        area_mean (float): Mean of integrated areas.
        area_std (float): Standard deviation (uncertainty) of areas.
    """
    amp_samples = np.random.normal(loc=amp, scale=amp_std, size=N)

    area_samples = [
        np.trapz(Gaussian_Fixed_Sigma(x_vals, a), x=x_vals)
        for a in zip(amp_samples)
    ]

    area_mean = np.mean(area_samples)
    area_std = np.std(area_samples)

    return area_mean, area_std

def Sersic_Fit_Fixed_N(x, I_e, r_e):
    """
    Computes a single Sérsic profile.
    """
    b_n = 2 * 1 - 1 / 3  # Approximate formula for b_n
    return I_e * np.exp(-b_n * ((x / r_e) ** (1 / 1) - 1))

def Monte_Carlo_Sersic_Fixed_N_Area(r_vals, I_e, r_e, I_e_std, r_e_std, N=5000):
    """
    Estimate the uncertainty in the area under a Sérsic profile using Monte Carlo sampling.
    """
    I_e_samples = np.random.normal(loc=I_e, scale=I_e_std, size=N)
    r_e_samples = np.random.normal(loc=r_e, scale=r_e_std, size=N)
    
    # Using numerical integration like in the Gaussian example
    area_samples = [
        np.trapz(2*np.pi*r_vals * Sersic_Fit_Fixed_N(r_vals, I, r), x=r_vals)
        for I, r in zip(I_e_samples, r_e_samples)
    ]
    
    area_mean = np.mean(area_samples)
    area_std = np.std(area_samples)
    return area_mean, area_std


def Monte_Carlo_Sersic_BG_Area(x_vals, I_e, r_e, n, bg,
                                I_e_std, r_e_std, n_std, bg_std,
                                N=5000):
    """
    Estimate the uncertainty in the area under the convolved Sérsic component
    using Monte Carlo sampling, including variation in background.

    Parameters:
    - x_vals: array of x (radial) values
    - I_e: central intensity
    - r_e: effective radius
    - n: Sérsic index
    - bg: background level
    - I_e_std, r_e_std, n_std, bg_std: standard deviations for each parameter
    - N: number of Monte Carlo samples

    Returns:
    - mean area and standard deviation of the Sérsic component (with bg included)
    """

    # Sample each parameter N times
    I_e_samples = np.random.normal(I_e, I_e_std, size=N)
    r_e_samples = np.random.normal(r_e, r_e_std, size=N)
    n_samples = np.random.normal(n, n_std, size=N)
    bg_samples = np.random.normal(bg, bg_std, size=N)

    # Compute area under each sampled model
    area_samples = [
        np.trapz(Sersic_Convolved_Model_BG(x_vals, I_e_s, r_e_s, n_s, bg_s), x=x_vals)
        for I_e_s, r_e_s, n_s, bg_s in zip(I_e_samples, r_e_samples, n_samples, bg_samples)
    ]

    return np.mean(area_samples), np.std(area_samples)


def Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_vals, bg, A, bg_std, A_std, N=5000):
    """
    Estimate the uncertainty in the area under the Gaussian component with background 
    using Monte Carlo sampling.

    Parameters:
    - x_vals: array of x values
    - bg: background value
    - A: amplitude of the Gaussian
    - bg_std: standard deviation of the background
    - A_std: standard deviation of the amplitude
    - N: number of Monte Carlo samples

    Returns:
    - mean area and standard deviation of the area
    """
    # Generate Monte Carlo samples for amplitude and background
    bg_samples = np.random.normal(loc=bg, scale=bg_std, size=N)
    A_samples = np.random.normal(loc=A, scale=A_std, size=N)

    # Compute area under each sampled Gaussian with background
    area_samples = [
        np.trapz(Gaussian_Component_BG(x_vals, bg_i, A_i), x=x_vals)
        for bg_i, A_i in zip(bg_samples, A_samples)
    ]

    # Return the mean and standard deviation of the areas
    return np.mean(area_samples), np.std(area_samples)


## <font color='#FF0B55' size=6 >Importing the file for the redshifts of each of the Sources</font>

In [8]:
File_Redshift_Confirmed = "Table_Spec_Redshift_Confirmed_New_Old.fits"


In [9]:
Data_Redshift_Confirmed = Table.read(File_Redshift_Confirmed, format = 'fits')


In [10]:
catlib = catalogs.CatalogLibrary()

## <font color='#FF0B55' size=6 > Splitting the data into the seven redshift bins</font>

In [11]:
# Binning confirmed redshift values into predefined intervals
# Each redshift value from Data_Redshift_Confirmed["z"] is placed into one of the following bins:
# 0.25–0.35, 0.35–0.45, 0.45–0.55, 0.55–0.65, 0.65–0.75, 0.75–0.85, 0.85–0.96

# Initialize redshift bins and their corresponding index lists
z_025_035, z_025_035_index = [], []
z_035_045, z_035_045_index = [], []
z_045_055, z_045_055_index = [], []
z_055_065, z_055_065_index = [], []
z_065_075, z_065_075_index = [], []
z_075_085, z_075_085_index = [], []
z_085_096, z_085_096_index = [], []

# Use enumerate to loop through redshift values while keeping track of their index.
# 'idx' is the index (position), and 'z' is the actual redshift value.
for idx, z in enumerate(Data_Redshift_Confirmed["z"]):
    if 0.25 <= z <= 0.35:
        z_025_035.append(z)
        z_025_035_index.append(idx)
    elif 0.35 <= z <= 0.45:
        z_035_045.append(z)
        z_035_045_index.append(idx)
    elif 0.45 <= z <= 0.55:
        z_045_055.append(z)
        z_045_055_index.append(idx)
    elif 0.55 <= z <= 0.65:
        z_055_065.append(z)
        z_055_065_index.append(idx)
    elif 0.65 <= z <= 0.75:
        z_065_075.append(z)
        z_065_075_index.append(idx)
    elif 0.75 <= z <= 0.85:
        z_075_085.append(z)
        z_075_085_index.append(idx)
    elif 0.85 <= z <= 0.96:
        z_085_096.append(z)
        z_085_096_index.append(idx)
    # Redshift values outside 0.25–0.96 are excluded


## <font color='#FF0B55' size=6 >Opening Files for the Images in each of the filters and redshifts</font>

In [12]:
G_File_Filters_025_035 = "Table_Filters_025_035_G.fits"
R_File_Filters_025_035 = "Table_Filters_025_035_R.fits"
I_File_Filters_025_035 = "Table_Filters_025_035_I.fits"
Z_File_Filters_025_035 = "Table_Filters_025_035_Z.fits"
Y_File_Filters_025_035 = "Table_Filters_025_035_Y.fits"


G_File_Filters_035_045 = "Table_Filters_035_045_G.fits"
R_File_Filters_035_045 = "Table_Filters_035_045_R.fits"
I_File_Filters_035_045 = "Table_Filters_035_045_I.fits"
Z_File_Filters_035_045 = "Table_Filters_035_045_Z.fits"
Y_File_Filters_035_045 = "Table_Filters_035_045_Y.fits"


G_File_Filters_045_055 = "Table_Filters_045_055_G.fits"
R_File_Filters_045_055 = "Table_Filters_045_055_R.fits"
I_File_Filters_045_055 = "Table_Filters_045_055_I.fits"
Z_File_Filters_045_055 = "Table_Filters_045_055_Z.fits"
Y_File_Filters_045_055 = "Table_Filters_045_055_Y.fits"

G_File_Filters_055_065 = "Table_Filters_055_065_G.fits"
R_File_Filters_055_065 = "Table_Filters_055_065_R.fits"
I_File_Filters_055_065 = "Table_Filters_055_065_I.fits"
Z_File_Filters_055_065 = "Table_Filters_055_065_Z.fits"
Y_File_Filters_055_065 = "Table_Filters_055_065_Y.fits"

G_File_Filters_065_075 = "Table_Filters_065_075_G.fits"
R_File_Filters_065_075 = "Table_Filters_065_075_R.fits"
I_File_Filters_065_075 = "Table_Filters_065_075_I.fits"
Z_File_Filters_065_075 = "Table_Filters_065_075_Z.fits"
Y_File_Filters_065_075 = "Table_Filters_065_075_Y.fits"

G_File_Filters_075_085 = "Table_Filters_075_085_G.fits"
R_File_Filters_075_085 = "Table_Filters_075_085_R.fits"
I_File_Filters_075_085 = "Table_Filters_075_085_I.fits"
Z_File_Filters_075_085 = "Table_Filters_075_085_Z.fits"
Y_File_Filters_075_085 = "Table_Filters_075_085_Y.fits"

G_File_Filters_085_096 = "Table_Filters_085_096_G.fits"
R_File_Filters_085_096 = "Table_Filters_085_096_R.fits"
I_File_Filters_085_096 = "Table_Filters_085_096_I.fits"
Z_File_Filters_085_096 = "Table_Filters_085_096_Z.fits"
Y_File_Filters_085_096 = "Table_Filters_085_096_Y.fits"



In [13]:
G_Filters_025_035 = Table.read(G_File_Filters_025_035, format = 'fits')
R_Filters_025_035 = Table.read(R_File_Filters_025_035, format = 'fits')
I_Filters_025_035 = Table.read(I_File_Filters_025_035, format = 'fits')
Z_Filters_025_035 = Table.read(Z_File_Filters_025_035, format = 'fits')
Y_Filters_025_035 = Table.read(Y_File_Filters_025_035, format = 'fits')


G_Filters_035_045 = Table.read(G_File_Filters_035_045, format = 'fits')
R_Filters_035_045 = Table.read(R_File_Filters_035_045, format = 'fits')
I_Filters_035_045 = Table.read(I_File_Filters_035_045, format = 'fits')
Z_Filters_035_045 = Table.read(Z_File_Filters_035_045, format = 'fits')
Y_Filters_035_045 = Table.read(Y_File_Filters_035_045, format = 'fits')


G_Filters_045_055 = Table.read(G_File_Filters_045_055, format = 'fits')
R_Filters_045_055 = Table.read(R_File_Filters_045_055, format = 'fits')
I_Filters_045_055 = Table.read(I_File_Filters_045_055, format = 'fits')
Z_Filters_045_055 = Table.read(Z_File_Filters_045_055, format = 'fits')
Y_Filters_045_055 = Table.read(Y_File_Filters_045_055, format = 'fits')


G_Filters_055_065 = Table.read(G_File_Filters_055_065, format = 'fits')
R_Filters_055_065 = Table.read(R_File_Filters_055_065, format = 'fits')
I_Filters_055_065 = Table.read(I_File_Filters_055_065, format = 'fits')
Z_Filters_055_065 = Table.read(Z_File_Filters_055_065, format = 'fits')
Y_Filters_055_065 = Table.read(Y_File_Filters_055_065, format = 'fits')


G_Filters_065_075 = Table.read(G_File_Filters_065_075, format = 'fits')
R_Filters_065_075 = Table.read(R_File_Filters_065_075, format = 'fits')
I_Filters_065_075 = Table.read(I_File_Filters_065_075, format = 'fits')
Z_Filters_065_075 = Table.read(Z_File_Filters_065_075, format = 'fits')
Y_Filters_065_075 = Table.read(Y_File_Filters_065_075, format = 'fits')


G_Filters_075_085 = Table.read(G_File_Filters_075_085, format = 'fits')
R_Filters_075_085 = Table.read(R_File_Filters_075_085, format = 'fits')
I_Filters_075_085 = Table.read(I_File_Filters_075_085, format = 'fits')
Z_Filters_075_085 = Table.read(Z_File_Filters_075_085, format = 'fits')
Y_Filters_075_085 = Table.read(Y_File_Filters_075_085, format = 'fits')

G_Filters_085_096 = Table.read(G_File_Filters_085_096, format = 'fits')
R_Filters_085_096 = Table.read(R_File_Filters_085_096, format = 'fits')
I_Filters_085_096 = Table.read(I_File_Filters_085_096, format = 'fits')
Z_Filters_085_096 = Table.read(Z_File_Filters_085_096, format = 'fits')
Y_Filters_085_096 = Table.read(Y_File_Filters_085_096, format = 'fits')

FileNotFoundError: [Errno 2] No such file or directory: 'Table_Filters_025_035_G.fits'

In [ ]:
G_Filters_025_035 = Table(G_Filters_025_035)
R_Filters_025_035 = Table(R_Filters_025_035)
I_Filters_025_035 = Table(I_Filters_025_035)
Z_Filters_025_035 = Table(Z_Filters_025_035)
Y_Filters_025_035 = Table(Y_Filters_025_035)

G_Filters_035_045 = Table(G_Filters_035_045)
R_Filters_035_045 = Table(R_Filters_035_045)
I_Filters_035_045 = Table(I_Filters_035_045)
Z_Filters_035_045 = Table(Z_Filters_035_045)
Y_Filters_035_045 = Table(Y_Filters_035_045)


G_Filters_045_055 = Table(G_Filters_045_055)
R_Filters_045_055 = Table(R_Filters_045_055)
I_Filters_045_055 = Table(I_Filters_045_055)
Z_Filters_045_055 = Table(Z_Filters_045_055)
Y_Filters_045_055 = Table(Y_Filters_045_055)


G_Filters_055_065 = Table(G_Filters_055_065)
R_Filters_055_065 = Table(R_Filters_055_065)
I_Filters_055_065 = Table(I_Filters_055_065)
Z_Filters_055_065 = Table(Z_Filters_055_065)
Y_Filters_055_065 = Table(Y_Filters_055_065)


G_Filters_065_075 = Table(G_Filters_065_075)
R_Filters_065_075 = Table(R_Filters_065_075)
I_Filters_065_075 = Table(I_Filters_065_075)
Z_Filters_065_075 = Table(Z_Filters_065_075)
Y_Filters_065_075 = Table(Y_Filters_065_075)


G_Filters_075_085 = Table(G_Filters_075_085)
R_Filters_075_085 = Table(R_Filters_075_085)
I_Filters_075_085 = Table(I_Filters_075_085)
Z_Filters_075_085 = Table(Z_Filters_075_085)
Y_Filters_075_085 = Table(Y_Filters_075_085)


G_Filters_085_096 = Table(G_Filters_085_096)
R_Filters_085_096 = Table(R_Filters_085_096)
I_Filters_085_096 = Table(I_Filters_085_096)
Z_Filters_085_096 = Table(Z_Filters_085_096)
Y_Filters_085_096 = Table(Y_Filters_085_096)


## <font color='#FF0B55' size=6 >Removeing the bad images</font>

In [ ]:
G_Filters_025_035.remove_row(1) #These are not the index numbers because removing a value changes the index for the next one
G_Filters_025_035.remove_row(6)
G_Filters_025_035.remove_row(25)
G_Filters_025_035.remove_row(25)

G_Filters_035_045.remove_row(18)
G_Filters_035_045.remove_row(24)
G_Filters_035_045.remove_row(29)
G_Filters_035_045.remove_row(37)
G_Filters_035_045.remove_row(49)
G_Filters_035_045.remove_row(60)
G_Filters_035_045.remove_row(77)
G_Filters_035_045.remove_row(80)


G_Filters_045_055.remove_row(3)
G_Filters_045_055.remove_row(14)
G_Filters_045_055.remove_row(23)
G_Filters_045_055.remove_row(35)
G_Filters_045_055.remove_row(50)

G_Filters_055_065.remove_row(12)
G_Filters_055_065.remove_row(13)
G_Filters_055_065.remove_row(21)
G_Filters_055_065.remove_row(37)
G_Filters_055_065.remove_row(39)
G_Filters_055_065.remove_row(48)


G_Filters_065_075.remove_row(6)
G_Filters_065_075.remove_row(9)
G_Filters_065_075.remove_row(53)

G_Filters_075_085.remove_row(16)
G_Filters_075_085.remove_row(17)
G_Filters_075_085.remove_row(43)
G_Filters_075_085.remove_row(59)


G_Filters_085_096.remove_row(5)
G_Filters_085_096.remove_row(8)
G_Filters_085_096.remove_row(29)
G_Filters_085_096.remove_row(40)
G_Filters_085_096.remove_row(59)
G_Filters_085_096.remove_row(84)







R_Filters_025_035.remove_row(1)
R_Filters_025_035.remove_row(26)
R_Filters_025_035.remove_row(26)

R_Filters_035_045.remove_row(18)
R_Filters_035_045.remove_row(22)
R_Filters_035_045.remove_row(23)
R_Filters_035_045.remove_row(28)
R_Filters_035_045.remove_row(36)
R_Filters_035_045.remove_row(41)
R_Filters_035_045.remove_row(47)
R_Filters_035_045.remove_row(47)
R_Filters_035_045.remove_row(52)
R_Filters_035_045.remove_row(56)
R_Filters_035_045.remove_row(73)
R_Filters_035_045.remove_row(75)
R_Filters_035_045.remove_row(75)

R_Filters_045_055.remove_row(3)
R_Filters_045_055.remove_row(10)
R_Filters_045_055.remove_row(13)
R_Filters_045_055.remove_row(15)
R_Filters_045_055.remove_row(21)
R_Filters_045_055.remove_row(33)
R_Filters_045_055.remove_row(43)
R_Filters_045_055.remove_row(47)

R_Filters_055_065.remove_row(7)
R_Filters_055_065.remove_row(11)
R_Filters_055_065.remove_row(12)
R_Filters_055_065.remove_row(12)
R_Filters_055_065.remove_row(19)
R_Filters_055_065.remove_row(25)
R_Filters_055_065.remove_row(34)
R_Filters_055_065.remove_row(36)
R_Filters_055_065.remove_row(45)

R_Filters_065_075.remove_row(6)
R_Filters_065_075.remove_row(9)
R_Filters_065_075.remove_row(15)
R_Filters_065_075.remove_row(26)

R_Filters_075_085.remove_row(16)
R_Filters_075_085.remove_row(17)
R_Filters_075_085.remove_row(18)
R_Filters_075_085.remove_row(18)
R_Filters_075_085.remove_row(18)
R_Filters_075_085.remove_row(40)
R_Filters_075_085.remove_row(48)
R_Filters_075_085.remove_row(55)
R_Filters_075_085.remove_row(74)

R_Filters_085_096.remove_row(5)
R_Filters_085_096.remove_row(8)
R_Filters_085_096.remove_row(35)
R_Filters_085_096.remove_row(40)
R_Filters_085_096.remove_row(59)
R_Filters_085_096.remove_row(84)





I_Filters_025_035.remove_row(1)
I_Filters_025_035.remove_row(23)

I_Filters_035_045.remove_row(1)
I_Filters_035_045.remove_row(15)
I_Filters_035_045.remove_row(15)
I_Filters_035_045.remove_row(16)
I_Filters_035_045.remove_row(20)
I_Filters_035_045.remove_row(28)
I_Filters_035_045.remove_row(35)
I_Filters_035_045.remove_row(46)
I_Filters_035_045.remove_row(60)
I_Filters_035_045.remove_row(60)
I_Filters_035_045.remove_row(62)
I_Filters_035_045.remove_row(62)

I_Filters_045_055.remove_row(3)
I_Filters_045_055.remove_row(6)
I_Filters_045_055.remove_row(8)
I_Filters_045_055.remove_row(10)
I_Filters_045_055.remove_row(15)
I_Filters_045_055.remove_row(22)
I_Filters_045_055.remove_row(24)
I_Filters_045_055.remove_row(31)
I_Filters_045_055.remove_row(35)

I_Filters_055_065.remove_row(10)
I_Filters_055_065.remove_row(11)
I_Filters_055_065.remove_row(11)
I_Filters_055_065.remove_row(13)
I_Filters_055_065.remove_row(16)
I_Filters_055_065.remove_row(24)
I_Filters_055_065.remove_row(28)
I_Filters_055_065.remove_row(35)
I_Filters_055_065.remove_row(35)

I_Filters_065_075.remove_row(6)
I_Filters_065_075.remove_row(8)
I_Filters_065_075.remove_row(8)
I_Filters_065_075.remove_row(13)
I_Filters_065_075.remove_row(13)
I_Filters_065_075.remove_row(13)
I_Filters_065_075.remove_row(18)
I_Filters_065_075.remove_row(20)
I_Filters_065_075.remove_row(27)
I_Filters_065_075.remove_row(35)
I_Filters_065_075.remove_row(38)

I_Filters_075_085.remove_row(5)
I_Filters_075_085.remove_row(12)
I_Filters_075_085.remove_row(12)
I_Filters_075_085.remove_row(13)
I_Filters_075_085.remove_row(13)
I_Filters_075_085.remove_row(13)
I_Filters_075_085.remove_row(30)
I_Filters_075_085.remove_row(30)
I_Filters_075_085.remove_row(36)
I_Filters_075_085.remove_row(41)
I_Filters_075_085.remove_row(41)
I_Filters_075_085.remove_row(47)

I_Filters_085_096.remove_row(5)
I_Filters_085_096.remove_row(7)
I_Filters_085_096.remove_row(14)
I_Filters_085_096.remove_row(14)
I_Filters_085_096.remove_row(28)
I_Filters_085_096.remove_row(32)
I_Filters_085_096.remove_row(35)
I_Filters_085_096.remove_row(48)
I_Filters_085_096.remove_row(58)
I_Filters_085_096.remove_row(70)
I_Filters_085_096.remove_row(70)








Z_Filters_025_035.remove_row(20)
Z_Filters_025_035.remove_row(26)

Z_Filters_035_045.remove_row(2)
Z_Filters_035_045.remove_row(17)
Z_Filters_035_045.remove_row(21)
Z_Filters_035_045.remove_row(28)
Z_Filters_035_045.remove_row(44)
Z_Filters_035_045.remove_row(47)
Z_Filters_035_045.remove_row(74)

Z_Filters_045_055.remove_row(11)
Z_Filters_045_055.remove_row(14)
Z_Filters_045_055.remove_row(23)
Z_Filters_045_055.remove_row(25)
Z_Filters_045_055.remove_row(41)
Z_Filters_045_055.remove_row(49)

Z_Filters_055_065.remove_row(12)
Z_Filters_055_065.remove_row(13)
Z_Filters_055_065.remove_row(21)
Z_Filters_055_065.remove_row(36)
Z_Filters_055_065.remove_row(38)
Z_Filters_055_065.remove_row(47)

Z_Filters_065_075.remove_row(6)
Z_Filters_065_075.remove_row(8)
Z_Filters_065_075.remove_row(8)
Z_Filters_065_075.remove_row(14)
Z_Filters_065_075.remove_row(25)
Z_Filters_065_075.remove_row(26)
Z_Filters_065_075.remove_row(43)
Z_Filters_065_075.remove_row(47)

Z_Filters_075_085.remove_row(16)
Z_Filters_075_085.remove_row(17)
Z_Filters_075_085.remove_row(18)
Z_Filters_075_085.remove_row(18)
Z_Filters_075_085.remove_row(18)
Z_Filters_075_085.remove_row(26)
Z_Filters_075_085.remove_row(39)
Z_Filters_075_085.remove_row(47)
Z_Filters_075_085.remove_row(52)
Z_Filters_075_085.remove_row(63)

Z_Filters_085_096.remove_row(5)
Z_Filters_085_096.remove_row(5)
Z_Filters_085_096.remove_row(7)
Z_Filters_085_096.remove_row(18)
Z_Filters_085_096.remove_row(39)
Z_Filters_085_096.remove_row(58)
Z_Filters_085_096.remove_row(67)
Z_Filters_085_096.remove_row(68)
Z_Filters_085_096.remove_row(81)





Y_Filters_025_035.remove_row(1)
Y_Filters_025_035.remove_row(20)
Y_Filters_025_035.remove_row(20)

Y_Filters_035_045.remove_row(2)
Y_Filters_035_045.remove_row(10)
Y_Filters_035_045.remove_row(12)
Y_Filters_035_045.remove_row(15)
Y_Filters_035_045.remove_row(22)
Y_Filters_035_045.remove_row(37)
Y_Filters_035_045.remove_row(49)
Y_Filters_035_045.remove_row(53)

Y_Filters_045_055.remove_row(6)
Y_Filters_045_055.remove_row(8)
Y_Filters_045_055.remove_row(10)
Y_Filters_045_055.remove_row(13)
Y_Filters_045_055.remove_row(21)
Y_Filters_045_055.remove_row(22)

Y_Filters_055_065.remove_row(2)
Y_Filters_055_065.remove_row(8)
Y_Filters_055_065.remove_row(12)
Y_Filters_055_065.remove_row(28)
Y_Filters_055_065.remove_row(28)

Y_Filters_065_075.remove_row(5)
Y_Filters_065_075.remove_row(12)
Y_Filters_065_075.remove_row(37)

Y_Filters_075_085.remove_row(4)
Y_Filters_075_085.remove_row(8)
Y_Filters_075_085.remove_row(8)
Y_Filters_075_085.remove_row(15)
Y_Filters_075_085.remove_row(25)
Y_Filters_075_085.remove_row(30)

Y_Filters_085_096.remove_row(7)
Y_Filters_085_096.remove_row(12)
Y_Filters_085_096.remove_row(17)
Y_Filters_085_096.remove_row(19)
Y_Filters_085_096.remove_row(42)
Y_Filters_085_096.remove_row(52)
Y_Filters_085_096.remove_row(59)




## <font color='#FF0B55' size=6 >Creating arrays that are the smae length for each of the unremoved files and creating a table of them</font>

In [ ]:
hstack_for_max = np.array([len(G_Filters_025_035["G_Detect_ID"]), len(G_Filters_025_035["G_Detect_ID"]), len(G_Filters_045_055["G_Detect_ID"]), len(G_Filters_055_065["G_Detect_ID"]), len(G_Filters_065_075["G_Detect_ID"]), len(G_Filters_075_085["G_Detect_ID"]), len(G_Filters_085_096["G_Detect_ID"]), len(R_Filters_025_035["R_Detect_ID"]), len(R_Filters_025_035["R_Detect_ID"]), len(R_Filters_045_055["R_Detect_ID"]), len(R_Filters_055_065["R_Detect_ID"]), len(R_Filters_065_075["R_Detect_ID"]), len(R_Filters_075_085["R_Detect_ID"]), len(R_Filters_085_096["R_Detect_ID"]), len(I_Filters_025_035["I_Detect_ID"]), len(I_Filters_025_035["I_Detect_ID"]), len(I_Filters_045_055["I_Detect_ID"]), len(I_Filters_055_065["I_Detect_ID"]), len(I_Filters_065_075["I_Detect_ID"]), len(I_Filters_075_085["I_Detect_ID"]), len(I_Filters_085_096["I_Detect_ID"]), len(Z_Filters_025_035["Z_Detect_ID"]), len(Z_Filters_025_035["Z_Detect_ID"]), len(Z_Filters_045_055["Z_Detect_ID"]), len(Z_Filters_055_065["Z_Detect_ID"]), len(Z_Filters_065_075["Z_Detect_ID"]), len(Z_Filters_075_085["Z_Detect_ID"]), len(Z_Filters_085_096["Z_Detect_ID"]), len(Y_Filters_025_035["Y_Detect_ID"]), len(Y_Filters_025_035["Y_Detect_ID"]), len(Y_Filters_045_055["Y_Detect_ID"]), len(Y_Filters_055_065["Y_Detect_ID"]), len(Y_Filters_065_075["Y_Detect_ID"]), len(Y_Filters_075_085["Y_Detect_ID"]), len(Y_Filters_085_096["Y_Detect_ID"])])
max_num_zeros = max(hstack_for_max)

G_Filters_For_Table_025_035 =  np.pad(G_Filters_025_035["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_025_035["G_Detect_ID"])), 'constant')
R_Filters_For_Table_025_035 =  np.pad(R_Filters_025_035["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_025_035["R_Detect_ID"])), 'constant')
I_Filters_For_Table_025_035 =  np.pad(I_Filters_025_035["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_025_035["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_025_035 =  np.pad(Z_Filters_025_035["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_025_035["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_025_035 =  np.pad(Y_Filters_025_035["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_025_035["Y_Detect_ID"])), 'constant')
G_Filters_For_Table_035_045 =  np.pad(G_Filters_035_045["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_035_045["G_Detect_ID"])), 'constant')
R_Filters_For_Table_035_045 =  np.pad(R_Filters_035_045["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_035_045["R_Detect_ID"])), 'constant')
I_Filters_For_Table_035_045 =  np.pad(I_Filters_035_045["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_035_045["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_035_045 =  np.pad(Z_Filters_035_045["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_035_045["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_035_045 =  np.pad(Y_Filters_035_045["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_035_045["Y_Detect_ID"])), 'constant')
G_Filters_For_Table_045_055 =  np.pad(G_Filters_045_055["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_045_055["G_Detect_ID"])), 'constant')
R_Filters_For_Table_045_055 =  np.pad(R_Filters_045_055["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_045_055["R_Detect_ID"])), 'constant')
I_Filters_For_Table_045_055 =  np.pad(I_Filters_045_055["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_045_055["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_045_055 =  np.pad(Z_Filters_045_055["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_045_055["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_045_055 =  np.pad(Y_Filters_045_055["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_045_055["Y_Detect_ID"])), 'constant')
G_Filters_For_Table_055_065 =  np.pad(G_Filters_055_065["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_055_065["G_Detect_ID"])), 'constant')
R_Filters_For_Table_055_065 =  np.pad(R_Filters_055_065["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_055_065["R_Detect_ID"])), 'constant')
I_Filters_For_Table_055_065 =  np.pad(I_Filters_055_065["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_055_065["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_055_065 =  np.pad(Z_Filters_055_065["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_055_065["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_055_065 =  np.pad(Y_Filters_055_065["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_055_065["Y_Detect_ID"])), 'constant')
G_Filters_For_Table_065_075 =  np.pad(G_Filters_065_075["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_065_075["G_Detect_ID"])), 'constant')
R_Filters_For_Table_065_075 =  np.pad(R_Filters_065_075["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_065_075["R_Detect_ID"])), 'constant')
I_Filters_For_Table_065_075 =  np.pad(I_Filters_065_075["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_065_075["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_065_075 =  np.pad(Z_Filters_065_075["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_065_075["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_065_075 =  np.pad(Y_Filters_065_075["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_065_075["Y_Detect_ID"])), 'constant')
G_Filters_For_Table_075_085 =  np.pad(G_Filters_075_085["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_075_085["G_Detect_ID"])), 'constant')
R_Filters_For_Table_075_085 =  np.pad(R_Filters_075_085["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_075_085["R_Detect_ID"])), 'constant')
I_Filters_For_Table_075_085 =  np.pad(I_Filters_075_085["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_075_085["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_075_085 =  np.pad(Z_Filters_075_085["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_075_085["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_075_085 =  np.pad(Y_Filters_075_085["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_075_085["Y_Detect_ID"])), 'constant')
G_Filters_For_Table_085_096 =  np.pad(G_Filters_085_096["G_Detect_ID"], (0, max_num_zeros - len(G_Filters_085_096["G_Detect_ID"])), 'constant')
R_Filters_For_Table_085_096 =  np.pad(R_Filters_085_096["R_Detect_ID"], (0, max_num_zeros - len(R_Filters_085_096["R_Detect_ID"])), 'constant')
I_Filters_For_Table_085_096 =  np.pad(I_Filters_085_096["I_Detect_ID"], (0, max_num_zeros - len(I_Filters_085_096["I_Detect_ID"])), 'constant')
Z_Filters_For_Table_085_096 =  np.pad(Z_Filters_085_096["Z_Detect_ID"], (0, max_num_zeros - len(Z_Filters_085_096["Z_Detect_ID"])), 'constant')
Y_Filters_For_Table_085_096 =  np.pad(Y_Filters_085_096["Y_Detect_ID"], (0, max_num_zeros - len(Y_Filters_085_096["Y_Detect_ID"])), 'constant')


In [ ]:
IDs = np.hstack((G_Filters_For_Table_025_035, R_Filters_For_Table_025_035, I_Filters_For_Table_025_035, Z_Filters_For_Table_025_035, Y_Filters_For_Table_025_035, G_Filters_For_Table_025_035, R_Filters_For_Table_025_035, I_Filters_For_Table_025_035, Z_Filters_For_Table_025_035, Y_Filters_For_Table_025_035, G_Filters_For_Table_045_055, R_Filters_For_Table_045_055, I_Filters_For_Table_045_055, Z_Filters_For_Table_045_055, Y_Filters_For_Table_045_055, G_Filters_For_Table_055_065, R_Filters_For_Table_055_065, I_Filters_For_Table_055_065, Z_Filters_For_Table_055_065, Y_Filters_For_Table_055_065, G_Filters_For_Table_065_075, R_Filters_For_Table_065_075, I_Filters_For_Table_065_075, Z_Filters_For_Table_065_075, Y_Filters_For_Table_065_075, G_Filters_For_Table_075_085, R_Filters_For_Table_075_085, I_Filters_For_Table_075_085, Z_Filters_For_Table_075_085, Y_Filters_For_Table_075_085, G_Filters_For_Table_085_096, R_Filters_For_Table_085_096, I_Filters_For_Table_085_096, Z_Filters_For_Table_085_096, Y_Filters_For_Table_085_096))
second_col = np.full(len(IDs), 0)


In [ ]:
Table_start_a = ([(IDs[0])])
Table_start_b = ([(second_col[0])])

Table_b = QTable([Table_start_a, Table_start_b], names=("Detect_ID", "Zeros"))

In [ ]:
for i in np.arange(1,len(IDs),1):
    a = ([(IDs[i])])
    b = ([(second_col[i])])
    
    Table_b.add_row([a,b])

In [ ]:
Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/Table_Detect_IDs_From_Images.fits", overwrite = True)

## <font color='#FF0B55' size=6 >Getting the data from the unremoved data</font>

In [ ]:
G_Data_025_035 = G_Filters_025_035["G_Data"]
G_RA_025_035 = G_Filters_025_035["G_RA"]
G_DEC_025_035 = G_Filters_025_035["G_DEC"]
G_Detect_ID_025_035 = G_Filters_025_035["G_Detect_ID"]
G_Z_025_035 = G_Filters_025_035["G_Z"]
G_Mag_025_035 = G_Filters_025_035["G_Mag"]
G_Mag_Err_025_035 = G_Filters_025_035["G_Mag_Err"]

R_Data_025_035 = R_Filters_025_035["R_Data"]
R_RA_025_035 = R_Filters_025_035["R_RA"]
R_DEC_025_035 = R_Filters_025_035["R_DEC"]
R_Detect_ID_025_035 = R_Filters_025_035["R_Detect_ID"]
R_Z_025_035 = R_Filters_025_035["R_Z"]
R_Mag_025_035 = R_Filters_025_035["R_Mag"]
R_Mag_Err_025_035 = R_Filters_025_035["R_Mag_Err"]


I_Data_025_035 = I_Filters_025_035["I_Data"]
I_RA_025_035 = I_Filters_025_035["I_RA"]
I_DEC_025_035 = I_Filters_025_035["I_DEC"]
I_Detect_ID_025_035 = I_Filters_025_035["I_Detect_ID"]
I_Z_025_035 = I_Filters_025_035["I_Z"]
I_Mag_025_035 = I_Filters_025_035["I_Mag"]
I_Mag_Err_025_035 = I_Filters_025_035["I_Mag_Err"]


Z_Data_025_035 = Z_Filters_025_035["Z_Data"]
Z_RA_025_035 = Z_Filters_025_035["Z_RA"]
Z_DEC_025_035 = Z_Filters_025_035["Z_DEC"]
Z_Detect_ID_025_035 = Z_Filters_025_035["Z_Detect_ID"]
Z_Z_025_035 = Z_Filters_025_035["Z_Z"]
Z_Mag_025_035 = Z_Filters_025_035["Z_Mag"]
Z_Mag_Err_025_035 = Z_Filters_025_035["Z_Mag_Err"]


Y_Data_025_035 = Y_Filters_025_035["Y_Data"]
Y_RA_025_035 = Y_Filters_025_035["Y_RA"]
Y_DEC_025_035 = Y_Filters_025_035["Y_DEC"]
Y_Detect_ID_025_035 = Y_Filters_025_035["Y_Detect_ID"]
Y_Z_025_035 = Y_Filters_025_035["Y_Z"]
Y_Mag_025_035 = Y_Filters_025_035["Y_Mag"]
Y_Mag_Err_025_035 = Y_Filters_025_035["Y_Mag_Err"]



In [ ]:
G_Data_035_045 = G_Filters_035_045["G_Data"]
G_RA_035_045 = G_Filters_035_045["G_RA"]
G_DEC_035_045 = G_Filters_035_045["G_DEC"]
G_Detect_ID_035_045 = G_Filters_035_045["G_Detect_ID"]
G_Z_035_045 = G_Filters_035_045["G_Z"]
G_Mag_035_045 = G_Filters_035_045["G_Mag"]
G_Mag_Err_035_045 = G_Filters_035_045["G_Mag_Err"]

R_Data_035_045 = R_Filters_035_045["R_Data"]
R_RA_035_045 = R_Filters_035_045["R_RA"]
R_DEC_035_045 = R_Filters_035_045["R_DEC"]
R_Detect_ID_035_045 = R_Filters_035_045["R_Detect_ID"]
R_Z_035_045 = R_Filters_035_045["R_Z"]
R_Mag_035_045 = R_Filters_035_045["R_Mag"]
R_Mag_Err_035_045 = R_Filters_035_045["R_Mag_Err"]


I_Data_035_045 = I_Filters_035_045["I_Data"]
I_RA_035_045 = I_Filters_035_045["I_RA"]
I_DEC_035_045 = I_Filters_035_045["I_DEC"]
I_Detect_ID_035_045 = I_Filters_035_045["I_Detect_ID"]
I_Z_035_045 = I_Filters_035_045["I_Z"]
I_Mag_035_045 = I_Filters_035_045["I_Mag"]
I_Mag_Err_035_045 = I_Filters_035_045["I_Mag_Err"]


Z_Data_035_045 = Z_Filters_035_045["Z_Data"]
Z_RA_035_045 = Z_Filters_035_045["Z_RA"]
Z_DEC_035_045 = Z_Filters_035_045["Z_DEC"]
Z_Detect_ID_035_045 = Z_Filters_035_045["Z_Detect_ID"]
Z_Z_035_045 = Z_Filters_035_045["Z_Z"]
Z_Mag_035_045 = Z_Filters_035_045["Z_Mag"]
Z_Mag_Err_035_045 = Z_Filters_035_045["Z_Mag_Err"]


Y_Data_035_045 = Y_Filters_035_045["Y_Data"]
Y_RA_035_045 = Y_Filters_035_045["Y_RA"]
Y_DEC_035_045 = Y_Filters_035_045["Y_DEC"]
Y_Detect_ID_035_045 = Y_Filters_035_045["Y_Detect_ID"]
Y_Z_035_045 = Y_Filters_035_045["Y_Z"]
Y_Mag_035_045 = Y_Filters_035_045["Y_Mag"]
Y_Mag_Err_035_045 = Y_Filters_035_045["Y_Mag_Err"]


In [ ]:
G_Data_045_055 = G_Filters_045_055["G_Data"]
G_RA_045_055 = G_Filters_045_055["G_RA"]
G_DEC_045_055 = G_Filters_045_055["G_DEC"]
G_Detect_ID_045_055 = G_Filters_045_055["G_Detect_ID"]
G_Z_045_055 = G_Filters_045_055["G_Z"]
G_Mag_045_055 = G_Filters_045_055["G_Mag"]
G_Mag_Err_045_055 = G_Filters_045_055["G_Mag_Err"]

R_Data_045_055 = R_Filters_045_055["R_Data"]
R_RA_045_055 = R_Filters_045_055["R_RA"]
R_DEC_045_055 = R_Filters_045_055["R_DEC"]
R_Detect_ID_045_055 = R_Filters_045_055["R_Detect_ID"]
R_Z_045_055 = R_Filters_045_055["R_Z"]
R_Mag_045_055 = R_Filters_045_055["R_Mag"]
R_Mag_Err_045_055 = R_Filters_045_055["R_Mag_Err"]


I_Data_045_055 = I_Filters_045_055["I_Data"]
I_RA_045_055 = I_Filters_045_055["I_RA"]
I_DEC_045_055 = I_Filters_045_055["I_DEC"]
I_Detect_ID_045_055 = I_Filters_045_055["I_Detect_ID"]
I_Z_045_055 = I_Filters_045_055["I_Z"]
I_Mag_045_055 = I_Filters_045_055["I_Mag"]
I_Mag_Err_045_055 = I_Filters_045_055["I_Mag_Err"]


Z_Data_045_055 = Z_Filters_045_055["Z_Data"]
Z_RA_045_055 = Z_Filters_045_055["Z_RA"]
Z_DEC_045_055 = Z_Filters_045_055["Z_DEC"]
Z_Detect_ID_045_055 = Z_Filters_045_055["Z_Detect_ID"]
Z_Z_045_055 = Z_Filters_045_055["Z_Z"]
Z_Mag_045_055 = Z_Filters_045_055["Z_Mag"]
Z_Mag_Err_045_055 = Z_Filters_045_055["Z_Mag_Err"]


Y_Data_045_055 = Y_Filters_045_055["Y_Data"]
Y_RA_045_055 = Y_Filters_045_055["Y_RA"]
Y_DEC_045_055 = Y_Filters_045_055["Y_DEC"]
Y_Detect_ID_045_055 = Y_Filters_045_055["Y_Detect_ID"]
Y_Z_045_055 = Y_Filters_045_055["Y_Z"]
Y_Mag_045_055 = Y_Filters_045_055["Y_Mag"]
Y_Mag_Err_045_055 = Y_Filters_045_055["Y_Mag_Err"]


In [ ]:
G_Data_055_065 = G_Filters_055_065["G_Data"]
G_RA_055_065 = G_Filters_055_065["G_RA"]
G_DEC_055_065 = G_Filters_055_065["G_DEC"]
G_Detect_ID_055_065 = G_Filters_055_065["G_Detect_ID"]
G_Z_055_065 = G_Filters_055_065["G_Z"]
G_Mag_055_065 = G_Filters_055_065["G_Mag"]
G_Mag_Err_055_065 = G_Filters_055_065["G_Mag_Err"]

R_Data_055_065 = R_Filters_055_065["R_Data"]
R_RA_055_065 = R_Filters_055_065["R_RA"]
R_DEC_055_065 = R_Filters_055_065["R_DEC"]
R_Detect_ID_055_065 = R_Filters_055_065["R_Detect_ID"]
R_Z_055_065 = R_Filters_055_065["R_Z"]
R_Mag_055_065 = R_Filters_055_065["R_Mag"]
R_Mag_Err_055_065 = R_Filters_055_065["R_Mag_Err"]


I_Data_055_065 = I_Filters_055_065["I_Data"]
I_RA_055_065 = I_Filters_055_065["I_RA"]
I_DEC_055_065 = I_Filters_055_065["I_DEC"]
I_Detect_ID_055_065 = I_Filters_055_065["I_Detect_ID"]
I_Z_055_065 = I_Filters_055_065["I_Z"]
I_Mag_055_065 = I_Filters_055_065["I_Mag"]
I_Mag_Err_055_065 = I_Filters_055_065["I_Mag_Err"]


Z_Data_055_065 = Z_Filters_055_065["Z_Data"]
Z_RA_055_065 = Z_Filters_055_065["Z_RA"]
Z_DEC_055_065 = Z_Filters_055_065["Z_DEC"]
Z_Detect_ID_055_065 = Z_Filters_055_065["Z_Detect_ID"]
Z_Z_055_065 = Z_Filters_055_065["Z_Z"]
Z_Mag_055_065 = Z_Filters_055_065["Z_Mag"]
Z_Mag_Err_055_065 = Z_Filters_055_065["Z_Mag_Err"]


Y_Data_055_065 = Y_Filters_055_065["Y_Data"]
Y_RA_055_065 = Y_Filters_055_065["Y_RA"]
Y_DEC_055_065 = Y_Filters_055_065["Y_DEC"]
Y_Detect_ID_055_065 = Y_Filters_055_065["Y_Detect_ID"]
Y_Z_055_065 = Y_Filters_055_065["Y_Z"]
Y_Mag_055_065 = Y_Filters_055_065["Y_Mag"]
Y_Mag_Err_055_065 = Y_Filters_055_065["Y_Mag_Err"]


In [ ]:
G_Data_065_075 = G_Filters_065_075["G_Data"]
G_RA_065_075 = G_Filters_065_075["G_RA"]
G_DEC_065_075 = G_Filters_065_075["G_DEC"]
G_Detect_ID_065_075 = G_Filters_065_075["G_Detect_ID"]
G_Z_065_075 = G_Filters_065_075["G_Z"]
G_Mag_065_075 = G_Filters_065_075["G_Mag"]
G_Mag_Err_065_075 = G_Filters_065_075["G_Mag_Err"]

R_Data_065_075 = R_Filters_065_075["R_Data"]
R_RA_065_075 = R_Filters_065_075["R_RA"]
R_DEC_065_075 = R_Filters_065_075["R_DEC"]
R_Detect_ID_065_075 = R_Filters_065_075["R_Detect_ID"]
R_Z_065_075 = R_Filters_065_075["R_Z"]
R_Mag_065_075 = R_Filters_065_075["R_Mag"]
R_Mag_Err_065_075 = R_Filters_065_075["R_Mag_Err"]


I_Data_065_075 = I_Filters_065_075["I_Data"]
I_RA_065_075 = I_Filters_065_075["I_RA"]
I_DEC_065_075 = I_Filters_065_075["I_DEC"]
I_Detect_ID_065_075 = I_Filters_065_075["I_Detect_ID"]
I_Z_065_075 = I_Filters_065_075["I_Z"]
I_Mag_065_075 = I_Filters_065_075["I_Mag"]
I_Mag_Err_065_075 = I_Filters_065_075["I_Mag_Err"]


Z_Data_065_075 = Z_Filters_065_075["Z_Data"]
Z_RA_065_075 = Z_Filters_065_075["Z_RA"]
Z_DEC_065_075 = Z_Filters_065_075["Z_DEC"]
Z_Detect_ID_065_075 = Z_Filters_065_075["Z_Detect_ID"]
Z_Z_065_075 = Z_Filters_065_075["Z_Z"]
Z_Mag_065_075 = Z_Filters_065_075["Z_Mag"]
Z_Mag_Err_065_075 = Z_Filters_065_075["Z_Mag_Err"]


Y_Data_065_075 = Y_Filters_065_075["Y_Data"]
Y_RA_065_075 = Y_Filters_065_075["Y_RA"]
Y_DEC_065_075 = Y_Filters_065_075["Y_DEC"]
Y_Detect_ID_065_075 = Y_Filters_065_075["Y_Detect_ID"]
Y_Z_065_075 = Y_Filters_065_075["Y_Z"]
Y_Mag_065_075 = Y_Filters_065_075["Y_Mag"]
Y_Mag_Err_065_075 = Y_Filters_065_075["Y_Mag_Err"]


In [ ]:
G_Data_075_085 = G_Filters_075_085["G_Data"]
G_RA_075_085 = G_Filters_075_085["G_RA"]
G_DEC_075_085 = G_Filters_075_085["G_DEC"]
G_Detect_ID_075_085 = G_Filters_075_085["G_Detect_ID"]
G_Z_075_085 = G_Filters_075_085["G_Z"]
G_Mag_075_085 = G_Filters_075_085["G_Mag"]
G_Mag_Err_075_085 = G_Filters_075_085["G_Mag_Err"]

R_Data_075_085 = R_Filters_075_085["R_Data"]
R_RA_075_085 = R_Filters_075_085["R_RA"]
R_DEC_075_085 = R_Filters_075_085["R_DEC"]
R_Detect_ID_075_085 = R_Filters_075_085["R_Detect_ID"]
R_Z_075_085 = R_Filters_075_085["R_Z"]
R_Mag_075_085 = R_Filters_075_085["R_Mag"]
R_Mag_Err_075_085 = R_Filters_075_085["R_Mag_Err"]


I_Data_075_085 = I_Filters_075_085["I_Data"]
I_RA_075_085 = I_Filters_075_085["I_RA"]
I_DEC_075_085 = I_Filters_075_085["I_DEC"]
I_Detect_ID_075_085 = I_Filters_075_085["I_Detect_ID"]
I_Z_075_085 = I_Filters_075_085["I_Z"]
I_Mag_075_085 = I_Filters_075_085["I_Mag"]
I_Mag_Err_075_085 = I_Filters_075_085["I_Mag_Err"]


Z_Data_075_085 = Z_Filters_075_085["Z_Data"]
Z_RA_075_085 = Z_Filters_075_085["Z_RA"]
Z_DEC_075_085 = Z_Filters_075_085["Z_DEC"]
Z_Detect_ID_075_085 = Z_Filters_075_085["Z_Detect_ID"]
Z_Z_075_085 = Z_Filters_075_085["Z_Z"]
Z_Mag_075_085 = Z_Filters_075_085["Z_Mag"]
Z_Mag_Err_075_085 = Z_Filters_075_085["Z_Mag_Err"]


Y_Data_075_085 = Y_Filters_075_085["Y_Data"]
Y_RA_075_085 = Y_Filters_075_085["Y_RA"]
Y_DEC_075_085 = Y_Filters_075_085["Y_DEC"]
Y_Detect_ID_075_085 = Y_Filters_075_085["Y_Detect_ID"]
Y_Z_075_085 = Y_Filters_075_085["Y_Z"]
Y_Mag_075_085 = Y_Filters_075_085["Y_Mag"]
Y_Mag_Err_075_085 = Y_Filters_075_085["Y_Mag_Err"]


In [ ]:
G_Data_085_096 = G_Filters_085_096["G_Data"]
G_RA_085_096 = G_Filters_085_096["G_RA"]
G_DEC_085_096 = G_Filters_085_096["G_DEC"]
G_Detect_ID_085_096 = G_Filters_085_096["G_Detect_ID"]
G_Z_085_096 = G_Filters_085_096["G_Z"]
G_Mag_085_096 = G_Filters_085_096["G_Mag"]
G_Mag_Err_085_096 = G_Filters_085_096["G_Mag_Err"]

R_Data_085_096 = R_Filters_085_096["R_Data"]
R_RA_085_096 = R_Filters_085_096["R_RA"]
R_DEC_085_096 = R_Filters_085_096["R_DEC"]
R_Detect_ID_085_096 = R_Filters_085_096["R_Detect_ID"]
R_Z_085_096 = R_Filters_085_096["R_Z"]
R_Mag_085_096 = R_Filters_085_096["R_Mag"]
R_Mag_Err_085_096 = R_Filters_085_096["R_Mag_Err"]


I_Data_085_096 = I_Filters_085_096["I_Data"]
I_RA_085_096 = I_Filters_085_096["I_RA"]
I_DEC_085_096 = I_Filters_085_096["I_DEC"]
I_Detect_ID_085_096 = I_Filters_085_096["I_Detect_ID"]
I_Z_085_096 = I_Filters_085_096["I_Z"]
I_Mag_085_096 = I_Filters_085_096["I_Mag"]
I_Mag_Err_085_096 = I_Filters_085_096["I_Mag_Err"]


Z_Data_085_096 = Z_Filters_085_096["Z_Data"]
Z_RA_085_096 = Z_Filters_085_096["Z_RA"]
Z_DEC_085_096 = Z_Filters_085_096["Z_DEC"]
Z_Detect_ID_085_096 = Z_Filters_085_096["Z_Detect_ID"]
Z_Z_085_096 = Z_Filters_085_096["Z_Z"]
Z_Mag_085_096 = Z_Filters_085_096["Z_Mag"]
Z_Mag_Err_085_096 = Z_Filters_085_096["Z_Mag_Err"]


Y_Data_085_096 = Y_Filters_085_096["Y_Data"]
Y_RA_085_096 = Y_Filters_085_096["Y_RA"]
Y_DEC_085_096 = Y_Filters_085_096["Y_DEC"]
Y_Detect_ID_085_096 = Y_Filters_085_096["Y_Detect_ID"]
Y_Z_085_096 = Y_Filters_085_096["Y_Z"]
Y_Mag_085_096 = Y_Filters_085_096["Y_Mag"]
Y_Mag_Err_085_096 = Y_Filters_085_096["Y_Mag_Err"]


## <font color='#FF0B55' size=6 >Getting everything shifted</font>

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(1,1, 1)

im = ax.imshow(R_Data_085_096[0], origin="lower", cmap="YlOrBr")

#plt.scatter(X, Y, color="skyblue", s=200, marker="+")

#sep_ap = cutout['details']['sep_objects'][cutout['details']['sep_obj_idx']]

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

plt.colorbar(im, ax=ax)

#plt.title(str(filename))

In [ ]:
G_X_Array_025_035, G_Y_Array_025_035 = getting_the_centers(G_Data_025_035)
R_X_Array_025_035, R_Y_Array_025_035 = getting_the_centers(R_Data_025_035)
I_X_Array_025_035, I_Y_Array_025_035 = getting_the_centers(I_Data_025_035)
Z_X_Array_025_035, Z_Y_Array_025_035 = getting_the_centers(Z_Data_025_035)
Y_X_Array_025_035, Y_Y_Array_025_035 = getting_the_centers(Y_Data_025_035)

In [ ]:
G_X_Array_035_045, G_Y_Array_035_045 = getting_the_centers(G_Data_035_045)
R_X_Array_035_045, R_Y_Array_035_045 = getting_the_centers(R_Data_035_045)
I_X_Array_035_045, I_Y_Array_035_045 = getting_the_centers(I_Data_035_045)
Z_X_Array_035_045, Z_Y_Array_035_045 = getting_the_centers(Z_Data_035_045)
Y_X_Array_035_045, Y_Y_Array_035_045 = getting_the_centers(Y_Data_035_045)

In [ ]:
G_X_Array_045_055, G_Y_Array_045_055 = getting_the_centers(G_Data_045_055)
R_X_Array_045_055, R_Y_Array_045_055 = getting_the_centers(R_Data_045_055)
I_X_Array_045_055, I_Y_Array_045_055 = getting_the_centers(I_Data_045_055)
Z_X_Array_045_055, Z_Y_Array_045_055 = getting_the_centers(Z_Data_045_055)
Y_X_Array_045_055, Y_Y_Array_045_055 = getting_the_centers(Y_Data_045_055)

In [ ]:
G_X_Array_055_065, G_Y_Array_055_065 = getting_the_centers(G_Data_055_065)
R_X_Array_055_065, R_Y_Array_055_065 = getting_the_centers(R_Data_055_065)
I_X_Array_055_065, I_Y_Array_055_065 = getting_the_centers(I_Data_055_065)
Z_X_Array_055_065, Z_Y_Array_055_065 = getting_the_centers(Z_Data_055_065)
Y_X_Array_055_065, Y_Y_Array_055_065 = getting_the_centers(Y_Data_055_065)

In [ ]:
G_X_Array_065_075, G_Y_Array_065_075 = getting_the_centers(G_Data_065_075)
R_X_Array_065_075, R_Y_Array_065_075 = getting_the_centers(R_Data_065_075)
I_X_Array_065_075, I_Y_Array_065_075 = getting_the_centers(I_Data_065_075)
Z_X_Array_065_075, Z_Y_Array_065_075 = getting_the_centers(Z_Data_065_075)
Y_X_Array_065_075, Y_Y_Array_065_075 = getting_the_centers(Y_Data_065_075)

In [ ]:
G_X_Array_075_085, G_Y_Array_075_085 = getting_the_centers(G_Data_075_085)
R_X_Array_075_085, R_Y_Array_075_085 = getting_the_centers(R_Data_075_085)
I_X_Array_075_085, I_Y_Array_075_085 = getting_the_centers(I_Data_075_085)
Z_X_Array_075_085, Z_Y_Array_075_085 = getting_the_centers(Z_Data_075_085)
Y_X_Array_075_085, Y_Y_Array_075_085 = getting_the_centers(Y_Data_075_085)

In [ ]:
G_X_Array_085_096, G_Y_Array_085_096 = getting_the_centers(G_Data_085_096)
R_X_Array_085_096, R_Y_Array_085_096 = getting_the_centers(R_Data_085_096)
I_X_Array_085_096, I_Y_Array_085_096 = getting_the_centers(I_Data_085_096)
Z_X_Array_085_096, Z_Y_Array_085_096 = getting_the_centers(Z_Data_085_096)
Y_X_Array_085_096, Y_Y_Array_085_096 = getting_the_centers(Y_Data_085_096)

In [ ]:
G_Shifted_X_Data_025_035 = shifting_in_the_x_direction(G_Data_025_035, G_X_Array_025_035)
R_Shifted_X_Data_025_035 = shifting_in_the_x_direction(R_Data_025_035, R_X_Array_025_035)
I_Shifted_X_Data_025_035 = shifting_in_the_x_direction(I_Data_025_035, I_X_Array_025_035)
Z_Shifted_X_Data_025_035 = shifting_in_the_x_direction(Z_Data_025_035, Z_X_Array_025_035)
Y_Shifted_X_Data_025_035 = shifting_in_the_x_direction(Y_Data_025_035, Y_X_Array_025_035)


In [ ]:
G_Shifted_X_Data_035_045 = shifting_in_the_x_direction(G_Data_035_045, G_X_Array_035_045)
R_Shifted_X_Data_035_045 = shifting_in_the_x_direction(R_Data_035_045, R_X_Array_035_045)
I_Shifted_X_Data_035_045 = shifting_in_the_x_direction(I_Data_035_045, I_X_Array_035_045)
Z_Shifted_X_Data_035_045 = shifting_in_the_x_direction(Z_Data_035_045, Z_X_Array_035_045)
Y_Shifted_X_Data_035_045 = shifting_in_the_x_direction(Y_Data_035_045, Y_X_Array_035_045)

In [ ]:
G_Shifted_X_Data_045_055 = shifting_in_the_x_direction(G_Data_045_055, G_X_Array_045_055)
R_Shifted_X_Data_045_055 = shifting_in_the_x_direction(R_Data_045_055, R_X_Array_045_055)
I_Shifted_X_Data_045_055 = shifting_in_the_x_direction(I_Data_045_055, I_X_Array_045_055)
Z_Shifted_X_Data_045_055 = shifting_in_the_x_direction(Z_Data_045_055, Z_X_Array_045_055)
Y_Shifted_X_Data_045_055 = shifting_in_the_x_direction(Y_Data_045_055, Y_X_Array_045_055)

In [ ]:
G_Shifted_X_Data_055_065 = shifting_in_the_x_direction(G_Data_055_065, G_X_Array_055_065)
R_Shifted_X_Data_055_065 = shifting_in_the_x_direction(R_Data_055_065, R_X_Array_055_065)
I_Shifted_X_Data_055_065 = shifting_in_the_x_direction(I_Data_055_065, I_X_Array_055_065)
Z_Shifted_X_Data_055_065 = shifting_in_the_x_direction(Z_Data_055_065, Z_X_Array_055_065)
Y_Shifted_X_Data_055_065 = shifting_in_the_x_direction(Y_Data_055_065, Y_X_Array_055_065)

In [ ]:
G_Shifted_X_Data_065_075 = shifting_in_the_x_direction(G_Data_065_075, G_X_Array_065_075)
R_Shifted_X_Data_065_075 = shifting_in_the_x_direction(R_Data_065_075, R_X_Array_065_075)
I_Shifted_X_Data_065_075 = shifting_in_the_x_direction(I_Data_065_075, I_X_Array_065_075)
Z_Shifted_X_Data_065_075 = shifting_in_the_x_direction(Z_Data_065_075, Z_X_Array_065_075)
Y_Shifted_X_Data_065_075 = shifting_in_the_x_direction(Y_Data_065_075, Y_X_Array_065_075)

In [ ]:
G_Shifted_X_Data_075_085 = shifting_in_the_x_direction(G_Data_075_085, G_X_Array_075_085)
R_Shifted_X_Data_075_085 = shifting_in_the_x_direction(R_Data_075_085, R_X_Array_075_085)
I_Shifted_X_Data_075_085 = shifting_in_the_x_direction(I_Data_075_085, I_X_Array_075_085)
Z_Shifted_X_Data_075_085 = shifting_in_the_x_direction(Z_Data_075_085, Z_X_Array_075_085)
Y_Shifted_X_Data_075_085 = shifting_in_the_x_direction(Y_Data_075_085, Y_X_Array_075_085)

In [ ]:
G_Shifted_X_Data_085_096 = shifting_in_the_x_direction(G_Data_085_096, G_X_Array_085_096)
R_Shifted_X_Data_085_096 = shifting_in_the_x_direction(R_Data_085_096, R_X_Array_085_096)
I_Shifted_X_Data_085_096 = shifting_in_the_x_direction(I_Data_085_096, I_X_Array_085_096)
Z_Shifted_X_Data_085_096 = shifting_in_the_x_direction(Z_Data_085_096, Z_X_Array_085_096)
Y_Shifted_X_Data_085_096 = shifting_in_the_x_direction(Y_Data_085_096, Y_X_Array_085_096)

In [ ]:
G_Shifted_Y_Data_025_035 = shifting_in_the_y_direction(G_Shifted_X_Data_025_035, G_Y_Array_025_035)
R_Shifted_Y_Data_025_035 = shifting_in_the_y_direction(R_Shifted_X_Data_025_035, R_Y_Array_025_035)
I_Shifted_Y_Data_025_035 = shifting_in_the_y_direction(I_Shifted_X_Data_025_035, I_Y_Array_025_035)
Z_Shifted_Y_Data_025_035 = shifting_in_the_y_direction(Z_Shifted_X_Data_025_035, Z_Y_Array_025_035)
Y_Shifted_Y_Data_025_035 = shifting_in_the_y_direction(Y_Shifted_X_Data_025_035, Y_Y_Array_025_035)

In [ ]:
G_Shifted_Y_Data_035_045 = shifting_in_the_y_direction(G_Shifted_X_Data_035_045, G_Y_Array_035_045)
R_Shifted_Y_Data_035_045 = shifting_in_the_y_direction(R_Shifted_X_Data_035_045, R_Y_Array_035_045)
I_Shifted_Y_Data_035_045 = shifting_in_the_y_direction(I_Shifted_X_Data_035_045, I_Y_Array_035_045)
Z_Shifted_Y_Data_035_045 = shifting_in_the_y_direction(Z_Shifted_X_Data_035_045, Z_Y_Array_035_045)
Y_Shifted_Y_Data_035_045 = shifting_in_the_y_direction(Y_Shifted_X_Data_035_045, Y_Y_Array_035_045)

In [ ]:
G_Shifted_Y_Data_045_055 = shifting_in_the_y_direction(G_Shifted_X_Data_045_055, G_Y_Array_045_055)
R_Shifted_Y_Data_045_055 = shifting_in_the_y_direction(R_Shifted_X_Data_045_055, R_Y_Array_045_055)
I_Shifted_Y_Data_045_055 = shifting_in_the_y_direction(I_Shifted_X_Data_045_055, I_Y_Array_045_055)
Z_Shifted_Y_Data_045_055 = shifting_in_the_y_direction(Z_Shifted_X_Data_045_055, Z_Y_Array_045_055)
Y_Shifted_Y_Data_045_055 = shifting_in_the_y_direction(Y_Shifted_X_Data_045_055, Y_Y_Array_045_055)

In [ ]:
G_Shifted_Y_Data_055_065 = shifting_in_the_y_direction(G_Shifted_X_Data_055_065, G_Y_Array_055_065)
R_Shifted_Y_Data_055_065 = shifting_in_the_y_direction(R_Shifted_X_Data_055_065, R_Y_Array_055_065)
I_Shifted_Y_Data_055_065 = shifting_in_the_y_direction(I_Shifted_X_Data_055_065, I_Y_Array_055_065)
Z_Shifted_Y_Data_055_065 = shifting_in_the_y_direction(Z_Shifted_X_Data_055_065, Z_Y_Array_055_065)
Y_Shifted_Y_Data_055_065 = shifting_in_the_y_direction(Y_Shifted_X_Data_055_065, Y_Y_Array_055_065)

In [ ]:
G_Shifted_Y_Data_065_075 = shifting_in_the_y_direction(G_Shifted_X_Data_065_075, G_Y_Array_065_075)
R_Shifted_Y_Data_065_075 = shifting_in_the_y_direction(R_Shifted_X_Data_065_075, R_Y_Array_065_075)
I_Shifted_Y_Data_065_075 = shifting_in_the_y_direction(I_Shifted_X_Data_065_075, I_Y_Array_065_075)
Z_Shifted_Y_Data_065_075 = shifting_in_the_y_direction(Z_Shifted_X_Data_065_075, Z_Y_Array_065_075)
Y_Shifted_Y_Data_065_075 = shifting_in_the_y_direction(Y_Shifted_X_Data_065_075, Y_Y_Array_065_075)

In [ ]:
G_Shifted_Y_Data_075_085 = shifting_in_the_y_direction(G_Shifted_X_Data_075_085, G_Y_Array_075_085)
R_Shifted_Y_Data_075_085 = shifting_in_the_y_direction(R_Shifted_X_Data_075_085, R_Y_Array_075_085)
I_Shifted_Y_Data_075_085 = shifting_in_the_y_direction(I_Shifted_X_Data_075_085, I_Y_Array_075_085)
Z_Shifted_Y_Data_075_085 = shifting_in_the_y_direction(Z_Shifted_X_Data_075_085, Z_Y_Array_075_085)
Y_Shifted_Y_Data_075_085 = shifting_in_the_y_direction(Y_Shifted_X_Data_075_085, Y_Y_Array_075_085)

In [ ]:
G_Shifted_Y_Data_085_096 = shifting_in_the_y_direction(G_Shifted_X_Data_085_096, G_Y_Array_085_096)
R_Shifted_Y_Data_085_096 = shifting_in_the_y_direction(R_Shifted_X_Data_085_096, R_Y_Array_085_096)
I_Shifted_Y_Data_085_096 = shifting_in_the_y_direction(I_Shifted_X_Data_085_096, I_Y_Array_085_096)
Z_Shifted_Y_Data_085_096 = shifting_in_the_y_direction(Z_Shifted_X_Data_085_096, Z_Y_Array_085_096)
Y_Shifted_Y_Data_085_096 = shifting_in_the_y_direction(Y_Shifted_X_Data_085_096, Y_Y_Array_085_096)

In [ ]:
num=6

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(1,1, 1)

im = ax.imshow(R_Shifted_Y_Data_025_035[num], origin="lower", cmap="YlOrBr", vmin=0, vmax=1.5)

plt.axhline(y=R_Y_Array_025_035[num], color='red', linewidth=2)
plt.axvline(x=42, color='green', linestyle='--', linewidth=2)
plt.axhline(y=42, color='green', linestyle='--', linewidth=2)

#sep_ap = cutout['details']['sep_objects'][cutout['details']['sep_obj_idx']]

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

plt.colorbar(im, ax=ax)


#plt.title(str(filename))

## <font color='#FF0B55' size=6 >Creating the final images to use for the analysis and writing them to files</font>

In [ ]:
Single_Rows_G_025_035 = all_the_rows(G_Shifted_Y_Data_025_035)
Single_Rows_R_025_035 = all_the_rows(R_Shifted_Y_Data_025_035)
Single_Rows_I_025_035 = all_the_rows(I_Shifted_Y_Data_025_035)
Single_Rows_Z_025_035 = all_the_rows(Z_Shifted_Y_Data_025_035)
Single_Rows_Y_025_035 = all_the_rows(Y_Shifted_Y_Data_025_035)

In [ ]:
Single_Rows_G_035_045 = all_the_rows(G_Shifted_Y_Data_035_045)
Single_Rows_R_035_045 = all_the_rows(R_Shifted_Y_Data_035_045)
Single_Rows_I_035_045 = all_the_rows(I_Shifted_Y_Data_035_045)
Single_Rows_Z_035_045 = all_the_rows(Z_Shifted_Y_Data_035_045)
Single_Rows_Y_035_045 = all_the_rows(Y_Shifted_Y_Data_035_045)

In [ ]:
Single_Rows_G_045_055 = all_the_rows(G_Shifted_Y_Data_045_055)
Single_Rows_R_045_055 = all_the_rows(R_Shifted_Y_Data_045_055)
Single_Rows_I_045_055 = all_the_rows(I_Shifted_Y_Data_045_055)
Single_Rows_Z_045_055 = all_the_rows(Z_Shifted_Y_Data_045_055)
Single_Rows_Y_045_055 = all_the_rows(Y_Shifted_Y_Data_045_055)

In [ ]:
Single_Rows_G_055_065 = all_the_rows(G_Shifted_Y_Data_055_065)
Single_Rows_R_055_065 = all_the_rows(R_Shifted_Y_Data_055_065)
Single_Rows_I_055_065 = all_the_rows(I_Shifted_Y_Data_055_065)
Single_Rows_Z_055_065 = all_the_rows(Z_Shifted_Y_Data_055_065)
Single_Rows_Y_055_065 = all_the_rows(Y_Shifted_Y_Data_055_065)

In [ ]:
Single_Rows_G_065_075 = all_the_rows(G_Shifted_Y_Data_065_075)
Single_Rows_R_065_075 = all_the_rows(R_Shifted_Y_Data_065_075)
Single_Rows_I_065_075 = all_the_rows(I_Shifted_Y_Data_065_075)
Single_Rows_Z_065_075 = all_the_rows(Z_Shifted_Y_Data_065_075)
Single_Rows_Y_065_075 = all_the_rows(Y_Shifted_Y_Data_065_075)

In [ ]:
Single_Rows_G_075_085 = all_the_rows(G_Shifted_Y_Data_075_085)
Single_Rows_R_075_085 = all_the_rows(R_Shifted_Y_Data_075_085)
Single_Rows_I_075_085 = all_the_rows(I_Shifted_Y_Data_075_085)
Single_Rows_Z_075_085 = all_the_rows(Z_Shifted_Y_Data_075_085)
Single_Rows_Y_075_085 = all_the_rows(Y_Shifted_Y_Data_075_085)

In [ ]:
Single_Rows_G_085_096 = all_the_rows(G_Shifted_Y_Data_085_096)
Single_Rows_R_085_096 = all_the_rows(R_Shifted_Y_Data_085_096)
Single_Rows_I_085_096 = all_the_rows(I_Shifted_Y_Data_085_096)
Single_Rows_Z_085_096 = all_the_rows(Z_Shifted_Y_Data_085_096)
Single_Rows_Y_085_096 = all_the_rows(Y_Shifted_Y_Data_085_096)

In [ ]:
Row_Median_G_025_035 = row_median(Single_Rows_G_025_035)
Row_Median_R_025_035 = row_median(Single_Rows_R_025_035)
Row_Median_I_025_035 = row_median(Single_Rows_I_025_035)
Row_Median_Z_025_035 = row_median(Single_Rows_Z_025_035)
Row_Median_Y_025_035 = row_median(Single_Rows_Y_025_035)


Row_Median_G_SD_025_035 = row_std_dev(Single_Rows_G_025_035)
Row_Median_R_SD_025_035 = row_std_dev(Single_Rows_R_025_035)
Row_Median_I_SD_025_035 = row_std_dev(Single_Rows_I_025_035)
Row_Median_Z_SD_025_035 = row_std_dev(Single_Rows_Z_025_035)
Row_Median_Y_SD_025_035 = row_std_dev(Single_Rows_Y_025_035)

In [ ]:
Row_Median_G_035_045 = row_median(Single_Rows_G_035_045)
Row_Median_R_035_045 = row_median(Single_Rows_R_035_045)
Row_Median_I_035_045 = row_median(Single_Rows_I_035_045)
Row_Median_Z_035_045 = row_median(Single_Rows_Z_035_045)
Row_Median_Y_035_045 = row_median(Single_Rows_Y_035_045)

Row_Median_G_SD_035_045 = row_std_dev(Single_Rows_G_035_045)
Row_Median_R_SD_035_045 = row_std_dev(Single_Rows_R_035_045)
Row_Median_I_SD_035_045 = row_std_dev(Single_Rows_I_035_045)
Row_Median_Z_SD_035_045 = row_std_dev(Single_Rows_Z_035_045)
Row_Median_Y_SD_035_045 = row_std_dev(Single_Rows_Y_035_045)

In [ ]:
Row_Median_G_045_055 = row_median(Single_Rows_G_045_055)
Row_Median_R_045_055 = row_median(Single_Rows_R_045_055)
Row_Median_I_045_055 = row_median(Single_Rows_I_045_055)
Row_Median_Z_045_055 = row_median(Single_Rows_Z_045_055)
Row_Median_Y_045_055 = row_median(Single_Rows_Y_045_055)

Row_Median_G_SD_045_055 = row_std_dev(Single_Rows_G_045_055)
Row_Median_R_SD_045_055 = row_std_dev(Single_Rows_R_045_055)
Row_Median_I_SD_045_055 = row_std_dev(Single_Rows_I_045_055)
Row_Median_Z_SD_045_055 = row_std_dev(Single_Rows_Z_045_055)
Row_Median_Y_SD_045_055 = row_std_dev(Single_Rows_Y_045_055)

In [ ]:
Row_Median_G_055_065 = row_median(Single_Rows_G_055_065)
Row_Median_R_055_065 = row_median(Single_Rows_R_055_065)
Row_Median_I_055_065 = row_median(Single_Rows_I_055_065)
Row_Median_Z_055_065 = row_median(Single_Rows_Z_055_065)
Row_Median_Y_055_065 = row_median(Single_Rows_Y_055_065)

Row_Median_G_SD_055_065 = row_std_dev(Single_Rows_G_055_065)
Row_Median_R_SD_055_065 = row_std_dev(Single_Rows_R_055_065)
Row_Median_I_SD_055_065 = row_std_dev(Single_Rows_I_055_065)
Row_Median_Z_SD_055_065 = row_std_dev(Single_Rows_Z_055_065)
Row_Median_Y_SD_055_065 = row_std_dev(Single_Rows_Y_055_065)

In [ ]:
Row_Median_G_065_075 = row_median(Single_Rows_G_065_075)
Row_Median_R_065_075 = row_median(Single_Rows_R_065_075)
Row_Median_I_065_075 = row_median(Single_Rows_I_065_075)
Row_Median_Z_065_075 = row_median(Single_Rows_Z_065_075)
Row_Median_Y_065_075 = row_median(Single_Rows_Y_065_075)

Row_Median_G_SD_065_075 = row_std_dev(Single_Rows_G_065_075)
Row_Median_R_SD_065_075 = row_std_dev(Single_Rows_R_065_075)
Row_Median_I_SD_065_075 = row_std_dev(Single_Rows_I_065_075)
Row_Median_Z_SD_065_075 = row_std_dev(Single_Rows_Z_065_075)
Row_Median_Y_SD_065_075 = row_std_dev(Single_Rows_Y_065_075)

In [ ]:
Row_Median_G_075_085 = row_median(Single_Rows_G_075_085)
Row_Median_R_075_085 = row_median(Single_Rows_R_075_085)
Row_Median_I_075_085 = row_median(Single_Rows_I_075_085)
Row_Median_Z_075_085 = row_median(Single_Rows_Z_075_085)
Row_Median_Y_075_085 = row_median(Single_Rows_Y_075_085)

Row_Median_G_SD_075_085 = row_std_dev(Single_Rows_G_075_085)
Row_Median_R_SD_075_085 = row_std_dev(Single_Rows_R_075_085)
Row_Median_I_SD_075_085 = row_std_dev(Single_Rows_I_075_085)
Row_Median_Z_SD_075_085 = row_std_dev(Single_Rows_Z_075_085)
Row_Median_Y_SD_075_085 = row_std_dev(Single_Rows_Y_075_085)

In [ ]:
Row_Median_G_085_096 = row_median(Single_Rows_G_085_096)
Row_Median_R_085_096 = row_median(Single_Rows_R_085_096)
Row_Median_I_085_096 = row_median(Single_Rows_I_085_096)
Row_Median_Z_085_096 = row_median(Single_Rows_Z_085_096)
Row_Median_Y_085_096 = row_median(Single_Rows_Y_085_096)

Row_Median_G_SD_085_096 = row_std_dev(Single_Rows_G_085_096)
Row_Median_R_SD_085_096 = row_std_dev(Single_Rows_R_085_096)
Row_Median_I_SD_085_096 = row_std_dev(Single_Rows_I_085_096)
Row_Median_Z_SD_085_096 = row_std_dev(Single_Rows_Z_085_096)
Row_Median_Y_SD_085_096 = row_std_dev(Single_Rows_Y_085_096)

In [ ]:
Table_A = ([(Row_Median_R_025_035)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_025_035_Median_Stack.fits", overwrite = True)

In [ ]:
Table_A = ([(Row_Median_R_035_045)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_035_045_Median_Stack.fits", overwrite = True)

In [ ]:
Table_A = ([(Row_Median_R_045_055)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_045_055_Median_Stack.fits", overwrite = True)

In [ ]:
Table_A = ([(Row_Median_R_055_065)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_055_065_Median_Stack.fits", overwrite = True)

In [ ]:
Table_A = ([(Row_Median_R_065_075)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_065_075_Median_Stack.fits", overwrite = True)

In [ ]:
Table_A = ([(Row_Median_R_075_085)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_075_085_Median_Stack.fits", overwrite = True)

In [ ]:
Table_A = ([(Row_Median_R_085_096)])

Table_b = QTable([Table_A], names=["Data"])

Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/R_085_096_Median_Stack.fits", overwrite = True)

In [ ]:
num=7

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(1,1, 1)

im = ax.imshow(Row_Median_R_085_096, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.05)

#plt.axhline(y=R_Y_Array_035_045[num], color='red', linewidth=2)
#plt.axvline(x=42, color='green', linestyle='--', linewidth=2)
#plt.axhline(y=42, color='green', linestyle='--', linewidth=2)

#sep_ap = cutout['details']['sep_objects'][cutout['details']['sep_obj_idx']]

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

plt.colorbar(im, ax=ax)

plt.plot([70, 82], [5,5], color="black", linewidth=4)
plt.text(73, 7, '2.00"')

plt.title("R        0.85 < Z < 0.96")

## <font color='#FF0B55' size=6 >Opening and extracting data from the 1_Analyzing notebook to make the big pannel plot</font>

In [ ]:
File_Table_3by7_025_035 = "Table_For_3by7_plot_025_035.fits"
File_Table_3by7_035_045 = "Table_For_3by7_plot_035_045.fits"
File_Table_3by7_045_055 = "Table_For_3by7_plot_045_055.fits"
File_Table_3by7_055_065 = "Table_For_3by7_plot_055_065.fits"
File_Table_3by7_065_075 = "Table_For_3by7_plot_065_075.fits"
File_Table_3by7_075_085 = "Table_For_3by7_plot_075_085.fits"
File_Table_3by7_085_096 = "Table_For_3by7_plot_085_096.fits"


In [ ]:
Table_3by7_025_035 = Table.read(File_Table_3by7_025_035, format = 'fits')
Table_3by7_035_045 = Table.read(File_Table_3by7_035_045, format = 'fits')
Table_3by7_045_055 = Table.read(File_Table_3by7_045_055, format = 'fits')
Table_3by7_055_065 = Table.read(File_Table_3by7_055_065, format = 'fits')
Table_3by7_065_075 = Table.read(File_Table_3by7_065_075, format = 'fits')
Table_3by7_075_085 = Table.read(File_Table_3by7_075_085, format = 'fits')
Table_3by7_085_096 = Table.read(File_Table_3by7_085_096, format = 'fits')


In [ ]:
Table_3by7_Velocity_025_035 = Table_3by7_025_035["Velocity_Array"][0]
Table_3by7_Flux_025_035 = Table_3by7_025_035["Flux_Array"][0]
Table_3by7_Narrow_025_035 = Table_3by7_025_035["Narrow"][0]
Table_3by7_Broad_025_035 = Table_3by7_025_035["Broad"][0]
Table_3by7_Exponential_025_035 = Table_3by7_025_035["Exponential"][0]
Table_3by7_Full_025_035 = Table_3by7_025_035["Full"][0]

In [ ]:
Table_3by7_Velocity_035_045 = Table_3by7_035_045["Velocity_Array"][0]
Table_3by7_Flux_035_045 = Table_3by7_035_045["Flux_Array"][0]
Table_3by7_Narrow_035_045 = Table_3by7_035_045["Narrow"][0]
Table_3by7_Broad_035_045 = Table_3by7_035_045["Broad"][0]
Table_3by7_Exponential_035_045 = Table_3by7_035_045["Exponential"][0]
Table_3by7_Full_035_045 = Table_3by7_035_045["Full"][0]

In [ ]:
Table_3by7_Velocity_045_055 = Table_3by7_045_055["Velocity_Array"][0]
Table_3by7_Flux_045_055 = Table_3by7_045_055["Flux_Array"][0]
Table_3by7_Narrow_045_055 = Table_3by7_045_055["Narrow"][0]
Table_3by7_Broad_045_055 = Table_3by7_045_055["Broad"][0]
Table_3by7_Exponential_045_055 = Table_3by7_045_055["Exponential"][0]
Table_3by7_Full_045_055 = Table_3by7_045_055["Full"][0]

In [ ]:
Table_3by7_Velocity_055_065 = Table_3by7_055_065["Velocity_Array"][0]
Table_3by7_Flux_055_065 = Table_3by7_055_065["Flux_Array"][0]
Table_3by7_Narrow_055_065 = Table_3by7_055_065["Narrow"][0]
Table_3by7_Broad_055_065 = Table_3by7_055_065["Broad"][0]
Table_3by7_Exponential_055_065 = Table_3by7_055_065["Exponential"][0]
Table_3by7_Full_055_065 = Table_3by7_055_065["Full"][0]

In [ ]:
Table_3by7_Velocity_065_075 = Table_3by7_065_075["Velocity_Array"][0]
Table_3by7_Flux_065_075 = Table_3by7_065_075["Flux_Array"][0]
Table_3by7_Narrow_065_075 = Table_3by7_065_075["Narrow"][0]
Table_3by7_Broad_065_075 = Table_3by7_065_075["Broad"][0]
Table_3by7_Exponential_065_075 = Table_3by7_065_075["Exponential"][0]
Table_3by7_Full_065_075 = Table_3by7_065_075["Full"][0]

In [ ]:
Table_3by7_Velocity_075_085 = Table_3by7_075_085["Velocity_Array"][0]
Table_3by7_Flux_075_085 = Table_3by7_075_085["Flux_Array"][0]
Table_3by7_Narrow_075_085 = Table_3by7_075_085["Narrow"][0]
Table_3by7_Broad_075_085 = Table_3by7_075_085["Broad"][0]
Table_3by7_Exponential_075_085 = Table_3by7_075_085["Exponential"][0]
Table_3by7_Full_075_085 = Table_3by7_075_085["Full"][0]

In [ ]:
Table_3by7_Velocity_085_096 = Table_3by7_085_096["Velocity_Array"][0]
Table_3by7_Flux_085_096 = Table_3by7_085_096["Flux_Array"][0]
Table_3by7_Narrow_085_096 = Table_3by7_085_096["Narrow"][0]
Table_3by7_Broad_085_096 = Table_3by7_085_096["Broad"][0]
Table_3by7_Exponential_085_096 = Table_3by7_085_096["Exponential"][0]
Table_3by7_Full_085_096 = Table_3by7_085_096["Full"][0]

In [ ]:
fig = plt.figure(figsize = (8,8), facecolor='white')
ax = plt.subplot(1,1,1)

plt.step(Table_3by7_Velocity_025_035, Table_3by7_Flux_025_035, where = 'mid', color = '#218380', label = "spectrum", linewidth=1)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')

plt.title("Uniform Narrow Broad")

#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

In [ ]:
fig = plt.figure(figsize = (30,45), facecolor='white')

ax = fig.add_subplot(7,3, 1)

im = ax.imshow(Row_Median_R_025_035, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.25<z<0.35"

plt.colorbar(im, ax=ax)

plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,2)

plt.step(Table_3by7_Velocity_025_035, Table_3by7_Flux_025_035, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,3)


plt.plot(Table_3by7_Velocity_025_035, Table_3by7_Full_025_035, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_025_035, Table_3by7_Flux_025_035, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_025_035, Table_3by7_Narrow_025_035, '--', linewidth=5, color='#d81159', label='Gaussian component 1', zorder=2)
plt.plot(Table_3by7_Velocity_025_035, Table_3by7_Broad_025_035, '--', linewidth=5, color='#0070be', label='Gaussian component 2', zorder=3)
plt.plot(Table_3by7_Velocity_025_035, Table_3by7_Exponential_025_035, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')

#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################

ax = fig.add_subplot(7,3, 4)

im = ax.imshow(Row_Median_R_035_045, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.35<z<0.45"

plt.colorbar(im, ax=ax)

plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,5)

plt.step(Table_3by7_Velocity_035_045, Table_3by7_Flux_035_045, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,6)


plt.plot(Table_3by7_Velocity_035_045, Table_3by7_Full_035_045, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_035_045, Table_3by7_Flux_035_045, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_035_045, Table_3by7_Narrow_035_045, '--', linewidth=5, color='#d81159', label='Gaussian component 1', zorder=2)
plt.plot(Table_3by7_Velocity_035_045, Table_3by7_Broad_035_045, '--', linewidth=5, color='#0070be', label='Gaussian component 2', zorder=3)
plt.plot(Table_3by7_Velocity_035_045, Table_3by7_Exponential_035_045, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################

ax = fig.add_subplot(7,3, 7)

im = ax.imshow(Row_Median_R_045_055, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.45<z<0.55"
plt.colorbar(im, ax=ax)


plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,8)

plt.step(Table_3by7_Velocity_045_055, Table_3by7_Flux_045_055, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,9)


plt.plot(Table_3by7_Velocity_045_055, Table_3by7_Full_045_055, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_045_055, Table_3by7_Flux_045_055, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_045_055, Table_3by7_Narrow_045_055, '--', linewidth=5, color='#d81159', label='Gaussian component 1', zorder=2)
plt.plot(Table_3by7_Velocity_045_055, Table_3by7_Broad_045_055, '--', linewidth=5, color='#0070be', label='Gaussian component 2', zorder=3)
plt.plot(Table_3by7_Velocity_045_055, Table_3by7_Exponential_045_055, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################

ax = fig.add_subplot(7,3, 10)

im = ax.imshow(Row_Median_R_055_065, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.55<z<0.65"
plt.colorbar(im, ax=ax)


plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,11)

plt.step(Table_3by7_Velocity_055_065, Table_3by7_Flux_055_065, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,12)


plt.plot(Table_3by7_Velocity_055_065, Table_3by7_Full_055_065, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_055_065, Table_3by7_Flux_055_065, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_055_065, Table_3by7_Narrow_055_065, '--', linewidth=5, color='#d81159', label='Gaussian component 1', zorder=2)
plt.plot(Table_3by7_Velocity_055_065, Table_3by7_Broad_055_065, '--', linewidth=5, color='#0070be', label='Gaussian component 2', zorder=3)
plt.plot(Table_3by7_Velocity_055_065, Table_3by7_Exponential_055_065, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################

ax = fig.add_subplot(7,3, 13)

im = ax.imshow(Row_Median_R_065_075, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.65<z<0.75"
plt.colorbar(im, ax=ax)


plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,14)

plt.step(Table_3by7_Velocity_065_075, Table_3by7_Flux_065_075, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,15)


plt.plot(Table_3by7_Velocity_065_075, Table_3by7_Full_065_075, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_065_075, Table_3by7_Flux_065_075, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_065_075, Table_3by7_Narrow_065_075, '--', linewidth=5, color='#d81159', label='Gaussian component 1', zorder=2)
plt.plot(Table_3by7_Velocity_065_075, Table_3by7_Broad_065_075, '--', linewidth=5, color='#0070be', label='Gaussian component 2', zorder=3)
plt.plot(Table_3by7_Velocity_065_075, Table_3by7_Exponential_065_075, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################

ax = fig.add_subplot(7,3, 16)

im = ax.imshow(Row_Median_R_075_085, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.75<z<0.85"

plt.colorbar(im, ax=ax)

plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,17)

plt.step(Table_3by7_Velocity_075_085, Table_3by7_Flux_075_085, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,18)


plt.plot(Table_3by7_Velocity_075_085, Table_3by7_Full_075_085, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_075_085, Table_3by7_Flux_075_085, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_075_085, Table_3by7_Narrow_075_085, '--', linewidth=5, color='#d81159', label='Gaussian component 1', zorder=2)
plt.plot(Table_3by7_Velocity_075_085, Table_3by7_Broad_075_085, '--', linewidth=5, color='#0070be', label='Gaussian component 2', zorder=3)
plt.plot(Table_3by7_Velocity_075_085, Table_3by7_Exponential_075_085, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################

ax = fig.add_subplot(7,3, 19)

im = ax.imshow(Row_Median_R_085_096, origin="lower", cmap="YlOrBr", vmin=0, vmax=0.5)

coord_offset = 0.5 * (ax.get_xlim()[0] + ax.get_xlim()[1])

filename = "R  0.85<z<0.96"

plt.colorbar(im, ax=ax)

plt.title(str(filename))

##########################################################################################

ax = plt.subplot(7,3,20)

plt.step(Table_3by7_Velocity_085_096, Table_3by7_Flux_085_096, where = 'mid', color = 'black', label = "spectrum", linewidth=2)

#plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)
 
##########################################################################################

ax = plt.subplot(7,3,21)


plt.plot(Table_3by7_Velocity_085_096, Table_3by7_Full_085_096, '-', linewidth=5, color='#7fbb54', label='best fit', zorder=1)

plt.step(Table_3by7_Velocity_085_096, Table_3by7_Flux_085_096, where='mid', linewidth=2, color='black', zorder=0)
plt.plot(Table_3by7_Velocity_085_096, Table_3by7_Narrow_085_096, '--', linewidth=5, color='#d81159', label='Gaussian component', zorder=2)
plt.plot(Table_3by7_Velocity_085_096, Table_3by7_Broad_085_096, '--', linewidth=5, color='#0070be', label='Gaussian component', zorder=3)
plt.plot(Table_3by7_Velocity_085_096, Table_3by7_Exponential_085_096, '--', linewidth=5, color='#ffbc42', label='Exponential component', zorder=4)

plt.legend()

ax.set_ylabel("Flux [$erg s^{-1} \AA^{-1}$]")
ax.set_xlabel("Velocity [km s$^{-1}$]")

#ax.set_yscale('log')
#ax.set_xscale('log')


#plt.xlim(3400, 3440)
#plt.ylim(0.1e41, 1.1e41)

##########################################################################################



## <font color='#FF0B55' size=6 >Getting the Magnitudes for the final images</font>

In [ ]:
# The total magnitude for the stacked images for each redshift

R_Flux_025_035, R_Flux_SD_025_035 = mag_to_flux(R_Mag_025_035, R_Mag_Err_025_035, Mag_Zero_Point, R_Band_Wave, R_RA_025_035, R_DEC_025_035)
R_Flux_035_045, R_Flux_SD_035_045 = mag_to_flux(R_Mag_035_045, R_Mag_Err_035_045, Mag_Zero_Point, R_Band_Wave, R_RA_035_045, R_DEC_035_045)
R_Flux_045_055, R_Flux_SD_045_055 = mag_to_flux(R_Mag_045_055, R_Mag_Err_045_055, Mag_Zero_Point, R_Band_Wave, R_RA_045_055, R_DEC_045_055)
R_Flux_055_065, R_Flux_SD_055_065 = mag_to_flux(R_Mag_055_065, R_Mag_Err_055_065, Mag_Zero_Point, R_Band_Wave, R_RA_055_065, R_DEC_055_065)
R_Flux_065_075, R_Flux_SD_065_075 = mag_to_flux(R_Mag_065_075, R_Mag_Err_065_075, Mag_Zero_Point, R_Band_Wave, R_RA_065_075, R_DEC_065_075)
R_Flux_075_085, R_Flux_SD_075_085 = mag_to_flux(R_Mag_075_085, R_Mag_Err_075_085, Mag_Zero_Point, R_Band_Wave, R_RA_075_085, R_DEC_075_085)
R_Flux_085_096, R_Flux_SD_085_096 = mag_to_flux(R_Mag_085_096, R_Mag_Err_085_096, Mag_Zero_Point, R_Band_Wave, R_RA_085_096, R_DEC_085_096)
    
Flux_Med_Total_025_035, Flux_Med_Total_SD_025_035 = median_with_uncertainty(R_Flux_025_035, R_Flux_SD_025_035)
Flux_Med_Total_035_045, Flux_Med_Total_SD_035_045 = median_with_uncertainty(R_Flux_035_045, R_Flux_SD_035_045)
Flux_Med_Total_045_055, Flux_Med_Total_SD_045_055 = median_with_uncertainty(R_Flux_045_055, R_Flux_SD_045_055)
Flux_Med_Total_055_065, Flux_Med_Total_SD_055_065 = median_with_uncertainty(R_Flux_055_065, R_Flux_SD_055_065)
Flux_Med_Total_065_075, Flux_Med_Total_SD_065_075 = median_with_uncertainty(R_Flux_065_075, R_Flux_SD_065_075)
Flux_Med_Total_075_085, Flux_Med_Total_SD_075_085 = median_with_uncertainty(R_Flux_075_085, R_Flux_SD_075_085)
Flux_Med_Total_085_096, Flux_Med_Total_SD_085_096 = median_with_uncertainty(R_Flux_085_096, R_Flux_SD_085_096)

Mag_Total_025_035, Mag_Total_SD_025_035 = flux_to_mag_and_uncertainty(Flux_Med_Total_025_035, Flux_Med_Total_SD_025_035, Mag_Zero_Point)
Mag_Total_035_045, Mag_Total_SD_035_045 = flux_to_mag_and_uncertainty(Flux_Med_Total_035_045, Flux_Med_Total_SD_035_045, Mag_Zero_Point)
Mag_Total_045_055, Mag_Total_SD_045_055 = flux_to_mag_and_uncertainty(Flux_Med_Total_045_055, Flux_Med_Total_SD_045_055, Mag_Zero_Point)
Mag_Total_055_065, Mag_Total_SD_055_065 = flux_to_mag_and_uncertainty(Flux_Med_Total_055_065, Flux_Med_Total_SD_055_065, Mag_Zero_Point)
Mag_Total_065_075, Mag_Total_SD_065_075 = flux_to_mag_and_uncertainty(Flux_Med_Total_065_075, Flux_Med_Total_SD_065_075, Mag_Zero_Point)
Mag_Total_075_085, Mag_Total_SD_075_085 = flux_to_mag_and_uncertainty(Flux_Med_Total_075_085, Flux_Med_Total_SD_075_085, Mag_Zero_Point)
Mag_Total_085_096, Mag_Total_SD_085_096 = flux_to_mag_and_uncertainty(Flux_Med_Total_085_096, Flux_Med_Total_SD_085_096, Mag_Zero_Point)

## <font color='#FF0B55' size=6 >Getting the central position for the final images and creating the radial profiles</font>

In [ ]:
# Combine median image data for different redshift bins into a single array
Row_Median_Array = np.array([
    Row_Median_R_025_035, Row_Median_R_035_045, Row_Median_R_045_055,
    Row_Median_R_055_065, Row_Median_R_065_075, Row_Median_R_075_085, Row_Median_R_085_096
])

# Initialize lists to store the centroid positions of detected sources
X_Pos_Array = []
Y_Pos_Array = []

# Loop through each median image (corresponding to a redshift bin)
for i in Row_Median_Array:
    image_data = i
    sources = None

    # Try different sigma values (for background estimation) in decreasing order
    # to ensure we detect sources even in low S/N images
    sigma_values = [5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5]

    for sigma in sigma_values:
        # Estimate background statistics using sigma-clipped statistics
        mean, median, std = sigma_clipped_stats(image_data, sigma=sigma)

        # Initialize the DAOStarFinder with a fixed FWHM and dynamic threshold
        daofind = DAOStarFinder(fwhm=Pixel_Scale, threshold=5. * std)

        # Subtract median background and attempt to detect sources
        sources = daofind(image_data - median)

        # If any sources are found, record the centroid of the first one and stop trying
        if sources and len(sources) > 0:
            x_pos, y_pos = sources[0]['xcentroid'], sources[0]['ycentroid']
            X_Pos_Array.append(x_pos)
            Y_Pos_Array.append(y_pos)
            break  # Stop trying different sigma values once we succeed

    # If no sources were found with any sigma, use a fallback approach
    if sources is None or len(sources) == 0:
        print("Sigmas didn't work")

        # Retry with a fixed sigma and lower threshold (more sensitive)
        mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
        daofind = DAOStarFinder(fwhm=Pixel_Scale, threshold=3. * std)
        sources = daofind(image_data - median)

        if sources and len(sources) > 0:
            x_pos, y_pos = sources[0]['xcentroid'], sources[0]['ycentroid']
            X_Pos_Array.append(x_pos)
            Y_Pos_Array.append(y_pos)
        else:
            # If still no sources found, store None (or consider using default/fallback values)
            X_Pos_Array.append(None)
            Y_Pos_Array.append(None)

# Output the collected source positions for all redshift bins
print(X_Pos_Array)
print(Y_Pos_Array)


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_025_035_Array = []
Total_Counts_025_035_Array = []
Total_Counts_025_035_Std_Array = []
Radius_Pixels_025_035_Array = []
Average_Counts_025_035 = []
Average_Counts_SD_025_035 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_025_035_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_025_035 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_025_035_Array.append(Apertures_025_035)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_025_035, Apertures_025_035)
    phot_table = aperture_photometry(Row_Median_R_025_035, Apertures_025_035)

    # Get the total counts within the aperture
    Total_Counts_025_035_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_025_035.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_025_035.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_025_035)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_025_035_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_025_035.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_025_035, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_025_035_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_035_045_Array = []
Total_Counts_035_045_Array = []
Total_Counts_035_045_Std_Array = []
Radius_Pixels_035_045_Array = []
Average_Counts_035_045 = []
Average_Counts_SD_035_045 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_035_045_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_035_045 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_035_045_Array.append(Apertures_035_045)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_035_045, Apertures_035_045)
    phot_table = aperture_photometry(Row_Median_R_035_045, Apertures_035_045)

    # Get the total counts within the aperture
    Total_Counts_035_045_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_035_045.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_035_045.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_035_045)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_035_045_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_035_045.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_035_045, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_035_045_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_045_055_Array = []
Total_Counts_045_055_Array = []
Total_Counts_045_055_Std_Array = []
Radius_Pixels_045_055_Array = []
Average_Counts_045_055 = []
Average_Counts_SD_045_055 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_045_055_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_045_055 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_045_055_Array.append(Apertures_045_055)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_045_055, Apertures_045_055)
    phot_table = aperture_photometry(Row_Median_R_045_055, Apertures_045_055)

    # Get the total counts within the aperture
    Total_Counts_045_055_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_045_055.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_045_055.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_045_055)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_045_055_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_045_055.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_045_055, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_045_055_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_055_065_Array = []
Total_Counts_055_065_Array = []
Total_Counts_055_065_Std_Array = []
Radius_Pixels_055_065_Array = []
Average_Counts_055_065 = []
Average_Counts_SD_055_065 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_055_065_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_055_065 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_055_065_Array.append(Apertures_055_065)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_055_065, Apertures_055_065)
    phot_table = aperture_photometry(Row_Median_R_055_065, Apertures_055_065)

    # Get the total counts within the aperture
    Total_Counts_055_065_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_055_065.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_055_065.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_055_065)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_055_065_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_055_065.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_055_065, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_055_065_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_065_075_Array = []
Total_Counts_065_075_Array = []
Total_Counts_065_075_Std_Array = []
Radius_Pixels_065_075_Array = []
Average_Counts_065_075 = []
Average_Counts_SD_065_075 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_065_075_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_065_075 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_065_075_Array.append(Apertures_065_075)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_065_075, Apertures_065_075)
    phot_table = aperture_photometry(Row_Median_R_065_075, Apertures_065_075)

    # Get the total counts within the aperture
    Total_Counts_065_075_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_065_075.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_065_075.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_065_075)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_065_075_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_065_075.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_065_075, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_065_075_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_075_085_Array = []
Total_Counts_075_085_Array = []
Total_Counts_075_085_Std_Array = []
Radius_Pixels_075_085_Array = []
Average_Counts_075_085 = []
Average_Counts_SD_075_085 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_075_085_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_075_085 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_075_085_Array.append(Apertures_075_085)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_075_085, Apertures_075_085)
    phot_table = aperture_photometry(Row_Median_R_075_085, Apertures_075_085)

    # Get the total counts within the aperture
    Total_Counts_075_085_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_075_085.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_075_085.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_075_085)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_075_085_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_075_085.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_075_085, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_075_085_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Number_Of_Rings = np.arange(1.0, 55, 1.0)

Apertures_085_096_Array = []
Total_Counts_085_096_Array = []
Total_Counts_085_096_Std_Array = []
Radius_Pixels_085_096_Array = []
Average_Counts_085_096 = []
Average_Counts_SD_085_096 = []

for i in Number_Of_Rings:
    
    radius_arcsec = i
    radius_pixels = i

    Radius_Pixels_085_096_Array.append(radius_pixels * Pixel_Scale)

    # Correct aperture for this set
    Apertures_085_096 = CircularAperture((X_Pos_Array[0], Y_Pos_Array[0]), r=radius_pixels)
    Apertures_085_096_Array.append(Apertures_085_096)
    
    # Perform aperture photometry
    aper_stats = ApertureStats(Row_Median_R_085_096, Apertures_085_096)
    phot_table = aperture_photometry(Row_Median_R_085_096, Apertures_085_096)

    # Get the total counts within the aperture
    Total_Counts_085_096_Array.append(phot_table['aperture_sum'][0])

    Average_Counts_085_096.append(aper_stats.mean)
    
    # Create a mask to get values inside the aperture
    mask = Apertures_085_096.to_mask(method='center')
    masked_data = mask.multiply(Row_Median_R_085_096)
    aperture_data = masked_data[mask.data > 0]

    # Compute std dev of pixel values inside aperture
    counts_std = np.std(aperture_data)

    Total_Counts_085_096_Std_Array.append(counts_std * np.sqrt(len(aperture_data)))



    # Compute std dev of pixel values inside aperture
    average_counts_std = counts_std / np.sqrt(len(aperture_data))  # Standard error of the mean
    
    Average_Counts_SD_085_096.append(average_counts_std)  # Store the uncertainty for each average



    
# Visualize the aperture
plt.figure(figsize=(8, 8))

plt.imshow(Row_Median_R_085_096, origin='lower', cmap='New_Trans', vmin=0, vmax=0.5)

for i in Apertures_085_096_Array:
    i.plot(color='red', lw=2)

plt.colorbar(label='Counts')
#plt.title(f'Image with Aperture (radius = {radius_arcsec}\" ≈ {radius_pixels:.2f} pixels)')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()


In [ ]:
Area_In_Rings_025_035_Array, Equal_Area_Radius_025_035_Array = calculate_ring_areas(Radius_Pixels_025_035_Array)
Area_In_Rings_035_045_Array, Equal_Area_Radius_035_045_Array = calculate_ring_areas(Radius_Pixels_035_045_Array)
Area_In_Rings_045_055_Array, Equal_Area_Radius_045_055_Array = calculate_ring_areas(Radius_Pixels_045_055_Array)
Area_In_Rings_055_065_Array, Equal_Area_Radius_055_065_Array = calculate_ring_areas(Radius_Pixels_055_065_Array)
Area_In_Rings_065_075_Array, Equal_Area_Radius_065_075_Array = calculate_ring_areas(Radius_Pixels_065_075_Array)
Area_In_Rings_075_085_Array, Equal_Area_Radius_075_085_Array = calculate_ring_areas(Radius_Pixels_075_085_Array)
Area_In_Rings_085_096_Array, Equal_Area_Radius_085_096_Array = calculate_ring_areas(Radius_Pixels_085_096_Array)

In [ ]:
Median_Mag_G_025_035, Median_Mag_G_SD_025_035, Median_Mag_G_SD_Use_025_035 = bootstrap_median_std_with_errors(G_Mag_025_035, G_Mag_Err_025_035)
Median_Mag_R_025_035, Median_Mag_R_SD_025_035, Median_Mag_R_SD_Use_025_035 = bootstrap_median_std_with_errors(R_Mag_025_035, R_Mag_Err_025_035)
Median_Mag_I_025_035, Median_Mag_I_SD_025_035, Median_Mag_I_SD_Use_025_035 = bootstrap_median_std_with_errors(I_Mag_025_035, I_Mag_Err_025_035)
Median_Mag_Z_025_035, Median_Mag_Z_SD_025_035, Median_Mag_Z_SD_Use_025_035 = bootstrap_median_std_with_errors(Z_Mag_025_035, Z_Mag_Err_025_035)
Median_Mag_Y_025_035, Median_Mag_Y_SD_025_035, Median_Mag_Y_SD_Use_025_035 = bootstrap_median_std_with_errors(Y_Mag_025_035, Y_Mag_Err_025_035)


In [ ]:
Median_Mag_G_035_045, Median_Mag_G_SD_035_045, Median_Mag_G_SD_Use_035_045 = bootstrap_median_std_with_errors(G_Mag_035_045, G_Mag_Err_035_045)
Median_Mag_R_035_045, Median_Mag_R_SD_035_045, Median_Mag_R_SD_Use_035_045 = bootstrap_median_std_with_errors(R_Mag_035_045, R_Mag_Err_035_045)
Median_Mag_I_035_045, Median_Mag_I_SD_035_045, Median_Mag_I_SD_Use_035_045 = bootstrap_median_std_with_errors(I_Mag_035_045, I_Mag_Err_035_045)
Median_Mag_Z_035_045, Median_Mag_Z_SD_035_045, Median_Mag_Z_SD_Use_035_045 = bootstrap_median_std_with_errors(Z_Mag_035_045, Z_Mag_Err_035_045)
Median_Mag_Y_035_045, Median_Mag_Y_SD_035_045, Median_Mag_Y_SD_Use_035_045 = bootstrap_median_std_with_errors(Y_Mag_035_045, Y_Mag_Err_035_045)


In [ ]:
Median_Mag_G_045_055, Median_Mag_G_SD_045_055, Median_Mag_G_SD_Use_045_055 = bootstrap_median_std_with_errors(G_Mag_045_055, G_Mag_Err_045_055)
Median_Mag_R_045_055, Median_Mag_R_SD_045_055, Median_Mag_R_SD_Use_045_055 = bootstrap_median_std_with_errors(R_Mag_045_055, R_Mag_Err_045_055)
Median_Mag_I_045_055, Median_Mag_I_SD_045_055, Median_Mag_I_SD_Use_045_055 = bootstrap_median_std_with_errors(I_Mag_045_055, I_Mag_Err_045_055)
Median_Mag_Z_045_055, Median_Mag_Z_SD_045_055, Median_Mag_Z_SD_Use_045_055 = bootstrap_median_std_with_errors(Z_Mag_045_055, Z_Mag_Err_045_055)
Median_Mag_Y_045_055, Median_Mag_Y_SD_045_055, Median_Mag_Y_SD_Use_045_055 = bootstrap_median_std_with_errors(Y_Mag_045_055, Y_Mag_Err_045_055)


In [ ]:
Median_Mag_G_055_065, Median_Mag_G_SD_055_065, Median_Mag_G_SD_Use_055_065 = bootstrap_median_std_with_errors(G_Mag_055_065, G_Mag_Err_055_065)
Median_Mag_R_055_065, Median_Mag_R_SD_055_065, Median_Mag_R_SD_Use_055_065 = bootstrap_median_std_with_errors(R_Mag_055_065, R_Mag_Err_055_065)
Median_Mag_I_055_065, Median_Mag_I_SD_055_065, Median_Mag_I_SD_Use_055_065 = bootstrap_median_std_with_errors(I_Mag_055_065, I_Mag_Err_055_065)
Median_Mag_Z_055_065, Median_Mag_Z_SD_055_065, Median_Mag_Z_SD_Use_055_065 = bootstrap_median_std_with_errors(Z_Mag_055_065, Z_Mag_Err_055_065)
Median_Mag_Y_055_065, Median_Mag_Y_SD_055_065, Median_Mag_Y_SD_Use_055_065 = bootstrap_median_std_with_errors(Y_Mag_055_065, Y_Mag_Err_055_065)


In [ ]:
Median_Mag_G_065_075, Median_Mag_G_SD_065_075, Median_Mag_G_SD_Use_065_075 = bootstrap_median_std_with_errors(G_Mag_065_075, G_Mag_Err_065_075)
Median_Mag_R_065_075, Median_Mag_R_SD_065_075, Median_Mag_R_SD_Use_065_075 = bootstrap_median_std_with_errors(R_Mag_065_075, R_Mag_Err_065_075)
Median_Mag_I_065_075, Median_Mag_I_SD_065_075, Median_Mag_I_SD_Use_065_075 = bootstrap_median_std_with_errors(I_Mag_065_075, I_Mag_Err_065_075)
Median_Mag_Z_065_075, Median_Mag_Z_SD_065_075, Median_Mag_Z_SD_Use_065_075 = bootstrap_median_std_with_errors(Z_Mag_065_075, Z_Mag_Err_065_075)
Median_Mag_Y_065_075, Median_Mag_Y_SD_065_075, Median_Mag_Y_SD_Use_065_075 = bootstrap_median_std_with_errors(Y_Mag_065_075, Y_Mag_Err_065_075)


In [ ]:
Median_Mag_G_075_085, Median_Mag_G_SD_075_085, Median_Mag_G_SD_Use_075_085 = bootstrap_median_std_with_errors(G_Mag_075_085, G_Mag_Err_075_085)
Median_Mag_R_075_085, Median_Mag_R_SD_075_085, Median_Mag_R_SD_Use_075_085 = bootstrap_median_std_with_errors(R_Mag_075_085, R_Mag_Err_075_085)
Median_Mag_I_075_085, Median_Mag_I_SD_075_085, Median_Mag_I_SD_Use_075_085 = bootstrap_median_std_with_errors(I_Mag_075_085, I_Mag_Err_075_085)
Median_Mag_Z_075_085, Median_Mag_Z_SD_075_085, Median_Mag_Z_SD_Use_075_085 = bootstrap_median_std_with_errors(Z_Mag_075_085, Z_Mag_Err_075_085)
Median_Mag_Y_075_085, Median_Mag_Y_SD_075_085, Median_Mag_Y_SD_Use_075_085 = bootstrap_median_std_with_errors(Y_Mag_075_085, Y_Mag_Err_075_085)


In [ ]:
Median_Mag_G_085_096, Median_Mag_G_SD_085_096, Median_Mag_G_SD_Use_085_096 = bootstrap_median_std_with_errors(G_Mag_085_096, G_Mag_Err_085_096)
Median_Mag_R_085_096, Median_Mag_R_SD_085_096, Median_Mag_R_SD_Use_085_096 = bootstrap_median_std_with_errors(R_Mag_085_096, R_Mag_Err_085_096)
Median_Mag_I_085_096, Median_Mag_I_SD_085_096, Median_Mag_I_SD_Use_085_096 = bootstrap_median_std_with_errors(I_Mag_085_096, I_Mag_Err_085_096)
Median_Mag_Z_085_096, Median_Mag_Z_SD_085_096, Median_Mag_Z_SD_Use_085_096 = bootstrap_median_std_with_errors(Z_Mag_085_096, Z_Mag_Err_085_096)
Median_Mag_Y_085_096, Median_Mag_Y_SD_085_096, Median_Mag_Y_SD_Use_085_096 = bootstrap_median_std_with_errors(Y_Mag_085_096, Y_Mag_Err_085_096)


## <font color='#FF0B55' size=6 >Mirroring the radial profiles around zero x-axis and fitting with two gaussians</font>

In [ ]:
Mirrored_Radius_array_025_035 = np.concatenate((np.flip(Average_Counts_025_035), Average_Counts_025_035))
Mirrored_Radius_array_035_045 = np.concatenate((np.flip(Average_Counts_035_045), Average_Counts_035_045))
Mirrored_Radius_array_045_055 = np.concatenate((np.flip(Average_Counts_045_055), Average_Counts_045_055))
Mirrored_Radius_array_055_065 = np.concatenate((np.flip(Average_Counts_055_065), Average_Counts_055_065))
Mirrored_Radius_array_065_075 = np.concatenate((np.flip(Average_Counts_065_075), Average_Counts_065_075))
Mirrored_Radius_array_075_085 = np.concatenate((np.flip(Average_Counts_075_085), Average_Counts_075_085))
Mirrored_Radius_array_085_096 = np.concatenate((np.flip(Average_Counts_085_096), Average_Counts_085_096))


In [ ]:
Mirrored_Radius_array_SD_025_035 = np.concatenate((np.flip(Average_Counts_SD_025_035), Average_Counts_SD_025_035))
Mirrored_Radius_array_SD_035_045 = np.concatenate((np.flip(Average_Counts_SD_035_045), Average_Counts_SD_035_045))
Mirrored_Radius_array_SD_045_055 = np.concatenate((np.flip(Average_Counts_SD_045_055), Average_Counts_SD_045_055))
Mirrored_Radius_array_SD_055_065 = np.concatenate((np.flip(Average_Counts_SD_055_065), Average_Counts_SD_055_065))
Mirrored_Radius_array_SD_065_075 = np.concatenate((np.flip(Average_Counts_SD_065_075), Average_Counts_SD_065_075))
Mirrored_Radius_array_SD_075_085 = np.concatenate((np.flip(Average_Counts_SD_075_085), Average_Counts_SD_075_085))
Mirrored_Radius_array_SD_085_096 = np.concatenate((np.flip(Average_Counts_SD_085_096), Average_Counts_SD_085_096))

In [ ]:
Mirrored_Equal_Area_Radius_025_035_Array = np.concatenate((-np.flip(Equal_Area_Radius_025_035_Array), Equal_Area_Radius_025_035_Array))
Mirrored_Equal_Area_Radius_035_045_Array = np.concatenate((-np.flip(Equal_Area_Radius_035_045_Array), Equal_Area_Radius_035_045_Array))
Mirrored_Equal_Area_Radius_045_055_Array = np.concatenate((-np.flip(Equal_Area_Radius_045_055_Array), Equal_Area_Radius_045_055_Array))
Mirrored_Equal_Area_Radius_055_065_Array = np.concatenate((-np.flip(Equal_Area_Radius_055_065_Array), Equal_Area_Radius_055_065_Array))
Mirrored_Equal_Area_Radius_065_075_Array = np.concatenate((-np.flip(Equal_Area_Radius_065_075_Array), Equal_Area_Radius_065_075_Array))
Mirrored_Equal_Area_Radius_075_085_Array = np.concatenate((-np.flip(Equal_Area_Radius_075_085_Array), Equal_Area_Radius_075_085_Array))
Mirrored_Equal_Area_Radius_085_096_Array = np.concatenate((-np.flip(Equal_Area_Radius_085_096_Array), Equal_Area_Radius_085_096_Array))


In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_025_035 = Mirrored_Equal_Area_Radius_025_035_Array
x_new_Mirror_Two_Gaussian_025_035 = np.linspace(min(x_data_Mirror_Two_Gaussian_025_035),max(x_data_Mirror_Two_Gaussian_025_035), 200)

y_data_Mirror_Two_Gaussian_025_035 = Mirrored_Radius_array_025_035
y_new_Mirror_Two_Gaussian_025_035 = CubicSpline(x_data_Mirror_Two_Gaussian_025_035, y_data_Mirror_Two_Gaussian_025_035)

yy_new_Mirror_Two_Gaussian_025_035 = y_new_Mirror_Two_Gaussian_025_035(x_new_Mirror_Two_Gaussian_025_035)


y_err_Mirror_Two_Gaussian_025_035 = Mirrored_Radius_array_SD_025_035

#From Popt --- amp1, wid1, amp2, wid2 = [20.10250112  0.44475915  6.25872261  1.77587814]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.33616256 0.00890189 0.28838261 0.05618633]

amp1_Two_Gaussian_025_035 = 20.10250112
wid1_Two_Gaussian_025_035 = 0.44475915

amp2_Two_Gaussian_025_035 = 6.25872261
wid2_Two_Gaussian_025_035 = 1.77587814

initial_Mirror_Two_Gaussian_025_035 = np.array([amp1_Two_Gaussian_025_035, wid1_Two_Gaussian_025_035, amp2_Two_Gaussian_025_035, wid2_Two_Gaussian_025_035]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_025_035 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_025_035 = amp1_Two_Gaussian_025_035 + (amp1_Two_Gaussian_025_035*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_025_035 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_025_035 = wid1_Two_Gaussian_025_035 + (wid1_Two_Gaussian_025_035*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_025_035 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_025_035 = amp2_Two_Gaussian_025_035 + (amp2_Two_Gaussian_025_035*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_025_035 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_025_035 = wid2_Two_Gaussian_025_035 + (wid2_Two_Gaussian_025_035*0.1)


Popt_Two_Gaussian_025_035, Pcov_Two_Gaussian_025_035 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_025_035, y_data_Mirror_Two_Gaussian_025_035, p0=[amp1_Two_Gaussian_025_035, wid1_Two_Gaussian_025_035, amp2_Two_Gaussian_025_035, wid2_Two_Gaussian_025_035], bounds=(0, np.inf))

Perr_Two_Gaussian_025_035 = np.sqrt(np.diag(Pcov_Two_Gaussian_025_035))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_025_035, wid1_Two_Gaussian_025_035, amp2_Two_Gaussian_025_035, wid2_Two_Gaussian_025_035 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_025_035, wid1_Two_Gaussian_025_035, amp2_Two_Gaussian_025_035, wid2_Two_Gaussian_025_035)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_025_035, wid1_Two_Gaussian_025_035, amp2_Two_Gaussian_025_035, wid2_Two_Gaussian_025_035 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_025_035 < amp1_Two_Gaussian_025_035 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_025_035 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_025_035 < amp2_Two_Gaussian_025_035 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_025_035 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_025_035 < wid1_Two_Gaussian_025_035 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_025_035 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_025_035 < wid2_Two_Gaussian_025_035 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_025_035):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_025_035 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_025_035, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_025_035), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_025_035, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_025_035), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_025_035, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_025_035), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_025_035, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_025_035)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_025_035 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_025_035, args=(x_data_Mirror_Two_Gaussian_025_035, y_data_Mirror_Two_Gaussian_025_035, y_err_Mirror_Two_Gaussian_025_035), bounds=bounds_Mirror_Two_Gaussian_025_035)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_025_035 = soln_Mirror_Two_Gaussian_025_035.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_025_035.x))
nwalkers_Mirror_Two_Gaussian_025_035, ndim_Mirror_Two_Gaussian_025_035 = pos_Mirror_Two_Gaussian_025_035.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_025_035 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_025_035, ndim_Mirror_Two_Gaussian_025_035, log_probability, args=(x_data_Mirror_Two_Gaussian_025_035, y_data_Mirror_Two_Gaussian_025_035, y_err_Mirror_Two_Gaussian_025_035))
sampler_Mirror_Two_Gaussian_025_035.run_mcmc(pos_Mirror_Two_Gaussian_025_035, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_025_035 = sampler_Mirror_Two_Gaussian_025_035.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_025_035 = []
for i in range(ndim_Mirror_Two_Gaussian_025_035):
    mcmc_Mirror_Two_Gaussian_025_035 = np.percentile(samples_Mirror_Two_Gaussian_025_035[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_025_035.append(mcmc_Mirror_Two_Gaussian_025_035[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_025_035 = []
for i in range(ndim_Mirror_Two_Gaussian_025_035):
     mcmc_Mirror_Two_Gaussian_025_035 = np.percentile(samples_Mirror_Two_Gaussian_025_035[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_025_035.append([mcmc_Mirror_Two_Gaussian_025_035[2]-mcmc_Mirror_Two_Gaussian_025_035[1],mcmc_Mirror_Two_Gaussian_025_035[1]-mcmc_Mirror_Two_Gaussian_025_035[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_025_035))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_025_035))

ax[0].plot(x_new_Mirror_Two_Gaussian_025_035, two_Gaussian(x_new_Mirror_Two_Gaussian_025_035, *best_fit_params_Mirror_Two_Gaussian_025_035), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_025_035, yy_new_Mirror_Two_Gaussian_025_035, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_025_035, y_data_Mirror_Two_Gaussian_025_035, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_025_035, Gaussian(x_new_Mirror_Two_Gaussian_025_035, best_fit_params_Mirror_Two_Gaussian_025_035[0], best_fit_params_Mirror_Two_Gaussian_025_035[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_025_035, Gaussian(x_new_Mirror_Two_Gaussian_025_035, best_fit_params_Mirror_Two_Gaussian_025_035[2], best_fit_params_Mirror_Two_Gaussian_025_035[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_025_035, yy_new_Mirror_Two_Gaussian_025_035 - two_Gaussian(x_new_Mirror_Two_Gaussian_025_035, *best_fit_params_Mirror_Two_Gaussian_025_035), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_025_035, (yy_new_Mirror_Two_Gaussian_025_035 - two_Gaussian(x_new_Mirror_Two_Gaussian_025_035, *best_fit_params_Mirror_Two_Gaussian_025_035))/yy_new_Mirror_Two_Gaussian_025_035, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_025_035, two_Gaussian(x_new_Mirror_Two_Gaussian_025_035, *Popt_Two_Gaussian_025_035), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_025_035, yy_new_Mirror_Two_Gaussian_025_035, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_025_035, y_data_Mirror_Two_Gaussian_025_035, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_025_035, Gaussian(x_new_Mirror_Two_Gaussian_025_035, Popt_Two_Gaussian_025_035[0], Popt_Two_Gaussian_025_035[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_025_035, Gaussian(x_new_Mirror_Two_Gaussian_025_035, Popt_Two_Gaussian_025_035[2], Popt_Two_Gaussian_025_035[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_025_035, (yy_new_Mirror_Two_Gaussian_025_035 - two_Gaussian(x_new_Mirror_Two_Gaussian_025_035, *Popt_Two_Gaussian_025_035)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_025_035, (yy_new_Mirror_Two_Gaussian_025_035 - two_Gaussian(x_new_Mirror_Two_Gaussian_025_035, *Popt_Two_Gaussian_025_035))/yy_new_Mirror_Two_Gaussian_025_035, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian - 0.25 < z < 0.35")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_025_035[0]) + " +" +str(errors_Mirror_Two_Gaussian_025_035[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_025_035[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_025_035[1]) + " +" +str(errors_Mirror_Two_Gaussian_025_035[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_025_035[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_025_035[2]) + " +" +str(errors_Mirror_Two_Gaussian_025_035[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_025_035[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_025_035[3]) + " +" +str(errors_Mirror_Two_Gaussian_025_035[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_025_035[3][1]))


In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_035_045 = Mirrored_Equal_Area_Radius_035_045_Array
x_new_Mirror_Two_Gaussian_035_045 = np.linspace(min(x_data_Mirror_Two_Gaussian_035_045),max(x_data_Mirror_Two_Gaussian_035_045), 200)

y_data_Mirror_Two_Gaussian_035_045 = Mirrored_Radius_array_035_045
y_new_Mirror_Two_Gaussian_035_045 = CubicSpline(x_data_Mirror_Two_Gaussian_035_045, y_data_Mirror_Two_Gaussian_035_045)

yy_new_Mirror_Two_Gaussian_035_045 = y_new_Mirror_Two_Gaussian_035_045(x_new_Mirror_Two_Gaussian_035_045)


y_err_Mirror_Two_Gaussian_035_045 = Mirrored_Radius_array_SD_035_045

#From Popt --- amp1, wid1, amp2, wid2 = [10.16983856  0.4119127   3.48297031  1.64743027]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.18259892 0.00886948 0.15625607 0.05079227]

amp1_Two_Gaussian_035_045 = 10.16983856
wid1_Two_Gaussian_035_045 = 0.4119127

amp2_Two_Gaussian_035_045 = 3.48297031
wid2_Two_Gaussian_035_045 = 1.64743027

initial_Mirror_Two_Gaussian_035_045 = np.array([amp1_Two_Gaussian_035_045, wid1_Two_Gaussian_035_045, amp2_Two_Gaussian_035_045, wid2_Two_Gaussian_035_045]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_035_045 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_035_045 = amp1_Two_Gaussian_035_045 + (amp1_Two_Gaussian_035_045*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_035_045 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_035_045 = wid1_Two_Gaussian_035_045 + (wid1_Two_Gaussian_035_045*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_035_045 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_035_045 = amp2_Two_Gaussian_035_045 + (amp2_Two_Gaussian_035_045*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_035_045 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_035_045 = wid2_Two_Gaussian_035_045 + (wid2_Two_Gaussian_035_045*0.1)


Popt_Two_Gaussian_035_045, Pcov_Two_Gaussian_035_045 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_035_045, y_data_Mirror_Two_Gaussian_035_045, p0=[amp1_Two_Gaussian_035_045, wid1_Two_Gaussian_035_045, amp2_Two_Gaussian_035_045, wid2_Two_Gaussian_035_045], bounds=(0, np.inf))

Perr_Two_Gaussian_035_045 = np.sqrt(np.diag(Pcov_Two_Gaussian_035_045))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_035_045, wid1_Two_Gaussian_035_045, amp2_Two_Gaussian_035_045, wid2_Two_Gaussian_035_045 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_035_045, wid1_Two_Gaussian_035_045, amp2_Two_Gaussian_035_045, wid2_Two_Gaussian_035_045)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_035_045, wid1_Two_Gaussian_035_045, amp2_Two_Gaussian_035_045, wid2_Two_Gaussian_035_045 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_035_045 < amp1_Two_Gaussian_035_045 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_035_045 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_035_045 < amp2_Two_Gaussian_035_045 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_035_045 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_035_045 < wid1_Two_Gaussian_035_045 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_035_045 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_035_045 < wid2_Two_Gaussian_035_045 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_035_045):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_035_045 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_035_045, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_035_045), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_035_045, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_035_045), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_035_045, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_035_045), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_035_045, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_035_045)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_035_045 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_035_045, args=(x_data_Mirror_Two_Gaussian_035_045, y_data_Mirror_Two_Gaussian_035_045, y_err_Mirror_Two_Gaussian_035_045), bounds=bounds_Mirror_Two_Gaussian_035_045)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_035_045 = soln_Mirror_Two_Gaussian_035_045.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_035_045.x))
nwalkers_Mirror_Two_Gaussian_035_045, ndim_Mirror_Two_Gaussian_035_045 = pos_Mirror_Two_Gaussian_035_045.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_035_045 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_035_045, ndim_Mirror_Two_Gaussian_035_045, log_probability, args=(x_data_Mirror_Two_Gaussian_035_045, y_data_Mirror_Two_Gaussian_035_045, y_err_Mirror_Two_Gaussian_035_045))
sampler_Mirror_Two_Gaussian_035_045.run_mcmc(pos_Mirror_Two_Gaussian_035_045, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_035_045 = sampler_Mirror_Two_Gaussian_035_045.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_035_045 = []
for i in range(ndim_Mirror_Two_Gaussian_035_045):
    mcmc_Mirror_Two_Gaussian_035_045 = np.percentile(samples_Mirror_Two_Gaussian_035_045[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_035_045.append(mcmc_Mirror_Two_Gaussian_035_045[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_035_045 = []
for i in range(ndim_Mirror_Two_Gaussian_035_045):
     mcmc_Mirror_Two_Gaussian_035_045 = np.percentile(samples_Mirror_Two_Gaussian_035_045[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_035_045.append([mcmc_Mirror_Two_Gaussian_035_045[2]-mcmc_Mirror_Two_Gaussian_035_045[1],mcmc_Mirror_Two_Gaussian_035_045[1]-mcmc_Mirror_Two_Gaussian_035_045[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_035_045))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_035_045))

ax[0].plot(x_new_Mirror_Two_Gaussian_035_045, two_Gaussian(x_new_Mirror_Two_Gaussian_035_045, *best_fit_params_Mirror_Two_Gaussian_035_045), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_035_045, yy_new_Mirror_Two_Gaussian_035_045, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_035_045, y_data_Mirror_Two_Gaussian_035_045, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_035_045, Gaussian(x_new_Mirror_Two_Gaussian_035_045, best_fit_params_Mirror_Two_Gaussian_035_045[0], best_fit_params_Mirror_Two_Gaussian_035_045[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_035_045, Gaussian(x_new_Mirror_Two_Gaussian_035_045, best_fit_params_Mirror_Two_Gaussian_035_045[2], best_fit_params_Mirror_Two_Gaussian_035_045[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_035_045, yy_new_Mirror_Two_Gaussian_035_045 - two_Gaussian(x_new_Mirror_Two_Gaussian_035_045, *best_fit_params_Mirror_Two_Gaussian_035_045), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_035_045, (yy_new_Mirror_Two_Gaussian_035_045 - two_Gaussian(x_new_Mirror_Two_Gaussian_035_045, *best_fit_params_Mirror_Two_Gaussian_035_045))/yy_new_Mirror_Two_Gaussian_035_045, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_035_045, two_Gaussian(x_new_Mirror_Two_Gaussian_035_045, *Popt_Two_Gaussian_035_045), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_035_045, yy_new_Mirror_Two_Gaussian_035_045, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_035_045, y_data_Mirror_Two_Gaussian_035_045, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_035_045, Gaussian(x_new_Mirror_Two_Gaussian_035_045, Popt_Two_Gaussian_035_045[0], Popt_Two_Gaussian_035_045[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_035_045, Gaussian(x_new_Mirror_Two_Gaussian_035_045, Popt_Two_Gaussian_035_045[2], Popt_Two_Gaussian_035_045[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_035_045, (yy_new_Mirror_Two_Gaussian_035_045 - two_Gaussian(x_new_Mirror_Two_Gaussian_035_045, *Popt_Two_Gaussian_035_045)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_035_045, (yy_new_Mirror_Two_Gaussian_035_045 - two_Gaussian(x_new_Mirror_Two_Gaussian_035_045, *Popt_Two_Gaussian_035_045))/yy_new_Mirror_Two_Gaussian_035_045, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian Fake Error - 0.35 < z < 0.45")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_035_045[0]) + " +" +str(errors_Mirror_Two_Gaussian_035_045[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_035_045[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_035_045[1]) + " +" +str(errors_Mirror_Two_Gaussian_035_045[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_035_045[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_035_045[2]) + " +" +str(errors_Mirror_Two_Gaussian_035_045[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_035_045[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_035_045[3]) + " +" +str(errors_Mirror_Two_Gaussian_035_045[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_035_045[3][1]))

In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_045_055 = Mirrored_Equal_Area_Radius_045_055_Array
x_new_Mirror_Two_Gaussian_045_055 = np.linspace(min(x_data_Mirror_Two_Gaussian_045_055),max(x_data_Mirror_Two_Gaussian_045_055), 200)

y_data_Mirror_Two_Gaussian_045_055 = Mirrored_Radius_array_045_055
y_new_Mirror_Two_Gaussian_045_055 = CubicSpline(x_data_Mirror_Two_Gaussian_045_055, y_data_Mirror_Two_Gaussian_045_055)

yy_new_Mirror_Two_Gaussian_045_055 = y_new_Mirror_Two_Gaussian_045_055(x_new_Mirror_Two_Gaussian_045_055)


y_err_Mirror_Two_Gaussian_045_055 = Mirrored_Radius_array_SD_045_055

#From Popt --- amp1, wid1, amp2, wid2 = [7.62532714 0.43522862 2.33885045 1.63937616]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.123544   0.00824165 0.11074941 0.05125452]

amp1_Two_Gaussian_045_055 = 7.62532714
wid1_Two_Gaussian_045_055 = 0.43522862

amp2_Two_Gaussian_045_055 = 2.33885045
wid2_Two_Gaussian_045_055 = 1.63937616

initial_Mirror_Two_Gaussian_045_055 = np.array([amp1_Two_Gaussian_045_055, wid1_Two_Gaussian_045_055, amp2_Two_Gaussian_045_055, wid2_Two_Gaussian_045_055]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_045_055 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_045_055 = amp1_Two_Gaussian_045_055 + (amp1_Two_Gaussian_045_055*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_045_055 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_045_055 = wid1_Two_Gaussian_045_055 + (wid1_Two_Gaussian_045_055*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_045_055 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_045_055 = amp2_Two_Gaussian_045_055 + (amp2_Two_Gaussian_045_055*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_045_055 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_045_055 = wid2_Two_Gaussian_045_055 + (wid2_Two_Gaussian_045_055*0.1)


Popt_Two_Gaussian_045_055, Pcov_Two_Gaussian_045_055 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_045_055, y_data_Mirror_Two_Gaussian_045_055, p0=[amp1_Two_Gaussian_045_055, wid1_Two_Gaussian_045_055, amp2_Two_Gaussian_045_055, wid2_Two_Gaussian_045_055], bounds=(0, np.inf))

Perr_Two_Gaussian_045_055 = np.sqrt(np.diag(Pcov_Two_Gaussian_045_055))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_045_055, wid1_Two_Gaussian_045_055, amp2_Two_Gaussian_045_055, wid2_Two_Gaussian_045_055 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_045_055, wid1_Two_Gaussian_045_055, amp2_Two_Gaussian_045_055, wid2_Two_Gaussian_045_055)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_045_055, wid1_Two_Gaussian_045_055, amp2_Two_Gaussian_045_055, wid2_Two_Gaussian_045_055 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_045_055 < amp1_Two_Gaussian_045_055 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_045_055 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_045_055 < amp2_Two_Gaussian_045_055 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_045_055 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_045_055 < wid1_Two_Gaussian_045_055 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_045_055 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_045_055 < wid2_Two_Gaussian_045_055 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_045_055):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_045_055 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_045_055, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_045_055), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_045_055, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_045_055), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_045_055, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_045_055), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_045_055, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_045_055)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_045_055 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_045_055, args=(x_data_Mirror_Two_Gaussian_045_055, y_data_Mirror_Two_Gaussian_045_055, y_err_Mirror_Two_Gaussian_045_055), bounds=bounds_Mirror_Two_Gaussian_045_055)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_045_055 = soln_Mirror_Two_Gaussian_045_055.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_045_055.x))
nwalkers_Mirror_Two_Gaussian_045_055, ndim_Mirror_Two_Gaussian_045_055 = pos_Mirror_Two_Gaussian_045_055.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_045_055 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_045_055, ndim_Mirror_Two_Gaussian_045_055, log_probability, args=(x_data_Mirror_Two_Gaussian_045_055, y_data_Mirror_Two_Gaussian_045_055, y_err_Mirror_Two_Gaussian_045_055))
sampler_Mirror_Two_Gaussian_045_055.run_mcmc(pos_Mirror_Two_Gaussian_045_055, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_045_055 = sampler_Mirror_Two_Gaussian_045_055.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_045_055 = []
for i in range(ndim_Mirror_Two_Gaussian_045_055):
    mcmc_Mirror_Two_Gaussian_045_055 = np.percentile(samples_Mirror_Two_Gaussian_045_055[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_045_055.append(mcmc_Mirror_Two_Gaussian_045_055[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_045_055 = []
for i in range(ndim_Mirror_Two_Gaussian_045_055):
     mcmc_Mirror_Two_Gaussian_045_055 = np.percentile(samples_Mirror_Two_Gaussian_045_055[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_045_055.append([mcmc_Mirror_Two_Gaussian_045_055[2]-mcmc_Mirror_Two_Gaussian_045_055[1],mcmc_Mirror_Two_Gaussian_045_055[1]-mcmc_Mirror_Two_Gaussian_045_055[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_045_055))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_045_055))

ax[0].plot(x_new_Mirror_Two_Gaussian_045_055, two_Gaussian(x_new_Mirror_Two_Gaussian_045_055, *best_fit_params_Mirror_Two_Gaussian_045_055), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_045_055, yy_new_Mirror_Two_Gaussian_045_055, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_045_055, y_data_Mirror_Two_Gaussian_045_055, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_045_055, Gaussian(x_new_Mirror_Two_Gaussian_045_055, best_fit_params_Mirror_Two_Gaussian_045_055[0], best_fit_params_Mirror_Two_Gaussian_045_055[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_045_055, Gaussian(x_new_Mirror_Two_Gaussian_045_055, best_fit_params_Mirror_Two_Gaussian_045_055[2], best_fit_params_Mirror_Two_Gaussian_045_055[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_045_055, yy_new_Mirror_Two_Gaussian_045_055 - two_Gaussian(x_new_Mirror_Two_Gaussian_045_055, *best_fit_params_Mirror_Two_Gaussian_045_055), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_045_055, (yy_new_Mirror_Two_Gaussian_045_055 - two_Gaussian(x_new_Mirror_Two_Gaussian_045_055, *best_fit_params_Mirror_Two_Gaussian_045_055))/yy_new_Mirror_Two_Gaussian_045_055, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_045_055, two_Gaussian(x_new_Mirror_Two_Gaussian_045_055, *Popt_Two_Gaussian_045_055), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_045_055, yy_new_Mirror_Two_Gaussian_045_055, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_045_055, y_data_Mirror_Two_Gaussian_045_055, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_045_055, Gaussian(x_new_Mirror_Two_Gaussian_045_055, Popt_Two_Gaussian_045_055[0], Popt_Two_Gaussian_045_055[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_045_055, Gaussian(x_new_Mirror_Two_Gaussian_045_055, Popt_Two_Gaussian_045_055[2], Popt_Two_Gaussian_045_055[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_045_055, (yy_new_Mirror_Two_Gaussian_045_055 - two_Gaussian(x_new_Mirror_Two_Gaussian_045_055, *Popt_Two_Gaussian_045_055)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_045_055, (yy_new_Mirror_Two_Gaussian_045_055 - two_Gaussian(x_new_Mirror_Two_Gaussian_045_055, *Popt_Two_Gaussian_045_055))/yy_new_Mirror_Two_Gaussian_045_055, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian Fake Error - 0.45 < z < 0.55")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_045_055[0]) + " +" +str(errors_Mirror_Two_Gaussian_045_055[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_045_055[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_045_055[1]) + " +" +str(errors_Mirror_Two_Gaussian_045_055[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_045_055[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_045_055[2]) + " +" +str(errors_Mirror_Two_Gaussian_045_055[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_045_055[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_045_055[3]) + " +" +str(errors_Mirror_Two_Gaussian_045_055[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_045_055[3][1]))

In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_055_065 = Mirrored_Equal_Area_Radius_055_065_Array
x_new_Mirror_Two_Gaussian_055_065 = np.linspace(min(x_data_Mirror_Two_Gaussian_055_065),max(x_data_Mirror_Two_Gaussian_055_065), 200)

y_data_Mirror_Two_Gaussian_055_065 = Mirrored_Radius_array_055_065
y_new_Mirror_Two_Gaussian_055_065 = CubicSpline(x_data_Mirror_Two_Gaussian_055_065, y_data_Mirror_Two_Gaussian_055_065)

yy_new_Mirror_Two_Gaussian_055_065 = y_new_Mirror_Two_Gaussian_055_065(x_new_Mirror_Two_Gaussian_055_065)


y_err_Mirror_Two_Gaussian_055_065 = Mirrored_Radius_array_SD_055_065

#From Popt --- amp1, wid1, amp2, wid2 = [8.70631936 0.39435141 2.41852383 1.48264752]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.12361988 0.00655527 0.11086506 0.04480188]

amp1_Two_Gaussian_055_065 = 8.70631936
wid1_Two_Gaussian_055_065 = 0.39435141

amp2_Two_Gaussian_055_065 = 2.41852383
wid2_Two_Gaussian_055_065 = 1.48264752

initial_Mirror_Two_Gaussian_055_065 = np.array([amp1_Two_Gaussian_055_065, wid1_Two_Gaussian_055_065, amp2_Two_Gaussian_055_065, wid2_Two_Gaussian_055_065]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_055_065 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_055_065 = amp1_Two_Gaussian_055_065 + (amp1_Two_Gaussian_055_065*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_055_065 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_055_065 = wid1_Two_Gaussian_055_065 + (wid1_Two_Gaussian_055_065*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_055_065 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_055_065 = amp2_Two_Gaussian_055_065 + (amp2_Two_Gaussian_055_065*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_055_065 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_055_065 = wid2_Two_Gaussian_055_065 + (wid2_Two_Gaussian_055_065*0.1)


Popt_Two_Gaussian_055_065, Pcov_Two_Gaussian_055_065 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_055_065, y_data_Mirror_Two_Gaussian_055_065, p0=[amp1_Two_Gaussian_055_065, wid1_Two_Gaussian_055_065, amp2_Two_Gaussian_055_065, wid2_Two_Gaussian_055_065], bounds=(0, np.inf))

Perr_Two_Gaussian_055_065 = np.sqrt(np.diag(Pcov_Two_Gaussian_055_065))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_055_065, wid1_Two_Gaussian_055_065, amp2_Two_Gaussian_055_065, wid2_Two_Gaussian_055_065 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_055_065, wid1_Two_Gaussian_055_065, amp2_Two_Gaussian_055_065, wid2_Two_Gaussian_055_065)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_055_065, wid1_Two_Gaussian_055_065, amp2_Two_Gaussian_055_065, wid2_Two_Gaussian_055_065 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_055_065 < amp1_Two_Gaussian_055_065 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_055_065 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_055_065 < amp2_Two_Gaussian_055_065 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_055_065 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_055_065 < wid1_Two_Gaussian_055_065 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_055_065 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_055_065 < wid2_Two_Gaussian_055_065 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_055_065):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_055_065 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_055_065, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_055_065), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_055_065, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_055_065), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_055_065, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_055_065), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_055_065, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_055_065)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_055_065 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_055_065, args=(x_data_Mirror_Two_Gaussian_055_065, y_data_Mirror_Two_Gaussian_055_065, y_err_Mirror_Two_Gaussian_055_065), bounds=bounds_Mirror_Two_Gaussian_055_065)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_055_065 = soln_Mirror_Two_Gaussian_055_065.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_055_065.x))
nwalkers_Mirror_Two_Gaussian_055_065, ndim_Mirror_Two_Gaussian_055_065 = pos_Mirror_Two_Gaussian_055_065.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_055_065 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_055_065, ndim_Mirror_Two_Gaussian_055_065, log_probability, args=(x_data_Mirror_Two_Gaussian_055_065, y_data_Mirror_Two_Gaussian_055_065, y_err_Mirror_Two_Gaussian_055_065))
sampler_Mirror_Two_Gaussian_055_065.run_mcmc(pos_Mirror_Two_Gaussian_055_065, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_055_065 = sampler_Mirror_Two_Gaussian_055_065.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_055_065 = []
for i in range(ndim_Mirror_Two_Gaussian_055_065):
    mcmc_Mirror_Two_Gaussian_055_065 = np.percentile(samples_Mirror_Two_Gaussian_055_065[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_055_065.append(mcmc_Mirror_Two_Gaussian_055_065[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_055_065 = []
for i in range(ndim_Mirror_Two_Gaussian_055_065):
     mcmc_Mirror_Two_Gaussian_055_065 = np.percentile(samples_Mirror_Two_Gaussian_055_065[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_055_065.append([mcmc_Mirror_Two_Gaussian_055_065[2]-mcmc_Mirror_Two_Gaussian_055_065[1],mcmc_Mirror_Two_Gaussian_055_065[1]-mcmc_Mirror_Two_Gaussian_055_065[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_055_065))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_055_065))

ax[0].plot(x_new_Mirror_Two_Gaussian_055_065, two_Gaussian(x_new_Mirror_Two_Gaussian_055_065, *best_fit_params_Mirror_Two_Gaussian_055_065), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_055_065, yy_new_Mirror_Two_Gaussian_055_065, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_055_065, y_data_Mirror_Two_Gaussian_055_065, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_055_065, Gaussian(x_new_Mirror_Two_Gaussian_055_065, best_fit_params_Mirror_Two_Gaussian_055_065[0], best_fit_params_Mirror_Two_Gaussian_055_065[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_055_065, Gaussian(x_new_Mirror_Two_Gaussian_055_065, best_fit_params_Mirror_Two_Gaussian_055_065[2], best_fit_params_Mirror_Two_Gaussian_055_065[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_055_065, yy_new_Mirror_Two_Gaussian_055_065 - two_Gaussian(x_new_Mirror_Two_Gaussian_055_065, *best_fit_params_Mirror_Two_Gaussian_055_065), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_055_065, (yy_new_Mirror_Two_Gaussian_055_065 - two_Gaussian(x_new_Mirror_Two_Gaussian_055_065, *best_fit_params_Mirror_Two_Gaussian_055_065))/yy_new_Mirror_Two_Gaussian_055_065, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_055_065, two_Gaussian(x_new_Mirror_Two_Gaussian_055_065, *Popt_Two_Gaussian_055_065), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_055_065, yy_new_Mirror_Two_Gaussian_055_065, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_055_065, y_data_Mirror_Two_Gaussian_055_065, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_055_065, Gaussian(x_new_Mirror_Two_Gaussian_055_065, Popt_Two_Gaussian_055_065[0], Popt_Two_Gaussian_055_065[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_055_065, Gaussian(x_new_Mirror_Two_Gaussian_055_065, Popt_Two_Gaussian_055_065[2], Popt_Two_Gaussian_055_065[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_055_065, (yy_new_Mirror_Two_Gaussian_055_065 - two_Gaussian(x_new_Mirror_Two_Gaussian_055_065, *Popt_Two_Gaussian_055_065)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_055_065, (yy_new_Mirror_Two_Gaussian_055_065 - two_Gaussian(x_new_Mirror_Two_Gaussian_055_065, *Popt_Two_Gaussian_055_065))/yy_new_Mirror_Two_Gaussian_055_065, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian - 0.55 < z < 0.65")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_055_065[0]) + " +" +str(errors_Mirror_Two_Gaussian_055_065[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_055_065[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_055_065[1]) + " +" +str(errors_Mirror_Two_Gaussian_055_065[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_055_065[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_055_065[2]) + " +" +str(errors_Mirror_Two_Gaussian_055_065[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_055_065[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_055_065[3]) + " +" +str(errors_Mirror_Two_Gaussian_055_065[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_055_065[3][1]))

In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_065_075 = Mirrored_Equal_Area_Radius_065_075_Array
x_new_Mirror_Two_Gaussian_065_075 = np.linspace(min(x_data_Mirror_Two_Gaussian_065_075),max(x_data_Mirror_Two_Gaussian_065_075), 200)

y_data_Mirror_Two_Gaussian_065_075 = Mirrored_Radius_array_065_075
y_new_Mirror_Two_Gaussian_065_075 = CubicSpline(x_data_Mirror_Two_Gaussian_065_075, y_data_Mirror_Two_Gaussian_065_075)

yy_new_Mirror_Two_Gaussian_065_075 = y_new_Mirror_Two_Gaussian_065_075(x_new_Mirror_Two_Gaussian_065_075)


y_err_Mirror_Two_Gaussian_065_075 = Mirrored_Radius_array_SD_065_075

#From Popt --- amp1, wid1, amp2, wid2 = [9.44170853 0.40670767 2.47531394 1.49263175]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.12574329 0.00625974 0.11475958 0.0448473 ]

amp1_Two_Gaussian_065_075 = 9.44170853
wid1_Two_Gaussian_065_075 = 0.40670767

amp2_Two_Gaussian_065_075 = 2.47531394
wid2_Two_Gaussian_065_075 = 1.49263175

initial_Mirror_Two_Gaussian_065_075 = np.array([amp1_Two_Gaussian_065_075, wid1_Two_Gaussian_065_075, amp2_Two_Gaussian_065_075, wid2_Two_Gaussian_065_075]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_065_075 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_065_075 = amp1_Two_Gaussian_065_075 + (amp1_Two_Gaussian_065_075*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_065_075 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_065_075 = wid1_Two_Gaussian_065_075 + (wid1_Two_Gaussian_065_075*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_065_075 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_065_075 = amp2_Two_Gaussian_065_075 + (amp2_Two_Gaussian_065_075*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_065_075 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_065_075 = wid2_Two_Gaussian_065_075 + (wid2_Two_Gaussian_065_075*0.1)


Popt_Two_Gaussian_065_075, Pcov_Two_Gaussian_065_075 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_065_075, y_data_Mirror_Two_Gaussian_065_075, p0=[amp1_Two_Gaussian_065_075, wid1_Two_Gaussian_065_075, amp2_Two_Gaussian_065_075, wid2_Two_Gaussian_065_075], bounds=(0, np.inf))

Perr_Two_Gaussian_065_075 = np.sqrt(np.diag(Pcov_Two_Gaussian_065_075))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_065_075, wid1_Two_Gaussian_065_075, amp2_Two_Gaussian_065_075, wid2_Two_Gaussian_065_075 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_065_075, wid1_Two_Gaussian_065_075, amp2_Two_Gaussian_065_075, wid2_Two_Gaussian_065_075)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_065_075, wid1_Two_Gaussian_065_075, amp2_Two_Gaussian_065_075, wid2_Two_Gaussian_065_075 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_065_075 < amp1_Two_Gaussian_065_075 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_065_075 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_065_075 < amp2_Two_Gaussian_065_075 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_065_075 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_065_075 < wid1_Two_Gaussian_065_075 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_065_075 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_065_075 < wid2_Two_Gaussian_065_075 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_065_075):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_065_075 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_065_075, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_065_075), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_065_075, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_065_075), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_065_075, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_065_075), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_065_075, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_065_075)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_065_075 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_065_075, args=(x_data_Mirror_Two_Gaussian_065_075, y_data_Mirror_Two_Gaussian_065_075, y_err_Mirror_Two_Gaussian_065_075), bounds=bounds_Mirror_Two_Gaussian_065_075)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_065_075 = soln_Mirror_Two_Gaussian_065_075.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_065_075.x))
nwalkers_Mirror_Two_Gaussian_065_075, ndim_Mirror_Two_Gaussian_065_075 = pos_Mirror_Two_Gaussian_065_075.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_065_075 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_065_075, ndim_Mirror_Two_Gaussian_065_075, log_probability, args=(x_data_Mirror_Two_Gaussian_065_075, y_data_Mirror_Two_Gaussian_065_075, y_err_Mirror_Two_Gaussian_065_075))
sampler_Mirror_Two_Gaussian_065_075.run_mcmc(pos_Mirror_Two_Gaussian_065_075, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_065_075 = sampler_Mirror_Two_Gaussian_065_075.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_065_075 = []
for i in range(ndim_Mirror_Two_Gaussian_065_075):
    mcmc_Mirror_Two_Gaussian_065_075 = np.percentile(samples_Mirror_Two_Gaussian_065_075[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_065_075.append(mcmc_Mirror_Two_Gaussian_065_075[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_065_075 = []
for i in range(ndim_Mirror_Two_Gaussian_065_075):
     mcmc_Mirror_Two_Gaussian_065_075 = np.percentile(samples_Mirror_Two_Gaussian_065_075[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_065_075.append([mcmc_Mirror_Two_Gaussian_065_075[2]-mcmc_Mirror_Two_Gaussian_065_075[1],mcmc_Mirror_Two_Gaussian_065_075[1]-mcmc_Mirror_Two_Gaussian_065_075[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_065_075))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_065_075))

ax[0].plot(x_new_Mirror_Two_Gaussian_065_075, two_Gaussian(x_new_Mirror_Two_Gaussian_065_075, *best_fit_params_Mirror_Two_Gaussian_065_075), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_065_075, yy_new_Mirror_Two_Gaussian_065_075, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_065_075, y_data_Mirror_Two_Gaussian_065_075, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_065_075, Gaussian(x_new_Mirror_Two_Gaussian_065_075, best_fit_params_Mirror_Two_Gaussian_065_075[0], best_fit_params_Mirror_Two_Gaussian_065_075[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_065_075, Gaussian(x_new_Mirror_Two_Gaussian_065_075, best_fit_params_Mirror_Two_Gaussian_065_075[2], best_fit_params_Mirror_Two_Gaussian_065_075[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_065_075, yy_new_Mirror_Two_Gaussian_065_075 - two_Gaussian(x_new_Mirror_Two_Gaussian_065_075, *best_fit_params_Mirror_Two_Gaussian_065_075), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_065_075, (yy_new_Mirror_Two_Gaussian_065_075 - two_Gaussian(x_new_Mirror_Two_Gaussian_065_075, *best_fit_params_Mirror_Two_Gaussian_065_075))/yy_new_Mirror_Two_Gaussian_065_075, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_065_075, two_Gaussian(x_new_Mirror_Two_Gaussian_065_075, *Popt_Two_Gaussian_065_075), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_065_075, yy_new_Mirror_Two_Gaussian_065_075, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_065_075, y_data_Mirror_Two_Gaussian_065_075, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_065_075, Gaussian(x_new_Mirror_Two_Gaussian_065_075, Popt_Two_Gaussian_065_075[0], Popt_Two_Gaussian_065_075[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_065_075, Gaussian(x_new_Mirror_Two_Gaussian_065_075, Popt_Two_Gaussian_065_075[2], Popt_Two_Gaussian_065_075[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_065_075, (yy_new_Mirror_Two_Gaussian_065_075 - two_Gaussian(x_new_Mirror_Two_Gaussian_065_075, *Popt_Two_Gaussian_065_075)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_065_075, (yy_new_Mirror_Two_Gaussian_065_075 - two_Gaussian(x_new_Mirror_Two_Gaussian_065_075, *Popt_Two_Gaussian_065_075))/yy_new_Mirror_Two_Gaussian_065_075, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian - 0.65 < z < 0.75")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_065_075[0]) + " +" +str(errors_Mirror_Two_Gaussian_065_075[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_065_075[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_065_075[1]) + " +" +str(errors_Mirror_Two_Gaussian_065_075[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_065_075[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_065_075[2]) + " +" +str(errors_Mirror_Two_Gaussian_065_075[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_065_075[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_065_075[3]) + " +" +str(errors_Mirror_Two_Gaussian_065_075[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_065_075[3][1]))

In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_075_085 = Mirrored_Equal_Area_Radius_075_085_Array
x_new_Mirror_Two_Gaussian_075_085 = np.linspace(min(x_data_Mirror_Two_Gaussian_075_085),max(x_data_Mirror_Two_Gaussian_075_085), 200)

y_data_Mirror_Two_Gaussian_075_085 = Mirrored_Radius_array_075_085
y_new_Mirror_Two_Gaussian_075_085 = CubicSpline(x_data_Mirror_Two_Gaussian_075_085, y_data_Mirror_Two_Gaussian_075_085)

yy_new_Mirror_Two_Gaussian_075_085 = y_new_Mirror_Two_Gaussian_075_085(x_new_Mirror_Two_Gaussian_075_085)


y_err_Mirror_Two_Gaussian_075_085 = Mirrored_Radius_array_SD_075_085

#From Popt --- amp1, wid1, amp2, wid2 = [8.67627767 0.40034398 2.23799643 1.46093976]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.1146613  0.00609875 0.10504736 0.04425605]

amp1_Two_Gaussian_075_085 = 8.67627767
wid1_Two_Gaussian_075_085 = 0.40034398

amp2_Two_Gaussian_075_085 = 2.23799643
wid2_Two_Gaussian_075_085 = 1.46093976

initial_Mirror_Two_Gaussian_075_085 = np.array([amp1_Two_Gaussian_075_085, wid1_Two_Gaussian_075_085, amp2_Two_Gaussian_075_085, wid2_Two_Gaussian_075_085]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_075_085 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_075_085 = amp1_Two_Gaussian_075_085 + (amp1_Two_Gaussian_075_085*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_075_085 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_075_085 = wid1_Two_Gaussian_075_085 + (wid1_Two_Gaussian_075_085*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_075_085 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_075_085 = amp2_Two_Gaussian_075_085 + (amp2_Two_Gaussian_075_085*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_075_085 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_075_085 = wid2_Two_Gaussian_075_085 + (wid2_Two_Gaussian_075_085*0.1)


Popt_Two_Gaussian_075_085, Pcov_Two_Gaussian_075_085 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_075_085, y_data_Mirror_Two_Gaussian_075_085, p0=[amp1_Two_Gaussian_075_085, wid1_Two_Gaussian_075_085, amp2_Two_Gaussian_075_085, wid2_Two_Gaussian_075_085], bounds=(0, np.inf))

Perr_Two_Gaussian_075_085 = np.sqrt(np.diag(Pcov_Two_Gaussian_075_085))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_075_085, wid1_Two_Gaussian_075_085, amp2_Two_Gaussian_075_085, wid2_Two_Gaussian_075_085 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_075_085, wid1_Two_Gaussian_075_085, amp2_Two_Gaussian_075_085, wid2_Two_Gaussian_075_085)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_075_085, wid1_Two_Gaussian_075_085, amp2_Two_Gaussian_075_085, wid2_Two_Gaussian_075_085 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_075_085 < amp1_Two_Gaussian_075_085 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_075_085 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_075_085 < amp2_Two_Gaussian_075_085 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_075_085 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_075_085 < wid1_Two_Gaussian_075_085 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_075_085 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_075_085 < wid2_Two_Gaussian_075_085 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_075_085):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_075_085 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_075_085, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_075_085), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_075_085, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_075_085), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_075_085, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_075_085), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_075_085, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_075_085)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_075_085 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_075_085, args=(x_data_Mirror_Two_Gaussian_075_085, y_data_Mirror_Two_Gaussian_075_085, y_err_Mirror_Two_Gaussian_075_085), bounds=bounds_Mirror_Two_Gaussian_075_085)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_075_085 = soln_Mirror_Two_Gaussian_075_085.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_075_085.x))
nwalkers_Mirror_Two_Gaussian_075_085, ndim_Mirror_Two_Gaussian_075_085 = pos_Mirror_Two_Gaussian_075_085.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_075_085 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_075_085, ndim_Mirror_Two_Gaussian_075_085, log_probability, args=(x_data_Mirror_Two_Gaussian_075_085, y_data_Mirror_Two_Gaussian_075_085, y_err_Mirror_Two_Gaussian_075_085))
sampler_Mirror_Two_Gaussian_075_085.run_mcmc(pos_Mirror_Two_Gaussian_075_085, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_075_085 = sampler_Mirror_Two_Gaussian_075_085.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_075_085 = []
for i in range(ndim_Mirror_Two_Gaussian_075_085):
    mcmc_Mirror_Two_Gaussian_075_085 = np.percentile(samples_Mirror_Two_Gaussian_075_085[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_075_085.append(mcmc_Mirror_Two_Gaussian_075_085[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_075_085 = []
for i in range(ndim_Mirror_Two_Gaussian_075_085):
     mcmc_Mirror_Two_Gaussian_075_085 = np.percentile(samples_Mirror_Two_Gaussian_075_085[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_075_085.append([mcmc_Mirror_Two_Gaussian_075_085[2]-mcmc_Mirror_Two_Gaussian_075_085[1],mcmc_Mirror_Two_Gaussian_075_085[1]-mcmc_Mirror_Two_Gaussian_075_085[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_075_085))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_075_085))

ax[0].plot(x_new_Mirror_Two_Gaussian_075_085, two_Gaussian(x_new_Mirror_Two_Gaussian_075_085, *best_fit_params_Mirror_Two_Gaussian_075_085), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_075_085, yy_new_Mirror_Two_Gaussian_075_085, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_075_085, y_data_Mirror_Two_Gaussian_075_085, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_075_085, Gaussian(x_new_Mirror_Two_Gaussian_075_085, best_fit_params_Mirror_Two_Gaussian_075_085[0], best_fit_params_Mirror_Two_Gaussian_075_085[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_075_085, Gaussian(x_new_Mirror_Two_Gaussian_075_085, best_fit_params_Mirror_Two_Gaussian_075_085[2], best_fit_params_Mirror_Two_Gaussian_075_085[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_075_085, yy_new_Mirror_Two_Gaussian_075_085 - two_Gaussian(x_new_Mirror_Two_Gaussian_075_085, *best_fit_params_Mirror_Two_Gaussian_075_085), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_075_085, (yy_new_Mirror_Two_Gaussian_075_085 - two_Gaussian(x_new_Mirror_Two_Gaussian_075_085, *best_fit_params_Mirror_Two_Gaussian_075_085))/yy_new_Mirror_Two_Gaussian_075_085, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_075_085, two_Gaussian(x_new_Mirror_Two_Gaussian_075_085, *Popt_Two_Gaussian_075_085), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_075_085, yy_new_Mirror_Two_Gaussian_075_085, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_075_085, y_data_Mirror_Two_Gaussian_075_085, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_075_085, Gaussian(x_new_Mirror_Two_Gaussian_075_085, Popt_Two_Gaussian_075_085[0], Popt_Two_Gaussian_075_085[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_075_085, Gaussian(x_new_Mirror_Two_Gaussian_075_085, Popt_Two_Gaussian_075_085[2], Popt_Two_Gaussian_075_085[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_075_085, (yy_new_Mirror_Two_Gaussian_075_085 - two_Gaussian(x_new_Mirror_Two_Gaussian_075_085, *Popt_Two_Gaussian_075_085)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_075_085, (yy_new_Mirror_Two_Gaussian_075_085 - two_Gaussian(x_new_Mirror_Two_Gaussian_075_085, *Popt_Two_Gaussian_075_085))/yy_new_Mirror_Two_Gaussian_075_085, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian - 0.75 < z < 0.85")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_075_085[0]) + " +" +str(errors_Mirror_Two_Gaussian_075_085[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_075_085[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_075_085[1]) + " +" +str(errors_Mirror_Two_Gaussian_075_085[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_075_085[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_075_085[2]) + " +" +str(errors_Mirror_Two_Gaussian_075_085[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_075_085[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_075_085[3]) + " +" +str(errors_Mirror_Two_Gaussian_075_085[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_075_085[3][1]))

In [ ]:
# 1. Generate or load data
x_data_Mirror_Two_Gaussian_085_096 = Mirrored_Equal_Area_Radius_085_096_Array
x_new_Mirror_Two_Gaussian_085_096 = np.linspace(min(x_data_Mirror_Two_Gaussian_085_096),max(x_data_Mirror_Two_Gaussian_085_096), 200)

y_data_Mirror_Two_Gaussian_085_096 = Mirrored_Radius_array_085_096
y_new_Mirror_Two_Gaussian_085_096 = CubicSpline(x_data_Mirror_Two_Gaussian_085_096, y_data_Mirror_Two_Gaussian_085_096)

yy_new_Mirror_Two_Gaussian_085_096 = y_new_Mirror_Two_Gaussian_085_096(x_new_Mirror_Two_Gaussian_085_096)


y_err_Mirror_Two_Gaussian_085_096 = Mirrored_Radius_array_SD_085_096

#From Popt --- amp1, wid1, amp2, wid2 = [10.12437085  0.36345423  2.85009951  1.32224147]
#From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = [0.14881953 0.00616822 0.13653721 0.04077331]

amp1_Two_Gaussian_085_096 = 10.12437085
wid1_Two_Gaussian_085_096 = 0.36345423

amp2_Two_Gaussian_085_096 = 2.85009951
wid2_Two_Gaussian_085_096 = 1.32224147

initial_Mirror_Two_Gaussian_085_096 = np.array([amp1_Two_Gaussian_085_096, wid1_Two_Gaussian_085_096, amp2_Two_Gaussian_085_096, wid2_Two_Gaussian_085_096]) #amp,wid ... red/narrow, blue/broad


Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_085_096 = 0
Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_085_096 = amp1_Two_Gaussian_085_096 + (amp1_Two_Gaussian_085_096*0.1)

Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_085_096 = 0
Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_085_096 = wid1_Two_Gaussian_085_096 + (wid1_Two_Gaussian_085_096*0.1)


Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_085_096 = 0
Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_085_096 = amp2_Two_Gaussian_085_096 + (amp2_Two_Gaussian_085_096*0.1)

Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_085_096 = 0
Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_085_096 = wid2_Two_Gaussian_085_096 + (wid2_Two_Gaussian_085_096*0.1)


Popt_Two_Gaussian_085_096, Pcov_Two_Gaussian_085_096 = curve_fit(two_Gaussian, x_data_Mirror_Two_Gaussian_085_096, y_data_Mirror_Two_Gaussian_085_096, p0=[amp1_Two_Gaussian_085_096, wid1_Two_Gaussian_085_096, amp2_Two_Gaussian_085_096, wid2_Two_Gaussian_085_096], bounds=(0, np.inf))

Perr_Two_Gaussian_085_096 = np.sqrt(np.diag(Pcov_Two_Gaussian_085_096))

# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    amp1_Two_Gaussian_085_096, wid1_Two_Gaussian_085_096, amp2_Two_Gaussian_085_096, wid2_Two_Gaussian_085_096 = theta
    model = two_Gaussian(x, amp1_Two_Gaussian_085_096, wid1_Two_Gaussian_085_096, amp2_Two_Gaussian_085_096, wid2_Two_Gaussian_085_096)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

# 4. Define the prior (optional, but recommended) ############################################################################################################################################
def log_prior(theta):
    amp1_Two_Gaussian_085_096, wid1_Two_Gaussian_085_096, amp2_Two_Gaussian_085_096, wid2_Two_Gaussian_085_096 = theta
    if (Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_085_096 < amp1_Two_Gaussian_085_096 < Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_085_096 and Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_085_096 < amp2_Two_Gaussian_085_096 <  Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_085_096 and Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_085_096 < wid1_Two_Gaussian_085_096 < Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_085_096 and Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_085_096 < wid2_Two_Gaussian_085_096 <  Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_085_096):
        return 0.0
    return -np.inf

# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Mirror_Two_Gaussian_085_096 = [(Amplitude_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_085_096, Amplitude_Narrow_Red_Upperr_Lim_Mirror_Two_Gaussian_085_096), (Width_Narrow_Red_Lower_Lim_Mirror_Two_Gaussian_085_096, Width_Narrow_Red_Upper_Lim_Mirror_Two_Gaussian_085_096), (Amplitude_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_085_096, Amplitude_Broad_Blue_Upperr_Lim_Mirror_Two_Gaussian_085_096), (Width_Broad_Blue_Lower_Lim_Mirror_Two_Gaussian_085_096, Width_Broad_Blue_Upper_Lim_Mirror_Two_Gaussian_085_096)] # Parameter bounds for optimization
soln_Mirror_Two_Gaussian_085_096 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Two_Gaussian_085_096, args=(x_data_Mirror_Two_Gaussian_085_096, y_data_Mirror_Two_Gaussian_085_096, y_err_Mirror_Two_Gaussian_085_096), bounds=bounds_Mirror_Two_Gaussian_085_096)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Mirror_Two_Gaussian_085_096 = soln_Mirror_Two_Gaussian_085_096.x + 1e-4 * np.random.randn(32, len(soln_Mirror_Two_Gaussian_085_096.x))
nwalkers_Mirror_Two_Gaussian_085_096, ndim_Mirror_Two_Gaussian_085_096 = pos_Mirror_Two_Gaussian_085_096.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Mirror_Two_Gaussian_085_096 = emcee.EnsembleSampler(nwalkers_Mirror_Two_Gaussian_085_096, ndim_Mirror_Two_Gaussian_085_096, log_probability, args=(x_data_Mirror_Two_Gaussian_085_096, y_data_Mirror_Two_Gaussian_085_096, y_err_Mirror_Two_Gaussian_085_096))
sampler_Mirror_Two_Gaussian_085_096.run_mcmc(pos_Mirror_Two_Gaussian_085_096, 5000, progress=True)

# 5. Analyze results
samples_Mirror_Two_Gaussian_085_096 = sampler_Mirror_Two_Gaussian_085_096.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Mirror_Two_Gaussian_085_096 = []
for i in range(ndim_Mirror_Two_Gaussian_085_096):
    mcmc_Mirror_Two_Gaussian_085_096 = np.percentile(samples_Mirror_Two_Gaussian_085_096[:, i], [68, 95, 99.7])
    best_fit_params_Mirror_Two_Gaussian_085_096.append(mcmc_Mirror_Two_Gaussian_085_096[1])

# 10. Error estimation ############################################################################################################################################
errors_Mirror_Two_Gaussian_085_096 = []
for i in range(ndim_Mirror_Two_Gaussian_085_096):
     mcmc_Mirror_Two_Gaussian_085_096 = np.percentile(samples_Mirror_Two_Gaussian_085_096[:, i], [68, 95, 99.7])
     errors_Mirror_Two_Gaussian_085_096.append([mcmc_Mirror_Two_Gaussian_085_096[2]-mcmc_Mirror_Two_Gaussian_085_096[1],mcmc_Mirror_Two_Gaussian_085_096[1]-mcmc_Mirror_Two_Gaussian_085_096[0]])
     
     
# 12. Plot results ############################################################################################################################################
fig, ax = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- amp1, wid1, amp2, wid2 = " +str(Popt_Two_Gaussian_085_096))
print("From Popt --- amp1_SD, wid1_SD, amp2_SD, wid2_SD = " +str(Perr_Two_Gaussian_085_096))

ax[0].plot(x_new_Mirror_Two_Gaussian_085_096, two_Gaussian(x_new_Mirror_Two_Gaussian_085_096, *best_fit_params_Mirror_Two_Gaussian_085_096), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[0].plot(x_new_Mirror_Two_Gaussian_085_096, yy_new_Mirror_Two_Gaussian_085_096, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[0].step(x_data_Mirror_Two_Gaussian_085_096, y_data_Mirror_Two_Gaussian_085_096, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[0].plot(x_new_Mirror_Two_Gaussian_085_096, Gaussian(x_new_Mirror_Two_Gaussian_085_096, best_fit_params_Mirror_Two_Gaussian_085_096[0], best_fit_params_Mirror_Two_Gaussian_085_096[1]), color="red", linestyle="--", label="Narrow")
ax[0].plot(x_new_Mirror_Two_Gaussian_085_096, Gaussian(x_new_Mirror_Two_Gaussian_085_096, best_fit_params_Mirror_Two_Gaussian_085_096[2], best_fit_params_Mirror_Two_Gaussian_085_096[3]), color="blue", linestyle="--", label="Broad")

ax[0].legend()


ax[1].scatter(x_new_Mirror_Two_Gaussian_085_096, yy_new_Mirror_Two_Gaussian_085_096 - two_Gaussian(x_new_Mirror_Two_Gaussian_085_096, *best_fit_params_Mirror_Two_Gaussian_085_096), c = "#A294F9", edgecolor="black") 


ax[2].scatter(x_new_Mirror_Two_Gaussian_085_096, (yy_new_Mirror_Two_Gaussian_085_096 - two_Gaussian(x_new_Mirror_Two_Gaussian_085_096, *best_fit_params_Mirror_Two_Gaussian_085_096))/yy_new_Mirror_Two_Gaussian_085_096, c = "#AA60C8", edgecolor="black") 


ax[3].plot(x_new_Mirror_Two_Gaussian_085_096, two_Gaussian(x_new_Mirror_Two_Gaussian_085_096, *Popt_Two_Gaussian_085_096), color = "#219B9D", linewidth=2.0, label="fit", zorder=10)
ax[3].plot(x_new_Mirror_Two_Gaussian_085_096, yy_new_Mirror_Two_Gaussian_085_096, color = "#FF8000", linewidth=2.0, label="smoothed data")
ax[3].step(x_data_Mirror_Two_Gaussian_085_096, y_data_Mirror_Two_Gaussian_085_096, color = "black", linewidth=3.0, where="mid", zorder=0, label="data")

ax[3].plot(x_new_Mirror_Two_Gaussian_085_096, Gaussian(x_new_Mirror_Two_Gaussian_085_096, Popt_Two_Gaussian_085_096[0], Popt_Two_Gaussian_085_096[1]), color="red", linestyle="--", label="Narrow")
ax[3].plot(x_new_Mirror_Two_Gaussian_085_096, Gaussian(x_new_Mirror_Two_Gaussian_085_096, Popt_Two_Gaussian_085_096[2], Popt_Two_Gaussian_085_096[3]), color="blue", linestyle="--", label="Broad")

ax[3].legend()

ax[4].scatter(x_new_Mirror_Two_Gaussian_085_096, (yy_new_Mirror_Two_Gaussian_085_096 - two_Gaussian(x_new_Mirror_Two_Gaussian_085_096, *Popt_Two_Gaussian_085_096)), c = "#AA60C8", edgecolor="black") 


ax[5].scatter(x_new_Mirror_Two_Gaussian_085_096, (yy_new_Mirror_Two_Gaussian_085_096 - two_Gaussian(x_new_Mirror_Two_Gaussian_085_096, *Popt_Two_Gaussian_085_096))/yy_new_Mirror_Two_Gaussian_085_096, c = "#AA60C8", edgecolor="black") 


ax[0].set_title("2 Guassian - 0.85 < z < 0.96")

ax[0].set_ylabel("Averaged Counts")
ax[1].set_ylabel("Residuals")
ax[2].set_ylabel("Percent Error")
ax[3].set_ylabel("Averaged Counts")
ax[4].set_ylabel("Residuals")
ax[5].set_ylabel("Percent Error")

ax[5].set_xlabel("Radius from center [arcsecond]")

ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()
ax[5].grid()

ax2 = ax[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = ax[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_Fit", rotation=360, labelpad=40)

plt.show(block=False)
############################################################################################################################################
############################################################################################################################################
print("Amplitude Red = " +str(best_fit_params_Mirror_Two_Gaussian_085_096[0]) + " +" +str(errors_Mirror_Two_Gaussian_085_096[0][0]) + " - " +str(errors_Mirror_Two_Gaussian_085_096[0][1]))
print("")
print("Width Red = " +str(best_fit_params_Mirror_Two_Gaussian_085_096[1]) + " +" +str(errors_Mirror_Two_Gaussian_085_096[1][0]) + " - " +str(errors_Mirror_Two_Gaussian_085_096[1][1]))
print("")
print("Amplitude Blue = " +str(best_fit_params_Mirror_Two_Gaussian_085_096[2]) + " +" +str(errors_Mirror_Two_Gaussian_085_096[2][0]) + " - " +str(errors_Mirror_Two_Gaussian_085_096[2][1]))
print("")
print("Width Blue = " +str(best_fit_params_Mirror_Two_Gaussian_085_096[3]) + " +" +str(errors_Mirror_Two_Gaussian_085_096[3][0]) + " - " +str(errors_Mirror_Two_Gaussian_085_096[3][1]))

## <font color='#FF0B55' size=6 > Finding the area under the two gaussian fit and finding the fraction of the two. Additionally, getting the galaxy component magnitude </font>

In [ ]:
Area_Under_Narrow_Two_Gaussian_025_035, Area_Under_Narrow_Two_Gaussian_SD_025_035 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_025_035, Popt_Two_Gaussian_025_035[0], Popt_Two_Gaussian_025_035[1], Perr_Two_Gaussian_025_035[0], Perr_Two_Gaussian_025_035[1])
Area_Under_Narrow_Two_Gaussian_035_045, Area_Under_Narrow_Two_Gaussian_SD_035_045 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_035_045, Popt_Two_Gaussian_035_045[0], Popt_Two_Gaussian_035_045[1], Perr_Two_Gaussian_035_045[0], Perr_Two_Gaussian_035_045[1])
Area_Under_Narrow_Two_Gaussian_045_055, Area_Under_Narrow_Two_Gaussian_SD_045_055 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_045_055, Popt_Two_Gaussian_045_055[0], Popt_Two_Gaussian_045_055[1], Perr_Two_Gaussian_045_055[0], Perr_Two_Gaussian_045_055[1])
Area_Under_Narrow_Two_Gaussian_055_065, Area_Under_Narrow_Two_Gaussian_SD_055_065 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_055_065, Popt_Two_Gaussian_055_065[0], Popt_Two_Gaussian_055_065[1], Perr_Two_Gaussian_055_065[0], Perr_Two_Gaussian_055_065[1])
Area_Under_Narrow_Two_Gaussian_065_075, Area_Under_Narrow_Two_Gaussian_SD_065_075 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_065_075, Popt_Two_Gaussian_065_075[0], Popt_Two_Gaussian_065_075[1], Perr_Two_Gaussian_065_075[0], Perr_Two_Gaussian_065_075[1])
Area_Under_Narrow_Two_Gaussian_075_085, Area_Under_Narrow_Two_Gaussian_SD_075_085 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_075_085, Popt_Two_Gaussian_075_085[0], Popt_Two_Gaussian_075_085[1], Perr_Two_Gaussian_075_085[0], Perr_Two_Gaussian_075_085[1])
Area_Under_Narrow_Two_Gaussian_085_096, Area_Under_Narrow_Two_Gaussian_SD_085_096 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_085_096, Popt_Two_Gaussian_085_096[0], Popt_Two_Gaussian_085_096[1], Perr_Two_Gaussian_085_096[0], Perr_Two_Gaussian_085_096[1])

Area_Under_Broad_Two_Gaussian_025_035, Area_Under_Broad_Two_Gaussian_SD_025_035 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_025_035, Popt_Two_Gaussian_025_035[2], Popt_Two_Gaussian_025_035[3], Perr_Two_Gaussian_025_035[2], Perr_Two_Gaussian_025_035[3])
Area_Under_Broad_Two_Gaussian_035_045, Area_Under_Broad_Two_Gaussian_SD_035_045 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_035_045, Popt_Two_Gaussian_035_045[2], Popt_Two_Gaussian_035_045[3], Perr_Two_Gaussian_035_045[2], Perr_Two_Gaussian_035_045[3])
Area_Under_Broad_Two_Gaussian_045_055, Area_Under_Broad_Two_Gaussian_SD_045_055 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_045_055, Popt_Two_Gaussian_045_055[2], Popt_Two_Gaussian_045_055[3], Perr_Two_Gaussian_045_055[2], Perr_Two_Gaussian_045_055[3])
Area_Under_Broad_Two_Gaussian_055_065, Area_Under_Broad_Two_Gaussian_SD_055_065 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_055_065, Popt_Two_Gaussian_055_065[2], Popt_Two_Gaussian_055_065[3], Perr_Two_Gaussian_055_065[2], Perr_Two_Gaussian_055_065[3])
Area_Under_Broad_Two_Gaussian_065_075, Area_Under_Broad_Two_Gaussian_SD_065_075 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_065_075, Popt_Two_Gaussian_065_075[2], Popt_Two_Gaussian_065_075[3], Perr_Two_Gaussian_065_075[2], Perr_Two_Gaussian_065_075[3])
Area_Under_Broad_Two_Gaussian_075_085, Area_Under_Broad_Two_Gaussian_SD_075_085 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_075_085, Popt_Two_Gaussian_075_085[2], Popt_Two_Gaussian_075_085[3], Perr_Two_Gaussian_075_085[2], Perr_Two_Gaussian_075_085[3])
Area_Under_Broad_Two_Gaussian_085_096, Area_Under_Broad_Two_Gaussian_SD_085_096 = monte_carlo_gaussian_area(x_new_Mirror_Two_Gaussian_085_096, Popt_Two_Gaussian_085_096[2], Popt_Two_Gaussian_085_096[3], Perr_Two_Gaussian_085_096[2], Perr_Two_Gaussian_085_096[3])

In [ ]:
# Fractional mag for the OG fits (Fake errors)
Fractional_Area_Broad_Two_Gaussian_025_035, Fractional_Area_Broad_Two_Gaussian_SD_025_035 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_025_035, Area_Under_Narrow_Two_Gaussian_025_035, Area_Under_Broad_Two_Gaussian_SD_025_035, Area_Under_Narrow_Two_Gaussian_SD_025_035)
Fractional_Area_Broad_Two_Gaussian_035_045, Fractional_Area_Broad_Two_Gaussian_SD_035_045 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_035_045, Area_Under_Narrow_Two_Gaussian_035_045, Area_Under_Broad_Two_Gaussian_SD_035_045, Area_Under_Narrow_Two_Gaussian_SD_035_045)
Fractional_Area_Broad_Two_Gaussian_045_055, Fractional_Area_Broad_Two_Gaussian_SD_045_055 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_045_055, Area_Under_Narrow_Two_Gaussian_045_055, Area_Under_Broad_Two_Gaussian_SD_045_055, Area_Under_Narrow_Two_Gaussian_SD_045_055)
Fractional_Area_Broad_Two_Gaussian_055_065, Fractional_Area_Broad_Two_Gaussian_SD_055_065 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_055_065, Area_Under_Narrow_Two_Gaussian_055_065, Area_Under_Broad_Two_Gaussian_SD_055_065, Area_Under_Narrow_Two_Gaussian_SD_055_065)
Fractional_Area_Broad_Two_Gaussian_065_075, Fractional_Area_Broad_Two_Gaussian_SD_065_075 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_065_075, Area_Under_Narrow_Two_Gaussian_065_075, Area_Under_Broad_Two_Gaussian_SD_065_075, Area_Under_Narrow_Two_Gaussian_SD_065_075)
Fractional_Area_Broad_Two_Gaussian_075_085, Fractional_Area_Broad_Two_Gaussian_SD_075_085 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_075_085, Area_Under_Narrow_Two_Gaussian_075_085, Area_Under_Broad_Two_Gaussian_SD_075_085, Area_Under_Narrow_Two_Gaussian_SD_075_085)
Fractional_Area_Broad_Two_Gaussian_085_096, Fractional_Area_Broad_Two_Gaussian_SD_085_096 = fractional_area_uncertainty(Area_Under_Broad_Two_Gaussian_085_096, Area_Under_Narrow_Two_Gaussian_085_096, Area_Under_Broad_Two_Gaussian_SD_085_096, Area_Under_Narrow_Two_Gaussian_SD_085_096)

In [ ]:
Gal_Mag_Two_Gaussian_025_035, Gal_Mag_Two_Gaussian_SD_025_035 = gal_mag_uncertainty(Mag_Total_025_035, Mag_Total_SD_025_035, Fractional_Area_Broad_Two_Gaussian_025_035, Fractional_Area_Broad_Two_Gaussian_SD_025_035)
Gal_Mag_Two_Gaussian_035_045, Gal_Mag_Two_Gaussian_SD_035_045 = gal_mag_uncertainty(Mag_Total_035_045, Mag_Total_SD_035_045, Fractional_Area_Broad_Two_Gaussian_035_045, Fractional_Area_Broad_Two_Gaussian_SD_035_045)
Gal_Mag_Two_Gaussian_045_055, Gal_Mag_Two_Gaussian_SD_045_055 = gal_mag_uncertainty(Mag_Total_045_055, Mag_Total_SD_045_055, Fractional_Area_Broad_Two_Gaussian_045_055, Fractional_Area_Broad_Two_Gaussian_SD_045_055)
Gal_Mag_Two_Gaussian_055_065, Gal_Mag_Two_Gaussian_SD_055_065 = gal_mag_uncertainty(Mag_Total_055_065, Mag_Total_SD_055_065, Fractional_Area_Broad_Two_Gaussian_055_065, Fractional_Area_Broad_Two_Gaussian_SD_055_065)
Gal_Mag_Two_Gaussian_065_075, Gal_Mag_Two_Gaussian_SD_065_075 = gal_mag_uncertainty(Mag_Total_065_075, Mag_Total_SD_065_075, Fractional_Area_Broad_Two_Gaussian_065_075, Fractional_Area_Broad_Two_Gaussian_SD_065_075)
Gal_Mag_Two_Gaussian_075_085, Gal_Mag_Two_Gaussian_SD_075_085 = gal_mag_uncertainty(Mag_Total_075_085, Mag_Total_SD_075_085, Fractional_Area_Broad_Two_Gaussian_075_085, Fractional_Area_Broad_Two_Gaussian_SD_075_085)
Gal_Mag_Two_Gaussian_085_096, Gal_Mag_Two_Gaussian_SD_085_096 = gal_mag_uncertainty(Mag_Total_085_096, Mag_Total_SD_085_096, Fractional_Area_Broad_Two_Gaussian_085_096, Fractional_Area_Broad_Two_Gaussian_SD_085_096)

In [ ]:

Gal_Mag_Two_Gaussian_Array = np.array([Gal_Mag_Two_Gaussian_025_035, Gal_Mag_Two_Gaussian_035_045, Gal_Mag_Two_Gaussian_045_055, Gal_Mag_Two_Gaussian_055_065, Gal_Mag_Two_Gaussian_065_075, Gal_Mag_Two_Gaussian_075_085, Gal_Mag_Two_Gaussian_085_096])


In [ ]:
Gal_Mag_Two_Gaussian_SD_Array = np.array([Gal_Mag_Two_Gaussian_SD_025_035, Gal_Mag_Two_Gaussian_SD_035_045, Gal_Mag_Two_Gaussian_SD_045_055, Gal_Mag_Two_Gaussian_SD_055_065, Gal_Mag_Two_Gaussian_SD_065_075, Gal_Mag_Two_Gaussian_SD_075_085, Gal_Mag_Two_Gaussian_SD_085_096])


## <font color='#FF0B55' size=6 >Fitting the radial profile using a gaussian + sersic component</font>

In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_025_035 = Equal_Area_Radius_025_035_Array
x_new_Sersic_Gaussian_025_035  = np.linspace(min(x_data_Sersic_Gaussian_025_035),max(x_data_Sersic_Gaussian_025_035), 200)

y_data_Sersic_Gaussian_025_035 = np.array(Average_Counts_025_035)
y_new_Sersic_Gaussian_025_035 = CubicSpline(x_data_Sersic_Gaussian_025_035, y_data_Sersic_Gaussian_025_035)

yy_new_Sersic_Gaussian_025_035 = y_new_Sersic_Gaussian_025_035(x_new_Sersic_Gaussian_025_035)


y_err_Sersic_Gaussian_025_035 = np.array(Average_Counts_SD_025_035)
y_new_err_Sersic_Gaussian_025_035 = CubicSpline(x_data_Sersic_Gaussian_025_035, y_err_Sersic_Gaussian_025_035)

yy_new_err_Sersic_Gaussian_025_035 = y_new_err_Sersic_Gaussian_025_035(x_new_Sersic_Gaussian_025_035)


#From Popt --- I_e, R_e, N, A, Sigma = [3.33459022e+00 1.88074997e+00 1.42134112e+00 1.82128292e-02 3.54539257e-04]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.1326097  0.05205382 0.03106795 0.         0.        ]

i_e_Sersic_Gaussian_025_035 = 3.33459022e+00
r_e_Sersic_Gaussian_025_035 = 1.88074997e+00
n_Sersic_Gaussian_025_035 = 1.42134112e+00

a_Sersic_Gaussian_025_035 = 1.82128292e-02
sigma_Sersic_Gaussian_025_035 = 3.54539257e-04

initial_Mirror_Sersic_Gaussian_025_035 = np.array([i_e_Sersic_Gaussian_025_035, r_e_Sersic_Gaussian_025_035, n_Sersic_Gaussian_025_035, a_Sersic_Gaussian_025_035, sigma_Sersic_Gaussian_025_035]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_025_035 = 0
I_e_u_Sersic_Gaussian_025_035 = i_e_Sersic_Gaussian_025_035 + (0.01*i_e_Sersic_Gaussian_025_035)

R_e_l_Sersic_Gaussian_025_035 = 0
R_e_u_Sersic_Gaussian_025_035 = r_e_Sersic_Gaussian_025_035 + (0.01*r_e_Sersic_Gaussian_025_035)

N_l_Sersic_Gaussian_025_035 = 0
N_u_Sersic_Gaussian_025_035 = n_Sersic_Gaussian_025_035 + (0.01*n_Sersic_Gaussian_025_035)


a_l_Sersic_Gaussian_025_035 = 0
a_u_Sersic_Gaussian_025_035 = a_Sersic_Gaussian_025_035 + (0.01*a_Sersic_Gaussian_025_035)

sigma_l_Sersic_Gaussian_025_035 = 0
sigma_u_Sersic_Gaussian_025_035 = sigma_Sersic_Gaussian_025_035 + (0.01*sigma_Sersic_Gaussian_025_035)


Popt_Sersic_Gaussian_025_035, Pcov_Sersic_Gaussian_025_035 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_025_035, yy_new_Sersic_Gaussian_025_035, p0=[i_e_Sersic_Gaussian_025_035, r_e_Sersic_Gaussian_025_035, n_Sersic_Gaussian_025_035, a_Sersic_Gaussian_025_035, sigma_Sersic_Gaussian_025_035], bounds=(0, np.inf))
Perr_Sersic_Gaussian_025_035 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_025_035))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_025_035, r_e_Sersic_Gaussian_025_035, n_Sersic_Gaussian_025_035, a_Sersic_Gaussian_025_035, sigma_Sersic_Gaussian_025_035 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_025_035, r_e_Sersic_Gaussian_025_035, n_Sersic_Gaussian_025_035, a_Sersic_Gaussian_025_035, sigma_Sersic_Gaussian_025_035)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_025_035, n_Sersic_Gaussian_025_035, a_Sersic_Gaussian_025_035, sigma_Sersic_Gaussian_025_035 = theta
    if (I_e_l_Sersic_Gaussian_025_035 < i_e_Sersic_Gaussian_025_035 < I_e_u_Sersic_Gaussian_025_035 and R_e_l_Sersic_Gaussian_025_035 < r_e_Sersic_Gaussian_025_035 < R_e_u_Sersic_Gaussian_025_035 and N_l_Sersic_Gaussian_025_035 < n_Sersic_Gaussian_025_035 < N_u_Sersic_Gaussian_025_035 and a_l_Sersic_Gaussian_025_035 < a_Sersic_Gaussian_025_035 < a_u_Sersic_Gaussian_025_035 and sigma_l_Sersic_Gaussian_025_035 < sigma_Sersic_Gaussian_025_035 < sigma_u_Sersic_Gaussian_025_035):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_025_035 = [(I_e_l_Sersic_Gaussian_025_035, I_e_u_Sersic_Gaussian_025_035), (R_e_l_Sersic_Gaussian_025_035, R_e_u_Sersic_Gaussian_025_035), (N_l_Sersic_Gaussian_025_035, N_u_Sersic_Gaussian_025_035), (a_l_Sersic_Gaussian_025_035, a_u_Sersic_Gaussian_025_035), (sigma_l_Sersic_Gaussian_025_035, sigma_u_Sersic_Gaussian_025_035)] # Parameter bounds for optimization
soln_Sersic_Gaussian_025_035 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_025_035, args=(x_new_Sersic_Gaussian_025_035, yy_new_Sersic_Gaussian_025_035, yy_new_err_Sersic_Gaussian_025_035), bounds=bounds_Sersic_Gaussian_025_035)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_025_035 = soln_Sersic_Gaussian_025_035.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_025_035.x))
nwalkers_Sersic_Gaussian_025_035, ndim_Sersic_Gaussian_025_035 = pos_Sersic_Gaussian_025_035.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_025_035 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_025_035, ndim_Sersic_Gaussian_025_035, log_probability, args=(x_new_Sersic_Gaussian_025_035, yy_new_Sersic_Gaussian_025_035, yy_new_err_Sersic_Gaussian_025_035))
sampler_Sersic_Gaussian_025_035.run_mcmc(pos_Sersic_Gaussian_025_035, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_025_035 = sampler_Sersic_Gaussian_025_035.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_025_035 = []
for i in range(ndim_Sersic_Gaussian_025_035):
    mcmc_Sersic_Gaussian_025_035 = np.percentile(samples_Sersic_Gaussian_025_035[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_025_035.append(mcmc_Sersic_Gaussian_025_035[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_025_035 = []
for i in range(ndim_Sersic_Gaussian_025_035):
     mcmc_Sersic_Gaussian_025_035 = np.percentile(samples_Sersic_Gaussian_025_035[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_025_035.append([mcmc_Sersic_Gaussian_025_035[2]-mcmc_Sersic_Gaussian_025_035[1],mcmc_Sersic_Gaussian_025_035[1]-mcmc_Sersic_Gaussian_025_035[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_025_035))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_025_035))

axes1[0].plot(x_new_Sersic_Gaussian_025_035, Sersic_Gauss(x_new_Sersic_Gaussian_025_035, *best_fit_params_Sersic_Gaussian_025_035[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_025_035, Gaussian(x_new_Sersic_Gaussian_025_035, *best_fit_params_Sersic_Gaussian_025_035[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_025_035, Sersic_Fit(x_new_Sersic_Gaussian_025_035, *best_fit_params_Sersic_Gaussian_025_035[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_025_035, yy_new_Sersic_Gaussian_025_035, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_025_035, y_data_Sersic_Gaussian_025_035, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_025_035, yy_new_Sersic_Gaussian_025_035 - Sersic_Gauss(x_new_Sersic_Gaussian_025_035, *best_fit_params_Sersic_Gaussian_025_035[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_025_035, (yy_new_Sersic_Gaussian_025_035 - Sersic_Gauss(x_new_Sersic_Gaussian_025_035, *best_fit_params_Sersic_Gaussian_025_035[:5]))/yy_new_Sersic_Gaussian_025_035, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_025_035, Sersic_Gauss(x_new_Sersic_Gaussian_025_035, *Popt_Sersic_Gaussian_025_035[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_025_035, Gaussian(x_new_Sersic_Gaussian_025_035, *Popt_Sersic_Gaussian_025_035[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_025_035, Sersic_Fit(x_new_Sersic_Gaussian_025_035, *Popt_Sersic_Gaussian_025_035[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_025_035, yy_new_Sersic_Gaussian_025_035, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_025_035, y_data_Sersic_Gaussian_025_035, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_025_035, (yy_new_Sersic_Gaussian_025_035 - Sersic_Gauss(x_new_Sersic_Gaussian_025_035, *Popt_Sersic_Gaussian_025_035[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_025_035, (yy_new_Sersic_Gaussian_025_035 - Sersic_Gauss(x_new_Sersic_Gaussian_025_035, *Popt_Sersic_Gaussian_025_035[:5]))/yy_new_Sersic_Gaussian_025_035, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.25 < z < 0.35")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_025_035 = best_fit_params_Sersic_Gaussian_025_035[0]
R_e_Sersic_025_035 = best_fit_params_Sersic_Gaussian_025_035[1]
N_Sersic_025_035 = best_fit_params_Sersic_Gaussian_025_035[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_025_035[0]) + " +" +str(errors_Sersic_Gaussian_025_035[0][0]) + " - " +str(errors_Sersic_Gaussian_025_035[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_025_035[1]) + " +" +str(errors_Sersic_Gaussian_025_035[1][0]) + " - " +str(errors_Sersic_Gaussian_025_035[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_025_035[2]) + " +" +str(errors_Sersic_Gaussian_025_035[2][0]) + " - " +str(errors_Sersic_Gaussian_025_035[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_025_035[3]) + " +" +str(errors_Sersic_Gaussian_025_035[3][0]) + " - " +str(errors_Sersic_Gaussian_025_035[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_025_035[4]) + " +" +str(errors_Sersic_Gaussian_025_035[4][0]) + " - " +str(errors_Sersic_Gaussian_025_035[4][1]))



In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_035_045 = Equal_Area_Radius_035_045_Array
x_new_Sersic_Gaussian_035_045 = np.linspace(min(x_data_Sersic_Gaussian_035_045),max(x_data_Sersic_Gaussian_035_045), 200)

y_data_Sersic_Gaussian_035_045 = np.array(Average_Counts_035_045)
y_new_Sersic_Gaussian_035_045 = CubicSpline(x_data_Sersic_Gaussian_035_045, y_data_Sersic_Gaussian_035_045)

yy_new_Sersic_Gaussian_035_045 = y_new_Sersic_Gaussian_035_045(x_new_Sersic_Gaussian_035_045)


y_err_Sersic_Gaussian_035_045 = np.array(Average_Counts_SD_035_045)
y_new_err_Sersic_Gaussian_035_045 = CubicSpline(x_data_Sersic_Gaussian_035_045, y_err_Sersic_Gaussian_035_045)

yy_new_err_Sersic_Gaussian_035_045 = y_new_err_Sersic_Gaussian_035_045(x_new_Sersic_Gaussian_035_045)


#From Popt --- I_e, R_e, N, A, Sigma = [0.79128168 2.92805117 1.63432883 4.28086086 0.47597196]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.03231013 0.07429088 0.03078563 0.13883122 0.0075032 ]

i_e_Sersic_Gaussian_035_045 = 0.79128168
r_e_Sersic_Gaussian_035_045 = 2.92805117
n_Sersic_Gaussian_035_045 = 1.63432883

a_Sersic_Gaussian_035_045 = 4.28086086
sigma_Sersic_Gaussian_035_045 = 0.47597196

initial_Mirror_Sersic_Gaussian_035_045 = np.array([i_e_Sersic_Gaussian_035_045, r_e_Sersic_Gaussian_035_045, n_Sersic_Gaussian_035_045, a_Sersic_Gaussian_035_045, sigma_Sersic_Gaussian_035_045]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_035_045 = 0
I_e_u_Sersic_Gaussian_035_045 = i_e_Sersic_Gaussian_035_045 + (0.01*i_e_Sersic_Gaussian_035_045)

R_e_l_Sersic_Gaussian_035_045 = 0
R_e_u_Sersic_Gaussian_035_045 = r_e_Sersic_Gaussian_035_045 + (0.01*r_e_Sersic_Gaussian_035_045)

N_l_Sersic_Gaussian_035_045 = 0
N_u_Sersic_Gaussian_035_045 = n_Sersic_Gaussian_035_045 + (0.01*n_Sersic_Gaussian_035_045)


a_l_Sersic_Gaussian_035_045 = 0
a_u_Sersic_Gaussian_035_045 = a_Sersic_Gaussian_035_045 + (0.01*a_Sersic_Gaussian_035_045)

sigma_l_Sersic_Gaussian_035_045 = 0
sigma_u_Sersic_Gaussian_035_045 = sigma_Sersic_Gaussian_035_045 + (0.01*sigma_Sersic_Gaussian_035_045)


Popt_Sersic_Gaussian_035_045, Pcov_Sersic_Gaussian_035_045 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_035_045, yy_new_Sersic_Gaussian_035_045, p0=[i_e_Sersic_Gaussian_035_045, r_e_Sersic_Gaussian_035_045, n_Sersic_Gaussian_035_045, a_Sersic_Gaussian_035_045, sigma_Sersic_Gaussian_035_045], bounds=(0, np.inf))
Perr_Sersic_Gaussian_035_045 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_035_045))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_035_045, r_e_Sersic_Gaussian_035_045, n_Sersic_Gaussian_035_045, a_Sersic_Gaussian_035_045, sigma_Sersic_Gaussian_035_045 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_035_045, r_e_Sersic_Gaussian_035_045, n_Sersic_Gaussian_035_045, a_Sersic_Gaussian_035_045, sigma_Sersic_Gaussian_035_045)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_035_045, n_Sersic_Gaussian_035_045, a_Sersic_Gaussian_035_045, sigma_Sersic_Gaussian_035_045 = theta
    if (I_e_l_Sersic_Gaussian_035_045 < i_e_Sersic_Gaussian_035_045 < I_e_u_Sersic_Gaussian_035_045 and R_e_l_Sersic_Gaussian_035_045 < r_e_Sersic_Gaussian_035_045 < R_e_u_Sersic_Gaussian_035_045 and N_l_Sersic_Gaussian_035_045 < n_Sersic_Gaussian_035_045 < N_u_Sersic_Gaussian_035_045 and a_l_Sersic_Gaussian_035_045 < a_Sersic_Gaussian_035_045 < a_u_Sersic_Gaussian_035_045 and sigma_l_Sersic_Gaussian_035_045 < sigma_Sersic_Gaussian_035_045 < sigma_u_Sersic_Gaussian_035_045):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_035_045 = [(I_e_l_Sersic_Gaussian_035_045, I_e_u_Sersic_Gaussian_035_045), (R_e_l_Sersic_Gaussian_035_045, R_e_u_Sersic_Gaussian_035_045), (N_l_Sersic_Gaussian_035_045, N_u_Sersic_Gaussian_035_045), (a_l_Sersic_Gaussian_035_045, a_u_Sersic_Gaussian_035_045), (sigma_l_Sersic_Gaussian_035_045, sigma_u_Sersic_Gaussian_035_045)] # Parameter bounds for optimization
soln_Sersic_Gaussian_035_045 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_035_045, args=(x_new_Sersic_Gaussian_035_045, yy_new_Sersic_Gaussian_035_045, yy_new_err_Sersic_Gaussian_035_045), bounds=bounds_Sersic_Gaussian_035_045)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_035_045 = soln_Sersic_Gaussian_035_045.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_035_045.x))
nwalkers_Sersic_Gaussian_035_045, ndim_Sersic_Gaussian_035_045 = pos_Sersic_Gaussian_035_045.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_035_045 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_035_045, ndim_Sersic_Gaussian_035_045, log_probability, args=(x_new_Sersic_Gaussian_035_045, yy_new_Sersic_Gaussian_035_045, yy_new_err_Sersic_Gaussian_035_045))
sampler_Sersic_Gaussian_035_045.run_mcmc(pos_Sersic_Gaussian_035_045, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_035_045 = sampler_Sersic_Gaussian_035_045.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_035_045 = []
for i in range(ndim_Sersic_Gaussian_035_045):
    mcmc_Sersic_Gaussian_035_045 = np.percentile(samples_Sersic_Gaussian_035_045[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_035_045.append(mcmc_Sersic_Gaussian_035_045[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_035_045 = []
for i in range(ndim_Sersic_Gaussian_035_045):
     mcmc_Sersic_Gaussian_035_045 = np.percentile(samples_Sersic_Gaussian_035_045[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_035_045.append([mcmc_Sersic_Gaussian_035_045[2]-mcmc_Sersic_Gaussian_035_045[1],mcmc_Sersic_Gaussian_035_045[1]-mcmc_Sersic_Gaussian_035_045[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_035_045))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_035_045))

axes1[0].plot(x_new_Sersic_Gaussian_035_045, Sersic_Gauss(x_new_Sersic_Gaussian_035_045, *best_fit_params_Sersic_Gaussian_035_045[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_035_045, Gaussian(x_new_Sersic_Gaussian_035_045, *best_fit_params_Sersic_Gaussian_035_045[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_035_045, Sersic_Fit(x_new_Sersic_Gaussian_035_045, *best_fit_params_Sersic_Gaussian_035_045[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_035_045, yy_new_Sersic_Gaussian_035_045, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_035_045, y_data_Sersic_Gaussian_035_045, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_035_045, yy_new_Sersic_Gaussian_035_045 - Sersic_Gauss(x_new_Sersic_Gaussian_035_045, *best_fit_params_Sersic_Gaussian_035_045[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_035_045, (yy_new_Sersic_Gaussian_035_045 - Sersic_Gauss(x_new_Sersic_Gaussian_035_045, *best_fit_params_Sersic_Gaussian_035_045[:5]))/yy_new_Sersic_Gaussian_035_045, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_035_045, Sersic_Gauss(x_new_Sersic_Gaussian_035_045, *Popt_Sersic_Gaussian_035_045[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_035_045, Gaussian(x_new_Sersic_Gaussian_035_045, *Popt_Sersic_Gaussian_035_045[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_035_045, Sersic_Fit(x_new_Sersic_Gaussian_035_045, *Popt_Sersic_Gaussian_035_045[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_035_045, yy_new_Sersic_Gaussian_035_045, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_035_045, y_data_Sersic_Gaussian_035_045, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_035_045, (yy_new_Sersic_Gaussian_035_045 - Sersic_Gauss(x_new_Sersic_Gaussian_035_045, *Popt_Sersic_Gaussian_035_045[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_035_045, (yy_new_Sersic_Gaussian_035_045 - Sersic_Gauss(x_new_Sersic_Gaussian_035_045, *Popt_Sersic_Gaussian_035_045[:5]))/yy_new_Sersic_Gaussian_035_045, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.35 < z < 0.45")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_035_045[0]
R_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_035_045[1]
N_Sersic_035_045 = best_fit_params_Sersic_Gaussian_035_045[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_035_045[0]) + " +" +str(errors_Sersic_Gaussian_035_045[0][0]) + " - " +str(errors_Sersic_Gaussian_035_045[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_035_045[1]) + " +" +str(errors_Sersic_Gaussian_035_045[1][0]) + " - " +str(errors_Sersic_Gaussian_035_045[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_035_045[2]) + " +" +str(errors_Sersic_Gaussian_035_045[2][0]) + " - " +str(errors_Sersic_Gaussian_035_045[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_035_045[3]) + " +" +str(errors_Sersic_Gaussian_035_045[3][0]) + " - " +str(errors_Sersic_Gaussian_035_045[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_035_045[4]) + " +" +str(errors_Sersic_Gaussian_035_045[4][0]) + " - " +str(errors_Sersic_Gaussian_035_045[4][1]))

In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_045_055 = Equal_Area_Radius_045_055_Array
x_new_Sersic_Gaussian_045_055 = np.linspace(min(x_data_Sersic_Gaussian_045_055),max(x_data_Sersic_Gaussian_045_055), 200)

y_data_Sersic_Gaussian_045_055 = np.array(Average_Counts_045_055)
y_new_Sersic_Gaussian_045_055 = CubicSpline(x_data_Sersic_Gaussian_045_055, y_data_Sersic_Gaussian_045_055)

yy_new_Sersic_Gaussian_045_055 = y_new_Sersic_Gaussian_045_055(x_new_Sersic_Gaussian_045_055)


y_err_Sersic_Gaussian_045_055 = np.array(Average_Counts_SD_045_055)
y_new_err_Sersic_Gaussian_045_055 = CubicSpline(x_data_Sersic_Gaussian_045_055, y_err_Sersic_Gaussian_045_055)

yy_new_err_Sersic_Gaussian_045_055 = y_new_err_Sersic_Gaussian_045_055(x_new_Sersic_Gaussian_045_055)


#From Popt --- I_e, R_e, N, A, Sigma = [0.56177463 2.81428788 1.5813704  3.83104019 0.49673328]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.02385884 0.07298566 0.03087954 0.09472077 0.00606346]

i_e_Sersic_Gaussian_045_055 = 0.56177463
r_e_Sersic_Gaussian_045_055 = 2.81428788
n_Sersic_Gaussian_045_055 = 1.5813704

a_Sersic_Gaussian_045_055 = 3.83104019
sigma_Sersic_Gaussian_045_055 = 0.49673328

initial_Mirror_Sersic_Gaussian_045_055 = np.array([i_e_Sersic_Gaussian_045_055, r_e_Sersic_Gaussian_045_055, n_Sersic_Gaussian_045_055, a_Sersic_Gaussian_045_055, sigma_Sersic_Gaussian_045_055]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_045_055 = 0
I_e_u_Sersic_Gaussian_045_055 = i_e_Sersic_Gaussian_045_055 + (0.01*i_e_Sersic_Gaussian_045_055)

R_e_l_Sersic_Gaussian_045_055 = 0
R_e_u_Sersic_Gaussian_045_055 = r_e_Sersic_Gaussian_045_055 + (0.01*r_e_Sersic_Gaussian_045_055)

N_l_Sersic_Gaussian_045_055 = 0
N_u_Sersic_Gaussian_045_055 = n_Sersic_Gaussian_045_055 + (0.01*n_Sersic_Gaussian_045_055)


a_l_Sersic_Gaussian_045_055 = 0
a_u_Sersic_Gaussian_045_055 = a_Sersic_Gaussian_045_055 + (0.01*a_Sersic_Gaussian_045_055)

sigma_l_Sersic_Gaussian_045_055 = 0
sigma_u_Sersic_Gaussian_045_055 = sigma_Sersic_Gaussian_045_055 + (0.01*sigma_Sersic_Gaussian_045_055)


Popt_Sersic_Gaussian_045_055, Pcov_Sersic_Gaussian_045_055 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_045_055, yy_new_Sersic_Gaussian_045_055, p0=[i_e_Sersic_Gaussian_045_055, r_e_Sersic_Gaussian_045_055, n_Sersic_Gaussian_045_055, a_Sersic_Gaussian_045_055, sigma_Sersic_Gaussian_045_055], bounds=(0, np.inf))
Perr_Sersic_Gaussian_045_055 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_045_055))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_045_055, r_e_Sersic_Gaussian_045_055, n_Sersic_Gaussian_045_055, a_Sersic_Gaussian_045_055, sigma_Sersic_Gaussian_045_055 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_045_055, r_e_Sersic_Gaussian_045_055, n_Sersic_Gaussian_045_055, a_Sersic_Gaussian_045_055, sigma_Sersic_Gaussian_045_055)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_045_055, n_Sersic_Gaussian_045_055, a_Sersic_Gaussian_045_055, sigma_Sersic_Gaussian_045_055 = theta
    if (I_e_l_Sersic_Gaussian_045_055 < i_e_Sersic_Gaussian_045_055 < I_e_u_Sersic_Gaussian_045_055 and R_e_l_Sersic_Gaussian_045_055 < r_e_Sersic_Gaussian_045_055 < R_e_u_Sersic_Gaussian_045_055 and N_l_Sersic_Gaussian_045_055 < n_Sersic_Gaussian_045_055 < N_u_Sersic_Gaussian_045_055 and a_l_Sersic_Gaussian_045_055 < a_Sersic_Gaussian_045_055 < a_u_Sersic_Gaussian_045_055 and sigma_l_Sersic_Gaussian_045_055 < sigma_Sersic_Gaussian_045_055 < sigma_u_Sersic_Gaussian_045_055):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_045_055 = [(I_e_l_Sersic_Gaussian_045_055, I_e_u_Sersic_Gaussian_045_055), (R_e_l_Sersic_Gaussian_045_055, R_e_u_Sersic_Gaussian_045_055), (N_l_Sersic_Gaussian_045_055, N_u_Sersic_Gaussian_045_055), (a_l_Sersic_Gaussian_045_055, a_u_Sersic_Gaussian_045_055), (sigma_l_Sersic_Gaussian_045_055, sigma_u_Sersic_Gaussian_045_055)] # Parameter bounds for optimization
soln_Sersic_Gaussian_045_055 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_045_055, args=(x_new_Sersic_Gaussian_045_055, yy_new_Sersic_Gaussian_045_055, yy_new_err_Sersic_Gaussian_045_055), bounds=bounds_Sersic_Gaussian_045_055)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_045_055 = soln_Sersic_Gaussian_045_055.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_045_055.x))
nwalkers_Sersic_Gaussian_045_055, ndim_Sersic_Gaussian_045_055 = pos_Sersic_Gaussian_045_055.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_045_055 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_045_055, ndim_Sersic_Gaussian_045_055, log_probability, args=(x_new_Sersic_Gaussian_045_055, yy_new_Sersic_Gaussian_045_055, yy_new_err_Sersic_Gaussian_045_055))
sampler_Sersic_Gaussian_045_055.run_mcmc(pos_Sersic_Gaussian_045_055, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_045_055 = sampler_Sersic_Gaussian_045_055.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_045_055 = []
for i in range(ndim_Sersic_Gaussian_045_055):
    mcmc_Sersic_Gaussian_045_055 = np.percentile(samples_Sersic_Gaussian_045_055[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_045_055.append(mcmc_Sersic_Gaussian_045_055[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_045_055 = []
for i in range(ndim_Sersic_Gaussian_045_055):
     mcmc_Sersic_Gaussian_045_055 = np.percentile(samples_Sersic_Gaussian_045_055[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_045_055.append([mcmc_Sersic_Gaussian_045_055[2]-mcmc_Sersic_Gaussian_045_055[1],mcmc_Sersic_Gaussian_045_055[1]-mcmc_Sersic_Gaussian_045_055[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_045_055))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_045_055))

axes1[0].plot(x_new_Sersic_Gaussian_045_055, Sersic_Gauss(x_new_Sersic_Gaussian_045_055, *best_fit_params_Sersic_Gaussian_045_055[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_045_055, Gaussian(x_new_Sersic_Gaussian_045_055, *best_fit_params_Sersic_Gaussian_045_055[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_045_055, Sersic_Fit(x_new_Sersic_Gaussian_045_055, *best_fit_params_Sersic_Gaussian_045_055[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_045_055, yy_new_Sersic_Gaussian_045_055, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_045_055, y_data_Sersic_Gaussian_045_055, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_045_055, yy_new_Sersic_Gaussian_045_055 - Sersic_Gauss(x_new_Sersic_Gaussian_045_055, *best_fit_params_Sersic_Gaussian_045_055[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_045_055, (yy_new_Sersic_Gaussian_045_055 - Sersic_Gauss(x_new_Sersic_Gaussian_045_055, *best_fit_params_Sersic_Gaussian_045_055[:5]))/yy_new_Sersic_Gaussian_045_055, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_045_055, Sersic_Gauss(x_new_Sersic_Gaussian_045_055, *Popt_Sersic_Gaussian_045_055[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_045_055, Gaussian(x_new_Sersic_Gaussian_045_055, *Popt_Sersic_Gaussian_045_055[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_045_055, Sersic_Fit(x_new_Sersic_Gaussian_045_055, *Popt_Sersic_Gaussian_045_055[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_045_055, yy_new_Sersic_Gaussian_045_055, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_045_055, y_data_Sersic_Gaussian_045_055, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_045_055, (yy_new_Sersic_Gaussian_045_055 - Sersic_Gauss(x_new_Sersic_Gaussian_045_055, *Popt_Sersic_Gaussian_045_055[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_045_055, (yy_new_Sersic_Gaussian_045_055 - Sersic_Gauss(x_new_Sersic_Gaussian_045_055, *Popt_Sersic_Gaussian_045_055[:5]))/yy_new_Sersic_Gaussian_045_055, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.45 < z < 0.55")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_045_055[0]
R_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_045_055[1]
N_Sersic_035_045 = best_fit_params_Sersic_Gaussian_045_055[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_045_055[0]) + " +" +str(errors_Sersic_Gaussian_045_055[0][0]) + " - " +str(errors_Sersic_Gaussian_045_055[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_045_055[1]) + " +" +str(errors_Sersic_Gaussian_045_055[1][0]) + " - " +str(errors_Sersic_Gaussian_045_055[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_045_055[2]) + " +" +str(errors_Sersic_Gaussian_045_055[2][0]) + " - " +str(errors_Sersic_Gaussian_045_055[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_045_055[3]) + " +" +str(errors_Sersic_Gaussian_045_055[3][0]) + " - " +str(errors_Sersic_Gaussian_045_055[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_045_055[4]) + " +" +str(errors_Sersic_Gaussian_045_055[4][0]) + " - " +str(errors_Sersic_Gaussian_045_055[4][1]))

In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_055_065 = Equal_Area_Radius_055_065_Array
x_new_Sersic_Gaussian_055_065 = np.linspace(min(x_data_Sersic_Gaussian_055_065),max(x_data_Sersic_Gaussian_055_065), 200)

y_data_Sersic_Gaussian_055_065 = np.array(Average_Counts_055_065)
y_new_Sersic_Gaussian_055_065 = CubicSpline(x_data_Sersic_Gaussian_055_065, y_data_Sersic_Gaussian_055_065)

yy_new_Sersic_Gaussian_055_065 = y_new_Sersic_Gaussian_055_065(x_new_Sersic_Gaussian_055_065)


y_err_Sersic_Gaussian_055_065 = np.array(Average_Counts_SD_055_065)
y_new_err_Sersic_Gaussian_055_065 = CubicSpline(x_data_Sersic_Gaussian_055_065, y_err_Sersic_Gaussian_055_065)

yy_new_err_Sersic_Gaussian_055_065 = y_new_err_Sersic_Gaussian_055_065(x_new_Sersic_Gaussian_055_065)


#From Popt --- I_e, R_e, N, A, Sigma = [0.64720827 2.40782359 1.49070718 4.98475642 0.42026838]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.03173512 0.07209185 0.04045171 0.15083045 0.00551747]

i_e_Sersic_Gaussian_055_065 = 0.64720827
r_e_Sersic_Gaussian_055_065 = 2.40782359
n_Sersic_Gaussian_055_065 = 1.49070718

a_Sersic_Gaussian_055_065 = 4.98475642
sigma_Sersic_Gaussian_055_065 = 0.42026838

initial_Mirror_Sersic_Gaussian_055_065 = np.array([i_e_Sersic_Gaussian_055_065, r_e_Sersic_Gaussian_055_065, n_Sersic_Gaussian_055_065, a_Sersic_Gaussian_055_065, sigma_Sersic_Gaussian_055_065]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_055_065 = 0
I_e_u_Sersic_Gaussian_055_065 = i_e_Sersic_Gaussian_055_065 + (0.01*i_e_Sersic_Gaussian_055_065)

R_e_l_Sersic_Gaussian_055_065 = 0
R_e_u_Sersic_Gaussian_055_065 = r_e_Sersic_Gaussian_055_065 + (0.01*r_e_Sersic_Gaussian_055_065)

N_l_Sersic_Gaussian_055_065 = 0
N_u_Sersic_Gaussian_055_065 = n_Sersic_Gaussian_055_065 + (0.01*n_Sersic_Gaussian_055_065)


a_l_Sersic_Gaussian_055_065 = 0
a_u_Sersic_Gaussian_055_065 = a_Sersic_Gaussian_055_065 + (0.01*a_Sersic_Gaussian_055_065)

sigma_l_Sersic_Gaussian_055_065 = 0
sigma_u_Sersic_Gaussian_055_065 = sigma_Sersic_Gaussian_055_065 + (0.01*sigma_Sersic_Gaussian_055_065)


Popt_Sersic_Gaussian_055_065, Pcov_Sersic_Gaussian_055_065 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_055_065, yy_new_Sersic_Gaussian_055_065, p0=[i_e_Sersic_Gaussian_055_065, r_e_Sersic_Gaussian_055_065, n_Sersic_Gaussian_055_065, a_Sersic_Gaussian_055_065, sigma_Sersic_Gaussian_055_065], bounds=(0, np.inf))
Perr_Sersic_Gaussian_055_065 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_055_065))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_055_065, r_e_Sersic_Gaussian_055_065, n_Sersic_Gaussian_055_065, a_Sersic_Gaussian_055_065, sigma_Sersic_Gaussian_055_065 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_055_065, r_e_Sersic_Gaussian_055_065, n_Sersic_Gaussian_055_065, a_Sersic_Gaussian_055_065, sigma_Sersic_Gaussian_055_065)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_055_065, n_Sersic_Gaussian_055_065, a_Sersic_Gaussian_055_065, sigma_Sersic_Gaussian_055_065 = theta
    if (I_e_l_Sersic_Gaussian_055_065 < i_e_Sersic_Gaussian_055_065 < I_e_u_Sersic_Gaussian_055_065 and R_e_l_Sersic_Gaussian_055_065 < r_e_Sersic_Gaussian_055_065 < R_e_u_Sersic_Gaussian_055_065 and N_l_Sersic_Gaussian_055_065 < n_Sersic_Gaussian_055_065 < N_u_Sersic_Gaussian_055_065 and a_l_Sersic_Gaussian_055_065 < a_Sersic_Gaussian_055_065 < a_u_Sersic_Gaussian_055_065 and sigma_l_Sersic_Gaussian_055_065 < sigma_Sersic_Gaussian_055_065 < sigma_u_Sersic_Gaussian_055_065):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_055_065 = [(I_e_l_Sersic_Gaussian_055_065, I_e_u_Sersic_Gaussian_055_065), (R_e_l_Sersic_Gaussian_055_065, R_e_u_Sersic_Gaussian_055_065), (N_l_Sersic_Gaussian_055_065, N_u_Sersic_Gaussian_055_065), (a_l_Sersic_Gaussian_055_065, a_u_Sersic_Gaussian_055_065), (sigma_l_Sersic_Gaussian_055_065, sigma_u_Sersic_Gaussian_055_065)] # Parameter bounds for optimization
soln_Sersic_Gaussian_055_065 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_055_065, args=(x_new_Sersic_Gaussian_055_065, yy_new_Sersic_Gaussian_055_065, yy_new_err_Sersic_Gaussian_055_065), bounds=bounds_Sersic_Gaussian_055_065)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_055_065 = soln_Sersic_Gaussian_055_065.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_055_065.x))
nwalkers_Sersic_Gaussian_055_065, ndim_Sersic_Gaussian_055_065 = pos_Sersic_Gaussian_055_065.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_055_065 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_055_065, ndim_Sersic_Gaussian_055_065, log_probability, args=(x_new_Sersic_Gaussian_055_065, yy_new_Sersic_Gaussian_055_065, yy_new_err_Sersic_Gaussian_055_065))
sampler_Sersic_Gaussian_055_065.run_mcmc(pos_Sersic_Gaussian_055_065, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_055_065 = sampler_Sersic_Gaussian_055_065.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_055_065 = []
for i in range(ndim_Sersic_Gaussian_055_065):
    mcmc_Sersic_Gaussian_055_065 = np.percentile(samples_Sersic_Gaussian_055_065[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_055_065.append(mcmc_Sersic_Gaussian_055_065[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_055_065 = []
for i in range(ndim_Sersic_Gaussian_055_065):
     mcmc_Sersic_Gaussian_055_065 = np.percentile(samples_Sersic_Gaussian_055_065[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_055_065.append([mcmc_Sersic_Gaussian_055_065[2]-mcmc_Sersic_Gaussian_055_065[1],mcmc_Sersic_Gaussian_055_065[1]-mcmc_Sersic_Gaussian_055_065[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_055_065))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_055_065))

axes1[0].plot(x_new_Sersic_Gaussian_055_065, Sersic_Gauss(x_new_Sersic_Gaussian_055_065, *best_fit_params_Sersic_Gaussian_055_065[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_055_065, Gaussian(x_new_Sersic_Gaussian_055_065, *best_fit_params_Sersic_Gaussian_055_065[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_055_065, Sersic_Fit(x_new_Sersic_Gaussian_055_065, *best_fit_params_Sersic_Gaussian_055_065[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_055_065, yy_new_Sersic_Gaussian_055_065, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_055_065, y_data_Sersic_Gaussian_055_065, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_055_065, yy_new_Sersic_Gaussian_055_065 - Sersic_Gauss(x_new_Sersic_Gaussian_055_065, *best_fit_params_Sersic_Gaussian_055_065[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_055_065, (yy_new_Sersic_Gaussian_055_065 - Sersic_Gauss(x_new_Sersic_Gaussian_055_065, *best_fit_params_Sersic_Gaussian_055_065[:5]))/yy_new_Sersic_Gaussian_055_065, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_055_065, Sersic_Gauss(x_new_Sersic_Gaussian_055_065, *Popt_Sersic_Gaussian_055_065[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_055_065, Gaussian(x_new_Sersic_Gaussian_055_065, *Popt_Sersic_Gaussian_055_065[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_055_065, Sersic_Fit(x_new_Sersic_Gaussian_055_065, *Popt_Sersic_Gaussian_055_065[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_055_065, yy_new_Sersic_Gaussian_055_065, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_055_065, y_data_Sersic_Gaussian_055_065, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_055_065, (yy_new_Sersic_Gaussian_055_065 - Sersic_Gauss(x_new_Sersic_Gaussian_055_065, *Popt_Sersic_Gaussian_055_065[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_055_065, (yy_new_Sersic_Gaussian_055_065 - Sersic_Gauss(x_new_Sersic_Gaussian_055_065, *Popt_Sersic_Gaussian_055_065[:5]))/yy_new_Sersic_Gaussian_055_065, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.55 < z < 0.65")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_055_065[0]
R_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_055_065[1]
N_Sersic_035_045 = best_fit_params_Sersic_Gaussian_055_065[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_055_065[0]) + " +" +str(errors_Sersic_Gaussian_055_065[0][0]) + " - " +str(errors_Sersic_Gaussian_055_065[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_055_065[1]) + " +" +str(errors_Sersic_Gaussian_055_065[1][0]) + " - " +str(errors_Sersic_Gaussian_055_065[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_055_065[2]) + " +" +str(errors_Sersic_Gaussian_055_065[2][0]) + " - " +str(errors_Sersic_Gaussian_055_065[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_055_065[3]) + " +" +str(errors_Sersic_Gaussian_055_065[3][0]) + " - " +str(errors_Sersic_Gaussian_055_065[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_055_065[4]) + " +" +str(errors_Sersic_Gaussian_055_065[4][0]) + " - " +str(errors_Sersic_Gaussian_055_065[4][1]))

In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_065_075 = Equal_Area_Radius_065_075_Array
x_new_Sersic_Gaussian_065_075 = np.linspace(min(x_data_Sersic_Gaussian_065_075),max(x_data_Sersic_Gaussian_065_075), 200)

y_data_Sersic_Gaussian_065_075 = np.array(Average_Counts_065_075)
y_new_Sersic_Gaussian_065_075 = CubicSpline(x_data_Sersic_Gaussian_065_075, y_data_Sersic_Gaussian_065_075)

yy_new_Sersic_Gaussian_065_075 = y_new_Sersic_Gaussian_065_075(x_new_Sersic_Gaussian_065_075)


y_err_Sersic_Gaussian_065_075 = np.array(Average_Counts_SD_065_075)
y_new_err_Sersic_Gaussian_065_075 = CubicSpline(x_data_Sersic_Gaussian_065_075, y_err_Sersic_Gaussian_065_075)

yy_new_err_Sersic_Gaussian_065_075 = y_new_err_Sersic_Gaussian_065_075(x_new_Sersic_Gaussian_065_075)


#From Popt --- I_e, R_e, N, A, Sigma = [0.73814808 2.28508746 1.38064103 5.95390071 0.42076486]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.03817755 0.07140324 0.04453649 0.18930339 0.00543095]

i_e_Sersic_Gaussian_065_075 = 0.73814808
r_e_Sersic_Gaussian_065_075 = 2.28508746
n_Sersic_Gaussian_065_075 = 1.38064103

a_Sersic_Gaussian_065_075 = 5.95390071
sigma_Sersic_Gaussian_065_075 = 0.42076486

initial_Mirror_Sersic_Gaussian_065_075 = np.array([i_e_Sersic_Gaussian_065_075, r_e_Sersic_Gaussian_065_075, n_Sersic_Gaussian_065_075, a_Sersic_Gaussian_065_075, sigma_Sersic_Gaussian_065_075]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_065_075 = 0
I_e_u_Sersic_Gaussian_065_075 = i_e_Sersic_Gaussian_065_075 + (0.01*i_e_Sersic_Gaussian_065_075)

R_e_l_Sersic_Gaussian_065_075 = 0
R_e_u_Sersic_Gaussian_065_075 = r_e_Sersic_Gaussian_065_075 + (0.01*r_e_Sersic_Gaussian_065_075)

N_l_Sersic_Gaussian_065_075 = 0
N_u_Sersic_Gaussian_065_075 = n_Sersic_Gaussian_065_075 + (0.01*n_Sersic_Gaussian_065_075)


a_l_Sersic_Gaussian_065_075 = 0
a_u_Sersic_Gaussian_065_075 = a_Sersic_Gaussian_065_075 + (0.01*a_Sersic_Gaussian_065_075)

sigma_l_Sersic_Gaussian_065_075 = 0
sigma_u_Sersic_Gaussian_065_075 = sigma_Sersic_Gaussian_065_075 + (0.01*sigma_Sersic_Gaussian_065_075)


Popt_Sersic_Gaussian_065_075, Pcov_Sersic_Gaussian_065_075 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_065_075, yy_new_Sersic_Gaussian_065_075, p0=[i_e_Sersic_Gaussian_065_075, r_e_Sersic_Gaussian_065_075, n_Sersic_Gaussian_065_075, a_Sersic_Gaussian_065_075, sigma_Sersic_Gaussian_065_075], bounds=(0, np.inf))
Perr_Sersic_Gaussian_065_075 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_065_075))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_065_075, r_e_Sersic_Gaussian_065_075, n_Sersic_Gaussian_065_075, a_Sersic_Gaussian_065_075, sigma_Sersic_Gaussian_065_075 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_065_075, r_e_Sersic_Gaussian_065_075, n_Sersic_Gaussian_065_075, a_Sersic_Gaussian_065_075, sigma_Sersic_Gaussian_065_075)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_065_075, n_Sersic_Gaussian_065_075, a_Sersic_Gaussian_065_075, sigma_Sersic_Gaussian_065_075 = theta
    if (I_e_l_Sersic_Gaussian_065_075 < i_e_Sersic_Gaussian_065_075 < I_e_u_Sersic_Gaussian_065_075 and R_e_l_Sersic_Gaussian_065_075 < r_e_Sersic_Gaussian_065_075 < R_e_u_Sersic_Gaussian_065_075 and N_l_Sersic_Gaussian_065_075 < n_Sersic_Gaussian_065_075 < N_u_Sersic_Gaussian_065_075 and a_l_Sersic_Gaussian_065_075 < a_Sersic_Gaussian_065_075 < a_u_Sersic_Gaussian_065_075 and sigma_l_Sersic_Gaussian_065_075 < sigma_Sersic_Gaussian_065_075 < sigma_u_Sersic_Gaussian_065_075):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_065_075 = [(I_e_l_Sersic_Gaussian_065_075, I_e_u_Sersic_Gaussian_065_075), (R_e_l_Sersic_Gaussian_065_075, R_e_u_Sersic_Gaussian_065_075), (N_l_Sersic_Gaussian_065_075, N_u_Sersic_Gaussian_065_075), (a_l_Sersic_Gaussian_065_075, a_u_Sersic_Gaussian_065_075), (sigma_l_Sersic_Gaussian_065_075, sigma_u_Sersic_Gaussian_065_075)] # Parameter bounds for optimization
soln_Sersic_Gaussian_065_075 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_065_075, args=(x_new_Sersic_Gaussian_065_075, yy_new_Sersic_Gaussian_065_075, yy_new_err_Sersic_Gaussian_065_075), bounds=bounds_Sersic_Gaussian_065_075)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_065_075 = soln_Sersic_Gaussian_065_075.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_065_075.x))
nwalkers_Sersic_Gaussian_065_075, ndim_Sersic_Gaussian_065_075 = pos_Sersic_Gaussian_065_075.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_065_075 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_065_075, ndim_Sersic_Gaussian_065_075, log_probability, args=(x_new_Sersic_Gaussian_065_075, yy_new_Sersic_Gaussian_065_075, yy_new_err_Sersic_Gaussian_065_075))
sampler_Sersic_Gaussian_065_075.run_mcmc(pos_Sersic_Gaussian_065_075, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_065_075 = sampler_Sersic_Gaussian_065_075.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_065_075 = []
for i in range(ndim_Sersic_Gaussian_065_075):
    mcmc_Sersic_Gaussian_065_075 = np.percentile(samples_Sersic_Gaussian_065_075[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_065_075.append(mcmc_Sersic_Gaussian_065_075[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_065_075 = []
for i in range(ndim_Sersic_Gaussian_065_075):
     mcmc_Sersic_Gaussian_065_075 = np.percentile(samples_Sersic_Gaussian_065_075[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_065_075.append([mcmc_Sersic_Gaussian_065_075[2]-mcmc_Sersic_Gaussian_065_075[1],mcmc_Sersic_Gaussian_065_075[1]-mcmc_Sersic_Gaussian_065_075[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_065_075))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_065_075))

axes1[0].plot(x_new_Sersic_Gaussian_065_075, Sersic_Gauss(x_new_Sersic_Gaussian_065_075, *best_fit_params_Sersic_Gaussian_065_075[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_065_075, Gaussian(x_new_Sersic_Gaussian_065_075, *best_fit_params_Sersic_Gaussian_065_075[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_065_075, Sersic_Fit(x_new_Sersic_Gaussian_065_075, *best_fit_params_Sersic_Gaussian_065_075[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_065_075, yy_new_Sersic_Gaussian_065_075, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_065_075, y_data_Sersic_Gaussian_065_075, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_065_075, yy_new_Sersic_Gaussian_065_075 - Sersic_Gauss(x_new_Sersic_Gaussian_065_075, *best_fit_params_Sersic_Gaussian_065_075[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_065_075, (yy_new_Sersic_Gaussian_065_075 - Sersic_Gauss(x_new_Sersic_Gaussian_065_075, *best_fit_params_Sersic_Gaussian_065_075[:5]))/yy_new_Sersic_Gaussian_065_075, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_065_075, Sersic_Gauss(x_new_Sersic_Gaussian_065_075, *Popt_Sersic_Gaussian_065_075[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_065_075, Gaussian(x_new_Sersic_Gaussian_065_075, *Popt_Sersic_Gaussian_065_075[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_065_075, Sersic_Fit(x_new_Sersic_Gaussian_065_075, *Popt_Sersic_Gaussian_065_075[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_065_075, yy_new_Sersic_Gaussian_065_075, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_065_075, y_data_Sersic_Gaussian_065_075, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_065_075, (yy_new_Sersic_Gaussian_065_075 - Sersic_Gauss(x_new_Sersic_Gaussian_065_075, *Popt_Sersic_Gaussian_065_075[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_065_075, (yy_new_Sersic_Gaussian_065_075 - Sersic_Gauss(x_new_Sersic_Gaussian_065_075, *Popt_Sersic_Gaussian_065_075[:5]))/yy_new_Sersic_Gaussian_065_075, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.65 < z < 0.75")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_065_075[0]
R_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_065_075[1]
N_Sersic_035_045 = best_fit_params_Sersic_Gaussian_065_075[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_065_075[0]) + " +" +str(errors_Sersic_Gaussian_065_075[0][0]) + " - " +str(errors_Sersic_Gaussian_065_075[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_065_075[1]) + " +" +str(errors_Sersic_Gaussian_065_075[1][0]) + " - " +str(errors_Sersic_Gaussian_065_075[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_065_075[2]) + " +" +str(errors_Sersic_Gaussian_065_075[2][0]) + " - " +str(errors_Sersic_Gaussian_065_075[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_065_075[3]) + " +" +str(errors_Sersic_Gaussian_065_075[3][0]) + " - " +str(errors_Sersic_Gaussian_065_075[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_065_075[4]) + " +" +str(errors_Sersic_Gaussian_065_075[4][0]) + " - " +str(errors_Sersic_Gaussian_065_075[4][1]))

In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_075_085 = Equal_Area_Radius_075_085_Array
x_new_Sersic_Gaussian_075_085 = np.linspace(min(x_data_Sersic_Gaussian_075_085),max(x_data_Sersic_Gaussian_075_085), 200)

y_data_Sersic_Gaussian_075_085 = np.array(Average_Counts_075_085)
y_new_Sersic_Gaussian_075_085 = CubicSpline(x_data_Sersic_Gaussian_075_085, y_data_Sersic_Gaussian_075_085)

yy_new_Sersic_Gaussian_075_085 = y_new_Sersic_Gaussian_075_085(x_new_Sersic_Gaussian_075_085)


y_err_Sersic_Gaussian_075_085 = np.array(Average_Counts_SD_075_085)
y_new_err_Sersic_Gaussian_075_085 = CubicSpline(x_data_Sersic_Gaussian_075_085, y_err_Sersic_Gaussian_075_085)

yy_new_err_Sersic_Gaussian_075_085 = y_new_err_Sersic_Gaussian_075_085(x_new_Sersic_Gaussian_075_085)


#From Popt --- I_e, R_e, N, A, Sigma = [0.62249014 2.31838635 1.45852294 5.30405082 0.42508707]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.03121059 0.0701967  0.04063364 0.14244454 0.00499532]

i_e_Sersic_Gaussian_075_085 = 0.62249014
r_e_Sersic_Gaussian_075_085 = 2.31838635
n_Sersic_Gaussian_075_085 = 1.45852294

a_Sersic_Gaussian_075_085 = 5.30405082
sigma_Sersic_Gaussian_075_085 = 0.42508707

initial_Mirror_Sersic_Gaussian_075_085 = np.array([i_e_Sersic_Gaussian_075_085, r_e_Sersic_Gaussian_075_085, n_Sersic_Gaussian_075_085, a_Sersic_Gaussian_075_085, sigma_Sersic_Gaussian_075_085]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_075_085 = 0
I_e_u_Sersic_Gaussian_075_085 = i_e_Sersic_Gaussian_075_085 + (0.01*i_e_Sersic_Gaussian_075_085)

R_e_l_Sersic_Gaussian_075_085 = 0
R_e_u_Sersic_Gaussian_075_085 = r_e_Sersic_Gaussian_075_085 + (0.01*r_e_Sersic_Gaussian_075_085)

N_l_Sersic_Gaussian_075_085 = 0
N_u_Sersic_Gaussian_075_085 = n_Sersic_Gaussian_075_085 + (0.01*n_Sersic_Gaussian_075_085)


a_l_Sersic_Gaussian_075_085 = 0
a_u_Sersic_Gaussian_075_085 = a_Sersic_Gaussian_075_085 + (0.01*a_Sersic_Gaussian_075_085)

sigma_l_Sersic_Gaussian_075_085 = 0
sigma_u_Sersic_Gaussian_075_085 = sigma_Sersic_Gaussian_075_085 + (0.01*sigma_Sersic_Gaussian_075_085)


Popt_Sersic_Gaussian_075_085, Pcov_Sersic_Gaussian_075_085 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_075_085, yy_new_Sersic_Gaussian_075_085, p0=[i_e_Sersic_Gaussian_075_085, r_e_Sersic_Gaussian_075_085, n_Sersic_Gaussian_075_085, a_Sersic_Gaussian_075_085, sigma_Sersic_Gaussian_075_085], bounds=(0, np.inf))
Perr_Sersic_Gaussian_075_085 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_075_085))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_075_085, r_e_Sersic_Gaussian_075_085, n_Sersic_Gaussian_075_085, a_Sersic_Gaussian_075_085, sigma_Sersic_Gaussian_075_085 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_075_085, r_e_Sersic_Gaussian_075_085, n_Sersic_Gaussian_075_085, a_Sersic_Gaussian_075_085, sigma_Sersic_Gaussian_075_085)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_075_085, n_Sersic_Gaussian_075_085, a_Sersic_Gaussian_075_085, sigma_Sersic_Gaussian_075_085 = theta
    if (I_e_l_Sersic_Gaussian_075_085 < i_e_Sersic_Gaussian_075_085 < I_e_u_Sersic_Gaussian_075_085 and R_e_l_Sersic_Gaussian_075_085 < r_e_Sersic_Gaussian_075_085 < R_e_u_Sersic_Gaussian_075_085 and N_l_Sersic_Gaussian_075_085 < n_Sersic_Gaussian_075_085 < N_u_Sersic_Gaussian_075_085 and a_l_Sersic_Gaussian_075_085 < a_Sersic_Gaussian_075_085 < a_u_Sersic_Gaussian_075_085 and sigma_l_Sersic_Gaussian_075_085 < sigma_Sersic_Gaussian_075_085 < sigma_u_Sersic_Gaussian_075_085):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_075_085 = [(I_e_l_Sersic_Gaussian_075_085, I_e_u_Sersic_Gaussian_075_085), (R_e_l_Sersic_Gaussian_075_085, R_e_u_Sersic_Gaussian_075_085), (N_l_Sersic_Gaussian_075_085, N_u_Sersic_Gaussian_075_085), (a_l_Sersic_Gaussian_075_085, a_u_Sersic_Gaussian_075_085), (sigma_l_Sersic_Gaussian_075_085, sigma_u_Sersic_Gaussian_075_085)] # Parameter bounds for optimization
soln_Sersic_Gaussian_075_085 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_075_085, args=(x_new_Sersic_Gaussian_075_085, yy_new_Sersic_Gaussian_075_085, yy_new_err_Sersic_Gaussian_075_085), bounds=bounds_Sersic_Gaussian_075_085)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_075_085 = soln_Sersic_Gaussian_075_085.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_075_085.x))
nwalkers_Sersic_Gaussian_075_085, ndim_Sersic_Gaussian_075_085 = pos_Sersic_Gaussian_075_085.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_075_085 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_075_085, ndim_Sersic_Gaussian_075_085, log_probability, args=(x_new_Sersic_Gaussian_075_085, yy_new_Sersic_Gaussian_075_085, yy_new_err_Sersic_Gaussian_075_085))
sampler_Sersic_Gaussian_075_085.run_mcmc(pos_Sersic_Gaussian_075_085, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_075_085 = sampler_Sersic_Gaussian_075_085.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_075_085 = []
for i in range(ndim_Sersic_Gaussian_075_085):
    mcmc_Sersic_Gaussian_075_085 = np.percentile(samples_Sersic_Gaussian_075_085[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_075_085.append(mcmc_Sersic_Gaussian_075_085[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_075_085 = []
for i in range(ndim_Sersic_Gaussian_075_085):
     mcmc_Sersic_Gaussian_075_085 = np.percentile(samples_Sersic_Gaussian_075_085[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_075_085.append([mcmc_Sersic_Gaussian_075_085[2]-mcmc_Sersic_Gaussian_075_085[1],mcmc_Sersic_Gaussian_075_085[1]-mcmc_Sersic_Gaussian_075_085[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_075_085))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_075_085))

axes1[0].plot(x_new_Sersic_Gaussian_075_085, Sersic_Gauss(x_new_Sersic_Gaussian_075_085, *best_fit_params_Sersic_Gaussian_075_085[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_075_085, Gaussian(x_new_Sersic_Gaussian_075_085, *best_fit_params_Sersic_Gaussian_075_085[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_075_085, Sersic_Fit(x_new_Sersic_Gaussian_075_085, *best_fit_params_Sersic_Gaussian_075_085[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_075_085, yy_new_Sersic_Gaussian_075_085, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_075_085, y_data_Sersic_Gaussian_075_085, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_075_085, yy_new_Sersic_Gaussian_075_085 - Sersic_Gauss(x_new_Sersic_Gaussian_075_085, *best_fit_params_Sersic_Gaussian_075_085[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_075_085, (yy_new_Sersic_Gaussian_075_085 - Sersic_Gauss(x_new_Sersic_Gaussian_075_085, *best_fit_params_Sersic_Gaussian_075_085[:5]))/yy_new_Sersic_Gaussian_075_085, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_075_085, Sersic_Gauss(x_new_Sersic_Gaussian_075_085, *Popt_Sersic_Gaussian_075_085[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_075_085, Gaussian(x_new_Sersic_Gaussian_075_085, *Popt_Sersic_Gaussian_075_085[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_075_085, Sersic_Fit(x_new_Sersic_Gaussian_075_085, *Popt_Sersic_Gaussian_075_085[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_075_085, yy_new_Sersic_Gaussian_075_085, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_075_085, y_data_Sersic_Gaussian_075_085, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_075_085, (yy_new_Sersic_Gaussian_075_085 - Sersic_Gauss(x_new_Sersic_Gaussian_075_085, *Popt_Sersic_Gaussian_075_085[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_075_085, (yy_new_Sersic_Gaussian_075_085 - Sersic_Gauss(x_new_Sersic_Gaussian_075_085, *Popt_Sersic_Gaussian_075_085[:5]))/yy_new_Sersic_Gaussian_075_085, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.75 < z < 0.85")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_075_085[0]
R_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_075_085[1]
N_Sersic_035_045 = best_fit_params_Sersic_Gaussian_075_085[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_075_085[0]) + " +" +str(errors_Sersic_Gaussian_075_085[0][0]) + " - " +str(errors_Sersic_Gaussian_075_085[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_075_085[1]) + " +" +str(errors_Sersic_Gaussian_075_085[1][0]) + " - " +str(errors_Sersic_Gaussian_075_085[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_075_085[2]) + " +" +str(errors_Sersic_Gaussian_075_085[2][0]) + " - " +str(errors_Sersic_Gaussian_075_085[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_075_085[3]) + " +" +str(errors_Sersic_Gaussian_075_085[3][0]) + " - " +str(errors_Sersic_Gaussian_075_085[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_075_085[4]) + " +" +str(errors_Sersic_Gaussian_075_085[4][0]) + " - " +str(errors_Sersic_Gaussian_075_085[4][1]))


In [ ]:
# 1. Generate or load data
x_data_Sersic_Gaussian_085_096 = Equal_Area_Radius_085_096_Array
x_new_Sersic_Gaussian_085_096 = np.linspace(min(x_data_Sersic_Gaussian_085_096),max(x_data_Sersic_Gaussian_085_096), 200)

y_data_Sersic_Gaussian_085_096 = np.array(Average_Counts_085_096)
y_new_Sersic_Gaussian_085_096 = CubicSpline(x_data_Sersic_Gaussian_085_096, y_data_Sersic_Gaussian_085_096)

yy_new_Sersic_Gaussian_085_096 = y_new_Sersic_Gaussian_085_096(x_new_Sersic_Gaussian_085_096)


y_err_Sersic_Gaussian_085_096 = np.array(Average_Counts_SD_085_096)
y_new_err_Sersic_Gaussian_085_096 = CubicSpline(x_data_Sersic_Gaussian_085_096, y_err_Sersic_Gaussian_085_096)

yy_new_err_Sersic_Gaussian_085_096 = y_new_err_Sersic_Gaussian_085_096(x_new_Sersic_Gaussian_085_096)


#From Popt --- I_e, R_e, N, A, Sigma = [0.72745817 2.19631573 1.55546547 5.58220012 0.40063247]
#From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = [0.03826408 0.06946756 0.04176369 0.16513572 0.00541166]

i_e_Sersic_Gaussian_085_096 = 0.72745817
r_e_Sersic_Gaussian_085_096 = 2.19631573
n_Sersic_Gaussian_085_096 = 1.55546547

a_Sersic_Gaussian_085_096 = 5.58220012
sigma_Sersic_Gaussian_085_096 = 0.40063247

initial_Mirror_Sersic_Gaussian_085_096 = np.array([i_e_Sersic_Gaussian_085_096, r_e_Sersic_Gaussian_085_096, n_Sersic_Gaussian_085_096, a_Sersic_Gaussian_085_096, sigma_Sersic_Gaussian_085_096]) #I_e,R_e,N


I_e_l_Sersic_Gaussian_085_096 = 0
I_e_u_Sersic_Gaussian_085_096 = i_e_Sersic_Gaussian_085_096 + (0.01*i_e_Sersic_Gaussian_085_096)

R_e_l_Sersic_Gaussian_085_096 = 0
R_e_u_Sersic_Gaussian_085_096 = r_e_Sersic_Gaussian_085_096 + (0.01*r_e_Sersic_Gaussian_085_096)

N_l_Sersic_Gaussian_085_096 = 0
N_u_Sersic_Gaussian_085_096 = n_Sersic_Gaussian_085_096 + (0.01*n_Sersic_Gaussian_085_096)


a_l_Sersic_Gaussian_085_096 = 0
a_u_Sersic_Gaussian_085_096 = a_Sersic_Gaussian_085_096 + (0.01*a_Sersic_Gaussian_085_096)

sigma_l_Sersic_Gaussian_085_096 = 0
sigma_u_Sersic_Gaussian_085_096 = sigma_Sersic_Gaussian_085_096 + (0.01*sigma_Sersic_Gaussian_085_096)


Popt_Sersic_Gaussian_085_096, Pcov_Sersic_Gaussian_085_096 = curve_fit(Sersic_Gauss, x_new_Sersic_Gaussian_085_096, yy_new_Sersic_Gaussian_085_096, p0=[i_e_Sersic_Gaussian_085_096, r_e_Sersic_Gaussian_085_096, n_Sersic_Gaussian_085_096, a_Sersic_Gaussian_085_096, sigma_Sersic_Gaussian_085_096], bounds=(0, np.inf))
Perr_Sersic_Gaussian_085_096 = np.sqrt(np.diag(Pcov_Sersic_Gaussian_085_096))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_Sersic_Gaussian_085_096, r_e_Sersic_Gaussian_085_096, n_Sersic_Gaussian_085_096, a_Sersic_Gaussian_085_096, sigma_Sersic_Gaussian_085_096 = theta
    model = Sersic_Gauss(x, i_e_Sersic_Gaussian_085_096, r_e_Sersic_Gaussian_085_096, n_Sersic_Gaussian_085_096, a_Sersic_Gaussian_085_096, sigma_Sersic_Gaussian_085_096)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_Sersic_Gaussian_085_096, n_Sersic_Gaussian_085_096, a_Sersic_Gaussian_085_096, sigma_Sersic_Gaussian_085_096 = theta
    if (I_e_l_Sersic_Gaussian_085_096 < i_e_Sersic_Gaussian_085_096 < I_e_u_Sersic_Gaussian_085_096 and R_e_l_Sersic_Gaussian_085_096 < r_e_Sersic_Gaussian_085_096 < R_e_u_Sersic_Gaussian_085_096 and N_l_Sersic_Gaussian_085_096 < n_Sersic_Gaussian_085_096 < N_u_Sersic_Gaussian_085_096 and a_l_Sersic_Gaussian_085_096 < a_Sersic_Gaussian_085_096 < a_u_Sersic_Gaussian_085_096 and sigma_l_Sersic_Gaussian_085_096 < sigma_Sersic_Gaussian_085_096 < sigma_u_Sersic_Gaussian_085_096):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_Sersic_Gaussian_085_096 = [(I_e_l_Sersic_Gaussian_085_096, I_e_u_Sersic_Gaussian_085_096), (R_e_l_Sersic_Gaussian_085_096, R_e_u_Sersic_Gaussian_085_096), (N_l_Sersic_Gaussian_085_096, N_u_Sersic_Gaussian_085_096), (a_l_Sersic_Gaussian_085_096, a_u_Sersic_Gaussian_085_096), (sigma_l_Sersic_Gaussian_085_096, sigma_u_Sersic_Gaussian_085_096)] # Parameter bounds for optimization
soln_Sersic_Gaussian_085_096 = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_Sersic_Gaussian_085_096, args=(x_new_Sersic_Gaussian_085_096, yy_new_Sersic_Gaussian_085_096, yy_new_err_Sersic_Gaussian_085_096), bounds=bounds_Sersic_Gaussian_085_096)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_Sersic_Gaussian_085_096 = soln_Sersic_Gaussian_085_096.x + 1e-4 * np.random.randn(32, len(soln_Sersic_Gaussian_085_096.x))
nwalkers_Sersic_Gaussian_085_096, ndim_Sersic_Gaussian_085_096 = pos_Sersic_Gaussian_085_096.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_Sersic_Gaussian_085_096 = emcee.EnsembleSampler(nwalkers_Sersic_Gaussian_085_096, ndim_Sersic_Gaussian_085_096, log_probability, args=(x_new_Sersic_Gaussian_085_096, yy_new_Sersic_Gaussian_085_096, yy_new_err_Sersic_Gaussian_085_096))
sampler_Sersic_Gaussian_085_096.run_mcmc(pos_Sersic_Gaussian_085_096, 5000, progress=True)

# 5. Analyze results
samples_Sersic_Gaussian_085_096 = sampler_Sersic_Gaussian_085_096.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_Sersic_Gaussian_085_096 = []
for i in range(ndim_Sersic_Gaussian_085_096):
    mcmc_Sersic_Gaussian_085_096 = np.percentile(samples_Sersic_Gaussian_085_096[:, i], [68, 95, 99.7])
    best_fit_params_Sersic_Gaussian_085_096.append(mcmc_Sersic_Gaussian_085_096[1])

# 10. Error estimation ############################################################################################################################################
errors_Sersic_Gaussian_085_096 = []
for i in range(ndim_Sersic_Gaussian_085_096):
     mcmc_Sersic_Gaussian_085_096 = np.percentile(samples_Sersic_Gaussian_085_096[:, i], [68, 95, 99.7])
     errors_Sersic_Gaussian_085_096.append([mcmc_Sersic_Gaussian_085_096[2]-mcmc_Sersic_Gaussian_085_096[1],mcmc_Sersic_Gaussian_085_096[1]-mcmc_Sersic_Gaussian_085_096[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, A, Sigma = " +str(Popt_Sersic_Gaussian_085_096))
print("From Popt Standard Deviation --- I_e, R_e, N, A, Sigma = " +str(Perr_Sersic_Gaussian_085_096))

axes1[0].plot(x_new_Sersic_Gaussian_085_096, Sersic_Gauss(x_new_Sersic_Gaussian_085_096, *best_fit_params_Sersic_Gaussian_085_096[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_Sersic_Gaussian_085_096, Gaussian(x_new_Sersic_Gaussian_085_096, *best_fit_params_Sersic_Gaussian_085_096[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_Sersic_Gaussian_085_096, Sersic_Fit(x_new_Sersic_Gaussian_085_096, *best_fit_params_Sersic_Gaussian_085_096[:3]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_Sersic_Gaussian_085_096, yy_new_Sersic_Gaussian_085_096, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_Sersic_Gaussian_085_096, y_data_Sersic_Gaussian_085_096, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_Sersic_Gaussian_085_096, yy_new_Sersic_Gaussian_085_096 - Sersic_Gauss(x_new_Sersic_Gaussian_085_096, *best_fit_params_Sersic_Gaussian_085_096[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_Sersic_Gaussian_085_096, (yy_new_Sersic_Gaussian_085_096 - Sersic_Gauss(x_new_Sersic_Gaussian_085_096, *best_fit_params_Sersic_Gaussian_085_096[:5]))/yy_new_Sersic_Gaussian_085_096, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_Sersic_Gaussian_085_096, Sersic_Gauss(x_new_Sersic_Gaussian_085_096, *Popt_Sersic_Gaussian_085_096[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_Sersic_Gaussian_085_096, Gaussian(x_new_Sersic_Gaussian_085_096, *Popt_Sersic_Gaussian_085_096[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_Sersic_Gaussian_085_096, Sersic_Fit(x_new_Sersic_Gaussian_085_096, *Popt_Sersic_Gaussian_085_096[:3]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_Sersic_Gaussian_085_096, yy_new_Sersic_Gaussian_085_096, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_Sersic_Gaussian_085_096, y_data_Sersic_Gaussian_085_096, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_Sersic_Gaussian_085_096, (yy_new_Sersic_Gaussian_085_096 - Sersic_Gauss(x_new_Sersic_Gaussian_085_096, *Popt_Sersic_Gaussian_085_096[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_Sersic_Gaussian_085_096, (yy_new_Sersic_Gaussian_085_096 - Sersic_Gauss(x_new_Sersic_Gaussian_085_096, *Popt_Sersic_Gaussian_085_096[:5]))/yy_new_Sersic_Gaussian_085_096, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian + Sersic - 0.85 < z < 0.96")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
I_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_085_096[0]
R_e_Sersic_035_045 = best_fit_params_Sersic_Gaussian_085_096[1]
N_Sersic_035_045 = best_fit_params_Sersic_Gaussian_085_096[2]
#############################################################################################################################################
print("I_e = " +str(best_fit_params_Sersic_Gaussian_085_096[0]) + " +" +str(errors_Sersic_Gaussian_085_096[0][0]) + " - " +str(errors_Sersic_Gaussian_085_096[0][1]))
print("")
print("R_e = " +str(best_fit_params_Sersic_Gaussian_085_096[1]) + " +" +str(errors_Sersic_Gaussian_085_096[1][0]) + " - " +str(errors_Sersic_Gaussian_085_096[1][1]))
print("")
print("N = " +str(best_fit_params_Sersic_Gaussian_085_096[2]) + " +" +str(errors_Sersic_Gaussian_085_096[2][0]) + " - " +str(errors_Sersic_Gaussian_085_096[2][1]))
print("")
print("A = " +str(best_fit_params_Sersic_Gaussian_085_096[3]) + " +" +str(errors_Sersic_Gaussian_085_096[3][0]) + " - " +str(errors_Sersic_Gaussian_085_096[3][1]))
print("")
print("Sigma = " +str(best_fit_params_Sersic_Gaussian_085_096[4]) + " +" +str(errors_Sersic_Gaussian_085_096[4][0]) + " - " +str(errors_Sersic_Gaussian_085_096[4][1]))

## <font color='#FF0B55' size=6 > Finding the area under the gaussian + sersic fit and finding the fraction of the two. Additionally, getting the galaxy component magnitude </font>

In [ ]:
Area_Under_Narrow_Sersic_Gaussian_025_035, Area_Under_Narrow_Sersic_Gaussian_SD_025_035 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_025_035[3], Popt_Sersic_Gaussian_025_035[4], Perr_Sersic_Gaussian_025_035[3], Perr_Sersic_Gaussian_025_035[4])
Area_Under_Narrow_Sersic_Gaussian_035_045, Area_Under_Narrow_Sersic_Gaussian_SD_035_045 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_035_045[3], Popt_Sersic_Gaussian_035_045[4], Perr_Sersic_Gaussian_035_045[3], Perr_Sersic_Gaussian_035_045[4])
Area_Under_Narrow_Sersic_Gaussian_045_055, Area_Under_Narrow_Sersic_Gaussian_SD_045_055 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_045_055[3], Popt_Sersic_Gaussian_045_055[4], Perr_Sersic_Gaussian_045_055[3], Perr_Sersic_Gaussian_045_055[4])
Area_Under_Narrow_Sersic_Gaussian_055_065, Area_Under_Narrow_Sersic_Gaussian_SD_055_065 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_055_065[3], Popt_Sersic_Gaussian_055_065[4], Perr_Sersic_Gaussian_055_065[3], Perr_Sersic_Gaussian_055_065[4])
Area_Under_Narrow_Sersic_Gaussian_065_075, Area_Under_Narrow_Sersic_Gaussian_SD_065_075 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_065_075[3], Popt_Sersic_Gaussian_065_075[4], Perr_Sersic_Gaussian_065_075[3], Perr_Sersic_Gaussian_065_075[4])
Area_Under_Narrow_Sersic_Gaussian_075_085, Area_Under_Narrow_Sersic_Gaussian_SD_075_085 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_075_085[3], Popt_Sersic_Gaussian_075_085[4], Perr_Sersic_Gaussian_075_085[3], Perr_Sersic_Gaussian_075_085[4])
Area_Under_Narrow_Sersic_Gaussian_085_096, Area_Under_Narrow_Sersic_Gaussian_SD_085_096 = monte_carlo_gaussian_area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_085_096[3], Popt_Sersic_Gaussian_085_096[4], Perr_Sersic_Gaussian_085_096[3], Perr_Sersic_Gaussian_085_096[4])

Area_Under_Broad_Sersic_Gaussian_025_035, Area_Under_Broad_Sersic_Gaussian_SD_025_035 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_025_035[0], Popt_Sersic_Gaussian_025_035[1], Popt_Sersic_Gaussian_025_035[2], Perr_Sersic_Gaussian_025_035[0], Perr_Sersic_Gaussian_025_035[1], n_std = Perr_Sersic_Gaussian_025_035[2])
Area_Under_Broad_Sersic_Gaussian_035_045, Area_Under_Broad_Sersic_Gaussian_SD_035_045 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_035_045[0], Popt_Sersic_Gaussian_035_045[1], Popt_Sersic_Gaussian_035_045[2], Perr_Sersic_Gaussian_035_045[0], Perr_Sersic_Gaussian_035_045[1], n_std = Perr_Sersic_Gaussian_035_045[2])
Area_Under_Broad_Sersic_Gaussian_045_055, Area_Under_Broad_Sersic_Gaussian_SD_045_055 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_045_055[0], Popt_Sersic_Gaussian_045_055[1], Popt_Sersic_Gaussian_045_055[2], Perr_Sersic_Gaussian_045_055[0], Perr_Sersic_Gaussian_045_055[1], n_std = Perr_Sersic_Gaussian_045_055[2])
Area_Under_Broad_Sersic_Gaussian_055_065, Area_Under_Broad_Sersic_Gaussian_SD_055_065 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_055_065[0], Popt_Sersic_Gaussian_055_065[1], Popt_Sersic_Gaussian_055_065[2], Perr_Sersic_Gaussian_055_065[0], Perr_Sersic_Gaussian_055_065[1], n_std = Perr_Sersic_Gaussian_055_065[2])
Area_Under_Broad_Sersic_Gaussian_065_075, Area_Under_Broad_Sersic_Gaussian_SD_065_075 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_065_075[0], Popt_Sersic_Gaussian_065_075[1], Popt_Sersic_Gaussian_065_075[2], Perr_Sersic_Gaussian_065_075[0], Perr_Sersic_Gaussian_065_075[1], n_std = Perr_Sersic_Gaussian_065_075[2])
Area_Under_Broad_Sersic_Gaussian_075_085, Area_Under_Broad_Sersic_Gaussian_SD_075_085 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_075_085[0], Popt_Sersic_Gaussian_075_085[1], Popt_Sersic_Gaussian_075_085[2], Perr_Sersic_Gaussian_075_085[0], Perr_Sersic_Gaussian_075_085[1], n_std = Perr_Sersic_Gaussian_075_085[2])
Area_Under_Broad_Sersic_Gaussian_085_096, Area_Under_Broad_Sersic_Gaussian_SD_085_096 = Monte_Carlo_Sersic_Area(x_new_Sersic_Gaussian_085_096, Popt_Sersic_Gaussian_085_096[0], Popt_Sersic_Gaussian_085_096[1], Popt_Sersic_Gaussian_085_096[2], Perr_Sersic_Gaussian_085_096[0], Perr_Sersic_Gaussian_085_096[1], n_std = Perr_Sersic_Gaussian_085_096[2])


In [ ]:
# Fractional mag for the OG fits (Fake errors)
Fractional_Area_Broad_Sersic_Gaussian_025_035, Fractional_Area_Broad_Sersic_Gaussian_SD_025_035 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_025_035, Area_Under_Narrow_Sersic_Gaussian_025_035, Area_Under_Broad_Sersic_Gaussian_SD_025_035, Area_Under_Narrow_Sersic_Gaussian_SD_025_035)
Fractional_Area_Broad_Sersic_Gaussian_035_045, Fractional_Area_Broad_Sersic_Gaussian_SD_035_045 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_035_045, Area_Under_Narrow_Sersic_Gaussian_035_045, Area_Under_Broad_Sersic_Gaussian_SD_035_045, Area_Under_Narrow_Sersic_Gaussian_SD_035_045)
Fractional_Area_Broad_Sersic_Gaussian_045_055, Fractional_Area_Broad_Sersic_Gaussian_SD_045_055 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_045_055, Area_Under_Narrow_Sersic_Gaussian_045_055, Area_Under_Broad_Sersic_Gaussian_SD_045_055, Area_Under_Narrow_Sersic_Gaussian_SD_045_055)
Fractional_Area_Broad_Sersic_Gaussian_055_065, Fractional_Area_Broad_Sersic_Gaussian_SD_055_065 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_055_065, Area_Under_Narrow_Sersic_Gaussian_055_065, Area_Under_Broad_Sersic_Gaussian_SD_055_065, Area_Under_Narrow_Sersic_Gaussian_SD_055_065)
Fractional_Area_Broad_Sersic_Gaussian_065_075, Fractional_Area_Broad_Sersic_Gaussian_SD_065_075 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_065_075, Area_Under_Narrow_Sersic_Gaussian_065_075, Area_Under_Broad_Sersic_Gaussian_SD_065_075, Area_Under_Narrow_Sersic_Gaussian_SD_065_075)
Fractional_Area_Broad_Sersic_Gaussian_075_085, Fractional_Area_Broad_Sersic_Gaussian_SD_075_085 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_075_085, Area_Under_Narrow_Sersic_Gaussian_075_085, Area_Under_Broad_Sersic_Gaussian_SD_075_085, Area_Under_Narrow_Sersic_Gaussian_SD_075_085)
Fractional_Area_Broad_Sersic_Gaussian_085_096, Fractional_Area_Broad_Sersic_Gaussian_SD_085_096 = fractional_area_uncertainty(Area_Under_Broad_Sersic_Gaussian_085_096, Area_Under_Narrow_Sersic_Gaussian_085_096, Area_Under_Broad_Sersic_Gaussian_SD_085_096, Area_Under_Narrow_Sersic_Gaussian_SD_085_096)


In [ ]:
# The Magnitude of the galaxy component for each redshift
Gal_Mag_Sersic_Gaussian_025_035, Gal_Mag_Sersic_Gaussian_SD_025_035 = gal_mag_uncertainty(Mag_Total_025_035, Mag_Total_SD_025_035, Fractional_Area_Broad_Sersic_Gaussian_025_035, Fractional_Area_Broad_Sersic_Gaussian_SD_025_035)
Gal_Mag_Sersic_Gaussian_035_045, Gal_Mag_Sersic_Gaussian_SD_035_045 = gal_mag_uncertainty(Mag_Total_035_045, Mag_Total_SD_035_045, Fractional_Area_Broad_Sersic_Gaussian_035_045, Fractional_Area_Broad_Sersic_Gaussian_SD_035_045)
Gal_Mag_Sersic_Gaussian_045_055, Gal_Mag_Sersic_Gaussian_SD_045_055 = gal_mag_uncertainty(Mag_Total_045_055, Mag_Total_SD_045_055, Fractional_Area_Broad_Sersic_Gaussian_045_055, Fractional_Area_Broad_Sersic_Gaussian_SD_045_055)
Gal_Mag_Sersic_Gaussian_055_065, Gal_Mag_Sersic_Gaussian_SD_055_065 = gal_mag_uncertainty(Mag_Total_055_065, Mag_Total_SD_055_065, Fractional_Area_Broad_Sersic_Gaussian_055_065, Fractional_Area_Broad_Sersic_Gaussian_SD_055_065)
Gal_Mag_Sersic_Gaussian_065_075, Gal_Mag_Sersic_Gaussian_SD_065_075 = gal_mag_uncertainty(Mag_Total_065_075, Mag_Total_SD_065_075, Fractional_Area_Broad_Sersic_Gaussian_065_075, Fractional_Area_Broad_Sersic_Gaussian_SD_065_075)
Gal_Mag_Sersic_Gaussian_075_085, Gal_Mag_Sersic_Gaussian_SD_075_085 = gal_mag_uncertainty(Mag_Total_075_085, Mag_Total_SD_075_085, Fractional_Area_Broad_Sersic_Gaussian_075_085, Fractional_Area_Broad_Sersic_Gaussian_SD_075_085)
Gal_Mag_Sersic_Gaussian_085_096, Gal_Mag_Sersic_Gaussian_SD_085_096 = gal_mag_uncertainty(Mag_Total_085_096, Mag_Total_SD_085_096, Fractional_Area_Broad_Sersic_Gaussian_085_096, Fractional_Area_Broad_Sersic_Gaussian_SD_085_096)


In [ ]:
Gal_Mag_Sersic_Gaussian_Array = np.array([Gal_Mag_Sersic_Gaussian_025_035, Gal_Mag_Sersic_Gaussian_035_045, Gal_Mag_Sersic_Gaussian_045_055, Gal_Mag_Sersic_Gaussian_055_065, Gal_Mag_Sersic_Gaussian_065_075, Gal_Mag_Sersic_Gaussian_075_085, Gal_Mag_Sersic_Gaussian_085_096])


In [ ]:
Gal_Mag_Sersic_Gaussian_SD_Array = np.array([Gal_Mag_Sersic_Gaussian_SD_025_035, Gal_Mag_Sersic_Gaussian_SD_035_045, Gal_Mag_Sersic_Gaussian_SD_045_055, Gal_Mag_Sersic_Gaussian_SD_055_065, Gal_Mag_Sersic_Gaussian_SD_065_075, Gal_Mag_Sersic_Gaussian_SD_075_085, Gal_Mag_Sersic_Gaussian_SD_085_096])


## <font color='#FF0B55' size=6 > Using a gaussian with fixed sigma (PSF/2.345) and a sersic with a fixed N of one to fit the radial profile </font>

In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_025_035 = Equal_Area_Radius_025_035_Array
x_new_SersicFixedN_GaussianFixedSigma_025_035 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_025_035),max(x_data_SersicFixedN_GaussianFixedSigma_025_035), 200)

y_data_SersicFixedN_GaussianFixedSigma_025_035 = np.array(Average_Counts_025_035)
y_new_SersicFixedN_GaussianFixedSigma_025_035 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_025_035, y_data_SersicFixedN_GaussianFixedSigma_025_035)

yy_new_SersicFixedN_GaussianFixedSigma_025_035 = y_new_SersicFixedN_GaussianFixedSigma_025_035(x_new_SersicFixedN_GaussianFixedSigma_025_035)


y_err_SersicFixedN_GaussianFixedSigma_025_035 = np.array(Average_Counts_SD_025_035)
y_new_err_SersicFixedN_GaussianFixedSigma_025_035 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_025_035, y_err_SersicFixedN_GaussianFixedSigma_025_035)

yy_new_err_SersicFixedN_GaussianFixedSigma_025_035 = y_new_err_SersicFixedN_GaussianFixedSigma_025_035(x_new_SersicFixedN_GaussianFixedSigma_025_035)

#From Popt --- I_e, R_e, A = [5.34629296 1.39269926 4.10350014]
#From Popt Standard Deviation --- I_e, R_e, A = [0.09329254 0.02394581 0.7780253 ]

i_e_SersicFixedN_GaussianFixedSigma_025_035 = 5.34629296
r_e_SersicFixedN_GaussianFixedSigma_025_035 = 1.39269926

a_SersicFixedN_GaussianFixedSigma_025_035 = 4.10350014


initial_Mirror_SersicFixedN_GaussianFixedSigma_025_035 = np.array([i_e_SersicFixedN_GaussianFixedSigma_025_035, r_e_SersicFixedN_GaussianFixedSigma_025_035, a_SersicFixedN_GaussianFixedSigma_025_035]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_025_035 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_025_035 = i_e_SersicFixedN_GaussianFixedSigma_025_035 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_025_035)

R_e_l_SersicFixedN_GaussianFixedSigma_025_035 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_025_035 = r_e_SersicFixedN_GaussianFixedSigma_025_035 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_025_035)

a_l_SersicFixedN_GaussianFixedSigma_025_035 = 0
a_u_SersicFixedN_GaussianFixedSigma_025_035 = a_SersicFixedN_GaussianFixedSigma_025_035 + (0.01*a_SersicFixedN_GaussianFixedSigma_025_035)


Popt_SersicFixedN_GaussianFixedSigma_025_035, Pcov_SersicFixedN_GaussianFixedSigma_025_035 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_SersicFixedN_GaussianFixedSigma_025_035, p0=[i_e_SersicFixedN_GaussianFixedSigma_025_035, r_e_SersicFixedN_GaussianFixedSigma_025_035, a_SersicFixedN_GaussianFixedSigma_025_035], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_025_035 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_025_035))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_025_035, r_e_SersicFixedN_GaussianFixedSigma_025_035, a_SersicFixedN_GaussianFixedSigma_025_035 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_025_035, r_e_SersicFixedN_GaussianFixedSigma_025_035, a_SersicFixedN_GaussianFixedSigma_025_035)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_025_035, a_SersicFixedN_GaussianFixedSigma_025_035 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_025_035 < i_e_SersicFixedN_GaussianFixedSigma_025_035 < I_e_u_SersicFixedN_GaussianFixedSigma_025_035 and R_e_l_SersicFixedN_GaussianFixedSigma_025_035 < r_e_SersicFixedN_GaussianFixedSigma_025_035 < R_e_u_SersicFixedN_GaussianFixedSigma_025_035 and a_l_SersicFixedN_GaussianFixedSigma_025_035 < a_SersicFixedN_GaussianFixedSigma_025_035 < a_u_SersicFixedN_GaussianFixedSigma_025_035):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_025_035 = [(I_e_l_SersicFixedN_GaussianFixedSigma_025_035, I_e_u_SersicFixedN_GaussianFixedSigma_025_035), (R_e_l_SersicFixedN_GaussianFixedSigma_025_035, R_e_u_SersicFixedN_GaussianFixedSigma_025_035), (a_l_SersicFixedN_GaussianFixedSigma_025_035, a_u_SersicFixedN_GaussianFixedSigma_025_035)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_025_035, args=(x_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_err_SersicFixedN_GaussianFixedSigma_025_035), bounds=bounds_SersicFixedN_GaussianFixedSigma_025_035)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_025_035 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_025_035, ndim_SersicFixedN_GaussianFixedSigma_025_035 = pos_SersicFixedN_GaussianFixedSigma_025_035.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_025_035 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_025_035, ndim_SersicFixedN_GaussianFixedSigma_025_035, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_err_SersicFixedN_GaussianFixedSigma_025_035))
sampler_SersicFixedN_GaussianFixedSigma_025_035.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_025_035, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_025_035 = sampler_SersicFixedN_GaussianFixedSigma_025_035.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_025_035 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_025_035):
    mcmc_SersicFixedN_GaussianFixedSigma_025_035 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_025_035[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_025_035.append(mcmc_SersicFixedN_GaussianFixedSigma_025_035[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_025_035 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_025_035):
     mcmc_SersicFixedN_GaussianFixedSigma_025_035 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_025_035[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_025_035.append([mcmc_SersicFixedN_GaussianFixedSigma_025_035[2]-mcmc_SersicFixedN_GaussianFixedSigma_025_035[1],mcmc_SersicFixedN_GaussianFixedSigma_025_035[1]-mcmc_SersicFixedN_GaussianFixedSigma_025_035[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_025_035))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_025_035))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_025_035, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_025_035, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_025_035, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_025_035, *best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_SersicFixedN_GaussianFixedSigma_025_035, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_025_035, y_data_SersicFixedN_GaussianFixedSigma_025_035, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_SersicFixedN_GaussianFixedSigma_025_035 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_025_035, (yy_new_SersicFixedN_GaussianFixedSigma_025_035 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_025_035, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_025_035, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *Popt_SersicFixedN_GaussianFixedSigma_025_035[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_025_035, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *Popt_SersicFixedN_GaussianFixedSigma_025_035[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_025_035, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_025_035, *Popt_SersicFixedN_GaussianFixedSigma_025_035[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_025_035, yy_new_SersicFixedN_GaussianFixedSigma_025_035, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_025_035, y_data_SersicFixedN_GaussianFixedSigma_025_035, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_025_035, (yy_new_SersicFixedN_GaussianFixedSigma_025_035 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *Popt_SersicFixedN_GaussianFixedSigma_025_035[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_025_035, (yy_new_SersicFixedN_GaussianFixedSigma_025_035 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_025_035, *Popt_SersicFixedN_GaussianFixedSigma_025_035[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_025_035, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.25 < z < 0.35")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_025_035[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_025_035[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_025_035[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_025_035[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_025_035[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_025_035[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_025_035[2][1]))



In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_035_045 = Equal_Area_Radius_035_045_Array
x_new_SersicFixedN_GaussianFixedSigma_035_045 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_035_045),max(x_data_SersicFixedN_GaussianFixedSigma_035_045), 200)

y_data_SersicFixedN_GaussianFixedSigma_035_045 = np.array(Average_Counts_035_045)
y_new_SersicFixedN_GaussianFixedSigma_035_045 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_035_045, y_data_SersicFixedN_GaussianFixedSigma_035_045)

yy_new_SersicFixedN_GaussianFixedSigma_035_045 = y_new_SersicFixedN_GaussianFixedSigma_035_045(x_new_SersicFixedN_GaussianFixedSigma_035_045)


y_err_SersicFixedN_GaussianFixedSigma_035_045 = np.array(Average_Counts_SD_035_045)
y_new_err_SersicFixedN_GaussianFixedSigma_035_045 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_035_045, y_err_SersicFixedN_GaussianFixedSigma_035_045)

yy_new_err_SersicFixedN_GaussianFixedSigma_035_045 = y_new_err_SersicFixedN_GaussianFixedSigma_035_045(x_new_SersicFixedN_GaussianFixedSigma_035_045)

#From Popt --- I_e, R_e, A = [2.64595103 1.39112177 2.79272094]
#From Popt Standard Deviation --- I_e, R_e, A = [0.04577467 0.02370381 0.38144615]

i_e_SersicFixedN_GaussianFixedSigma_035_045 = 2.64595103
r_e_SersicFixedN_GaussianFixedSigma_035_045 = 1.39112177

a_SersicFixedN_GaussianFixedSigma_035_045 = 2.79272094


initial_Mirror_SersicFixedN_GaussianFixedSigma_035_045 = np.array([i_e_SersicFixedN_GaussianFixedSigma_035_045, r_e_SersicFixedN_GaussianFixedSigma_035_045, a_SersicFixedN_GaussianFixedSigma_035_045]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_035_045 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_035_045 = i_e_SersicFixedN_GaussianFixedSigma_035_045 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_035_045)

R_e_l_SersicFixedN_GaussianFixedSigma_035_045 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_035_045 = r_e_SersicFixedN_GaussianFixedSigma_035_045 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_035_045)

a_l_SersicFixedN_GaussianFixedSigma_035_045 = 0
a_u_SersicFixedN_GaussianFixedSigma_035_045 = a_SersicFixedN_GaussianFixedSigma_035_045 + (0.01*a_SersicFixedN_GaussianFixedSigma_035_045)


Popt_SersicFixedN_GaussianFixedSigma_035_045, Pcov_SersicFixedN_GaussianFixedSigma_035_045 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_SersicFixedN_GaussianFixedSigma_035_045, p0=[i_e_SersicFixedN_GaussianFixedSigma_035_045, r_e_SersicFixedN_GaussianFixedSigma_035_045, a_SersicFixedN_GaussianFixedSigma_035_045], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_035_045 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_035_045))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_035_045, r_e_SersicFixedN_GaussianFixedSigma_035_045, a_SersicFixedN_GaussianFixedSigma_035_045 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_035_045, r_e_SersicFixedN_GaussianFixedSigma_035_045, a_SersicFixedN_GaussianFixedSigma_035_045)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_035_045, a_SersicFixedN_GaussianFixedSigma_035_045 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_035_045 < i_e_SersicFixedN_GaussianFixedSigma_035_045 < I_e_u_SersicFixedN_GaussianFixedSigma_035_045 and R_e_l_SersicFixedN_GaussianFixedSigma_035_045 < r_e_SersicFixedN_GaussianFixedSigma_035_045 < R_e_u_SersicFixedN_GaussianFixedSigma_035_045 and a_l_SersicFixedN_GaussianFixedSigma_035_045 < a_SersicFixedN_GaussianFixedSigma_035_045 < a_u_SersicFixedN_GaussianFixedSigma_035_045):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_035_045 = [(I_e_l_SersicFixedN_GaussianFixedSigma_035_045, I_e_u_SersicFixedN_GaussianFixedSigma_035_045), (R_e_l_SersicFixedN_GaussianFixedSigma_035_045, R_e_u_SersicFixedN_GaussianFixedSigma_035_045), (a_l_SersicFixedN_GaussianFixedSigma_035_045, a_u_SersicFixedN_GaussianFixedSigma_035_045)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_035_045, args=(x_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_err_SersicFixedN_GaussianFixedSigma_035_045), bounds=bounds_SersicFixedN_GaussianFixedSigma_035_045)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_035_045 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_035_045, ndim_SersicFixedN_GaussianFixedSigma_035_045 = pos_SersicFixedN_GaussianFixedSigma_035_045.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_035_045 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_035_045, ndim_SersicFixedN_GaussianFixedSigma_035_045, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_err_SersicFixedN_GaussianFixedSigma_035_045))
sampler_SersicFixedN_GaussianFixedSigma_035_045.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_035_045, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_035_045 = sampler_SersicFixedN_GaussianFixedSigma_035_045.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_035_045 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_035_045):
    mcmc_SersicFixedN_GaussianFixedSigma_035_045 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_035_045[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_035_045.append(mcmc_SersicFixedN_GaussianFixedSigma_035_045[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_035_045 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_035_045):
     mcmc_SersicFixedN_GaussianFixedSigma_035_045 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_035_045[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_035_045.append([mcmc_SersicFixedN_GaussianFixedSigma_035_045[2]-mcmc_SersicFixedN_GaussianFixedSigma_035_045[1],mcmc_SersicFixedN_GaussianFixedSigma_035_045[1]-mcmc_SersicFixedN_GaussianFixedSigma_035_045[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_035_045))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_035_045))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_035_045, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_035_045, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_035_045, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_035_045, *best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_SersicFixedN_GaussianFixedSigma_035_045, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_035_045, y_data_SersicFixedN_GaussianFixedSigma_035_045, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_SersicFixedN_GaussianFixedSigma_035_045 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_035_045, (yy_new_SersicFixedN_GaussianFixedSigma_035_045 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_035_045, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_035_045, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *Popt_SersicFixedN_GaussianFixedSigma_035_045[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_035_045, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *Popt_SersicFixedN_GaussianFixedSigma_035_045[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_035_045, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_035_045, *Popt_SersicFixedN_GaussianFixedSigma_035_045[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_035_045, yy_new_SersicFixedN_GaussianFixedSigma_035_045, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_035_045, y_data_SersicFixedN_GaussianFixedSigma_035_045, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_035_045, (yy_new_SersicFixedN_GaussianFixedSigma_035_045 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *Popt_SersicFixedN_GaussianFixedSigma_035_045[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_035_045, (yy_new_SersicFixedN_GaussianFixedSigma_035_045 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_035_045, *Popt_SersicFixedN_GaussianFixedSigma_035_045[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_035_045, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.35 < z < 0.45")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_035_045[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_035_045[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_035_045[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_035_045[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_035_045[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_035_045[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_035_045[2][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_045_055 = Equal_Area_Radius_045_055_Array
x_new_SersicFixedN_GaussianFixedSigma_045_055 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_045_055),max(x_data_SersicFixedN_GaussianFixedSigma_045_055), 200)

y_data_SersicFixedN_GaussianFixedSigma_045_055 = np.array(Average_Counts_045_055)
y_new_SersicFixedN_GaussianFixedSigma_045_055 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_045_055, y_data_SersicFixedN_GaussianFixedSigma_045_055)

yy_new_SersicFixedN_GaussianFixedSigma_045_055 = y_new_SersicFixedN_GaussianFixedSigma_045_055(x_new_SersicFixedN_GaussianFixedSigma_045_055)


y_err_SersicFixedN_GaussianFixedSigma_045_055 = np.array(Average_Counts_SD_045_055)
y_new_err_SersicFixedN_GaussianFixedSigma_045_055 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_045_055, y_err_SersicFixedN_GaussianFixedSigma_045_055)

yy_new_err_SersicFixedN_GaussianFixedSigma_045_055 = y_new_err_SersicFixedN_GaussianFixedSigma_045_055(x_new_SersicFixedN_GaussianFixedSigma_045_055)

#From Popt --- I_e, R_e, A = [2.11710903 1.27258875 1.00297097]
#From Popt Standard Deviation --- I_e, R_e, A = [0.03543995 0.02032712 0.277624  ]

i_e_SersicFixedN_GaussianFixedSigma_045_055 = 2.11710903
r_e_SersicFixedN_GaussianFixedSigma_045_055 = 1.27258875

a_SersicFixedN_GaussianFixedSigma_045_055 = 1.00297097


initial_Mirror_SersicFixedN_GaussianFixedSigma_045_055 = np.array([i_e_SersicFixedN_GaussianFixedSigma_045_055, r_e_SersicFixedN_GaussianFixedSigma_045_055, a_SersicFixedN_GaussianFixedSigma_045_055]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_045_055 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_045_055 = i_e_SersicFixedN_GaussianFixedSigma_045_055 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_045_055)

R_e_l_SersicFixedN_GaussianFixedSigma_045_055 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_045_055 = r_e_SersicFixedN_GaussianFixedSigma_045_055 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_045_055)

a_l_SersicFixedN_GaussianFixedSigma_045_055 = 0
a_u_SersicFixedN_GaussianFixedSigma_045_055 = a_SersicFixedN_GaussianFixedSigma_045_055 + (0.01*a_SersicFixedN_GaussianFixedSigma_045_055)


Popt_SersicFixedN_GaussianFixedSigma_045_055, Pcov_SersicFixedN_GaussianFixedSigma_045_055 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_SersicFixedN_GaussianFixedSigma_045_055, p0=[i_e_SersicFixedN_GaussianFixedSigma_045_055, r_e_SersicFixedN_GaussianFixedSigma_045_055, a_SersicFixedN_GaussianFixedSigma_045_055], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_045_055 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_045_055))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_045_055, r_e_SersicFixedN_GaussianFixedSigma_045_055, a_SersicFixedN_GaussianFixedSigma_045_055 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_045_055, r_e_SersicFixedN_GaussianFixedSigma_045_055, a_SersicFixedN_GaussianFixedSigma_045_055)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_045_055, a_SersicFixedN_GaussianFixedSigma_045_055 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_045_055 < i_e_SersicFixedN_GaussianFixedSigma_045_055 < I_e_u_SersicFixedN_GaussianFixedSigma_045_055 and R_e_l_SersicFixedN_GaussianFixedSigma_045_055 < r_e_SersicFixedN_GaussianFixedSigma_045_055 < R_e_u_SersicFixedN_GaussianFixedSigma_045_055 and a_l_SersicFixedN_GaussianFixedSigma_045_055 < a_SersicFixedN_GaussianFixedSigma_045_055 < a_u_SersicFixedN_GaussianFixedSigma_045_055):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_045_055 = [(I_e_l_SersicFixedN_GaussianFixedSigma_045_055, I_e_u_SersicFixedN_GaussianFixedSigma_045_055), (R_e_l_SersicFixedN_GaussianFixedSigma_045_055, R_e_u_SersicFixedN_GaussianFixedSigma_045_055), (a_l_SersicFixedN_GaussianFixedSigma_045_055, a_u_SersicFixedN_GaussianFixedSigma_045_055)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_045_055, args=(x_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_err_SersicFixedN_GaussianFixedSigma_045_055), bounds=bounds_SersicFixedN_GaussianFixedSigma_045_055)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_045_055 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_045_055, ndim_SersicFixedN_GaussianFixedSigma_045_055 = pos_SersicFixedN_GaussianFixedSigma_045_055.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_045_055 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_045_055, ndim_SersicFixedN_GaussianFixedSigma_045_055, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_err_SersicFixedN_GaussianFixedSigma_045_055))
sampler_SersicFixedN_GaussianFixedSigma_045_055.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_045_055, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_045_055 = sampler_SersicFixedN_GaussianFixedSigma_045_055.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_045_055 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_045_055):
    mcmc_SersicFixedN_GaussianFixedSigma_045_055 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_045_055[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_045_055.append(mcmc_SersicFixedN_GaussianFixedSigma_045_055[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_045_055 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_045_055):
     mcmc_SersicFixedN_GaussianFixedSigma_045_055 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_045_055[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_045_055.append([mcmc_SersicFixedN_GaussianFixedSigma_045_055[2]-mcmc_SersicFixedN_GaussianFixedSigma_045_055[1],mcmc_SersicFixedN_GaussianFixedSigma_045_055[1]-mcmc_SersicFixedN_GaussianFixedSigma_045_055[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_045_055))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_045_055))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_045_055, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_045_055, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_045_055, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_045_055, *best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_SersicFixedN_GaussianFixedSigma_045_055, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_045_055, y_data_SersicFixedN_GaussianFixedSigma_045_055, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_SersicFixedN_GaussianFixedSigma_045_055 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_045_055, (yy_new_SersicFixedN_GaussianFixedSigma_045_055 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_045_055, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_045_055, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *Popt_SersicFixedN_GaussianFixedSigma_045_055[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_045_055, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *Popt_SersicFixedN_GaussianFixedSigma_045_055[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_045_055, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_045_055, *Popt_SersicFixedN_GaussianFixedSigma_045_055[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_045_055, yy_new_SersicFixedN_GaussianFixedSigma_045_055, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_045_055, y_data_SersicFixedN_GaussianFixedSigma_045_055, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_045_055, (yy_new_SersicFixedN_GaussianFixedSigma_045_055 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *Popt_SersicFixedN_GaussianFixedSigma_045_055[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_045_055, (yy_new_SersicFixedN_GaussianFixedSigma_045_055 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_045_055, *Popt_SersicFixedN_GaussianFixedSigma_045_055[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_045_055, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.45 < z < 0.55")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_045_055[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_045_055[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_045_055[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_045_055[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_045_055[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_045_055[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_045_055[2][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_055_065 = Equal_Area_Radius_055_065_Array
x_new_SersicFixedN_GaussianFixedSigma_055_065 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_055_065),max(x_data_SersicFixedN_GaussianFixedSigma_055_065), 200)

y_data_SersicFixedN_GaussianFixedSigma_055_065 = np.array(Average_Counts_055_065)
y_new_SersicFixedN_GaussianFixedSigma_055_065 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_055_065, y_data_SersicFixedN_GaussianFixedSigma_055_065)

yy_new_SersicFixedN_GaussianFixedSigma_055_065 = y_new_SersicFixedN_GaussianFixedSigma_055_065(x_new_SersicFixedN_GaussianFixedSigma_055_065)


y_err_SersicFixedN_GaussianFixedSigma_055_065 = np.array(Average_Counts_SD_055_065)
y_new_err_SersicFixedN_GaussianFixedSigma_055_065 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_055_065, y_err_SersicFixedN_GaussianFixedSigma_055_065)

yy_new_err_SersicFixedN_GaussianFixedSigma_055_065 = y_new_err_SersicFixedN_GaussianFixedSigma_055_065(x_new_SersicFixedN_GaussianFixedSigma_055_065)

#From Popt --- I_e, R_e, A = [2.41825068 1.09218047 1.03569826]
#From Popt Standard Deviation --- I_e, R_e, A = [0.04800456 0.01947496 0.33740315]

i_e_SersicFixedN_GaussianFixedSigma_055_065 = 2.41825068
r_e_SersicFixedN_GaussianFixedSigma_055_065 = 1.09218047

a_SersicFixedN_GaussianFixedSigma_055_065 = 1.03569826


initial_Mirror_SersicFixedN_GaussianFixedSigma_055_065 = np.array([i_e_SersicFixedN_GaussianFixedSigma_055_065, r_e_SersicFixedN_GaussianFixedSigma_055_065, a_SersicFixedN_GaussianFixedSigma_055_065]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_055_065 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_055_065 = i_e_SersicFixedN_GaussianFixedSigma_055_065 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_055_065)

R_e_l_SersicFixedN_GaussianFixedSigma_055_065 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_055_065 = r_e_SersicFixedN_GaussianFixedSigma_055_065 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_055_065)

a_l_SersicFixedN_GaussianFixedSigma_055_065 = 0
a_u_SersicFixedN_GaussianFixedSigma_055_065 = a_SersicFixedN_GaussianFixedSigma_055_065 + (0.01*a_SersicFixedN_GaussianFixedSigma_055_065)


Popt_SersicFixedN_GaussianFixedSigma_055_065, Pcov_SersicFixedN_GaussianFixedSigma_055_065 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_SersicFixedN_GaussianFixedSigma_055_065, p0=[i_e_SersicFixedN_GaussianFixedSigma_055_065, r_e_SersicFixedN_GaussianFixedSigma_055_065, a_SersicFixedN_GaussianFixedSigma_055_065], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_055_065 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_055_065))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_055_065, r_e_SersicFixedN_GaussianFixedSigma_055_065, a_SersicFixedN_GaussianFixedSigma_055_065 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_055_065, r_e_SersicFixedN_GaussianFixedSigma_055_065, a_SersicFixedN_GaussianFixedSigma_055_065)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_055_065, a_SersicFixedN_GaussianFixedSigma_055_065 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_055_065 < i_e_SersicFixedN_GaussianFixedSigma_055_065 < I_e_u_SersicFixedN_GaussianFixedSigma_055_065 and R_e_l_SersicFixedN_GaussianFixedSigma_055_065 < r_e_SersicFixedN_GaussianFixedSigma_055_065 < R_e_u_SersicFixedN_GaussianFixedSigma_055_065 and a_l_SersicFixedN_GaussianFixedSigma_055_065 < a_SersicFixedN_GaussianFixedSigma_055_065 < a_u_SersicFixedN_GaussianFixedSigma_055_065):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_055_065 = [(I_e_l_SersicFixedN_GaussianFixedSigma_055_065, I_e_u_SersicFixedN_GaussianFixedSigma_055_065), (R_e_l_SersicFixedN_GaussianFixedSigma_055_065, R_e_u_SersicFixedN_GaussianFixedSigma_055_065), (a_l_SersicFixedN_GaussianFixedSigma_055_065, a_u_SersicFixedN_GaussianFixedSigma_055_065)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_055_065, args=(x_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_err_SersicFixedN_GaussianFixedSigma_055_065), bounds=bounds_SersicFixedN_GaussianFixedSigma_055_065)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_055_065 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_055_065, ndim_SersicFixedN_GaussianFixedSigma_055_065 = pos_SersicFixedN_GaussianFixedSigma_055_065.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_055_065 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_055_065, ndim_SersicFixedN_GaussianFixedSigma_055_065, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_err_SersicFixedN_GaussianFixedSigma_055_065))
sampler_SersicFixedN_GaussianFixedSigma_055_065.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_055_065, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_055_065 = sampler_SersicFixedN_GaussianFixedSigma_055_065.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_055_065 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_055_065):
    mcmc_SersicFixedN_GaussianFixedSigma_055_065 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_055_065[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_055_065.append(mcmc_SersicFixedN_GaussianFixedSigma_055_065[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_055_065 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_055_065):
     mcmc_SersicFixedN_GaussianFixedSigma_055_065 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_055_065[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_055_065.append([mcmc_SersicFixedN_GaussianFixedSigma_055_065[2]-mcmc_SersicFixedN_GaussianFixedSigma_055_065[1],mcmc_SersicFixedN_GaussianFixedSigma_055_065[1]-mcmc_SersicFixedN_GaussianFixedSigma_055_065[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_055_065))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_055_065))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_055_065, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_055_065, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_055_065, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_055_065, *best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_SersicFixedN_GaussianFixedSigma_055_065, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_055_065, y_data_SersicFixedN_GaussianFixedSigma_055_065, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_SersicFixedN_GaussianFixedSigma_055_065 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_055_065, (yy_new_SersicFixedN_GaussianFixedSigma_055_065 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_055_065, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_055_065, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *Popt_SersicFixedN_GaussianFixedSigma_055_065[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_055_065, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *Popt_SersicFixedN_GaussianFixedSigma_055_065[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_055_065, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_055_065, *Popt_SersicFixedN_GaussianFixedSigma_055_065[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_055_065, yy_new_SersicFixedN_GaussianFixedSigma_055_065, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_055_065, y_data_SersicFixedN_GaussianFixedSigma_055_065, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_055_065, (yy_new_SersicFixedN_GaussianFixedSigma_055_065 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *Popt_SersicFixedN_GaussianFixedSigma_055_065[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_055_065, (yy_new_SersicFixedN_GaussianFixedSigma_055_065 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_055_065, *Popt_SersicFixedN_GaussianFixedSigma_055_065[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_055_065, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.55 < z < 0.65")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_055_065[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_055_065[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_055_065[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_055_065[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_055_065[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_055_065[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_055_065[2][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_065_075 = Equal_Area_Radius_065_075_Array
x_new_SersicFixedN_GaussianFixedSigma_065_075 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_065_075),max(x_data_SersicFixedN_GaussianFixedSigma_065_075), 200)

y_data_SersicFixedN_GaussianFixedSigma_065_075 = np.array(Average_Counts_065_075)
y_new_SersicFixedN_GaussianFixedSigma_065_075 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_065_075, y_data_SersicFixedN_GaussianFixedSigma_065_075)

yy_new_SersicFixedN_GaussianFixedSigma_065_075 = y_new_SersicFixedN_GaussianFixedSigma_065_075(x_new_SersicFixedN_GaussianFixedSigma_065_075)


y_err_SersicFixedN_GaussianFixedSigma_065_075 = np.array(Average_Counts_SD_065_075)
y_new_err_SersicFixedN_GaussianFixedSigma_065_075 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_065_075, y_err_SersicFixedN_GaussianFixedSigma_065_075)

yy_new_err_SersicFixedN_GaussianFixedSigma_065_075 = y_new_err_SersicFixedN_GaussianFixedSigma_065_075(x_new_SersicFixedN_GaussianFixedSigma_065_075)

#From Popt --- I_e, R_e, A = [2.74565468 1.04782953 0.30556623]
#From Popt Standard Deviation --- I_e, R_e, A = [0.05469371 0.01841891 0.37309308]

i_e_SersicFixedN_GaussianFixedSigma_065_075 = 2.74565468
r_e_SersicFixedN_GaussianFixedSigma_065_075 = 1.04782953

a_SersicFixedN_GaussianFixedSigma_065_075 = 0.30556623


initial_Mirror_SersicFixedN_GaussianFixedSigma_065_075 = np.array([i_e_SersicFixedN_GaussianFixedSigma_065_075, r_e_SersicFixedN_GaussianFixedSigma_065_075, a_SersicFixedN_GaussianFixedSigma_065_075]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_065_075 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_065_075 = i_e_SersicFixedN_GaussianFixedSigma_065_075 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_065_075)

R_e_l_SersicFixedN_GaussianFixedSigma_065_075 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_065_075 = r_e_SersicFixedN_GaussianFixedSigma_065_075 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_065_075)

a_l_SersicFixedN_GaussianFixedSigma_065_075 = 0
a_u_SersicFixedN_GaussianFixedSigma_065_075 = a_SersicFixedN_GaussianFixedSigma_065_075 + (0.01*a_SersicFixedN_GaussianFixedSigma_065_075)


Popt_SersicFixedN_GaussianFixedSigma_065_075, Pcov_SersicFixedN_GaussianFixedSigma_065_075 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_SersicFixedN_GaussianFixedSigma_065_075, p0=[i_e_SersicFixedN_GaussianFixedSigma_065_075, r_e_SersicFixedN_GaussianFixedSigma_065_075, a_SersicFixedN_GaussianFixedSigma_065_075], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_065_075 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_065_075))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_065_075, r_e_SersicFixedN_GaussianFixedSigma_065_075, a_SersicFixedN_GaussianFixedSigma_065_075 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_065_075, r_e_SersicFixedN_GaussianFixedSigma_065_075, a_SersicFixedN_GaussianFixedSigma_065_075)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_065_075, a_SersicFixedN_GaussianFixedSigma_065_075 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_065_075 < i_e_SersicFixedN_GaussianFixedSigma_065_075 < I_e_u_SersicFixedN_GaussianFixedSigma_065_075 and R_e_l_SersicFixedN_GaussianFixedSigma_065_075 < r_e_SersicFixedN_GaussianFixedSigma_065_075 < R_e_u_SersicFixedN_GaussianFixedSigma_065_075 and a_l_SersicFixedN_GaussianFixedSigma_065_075 < a_SersicFixedN_GaussianFixedSigma_065_075 < a_u_SersicFixedN_GaussianFixedSigma_065_075):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_065_075 = [(I_e_l_SersicFixedN_GaussianFixedSigma_065_075, I_e_u_SersicFixedN_GaussianFixedSigma_065_075), (R_e_l_SersicFixedN_GaussianFixedSigma_065_075, R_e_u_SersicFixedN_GaussianFixedSigma_065_075), (a_l_SersicFixedN_GaussianFixedSigma_065_075, a_u_SersicFixedN_GaussianFixedSigma_065_075)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_065_075, args=(x_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_err_SersicFixedN_GaussianFixedSigma_065_075), bounds=bounds_SersicFixedN_GaussianFixedSigma_065_075)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_065_075 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_065_075, ndim_SersicFixedN_GaussianFixedSigma_065_075 = pos_SersicFixedN_GaussianFixedSigma_065_075.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_065_075 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_065_075, ndim_SersicFixedN_GaussianFixedSigma_065_075, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_err_SersicFixedN_GaussianFixedSigma_065_075))
sampler_SersicFixedN_GaussianFixedSigma_065_075.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_065_075, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_065_075 = sampler_SersicFixedN_GaussianFixedSigma_065_075.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_065_075 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_065_075):
    mcmc_SersicFixedN_GaussianFixedSigma_065_075 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_065_075[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_065_075.append(mcmc_SersicFixedN_GaussianFixedSigma_065_075[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_065_075 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_065_075):
     mcmc_SersicFixedN_GaussianFixedSigma_065_075 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_065_075[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_065_075.append([mcmc_SersicFixedN_GaussianFixedSigma_065_075[2]-mcmc_SersicFixedN_GaussianFixedSigma_065_075[1],mcmc_SersicFixedN_GaussianFixedSigma_065_075[1]-mcmc_SersicFixedN_GaussianFixedSigma_065_075[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_065_075))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_065_075))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_065_075, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_065_075, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_065_075, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_065_075, *best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_SersicFixedN_GaussianFixedSigma_065_075, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_065_075, y_data_SersicFixedN_GaussianFixedSigma_065_075, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_SersicFixedN_GaussianFixedSigma_065_075 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_065_075, (yy_new_SersicFixedN_GaussianFixedSigma_065_075 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_065_075, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_065_075, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *Popt_SersicFixedN_GaussianFixedSigma_065_075[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_065_075, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *Popt_SersicFixedN_GaussianFixedSigma_065_075[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_065_075, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_065_075, *Popt_SersicFixedN_GaussianFixedSigma_065_075[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_065_075, yy_new_SersicFixedN_GaussianFixedSigma_065_075, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_065_075, y_data_SersicFixedN_GaussianFixedSigma_065_075, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_065_075, (yy_new_SersicFixedN_GaussianFixedSigma_065_075 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *Popt_SersicFixedN_GaussianFixedSigma_065_075[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_065_075, (yy_new_SersicFixedN_GaussianFixedSigma_065_075 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_065_075, *Popt_SersicFixedN_GaussianFixedSigma_065_075[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_065_075, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.65 < z < 0.75")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_065_075[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_065_075[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_065_075[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_065_075[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_065_075[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_065_075[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_065_075[2][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_075_085 = Equal_Area_Radius_075_085_Array
x_new_SersicFixedN_GaussianFixedSigma_075_085 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_075_085),max(x_data_SersicFixedN_GaussianFixedSigma_075_085), 200)

y_data_SersicFixedN_GaussianFixedSigma_075_085 = np.array(Average_Counts_075_085)
y_new_SersicFixedN_GaussianFixedSigma_075_085 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_075_085, y_data_SersicFixedN_GaussianFixedSigma_075_085)

yy_new_SersicFixedN_GaussianFixedSigma_075_085 = y_new_SersicFixedN_GaussianFixedSigma_075_085(x_new_SersicFixedN_GaussianFixedSigma_075_085)


y_err_SersicFixedN_GaussianFixedSigma_075_085 = np.array(Average_Counts_SD_075_085)
y_new_err_SersicFixedN_GaussianFixedSigma_075_085 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_075_085, y_err_SersicFixedN_GaussianFixedSigma_075_085)

yy_new_err_SersicFixedN_GaussianFixedSigma_075_085 = y_new_err_SersicFixedN_GaussianFixedSigma_075_085(x_new_SersicFixedN_GaussianFixedSigma_075_085)

#From Popt --- I_e, R_e, A = [2.51301854 1.02458463 0.31195577]
#From Popt Standard Deviation --- I_e, R_e, A = [0.0492029  0.01752808 0.3302217 ]

i_e_SersicFixedN_GaussianFixedSigma_075_085 = 2.51301854
r_e_SersicFixedN_GaussianFixedSigma_075_085 = 1.02458463

a_SersicFixedN_GaussianFixedSigma_075_085 = 0.31195577


initial_Mirror_SersicFixedN_GaussianFixedSigma_075_085 = np.array([i_e_SersicFixedN_GaussianFixedSigma_075_085, r_e_SersicFixedN_GaussianFixedSigma_075_085, a_SersicFixedN_GaussianFixedSigma_075_085]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_075_085 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_075_085 = i_e_SersicFixedN_GaussianFixedSigma_075_085 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_075_085)

R_e_l_SersicFixedN_GaussianFixedSigma_075_085 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_075_085 = r_e_SersicFixedN_GaussianFixedSigma_075_085 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_075_085)

a_l_SersicFixedN_GaussianFixedSigma_075_085 = 0
a_u_SersicFixedN_GaussianFixedSigma_075_085 = a_SersicFixedN_GaussianFixedSigma_075_085 + (0.01*a_SersicFixedN_GaussianFixedSigma_075_085)


Popt_SersicFixedN_GaussianFixedSigma_075_085, Pcov_SersicFixedN_GaussianFixedSigma_075_085 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_SersicFixedN_GaussianFixedSigma_075_085, p0=[i_e_SersicFixedN_GaussianFixedSigma_075_085, r_e_SersicFixedN_GaussianFixedSigma_075_085, a_SersicFixedN_GaussianFixedSigma_075_085], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_075_085 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_075_085))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_075_085, r_e_SersicFixedN_GaussianFixedSigma_075_085, a_SersicFixedN_GaussianFixedSigma_075_085 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_075_085, r_e_SersicFixedN_GaussianFixedSigma_075_085, a_SersicFixedN_GaussianFixedSigma_075_085)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_075_085, a_SersicFixedN_GaussianFixedSigma_075_085 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_075_085 < i_e_SersicFixedN_GaussianFixedSigma_075_085 < I_e_u_SersicFixedN_GaussianFixedSigma_075_085 and R_e_l_SersicFixedN_GaussianFixedSigma_075_085 < r_e_SersicFixedN_GaussianFixedSigma_075_085 < R_e_u_SersicFixedN_GaussianFixedSigma_075_085 and a_l_SersicFixedN_GaussianFixedSigma_075_085 < a_SersicFixedN_GaussianFixedSigma_075_085 < a_u_SersicFixedN_GaussianFixedSigma_075_085):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_075_085 = [(I_e_l_SersicFixedN_GaussianFixedSigma_075_085, I_e_u_SersicFixedN_GaussianFixedSigma_075_085), (R_e_l_SersicFixedN_GaussianFixedSigma_075_085, R_e_u_SersicFixedN_GaussianFixedSigma_075_085), (a_l_SersicFixedN_GaussianFixedSigma_075_085, a_u_SersicFixedN_GaussianFixedSigma_075_085)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_075_085, args=(x_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_err_SersicFixedN_GaussianFixedSigma_075_085), bounds=bounds_SersicFixedN_GaussianFixedSigma_075_085)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_075_085 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_075_085, ndim_SersicFixedN_GaussianFixedSigma_075_085 = pos_SersicFixedN_GaussianFixedSigma_075_085.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_075_085 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_075_085, ndim_SersicFixedN_GaussianFixedSigma_075_085, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_err_SersicFixedN_GaussianFixedSigma_075_085))
sampler_SersicFixedN_GaussianFixedSigma_075_085.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_075_085, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_075_085 = sampler_SersicFixedN_GaussianFixedSigma_075_085.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_075_085 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_075_085):
    mcmc_SersicFixedN_GaussianFixedSigma_075_085 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_075_085[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_075_085.append(mcmc_SersicFixedN_GaussianFixedSigma_075_085[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_075_085 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_075_085):
     mcmc_SersicFixedN_GaussianFixedSigma_075_085 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_075_085[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_075_085.append([mcmc_SersicFixedN_GaussianFixedSigma_075_085[2]-mcmc_SersicFixedN_GaussianFixedSigma_075_085[1],mcmc_SersicFixedN_GaussianFixedSigma_075_085[1]-mcmc_SersicFixedN_GaussianFixedSigma_075_085[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_075_085))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_075_085))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_075_085, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_075_085, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_075_085, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_075_085, *best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_SersicFixedN_GaussianFixedSigma_075_085, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_075_085, y_data_SersicFixedN_GaussianFixedSigma_075_085, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_SersicFixedN_GaussianFixedSigma_075_085 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_075_085, (yy_new_SersicFixedN_GaussianFixedSigma_075_085 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_075_085, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_075_085, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *Popt_SersicFixedN_GaussianFixedSigma_075_085[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_075_085, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *Popt_SersicFixedN_GaussianFixedSigma_075_085[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_075_085, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_075_085, *Popt_SersicFixedN_GaussianFixedSigma_075_085[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_075_085, yy_new_SersicFixedN_GaussianFixedSigma_075_085, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_075_085, y_data_SersicFixedN_GaussianFixedSigma_075_085, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_075_085, (yy_new_SersicFixedN_GaussianFixedSigma_075_085 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *Popt_SersicFixedN_GaussianFixedSigma_075_085[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_075_085, (yy_new_SersicFixedN_GaussianFixedSigma_075_085 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_075_085, *Popt_SersicFixedN_GaussianFixedSigma_075_085[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_075_085, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.75 < z < 0.85")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_075_085[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_075_085[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_075_085[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_075_085[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_075_085[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_075_085[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_075_085[2][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicFixedN_GaussianFixedSigma_085_096 = Equal_Area_Radius_085_096_Array
x_new_SersicFixedN_GaussianFixedSigma_085_096 = np.linspace(min(x_data_SersicFixedN_GaussianFixedSigma_085_096),max(x_data_SersicFixedN_GaussianFixedSigma_085_096), 200)

y_data_SersicFixedN_GaussianFixedSigma_085_096 = np.array(Average_Counts_085_096)
y_new_SersicFixedN_GaussianFixedSigma_085_096 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_085_096, y_data_SersicFixedN_GaussianFixedSigma_085_096)

yy_new_SersicFixedN_GaussianFixedSigma_085_096 = y_new_SersicFixedN_GaussianFixedSigma_085_096(x_new_SersicFixedN_GaussianFixedSigma_085_096)


y_err_SersicFixedN_GaussianFixedSigma_085_096 = np.array(Average_Counts_SD_085_096)
y_new_err_SersicFixedN_GaussianFixedSigma_085_096 = CubicSpline(x_data_SersicFixedN_GaussianFixedSigma_085_096, y_err_SersicFixedN_GaussianFixedSigma_085_096)

yy_new_err_SersicFixedN_GaussianFixedSigma_085_096 = y_new_err_SersicFixedN_GaussianFixedSigma_085_096(x_new_SersicFixedN_GaussianFixedSigma_085_096)

#From Popt --- I_e, R_e, A = [2.80567329 1.00142684 1.41190753]
#From Popt Standard Deviation --- I_e, R_e, A = [0.05630143 0.01737949 0.37162795]

i_e_SersicFixedN_GaussianFixedSigma_085_096 = 2.80567329
r_e_SersicFixedN_GaussianFixedSigma_085_096 = 1.00142684

a_SersicFixedN_GaussianFixedSigma_085_096 = 1.41190753


initial_Mirror_SersicFixedN_GaussianFixedSigma_085_096 = np.array([i_e_SersicFixedN_GaussianFixedSigma_085_096, r_e_SersicFixedN_GaussianFixedSigma_085_096, a_SersicFixedN_GaussianFixedSigma_085_096]) #I_e,R_e,


I_e_l_SersicFixedN_GaussianFixedSigma_085_096 = 0
I_e_u_SersicFixedN_GaussianFixedSigma_085_096 = i_e_SersicFixedN_GaussianFixedSigma_085_096 + (0.01*i_e_SersicFixedN_GaussianFixedSigma_085_096)

R_e_l_SersicFixedN_GaussianFixedSigma_085_096 = 0
R_e_u_SersicFixedN_GaussianFixedSigma_085_096 = r_e_SersicFixedN_GaussianFixedSigma_085_096 + (0.01*r_e_SersicFixedN_GaussianFixedSigma_085_096)

a_l_SersicFixedN_GaussianFixedSigma_085_096 = 0
a_u_SersicFixedN_GaussianFixedSigma_085_096 = a_SersicFixedN_GaussianFixedSigma_085_096 + (0.01*a_SersicFixedN_GaussianFixedSigma_085_096)


Popt_SersicFixedN_GaussianFixedSigma_085_096, Pcov_SersicFixedN_GaussianFixedSigma_085_096 = curve_fit(Sersic_Fixed_N_Gaussian_Fixed_Sigma, x_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_SersicFixedN_GaussianFixedSigma_085_096, p0=[i_e_SersicFixedN_GaussianFixedSigma_085_096, r_e_SersicFixedN_GaussianFixedSigma_085_096, a_SersicFixedN_GaussianFixedSigma_085_096], bounds=(0, np.inf))
Perr_SersicFixedN_GaussianFixedSigma_085_096 = np.sqrt(np.diag(Pcov_SersicFixedN_GaussianFixedSigma_085_096))


# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    i_e_SersicFixedN_GaussianFixedSigma_085_096, r_e_SersicFixedN_GaussianFixedSigma_085_096, a_SersicFixedN_GaussianFixedSigma_085_096 = theta
    model = Sersic_Fixed_N_Gaussian_Fixed_Sigma(x, i_e_SersicFixedN_GaussianFixedSigma_085_096, r_e_SersicFixedN_GaussianFixedSigma_085_096, a_SersicFixedN_GaussianFixedSigma_085_096)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e, r_e_SersicFixedN_GaussianFixedSigma_085_096, a_SersicFixedN_GaussianFixedSigma_085_096 = theta
    if (I_e_l_SersicFixedN_GaussianFixedSigma_085_096 < i_e_SersicFixedN_GaussianFixedSigma_085_096 < I_e_u_SersicFixedN_GaussianFixedSigma_085_096 and R_e_l_SersicFixedN_GaussianFixedSigma_085_096 < r_e_SersicFixedN_GaussianFixedSigma_085_096 < R_e_u_SersicFixedN_GaussianFixedSigma_085_096 and a_l_SersicFixedN_GaussianFixedSigma_085_096 < a_SersicFixedN_GaussianFixedSigma_085_096 < a_u_SersicFixedN_GaussianFixedSigma_085_096):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicFixedN_GaussianFixedSigma_085_096 = [(I_e_l_SersicFixedN_GaussianFixedSigma_085_096, I_e_u_SersicFixedN_GaussianFixedSigma_085_096), (R_e_l_SersicFixedN_GaussianFixedSigma_085_096, R_e_u_SersicFixedN_GaussianFixedSigma_085_096), (a_l_SersicFixedN_GaussianFixedSigma_085_096, a_u_SersicFixedN_GaussianFixedSigma_085_096)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_Mirror_SersicFixedN_GaussianFixedSigma_085_096, args=(x_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_err_SersicFixedN_GaussianFixedSigma_085_096), bounds=bounds_SersicFixedN_GaussianFixedSigma_085_096)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicFixedN_GaussianFixedSigma_085_096 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicFixedN_GaussianFixedSigma_085_096, ndim_SersicFixedN_GaussianFixedSigma_085_096 = pos_SersicFixedN_GaussianFixedSigma_085_096.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicFixedN_GaussianFixedSigma_085_096 = emcee.EnsembleSampler(nwalkers_SersicFixedN_GaussianFixedSigma_085_096, ndim_SersicFixedN_GaussianFixedSigma_085_096, log_probability, args=(x_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_err_SersicFixedN_GaussianFixedSigma_085_096))
sampler_SersicFixedN_GaussianFixedSigma_085_096.run_mcmc(pos_SersicFixedN_GaussianFixedSigma_085_096, 5000, progress=True)

# 5. Analyze results
samples_SersicFixedN_GaussianFixedSigma_085_096 = sampler_SersicFixedN_GaussianFixedSigma_085_096.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicFixedN_GaussianFixedSigma_085_096 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_085_096):
    mcmc_SersicFixedN_GaussianFixedSigma_085_096 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_085_096[:, i], [68, 95, 99.7])
    best_fit_params_SersicFixedN_GaussianFixedSigma_085_096.append(mcmc_SersicFixedN_GaussianFixedSigma_085_096[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicFixedN_GaussianFixedSigma_085_096 = []
for i in range(ndim_SersicFixedN_GaussianFixedSigma_085_096):
     mcmc_SersicFixedN_GaussianFixedSigma_085_096 = np.percentile(samples_SersicFixedN_GaussianFixedSigma_085_096[:, i], [68, 95, 99.7])
     errors_SersicFixedN_GaussianFixedSigma_085_096.append([mcmc_SersicFixedN_GaussianFixedSigma_085_096[2]-mcmc_SersicFixedN_GaussianFixedSigma_085_096[1],mcmc_SersicFixedN_GaussianFixedSigma_085_096[1]-mcmc_SersicFixedN_GaussianFixedSigma_085_096[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, A = " +str(Popt_SersicFixedN_GaussianFixedSigma_085_096))
print("From Popt Standard Deviation --- I_e, R_e, A = " +str(Perr_SersicFixedN_GaussianFixedSigma_085_096))

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_085_096, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_085_096, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicFixedN_GaussianFixedSigma_085_096, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_085_096, *best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[:2]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_SersicFixedN_GaussianFixedSigma_085_096, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicFixedN_GaussianFixedSigma_085_096, y_data_SersicFixedN_GaussianFixedSigma_085_096, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_SersicFixedN_GaussianFixedSigma_085_096 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicFixedN_GaussianFixedSigma_085_096, (yy_new_SersicFixedN_GaussianFixedSigma_085_096 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[:5]))/yy_new_SersicFixedN_GaussianFixedSigma_085_096, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_085_096, Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *Popt_SersicFixedN_GaussianFixedSigma_085_096[:3]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_085_096, Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *Popt_SersicFixedN_GaussianFixedSigma_085_096[2:3]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicFixedN_GaussianFixedSigma_085_096, Sersic_Fit_Fixed_N(x_new_SersicFixedN_GaussianFixedSigma_085_096, *Popt_SersicFixedN_GaussianFixedSigma_085_096[:2]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicFixedN_GaussianFixedSigma_085_096, yy_new_SersicFixedN_GaussianFixedSigma_085_096, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicFixedN_GaussianFixedSigma_085_096, y_data_SersicFixedN_GaussianFixedSigma_085_096, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicFixedN_GaussianFixedSigma_085_096, (yy_new_SersicFixedN_GaussianFixedSigma_085_096 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *Popt_SersicFixedN_GaussianFixedSigma_085_096[:3])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicFixedN_GaussianFixedSigma_085_096, (yy_new_SersicFixedN_GaussianFixedSigma_085_096 - Sersic_Fixed_N_Gaussian_Fixed_Sigma(x_new_SersicFixedN_GaussianFixedSigma_085_096, *Popt_SersicFixedN_GaussianFixedSigma_085_096[:3]))/yy_new_SersicFixedN_GaussianFixedSigma_085_096, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.85 < z < 0.96")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[0]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_085_096[0][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_085_096[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[1]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_085_096[1][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_085_096[1][1]))
print("")
print("A = " +str(best_fit_params_SersicFixedN_GaussianFixedSigma_085_096[2]) + " +" +str(errors_SersicFixedN_GaussianFixedSigma_085_096[2][0]) + " - " +str(errors_SersicFixedN_GaussianFixedSigma_085_096[2][1]))

## <font color='#FF0B55' size=6 > Finding the area under the gaussian with fixed sigama (PSF/2.345) and sersic fit and finding the fraction of the two. Additionally, getting the galaxy component magnitude </font>

In [ ]:
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_025_035, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_025_035 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_025_035[2], Perr_SersicFixedN_GaussianFixedSigma_025_035[2])
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_035_045, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_035_045 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_035_045[2], Perr_SersicFixedN_GaussianFixedSigma_035_045[2])
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_045_055, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_045_055 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_045_055[2], Perr_SersicFixedN_GaussianFixedSigma_045_055[2])
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_055_065, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_055_065 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_055_065[2], Perr_SersicFixedN_GaussianFixedSigma_055_065[2])
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_065_075, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_065_075 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_065_075[2], Perr_SersicFixedN_GaussianFixedSigma_065_075[2])
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_075_085, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_075_085 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_075_085[2], Perr_SersicFixedN_GaussianFixedSigma_075_085[2])
Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_085_096, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_085_096 = Monte_Carlo_Gaussian_Fixed_Sigma_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_085_096[2], Perr_SersicFixedN_GaussianFixedSigma_085_096[2])

Area_Under_Broad_SersicFixedN_GaussianFixedSigma_025_035, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_025_035 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_025_035[0], Popt_SersicFixedN_GaussianFixedSigma_025_035[1], Perr_SersicFixedN_GaussianFixedSigma_025_035[0], Perr_SersicFixedN_GaussianFixedSigma_025_035[1])
Area_Under_Broad_SersicFixedN_GaussianFixedSigma_035_045, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_035_045 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_035_045[0], Popt_SersicFixedN_GaussianFixedSigma_035_045[1], Perr_SersicFixedN_GaussianFixedSigma_035_045[0], Perr_SersicFixedN_GaussianFixedSigma_035_045[1])
Area_Under_Broad_SersicFixedN_GaussianFixedSigma_045_055, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_045_055 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_045_055[0], Popt_SersicFixedN_GaussianFixedSigma_045_055[1], Perr_SersicFixedN_GaussianFixedSigma_045_055[0], Perr_SersicFixedN_GaussianFixedSigma_045_055[1])
Area_Under_Broad_SersicFixedN_GaussianFixedSigma_055_065, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_055_065 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_055_065[0], Popt_SersicFixedN_GaussianFixedSigma_055_065[1], Perr_SersicFixedN_GaussianFixedSigma_055_065[0], Perr_SersicFixedN_GaussianFixedSigma_055_065[1])
Area_Under_Broad_SersicFixedN_GaussianFixedSigma_065_075, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_065_075 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_065_075[0], Popt_SersicFixedN_GaussianFixedSigma_065_075[1], Perr_SersicFixedN_GaussianFixedSigma_065_075[0], Perr_SersicFixedN_GaussianFixedSigma_065_075[1])
Area_Under_Broad_SersicFixedN_GaussianFixedSigma_075_085, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_075_085 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_075_085[0], Popt_SersicFixedN_GaussianFixedSigma_075_085[1], Perr_SersicFixedN_GaussianFixedSigma_075_085[0], Perr_SersicFixedN_GaussianFixedSigma_075_085[1])
Area_Under_Broad_SersicFixedN_GaussianFixedSigma_085_096, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_085_096 = Monte_Carlo_Sersic_Fixed_N_Area(x_new_SersicFixedN_GaussianFixedSigma_085_096, Popt_SersicFixedN_GaussianFixedSigma_085_096[0], Popt_SersicFixedN_GaussianFixedSigma_085_096[1], Perr_SersicFixedN_GaussianFixedSigma_085_096[0], Perr_SersicFixedN_GaussianFixedSigma_085_096[1])

In [ ]:
# Fractional mag for the OG fits (Fake errors)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_025_035, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_025_035 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_025_035, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_025_035, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_025_035, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_025_035)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_035_045, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_035_045 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_035_045, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_035_045, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_035_045, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_035_045)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_045_055, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_045_055 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_045_055, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_045_055, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_045_055, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_045_055)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_055_065, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_055_065 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_055_065, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_055_065, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_055_065, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_055_065)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_065_075, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_065_075 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_065_075, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_065_075, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_065_075, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_065_075)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_075_085, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_075_085 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_075_085, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_075_085, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_075_085, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_075_085)
Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_085_096, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_085_096 = fractional_area_uncertainty(Area_Under_Broad_SersicFixedN_GaussianFixedSigma_085_096, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_085_096, Area_Under_Broad_SersicFixedN_GaussianFixedSigma_SD_085_096, Area_Under_Narrow_SersicFixedN_GaussianFixedSigma_SD_085_096)


In [ ]:
# The Magnitude of the galaxy component for each redshift
Gal_Mag_SersicFixedN_GaussianFixedSigma_025_035, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_025_035 = gal_mag_uncertainty(Mag_Total_025_035, Mag_Total_SD_025_035, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_025_035, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_025_035)
Gal_Mag_SersicFixedN_GaussianFixedSigma_035_045, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_035_045 = gal_mag_uncertainty(Mag_Total_035_045, Mag_Total_SD_035_045, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_035_045, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_035_045)
Gal_Mag_SersicFixedN_GaussianFixedSigma_045_055, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_045_055 = gal_mag_uncertainty(Mag_Total_045_055, Mag_Total_SD_045_055, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_045_055, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_045_055)
Gal_Mag_SersicFixedN_GaussianFixedSigma_055_065, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_055_065 = gal_mag_uncertainty(Mag_Total_055_065, Mag_Total_SD_055_065, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_055_065, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_055_065)
Gal_Mag_SersicFixedN_GaussianFixedSigma_065_075, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_065_075 = gal_mag_uncertainty(Mag_Total_065_075, Mag_Total_SD_065_075, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_065_075, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_065_075)
Gal_Mag_SersicFixedN_GaussianFixedSigma_075_085, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_075_085 = gal_mag_uncertainty(Mag_Total_075_085, Mag_Total_SD_075_085, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_075_085, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_075_085)
Gal_Mag_SersicFixedN_GaussianFixedSigma_085_096, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_085_096 = gal_mag_uncertainty(Mag_Total_085_096, Mag_Total_SD_085_096, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_085_096, Fractional_Area_Broad_SersicFixedN_GaussianFixedSigma_SD_085_096)


In [ ]:
Gal_Mag_SersicFixedN_GaussianFixedSigma_Array = np.array([Gal_Mag_SersicFixedN_GaussianFixedSigma_025_035, Gal_Mag_SersicFixedN_GaussianFixedSigma_035_045, Gal_Mag_SersicFixedN_GaussianFixedSigma_045_055, Gal_Mag_SersicFixedN_GaussianFixedSigma_055_065, Gal_Mag_SersicFixedN_GaussianFixedSigma_065_075, Gal_Mag_SersicFixedN_GaussianFixedSigma_075_085, Gal_Mag_SersicFixedN_GaussianFixedSigma_085_096])


In [ ]:
Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_Array = np.array([Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_025_035, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_035_045, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_045_055, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_055_065, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_065_075, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_075_085, Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_085_096])


## <font color='#FF0B55' size=6 > Fitting the radial profile with a gaussian with fixed sigma (PSF/2.345) and sersic with a fixed N to be above one </font>

In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_025_035 = Equal_Area_Radius_025_035_Array
x_new_SersicNOne_GaussianFixedSigma_025_035 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_025_035),max(x_data_SersicNOne_GaussianFixedSigma_025_035), 200)

y_data_SersicNOne_GaussianFixedSigma_025_035 = np.array(Average_Counts_025_035)
y_new_SersicNOne_GaussianFixedSigma_025_035 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_025_035, y_data_SersicNOne_GaussianFixedSigma_025_035)

yy_new_SersicNOne_GaussianFixedSigma_025_035 = y_new_SersicNOne_GaussianFixedSigma_025_035(x_new_SersicNOne_GaussianFixedSigma_025_035)


y_err_SersicNOne_GaussianFixedSigma_025_035 = np.array(Average_Counts_SD_025_035)
y_new_err_SersicNOne_GaussianFixedSigma_025_035 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_025_035, y_err_SersicNOne_GaussianFixedSigma_025_035)

yy_new_err_SersicNOne_GaussianFixedSigma_025_035 = y_new_err_SersicNOne_GaussianFixedSigma_025_035(x_new_SersicNOne_GaussianFixedSigma_025_035)

#From Popt --- I_e, R_e, N, BG, A = [ 2.76047969  1.60026062  1.13066268  0.25105957 13.62608638]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.32678547 0.10900932 0.1082596  0.01770488 0.3136965 ]

i_e_SersicNOne_GaussianFixedSigma_025_035 = 2.76047969
r_e_SersicNOne_GaussianFixedSigma_025_035 = 1.60026062

n_SersicNOne_GaussianFixedSigma_025_035 = 1.13066268

bg_SersicNOne_GaussianFixedSigma_025_035 = min(yy_new_SersicNOne_GaussianFixedSigma_025_035)

a_SersicNOne_GaussianFixedSigma_025_035 = 13.62608638

initial_SersicNOne_GaussianFixedSigma_025_035 = np.array([i_e_SersicNOne_GaussianFixedSigma_025_035, r_e_SersicNOne_GaussianFixedSigma_025_035, n_SersicNOne_GaussianFixedSigma_025_035, bg_SersicNOne_GaussianFixedSigma_025_035, a_SersicNOne_GaussianFixedSigma_025_035]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_025_035 = 0
I_e_u_SersicNOne_GaussianFixedSigma_025_035 = i_e_SersicNOne_GaussianFixedSigma_025_035 + (0.01*i_e_SersicNOne_GaussianFixedSigma_025_035)

R_e_l_SersicNOne_GaussianFixedSigma_025_035 = 0
R_e_u_SersicNOne_GaussianFixedSigma_025_035 = r_e_SersicNOne_GaussianFixedSigma_025_035 + (0.01*r_e_SersicNOne_GaussianFixedSigma_025_035)

a_l_SersicNOne_GaussianFixedSigma_025_035 = 0
a_u_SersicNOne_GaussianFixedSigma_025_035 = a_SersicNOne_GaussianFixedSigma_025_035 + (0.01*a_SersicNOne_GaussianFixedSigma_025_035)

n_l_SersicNOne_GaussianFixedSigma_025_035 = 1.0
n_u_SersicNOne_GaussianFixedSigma_025_035 = n_SersicNOne_GaussianFixedSigma_025_035 + (0.01*n_SersicNOne_GaussianFixedSigma_025_035)

bg_l_SersicNOne_GaussianFixedSigma_025_035 = bg_SersicNOne_GaussianFixedSigma_025_035 - (bg_SersicNOne_GaussianFixedSigma_025_035*0.001)
bg_u_SersicNOne_GaussianFixedSigma_025_035 = bg_SersicNOne_GaussianFixedSigma_025_035 + (bg_SersicNOne_GaussianFixedSigma_025_035*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_025_035 = [I_e_l_SersicNOne_GaussianFixedSigma_025_035, R_e_l_SersicNOne_GaussianFixedSigma_025_035, n_l_SersicNOne_GaussianFixedSigma_025_035, bg_l_SersicNOne_GaussianFixedSigma_025_035, a_l_SersicNOne_GaussianFixedSigma_025_035]
Bounds_Upper_SersicNOne_GaussianFixedSigma_025_035 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_025_035, 500]


Popt_SersicNOne_GaussianFixedSigma_025_035, Pcov_SersicNOne_GaussianFixedSigma_025_035 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_025_035, p0=[i_e_SersicNOne_GaussianFixedSigma_025_035, r_e_SersicNOne_GaussianFixedSigma_025_035, n_SersicNOne_GaussianFixedSigma_025_035, bg_SersicNOne_GaussianFixedSigma_025_035, a_SersicNOne_GaussianFixedSigma_025_035], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_025_035, Bounds_Upper_SersicNOne_GaussianFixedSigma_025_035), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_025_035 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_025_035))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_025_035, R_e_l_SersicNOne_GaussianFixedSigma_025_035, n_l_SersicNOne_GaussianFixedSigma_025_035, bg_l_SersicNOne_GaussianFixedSigma_025_035, a_l_SersicNOne_GaussianFixedSigma_025_035 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_025_035, R_e_l_SersicNOne_GaussianFixedSigma_025_035, n_l_SersicNOne_GaussianFixedSigma_025_035, bg_l_SersicNOne_GaussianFixedSigma_025_035, a_l_SersicNOne_GaussianFixedSigma_025_035)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_025_035, R_e_l_SersicNOne_GaussianFixedSigma_025_035, n_l_SersicNOne_GaussianFixedSigma_025_035, bg_l_SersicNOne_GaussianFixedSigma_025_035, a_l_SersicNOne_GaussianFixedSigma_025_035 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_025_035 < i_e_SersicNOne_GaussianFixedSigma_025_035 < I_e_u_SersicNOne_GaussianFixedSigma_025_035 and R_e_l_SersicNOne_GaussianFixedSigma_025_035 < r_e_SersicNOne_GaussianFixedSigma_025_035 < R_e_u_SersicNOne_GaussianFixedSigma_025_035 and n_l_SersicNOne_GaussianFixedSigma_025_035 < n_SersicNOne_GaussianFixedSigma_025_035 < n_u_SersicNOne_GaussianFixedSigma_025_035 and bg_l_SersicNOne_GaussianFixedSigma_025_035 < bg_SersicNOne_GaussianFixedSigma_025_035 < bg_u_SersicNOne_GaussianFixedSigma_025_035 and a_l_SersicNOne_GaussianFixedSigma_025_035 < a_SersicNOne_GaussianFixedSigma_025_035 < a_u_SersicNOne_GaussianFixedSigma_025_035):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_025_035 = [(I_e_l_SersicNOne_GaussianFixedSigma_025_035, I_e_u_SersicNOne_GaussianFixedSigma_025_035), (R_e_l_SersicNOne_GaussianFixedSigma_025_035, R_e_u_SersicNOne_GaussianFixedSigma_025_035), (n_l_SersicNOne_GaussianFixedSigma_025_035, n_u_SersicNOne_GaussianFixedSigma_025_035), (bg_l_SersicNOne_GaussianFixedSigma_025_035, bg_u_SersicNOne_GaussianFixedSigma_025_035), (a_l_SersicNOne_GaussianFixedSigma_025_035, a_u_SersicNOne_GaussianFixedSigma_025_035)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_025_035, args=(x_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_err_SersicNOne_GaussianFixedSigma_025_035), bounds=bounds_SersicNOne_GaussianFixedSigma_025_035)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_025_035 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_025_035, ndim_SersicNOne_GaussianFixedSigma_025_035 = pos_SersicNOne_GaussianFixedSigma_025_035.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_025_035 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_025_035, ndim_SersicNOne_GaussianFixedSigma_025_035, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_err_SersicNOne_GaussianFixedSigma_025_035))
sampler_SersicNOne_GaussianFixedSigma_025_035.run_mcmc(pos_SersicNOne_GaussianFixedSigma_025_035, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_025_035 = sampler_SersicNOne_GaussianFixedSigma_025_035.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_025_035 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_025_035):
    mcmc_SersicNOne_GaussianFixedSigma_025_035 = np.percentile(samples_SersicNOne_GaussianFixedSigma_025_035[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_025_035.append(mcmc_SersicNOne_GaussianFixedSigma_025_035[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_025_035 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_025_035):
     mcmc_SersicNOne_GaussianFixedSigma_025_035 = np.percentile(samples_SersicNOne_GaussianFixedSigma_025_035[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_025_035.append([mcmc_SersicNOne_GaussianFixedSigma_025_035[2]-mcmc_SersicNOne_GaussianFixedSigma_025_035[1],mcmc_SersicNOne_GaussianFixedSigma_025_035[1]-mcmc_SersicNOne_GaussianFixedSigma_025_035[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_025_035))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_025_035))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *best_fit_params_SersicNOne_GaussianFixedSigma_025_035[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *best_fit_params_SersicNOne_GaussianFixedSigma_025_035[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *best_fit_params_SersicNOne_GaussianFixedSigma_025_035[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_025_035, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_025_035, y_data_SersicNOne_GaussianFixedSigma_025_035, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_025_035 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *best_fit_params_SersicNOne_GaussianFixedSigma_025_035[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, (yy_new_SersicNOne_GaussianFixedSigma_025_035 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *best_fit_params_SersicNOne_GaussianFixedSigma_025_035[:5]))/yy_new_SersicNOne_GaussianFixedSigma_025_035, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_025_035, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_025_035, y_data_SersicNOne_GaussianFixedSigma_025_035, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, (yy_new_SersicNOne_GaussianFixedSigma_025_035 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, (yy_new_SersicNOne_GaussianFixedSigma_025_035 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:5]))/yy_new_SersicNOne_GaussianFixedSigma_025_035, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.25 < z < 0.35")
axes1[0].set_title("Guassian Fixed Sigma + Sersic Fixed N -- 0.25 < z < 0.35")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_025_035[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_025_035[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_025_035[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_025_035[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_025_035[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_025_035[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_025_035[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_025_035[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_025_035[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_025_035[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_025_035[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_025_035[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_025_035[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_025_035[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_025_035[4][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_035_045 = Equal_Area_Radius_035_045_Array
x_new_SersicNOne_GaussianFixedSigma_035_045 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_035_045),max(x_data_SersicNOne_GaussianFixedSigma_035_045), 200)

y_data_SersicNOne_GaussianFixedSigma_035_045 = np.array(Average_Counts_035_045)
y_new_SersicNOne_GaussianFixedSigma_035_045 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_035_045, y_data_SersicNOne_GaussianFixedSigma_035_045)

yy_new_SersicNOne_GaussianFixedSigma_035_045 = y_new_SersicNOne_GaussianFixedSigma_035_045(x_new_SersicNOne_GaussianFixedSigma_035_045)


y_err_SersicNOne_GaussianFixedSigma_035_045 = np.array(Average_Counts_SD_035_045)
y_new_err_SersicNOne_GaussianFixedSigma_035_045 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_035_045, y_err_SersicNOne_GaussianFixedSigma_035_045)

yy_new_err_SersicNOne_GaussianFixedSigma_035_045 = y_new_err_SersicNOne_GaussianFixedSigma_035_045(x_new_SersicNOne_GaussianFixedSigma_035_045)

#From Popt --- I_e, R_e, N, BG, A = [1.47531797 1.56445851 1.01598273 0.12143001 7.31372356]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.1767533  0.10557886 0.10955562 0.00968288 0.17514191]

i_e_SersicNOne_GaussianFixedSigma_035_045 = 1.47531797
r_e_SersicNOne_GaussianFixedSigma_035_045 = 1.56445851

n_SersicNOne_GaussianFixedSigma_035_045 = 1.01598273

bg_SersicNOne_GaussianFixedSigma_035_045 = min(yy_new_SersicNOne_GaussianFixedSigma_035_045)

a_SersicNOne_GaussianFixedSigma_035_045 = 7.31372356

initial_SersicNOne_GaussianFixedSigma_035_045 = np.array([i_e_SersicNOne_GaussianFixedSigma_035_045, r_e_SersicNOne_GaussianFixedSigma_035_045, n_SersicNOne_GaussianFixedSigma_035_045, bg_SersicNOne_GaussianFixedSigma_035_045, a_SersicNOne_GaussianFixedSigma_035_045]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_035_045 = 0
I_e_u_SersicNOne_GaussianFixedSigma_035_045 = i_e_SersicNOne_GaussianFixedSigma_035_045 + (0.01*i_e_SersicNOne_GaussianFixedSigma_035_045)

R_e_l_SersicNOne_GaussianFixedSigma_035_045 = 0
R_e_u_SersicNOne_GaussianFixedSigma_035_045 = r_e_SersicNOne_GaussianFixedSigma_035_045 + (0.01*r_e_SersicNOne_GaussianFixedSigma_035_045)

a_l_SersicNOne_GaussianFixedSigma_035_045 = 0
a_u_SersicNOne_GaussianFixedSigma_035_045 = a_SersicNOne_GaussianFixedSigma_035_045 + (0.01*a_SersicNOne_GaussianFixedSigma_035_045)

n_l_SersicNOne_GaussianFixedSigma_035_045 = 1.0
n_u_SersicNOne_GaussianFixedSigma_035_045 = n_SersicNOne_GaussianFixedSigma_035_045 + (0.01*n_SersicNOne_GaussianFixedSigma_035_045)

bg_l_SersicNOne_GaussianFixedSigma_035_045 = bg_SersicNOne_GaussianFixedSigma_035_045 - (bg_SersicNOne_GaussianFixedSigma_035_045*0.001)
bg_u_SersicNOne_GaussianFixedSigma_035_045 = bg_SersicNOne_GaussianFixedSigma_035_045 + (bg_SersicNOne_GaussianFixedSigma_035_045*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_035_045 = [I_e_l_SersicNOne_GaussianFixedSigma_035_045, R_e_l_SersicNOne_GaussianFixedSigma_035_045, n_l_SersicNOne_GaussianFixedSigma_035_045, bg_l_SersicNOne_GaussianFixedSigma_035_045, a_l_SersicNOne_GaussianFixedSigma_035_045]
Bounds_Upper_SersicNOne_GaussianFixedSigma_035_045 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_035_045, 500]


Popt_SersicNOne_GaussianFixedSigma_035_045, Pcov_SersicNOne_GaussianFixedSigma_035_045 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_035_045, p0=[i_e_SersicNOne_GaussianFixedSigma_035_045, r_e_SersicNOne_GaussianFixedSigma_035_045, n_SersicNOne_GaussianFixedSigma_035_045, bg_SersicNOne_GaussianFixedSigma_035_045, a_SersicNOne_GaussianFixedSigma_035_045], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_035_045, Bounds_Upper_SersicNOne_GaussianFixedSigma_035_045), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_035_045 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_035_045))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_035_045, R_e_l_SersicNOne_GaussianFixedSigma_035_045, n_l_SersicNOne_GaussianFixedSigma_035_045, bg_l_SersicNOne_GaussianFixedSigma_035_045, a_l_SersicNOne_GaussianFixedSigma_035_045 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_035_045, R_e_l_SersicNOne_GaussianFixedSigma_035_045, n_l_SersicNOne_GaussianFixedSigma_035_045, bg_l_SersicNOne_GaussianFixedSigma_035_045, a_l_SersicNOne_GaussianFixedSigma_035_045)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_035_045, R_e_l_SersicNOne_GaussianFixedSigma_035_045, n_l_SersicNOne_GaussianFixedSigma_035_045, bg_l_SersicNOne_GaussianFixedSigma_035_045, a_l_SersicNOne_GaussianFixedSigma_035_045 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_035_045 < i_e_SersicNOne_GaussianFixedSigma_035_045 < I_e_u_SersicNOne_GaussianFixedSigma_035_045 and R_e_l_SersicNOne_GaussianFixedSigma_035_045 < r_e_SersicNOne_GaussianFixedSigma_035_045 < R_e_u_SersicNOne_GaussianFixedSigma_035_045 and n_l_SersicNOne_GaussianFixedSigma_035_045 < n_SersicNOne_GaussianFixedSigma_035_045 < n_u_SersicNOne_GaussianFixedSigma_035_045 and bg_l_SersicNOne_GaussianFixedSigma_035_045 < bg_SersicNOne_GaussianFixedSigma_035_045 < bg_u_SersicNOne_GaussianFixedSigma_035_045 and a_l_SersicNOne_GaussianFixedSigma_035_045 < a_SersicNOne_GaussianFixedSigma_035_045 < a_u_SersicNOne_GaussianFixedSigma_035_045):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_035_045 = [(I_e_l_SersicNOne_GaussianFixedSigma_035_045, I_e_u_SersicNOne_GaussianFixedSigma_035_045), (R_e_l_SersicNOne_GaussianFixedSigma_035_045, R_e_u_SersicNOne_GaussianFixedSigma_035_045), (n_l_SersicNOne_GaussianFixedSigma_035_045, n_u_SersicNOne_GaussianFixedSigma_035_045), (bg_l_SersicNOne_GaussianFixedSigma_035_045, bg_u_SersicNOne_GaussianFixedSigma_035_045), (a_l_SersicNOne_GaussianFixedSigma_035_045, a_u_SersicNOne_GaussianFixedSigma_035_045)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_035_045, args=(x_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_err_SersicNOne_GaussianFixedSigma_035_045), bounds=bounds_SersicNOne_GaussianFixedSigma_035_045)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_035_045 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_035_045, ndim_SersicNOne_GaussianFixedSigma_035_045 = pos_SersicNOne_GaussianFixedSigma_035_045.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_035_045 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_035_045, ndim_SersicNOne_GaussianFixedSigma_035_045, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_err_SersicNOne_GaussianFixedSigma_035_045))
sampler_SersicNOne_GaussianFixedSigma_035_045.run_mcmc(pos_SersicNOne_GaussianFixedSigma_035_045, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_035_045 = sampler_SersicNOne_GaussianFixedSigma_035_045.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_035_045 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_035_045):
    mcmc_SersicNOne_GaussianFixedSigma_035_045 = np.percentile(samples_SersicNOne_GaussianFixedSigma_035_045[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_035_045.append(mcmc_SersicNOne_GaussianFixedSigma_035_045[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_035_045 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_035_045):
     mcmc_SersicNOne_GaussianFixedSigma_035_045 = np.percentile(samples_SersicNOne_GaussianFixedSigma_035_045[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_035_045.append([mcmc_SersicNOne_GaussianFixedSigma_035_045[2]-mcmc_SersicNOne_GaussianFixedSigma_035_045[1],mcmc_SersicNOne_GaussianFixedSigma_035_045[1]-mcmc_SersicNOne_GaussianFixedSigma_035_045[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_035_045))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_035_045))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *best_fit_params_SersicNOne_GaussianFixedSigma_035_045[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *best_fit_params_SersicNOne_GaussianFixedSigma_035_045[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *best_fit_params_SersicNOne_GaussianFixedSigma_035_045[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_035_045, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_035_045, y_data_SersicNOne_GaussianFixedSigma_035_045, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_035_045 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *best_fit_params_SersicNOne_GaussianFixedSigma_035_045[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, (yy_new_SersicNOne_GaussianFixedSigma_035_045 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *best_fit_params_SersicNOne_GaussianFixedSigma_035_045[:5]))/yy_new_SersicNOne_GaussianFixedSigma_035_045, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_035_045, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_035_045, y_data_SersicNOne_GaussianFixedSigma_035_045, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, (yy_new_SersicNOne_GaussianFixedSigma_035_045 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, (yy_new_SersicNOne_GaussianFixedSigma_035_045 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:5]))/yy_new_SersicNOne_GaussianFixedSigma_035_045, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.35 < z < 0.45")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_035_045[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_035_045[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_035_045[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_035_045[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_035_045[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_035_045[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_035_045[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_035_045[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_035_045[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_035_045[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_035_045[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_035_045[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_035_045[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_035_045[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_035_045[4][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_045_055 = Equal_Area_Radius_045_055_Array
x_new_SersicNOne_GaussianFixedSigma_045_055 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_045_055),max(x_data_SersicNOne_GaussianFixedSigma_045_055), 200)

y_data_SersicNOne_GaussianFixedSigma_045_055 = np.array(Average_Counts_045_055)
y_new_SersicNOne_GaussianFixedSigma_045_055 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_045_055, y_data_SersicNOne_GaussianFixedSigma_045_055)

yy_new_SersicNOne_GaussianFixedSigma_045_055 = y_new_SersicNOne_GaussianFixedSigma_045_055(x_new_SersicNOne_GaussianFixedSigma_045_055)


y_err_SersicNOne_GaussianFixedSigma_045_055 = np.array(Average_Counts_SD_045_055)
y_new_err_SersicNOne_GaussianFixedSigma_045_055 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_045_055, y_err_SersicNOne_GaussianFixedSigma_045_055)

yy_new_err_SersicNOne_GaussianFixedSigma_045_055 = y_new_err_SersicNOne_GaussianFixedSigma_045_055(x_new_SersicNOne_GaussianFixedSigma_045_055)

#From Popt --- I_e, R_e, N, BG, A = [0.97955996 1.53661805 1.1907463  0.07874492 5.22276602]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.12759009 0.11331761 0.12069794 0.00625854 0.11642905]

i_e_SersicNOne_GaussianFixedSigma_045_055 = 0.97955996
r_e_SersicNOne_GaussianFixedSigma_045_055 = 1.53661805

n_SersicNOne_GaussianFixedSigma_045_055 = 1.1907463

bg_SersicNOne_GaussianFixedSigma_045_055 = min(yy_new_SersicNOne_GaussianFixedSigma_045_055)

a_SersicNOne_GaussianFixedSigma_045_055 = 5.22276602

initial_SersicNOne_GaussianFixedSigma_045_055 = np.array([i_e_SersicNOne_GaussianFixedSigma_045_055, r_e_SersicNOne_GaussianFixedSigma_045_055, n_SersicNOne_GaussianFixedSigma_045_055, bg_SersicNOne_GaussianFixedSigma_045_055, a_SersicNOne_GaussianFixedSigma_045_055]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_045_055 = 0
I_e_u_SersicNOne_GaussianFixedSigma_045_055 = i_e_SersicNOne_GaussianFixedSigma_045_055 + (0.01*i_e_SersicNOne_GaussianFixedSigma_045_055)

R_e_l_SersicNOne_GaussianFixedSigma_045_055 = 0
R_e_u_SersicNOne_GaussianFixedSigma_045_055 = r_e_SersicNOne_GaussianFixedSigma_045_055 + (0.01*r_e_SersicNOne_GaussianFixedSigma_045_055)

a_l_SersicNOne_GaussianFixedSigma_045_055 = 0
a_u_SersicNOne_GaussianFixedSigma_045_055 = a_SersicNOne_GaussianFixedSigma_045_055 + (0.01*a_SersicNOne_GaussianFixedSigma_045_055)

n_l_SersicNOne_GaussianFixedSigma_045_055 = 1.0
n_u_SersicNOne_GaussianFixedSigma_045_055 = n_SersicNOne_GaussianFixedSigma_045_055 + (0.01*n_SersicNOne_GaussianFixedSigma_045_055)

bg_l_SersicNOne_GaussianFixedSigma_045_055 = bg_SersicNOne_GaussianFixedSigma_045_055 - (bg_SersicNOne_GaussianFixedSigma_045_055*0.001)
bg_u_SersicNOne_GaussianFixedSigma_045_055 = bg_SersicNOne_GaussianFixedSigma_045_055 + (bg_SersicNOne_GaussianFixedSigma_045_055*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_045_055 = [I_e_l_SersicNOne_GaussianFixedSigma_045_055, R_e_l_SersicNOne_GaussianFixedSigma_045_055, n_l_SersicNOne_GaussianFixedSigma_045_055, bg_l_SersicNOne_GaussianFixedSigma_045_055, a_l_SersicNOne_GaussianFixedSigma_045_055]
Bounds_Upper_SersicNOne_GaussianFixedSigma_045_055 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_045_055, 500]


Popt_SersicNOne_GaussianFixedSigma_045_055, Pcov_SersicNOne_GaussianFixedSigma_045_055 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_045_055, p0=[i_e_SersicNOne_GaussianFixedSigma_045_055, r_e_SersicNOne_GaussianFixedSigma_045_055, n_SersicNOne_GaussianFixedSigma_045_055, bg_SersicNOne_GaussianFixedSigma_045_055, a_SersicNOne_GaussianFixedSigma_045_055], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_045_055, Bounds_Upper_SersicNOne_GaussianFixedSigma_045_055), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_045_055 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_045_055))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_045_055, R_e_l_SersicNOne_GaussianFixedSigma_045_055, n_l_SersicNOne_GaussianFixedSigma_045_055, bg_l_SersicNOne_GaussianFixedSigma_045_055, a_l_SersicNOne_GaussianFixedSigma_045_055 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_045_055, R_e_l_SersicNOne_GaussianFixedSigma_045_055, n_l_SersicNOne_GaussianFixedSigma_045_055, bg_l_SersicNOne_GaussianFixedSigma_045_055, a_l_SersicNOne_GaussianFixedSigma_045_055)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_045_055, R_e_l_SersicNOne_GaussianFixedSigma_045_055, n_l_SersicNOne_GaussianFixedSigma_045_055, bg_l_SersicNOne_GaussianFixedSigma_045_055, a_l_SersicNOne_GaussianFixedSigma_045_055 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_045_055 < i_e_SersicNOne_GaussianFixedSigma_045_055 < I_e_u_SersicNOne_GaussianFixedSigma_045_055 and R_e_l_SersicNOne_GaussianFixedSigma_045_055 < r_e_SersicNOne_GaussianFixedSigma_045_055 < R_e_u_SersicNOne_GaussianFixedSigma_045_055 and n_l_SersicNOne_GaussianFixedSigma_045_055 < n_SersicNOne_GaussianFixedSigma_045_055 < n_u_SersicNOne_GaussianFixedSigma_045_055 and bg_l_SersicNOne_GaussianFixedSigma_045_055 < bg_SersicNOne_GaussianFixedSigma_045_055 < bg_u_SersicNOne_GaussianFixedSigma_045_055 and a_l_SersicNOne_GaussianFixedSigma_045_055 < a_SersicNOne_GaussianFixedSigma_045_055 < a_u_SersicNOne_GaussianFixedSigma_045_055):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_045_055 = [(I_e_l_SersicNOne_GaussianFixedSigma_045_055, I_e_u_SersicNOne_GaussianFixedSigma_045_055), (R_e_l_SersicNOne_GaussianFixedSigma_045_055, R_e_u_SersicNOne_GaussianFixedSigma_045_055), (n_l_SersicNOne_GaussianFixedSigma_045_055, n_u_SersicNOne_GaussianFixedSigma_045_055), (bg_l_SersicNOne_GaussianFixedSigma_045_055, bg_u_SersicNOne_GaussianFixedSigma_045_055), (a_l_SersicNOne_GaussianFixedSigma_045_055, a_u_SersicNOne_GaussianFixedSigma_045_055)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_045_055, args=(x_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_err_SersicNOne_GaussianFixedSigma_045_055), bounds=bounds_SersicNOne_GaussianFixedSigma_045_055)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_045_055 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_045_055, ndim_SersicNOne_GaussianFixedSigma_045_055 = pos_SersicNOne_GaussianFixedSigma_045_055.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_045_055 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_045_055, ndim_SersicNOne_GaussianFixedSigma_045_055, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_err_SersicNOne_GaussianFixedSigma_045_055))
sampler_SersicNOne_GaussianFixedSigma_045_055.run_mcmc(pos_SersicNOne_GaussianFixedSigma_045_055, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_045_055 = sampler_SersicNOne_GaussianFixedSigma_045_055.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_045_055 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_045_055):
    mcmc_SersicNOne_GaussianFixedSigma_045_055 = np.percentile(samples_SersicNOne_GaussianFixedSigma_045_055[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_045_055.append(mcmc_SersicNOne_GaussianFixedSigma_045_055[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_045_055 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_045_055):
     mcmc_SersicNOne_GaussianFixedSigma_045_055 = np.percentile(samples_SersicNOne_GaussianFixedSigma_045_055[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_045_055.append([mcmc_SersicNOne_GaussianFixedSigma_045_055[2]-mcmc_SersicNOne_GaussianFixedSigma_045_055[1],mcmc_SersicNOne_GaussianFixedSigma_045_055[1]-mcmc_SersicNOne_GaussianFixedSigma_045_055[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_045_055))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_045_055))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *best_fit_params_SersicNOne_GaussianFixedSigma_045_055[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *best_fit_params_SersicNOne_GaussianFixedSigma_045_055[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *best_fit_params_SersicNOne_GaussianFixedSigma_045_055[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_045_055, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_045_055, y_data_SersicNOne_GaussianFixedSigma_045_055, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_045_055 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *best_fit_params_SersicNOne_GaussianFixedSigma_045_055[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, (yy_new_SersicNOne_GaussianFixedSigma_045_055 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *best_fit_params_SersicNOne_GaussianFixedSigma_045_055[:5]))/yy_new_SersicNOne_GaussianFixedSigma_045_055, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_045_055, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_045_055, y_data_SersicNOne_GaussianFixedSigma_045_055, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, (yy_new_SersicNOne_GaussianFixedSigma_045_055 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, (yy_new_SersicNOne_GaussianFixedSigma_045_055 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:5]))/yy_new_SersicNOne_GaussianFixedSigma_045_055, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.45 < z < 0.55")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_045_055[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_045_055[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_045_055[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_045_055[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_045_055[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_045_055[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_045_055[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_045_055[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_045_055[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_045_055[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_045_055[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_045_055[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_045_055[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_045_055[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_045_055[4][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_055_065 = Equal_Area_Radius_055_065_Array
x_new_SersicNOne_GaussianFixedSigma_055_065 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_055_065),max(x_data_SersicNOne_GaussianFixedSigma_055_065), 200)

y_data_SersicNOne_GaussianFixedSigma_055_065 = np.array(Average_Counts_055_065)
y_new_SersicNOne_GaussianFixedSigma_055_065 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_055_065, y_data_SersicNOne_GaussianFixedSigma_055_065)

yy_new_SersicNOne_GaussianFixedSigma_055_065 = y_new_SersicNOne_GaussianFixedSigma_055_065(x_new_SersicNOne_GaussianFixedSigma_055_065)


y_err_SersicNOne_GaussianFixedSigma_055_065 = np.array(Average_Counts_SD_055_065)
y_new_err_SersicNOne_GaussianFixedSigma_055_065 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_055_065, y_err_SersicNOne_GaussianFixedSigma_055_065)

yy_new_err_SersicNOne_GaussianFixedSigma_055_065 = y_new_err_SersicNOne_GaussianFixedSigma_055_065(x_new_SersicNOne_GaussianFixedSigma_055_065)

#From Popt --- I_e, R_e, N, BG, A = [0.97754285 1.46444587 1.03438736 0.0685562  6.73242343]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.14526892 0.118546   0.13830954 0.00699385 0.13640816]

i_e_SersicNOne_GaussianFixedSigma_055_065 = 0.97754285
r_e_SersicNOne_GaussianFixedSigma_055_065 = 1.46444587

n_SersicNOne_GaussianFixedSigma_055_065 = 1.03438736

bg_SersicNOne_GaussianFixedSigma_055_065 = min(yy_new_SersicNOne_GaussianFixedSigma_055_065)

a_SersicNOne_GaussianFixedSigma_055_065 = 6.73242343

initial_SersicNOne_GaussianFixedSigma_055_065 = np.array([i_e_SersicNOne_GaussianFixedSigma_055_065, r_e_SersicNOne_GaussianFixedSigma_055_065, n_SersicNOne_GaussianFixedSigma_055_065, bg_SersicNOne_GaussianFixedSigma_055_065, a_SersicNOne_GaussianFixedSigma_055_065]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_055_065 = 0
I_e_u_SersicNOne_GaussianFixedSigma_055_065 = i_e_SersicNOne_GaussianFixedSigma_055_065 + (0.01*i_e_SersicNOne_GaussianFixedSigma_055_065)

R_e_l_SersicNOne_GaussianFixedSigma_055_065 = 0
R_e_u_SersicNOne_GaussianFixedSigma_055_065 = r_e_SersicNOne_GaussianFixedSigma_055_065 + (0.01*r_e_SersicNOne_GaussianFixedSigma_055_065)

a_l_SersicNOne_GaussianFixedSigma_055_065 = 0
a_u_SersicNOne_GaussianFixedSigma_055_065 = a_SersicNOne_GaussianFixedSigma_055_065 + (0.01*a_SersicNOne_GaussianFixedSigma_055_065)

n_l_SersicNOne_GaussianFixedSigma_055_065 = 1.0
n_u_SersicNOne_GaussianFixedSigma_055_065 = n_SersicNOne_GaussianFixedSigma_055_065 + (0.01*n_SersicNOne_GaussianFixedSigma_055_065)

bg_l_SersicNOne_GaussianFixedSigma_055_065 = bg_SersicNOne_GaussianFixedSigma_055_065 - (bg_SersicNOne_GaussianFixedSigma_055_065*0.001)
bg_u_SersicNOne_GaussianFixedSigma_055_065 = bg_SersicNOne_GaussianFixedSigma_055_065 + (bg_SersicNOne_GaussianFixedSigma_055_065*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_055_065 = [I_e_l_SersicNOne_GaussianFixedSigma_055_065, R_e_l_SersicNOne_GaussianFixedSigma_055_065, n_l_SersicNOne_GaussianFixedSigma_055_065, bg_l_SersicNOne_GaussianFixedSigma_055_065, a_l_SersicNOne_GaussianFixedSigma_055_065]
Bounds_Upper_SersicNOne_GaussianFixedSigma_055_065 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_055_065, 500]


Popt_SersicNOne_GaussianFixedSigma_055_065, Pcov_SersicNOne_GaussianFixedSigma_055_065 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_055_065, p0=[i_e_SersicNOne_GaussianFixedSigma_055_065, r_e_SersicNOne_GaussianFixedSigma_055_065, n_SersicNOne_GaussianFixedSigma_055_065, bg_SersicNOne_GaussianFixedSigma_055_065, a_SersicNOne_GaussianFixedSigma_055_065], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_055_065, Bounds_Upper_SersicNOne_GaussianFixedSigma_055_065), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_055_065 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_055_065))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_055_065, R_e_l_SersicNOne_GaussianFixedSigma_055_065, n_l_SersicNOne_GaussianFixedSigma_055_065, bg_l_SersicNOne_GaussianFixedSigma_055_065, a_l_SersicNOne_GaussianFixedSigma_055_065 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_055_065, R_e_l_SersicNOne_GaussianFixedSigma_055_065, n_l_SersicNOne_GaussianFixedSigma_055_065, bg_l_SersicNOne_GaussianFixedSigma_055_065, a_l_SersicNOne_GaussianFixedSigma_055_065)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_055_065, R_e_l_SersicNOne_GaussianFixedSigma_055_065, n_l_SersicNOne_GaussianFixedSigma_055_065, bg_l_SersicNOne_GaussianFixedSigma_055_065, a_l_SersicNOne_GaussianFixedSigma_055_065 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_055_065 < i_e_SersicNOne_GaussianFixedSigma_055_065 < I_e_u_SersicNOne_GaussianFixedSigma_055_065 and R_e_l_SersicNOne_GaussianFixedSigma_055_065 < r_e_SersicNOne_GaussianFixedSigma_055_065 < R_e_u_SersicNOne_GaussianFixedSigma_055_065 and n_l_SersicNOne_GaussianFixedSigma_055_065 < n_SersicNOne_GaussianFixedSigma_055_065 < n_u_SersicNOne_GaussianFixedSigma_055_065 and bg_l_SersicNOne_GaussianFixedSigma_055_065 < bg_SersicNOne_GaussianFixedSigma_055_065 < bg_u_SersicNOne_GaussianFixedSigma_055_065 and a_l_SersicNOne_GaussianFixedSigma_055_065 < a_SersicNOne_GaussianFixedSigma_055_065 < a_u_SersicNOne_GaussianFixedSigma_055_065):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_055_065 = [(I_e_l_SersicNOne_GaussianFixedSigma_055_065, I_e_u_SersicNOne_GaussianFixedSigma_055_065), (R_e_l_SersicNOne_GaussianFixedSigma_055_065, R_e_u_SersicNOne_GaussianFixedSigma_055_065), (n_l_SersicNOne_GaussianFixedSigma_055_065, n_u_SersicNOne_GaussianFixedSigma_055_065), (bg_l_SersicNOne_GaussianFixedSigma_055_065, bg_u_SersicNOne_GaussianFixedSigma_055_065), (a_l_SersicNOne_GaussianFixedSigma_055_065, a_u_SersicNOne_GaussianFixedSigma_055_065)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_055_065, args=(x_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_err_SersicNOne_GaussianFixedSigma_055_065), bounds=bounds_SersicNOne_GaussianFixedSigma_055_065)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_055_065 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_055_065, ndim_SersicNOne_GaussianFixedSigma_055_065 = pos_SersicNOne_GaussianFixedSigma_055_065.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_055_065 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_055_065, ndim_SersicNOne_GaussianFixedSigma_055_065, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_err_SersicNOne_GaussianFixedSigma_055_065))
sampler_SersicNOne_GaussianFixedSigma_055_065.run_mcmc(pos_SersicNOne_GaussianFixedSigma_055_065, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_055_065 = sampler_SersicNOne_GaussianFixedSigma_055_065.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_055_065 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_055_065):
    mcmc_SersicNOne_GaussianFixedSigma_055_065 = np.percentile(samples_SersicNOne_GaussianFixedSigma_055_065[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_055_065.append(mcmc_SersicNOne_GaussianFixedSigma_055_065[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_055_065 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_055_065):
     mcmc_SersicNOne_GaussianFixedSigma_055_065 = np.percentile(samples_SersicNOne_GaussianFixedSigma_055_065[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_055_065.append([mcmc_SersicNOne_GaussianFixedSigma_055_065[2]-mcmc_SersicNOne_GaussianFixedSigma_055_065[1],mcmc_SersicNOne_GaussianFixedSigma_055_065[1]-mcmc_SersicNOne_GaussianFixedSigma_055_065[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_055_065))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_055_065))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *best_fit_params_SersicNOne_GaussianFixedSigma_055_065[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *best_fit_params_SersicNOne_GaussianFixedSigma_055_065[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *best_fit_params_SersicNOne_GaussianFixedSigma_055_065[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_055_065, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_055_065, y_data_SersicNOne_GaussianFixedSigma_055_065, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_055_065 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *best_fit_params_SersicNOne_GaussianFixedSigma_055_065[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, (yy_new_SersicNOne_GaussianFixedSigma_055_065 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *best_fit_params_SersicNOne_GaussianFixedSigma_055_065[:5]))/yy_new_SersicNOne_GaussianFixedSigma_055_065, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_055_065, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_055_065, y_data_SersicNOne_GaussianFixedSigma_055_065, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, (yy_new_SersicNOne_GaussianFixedSigma_055_065 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, (yy_new_SersicNOne_GaussianFixedSigma_055_065 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:5]))/yy_new_SersicNOne_GaussianFixedSigma_055_065, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.55 < z < 0.65")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_055_065[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_055_065[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_055_065[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_055_065[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_055_065[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_055_065[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_055_065[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_055_065[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_055_065[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_055_065[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_055_065[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_055_065[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_055_065[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_055_065[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_055_065[4][1]))


In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_065_075 = Equal_Area_Radius_065_075_Array
x_new_SersicNOne_GaussianFixedSigma_065_075 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_065_075),max(x_data_SersicNOne_GaussianFixedSigma_065_075), 200)

y_data_SersicNOne_GaussianFixedSigma_065_075 = np.array(Average_Counts_065_075)
y_new_SersicNOne_GaussianFixedSigma_065_075 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_065_075, y_data_SersicNOne_GaussianFixedSigma_065_075)

yy_new_SersicNOne_GaussianFixedSigma_065_075 = y_new_SersicNOne_GaussianFixedSigma_065_075(x_new_SersicNOne_GaussianFixedSigma_065_075)


y_err_SersicNOne_GaussianFixedSigma_065_075 = np.array(Average_Counts_SD_065_075)
y_new_err_SersicNOne_GaussianFixedSigma_065_075 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_065_075, y_err_SersicNOne_GaussianFixedSigma_065_075)

yy_new_err_SersicNOne_GaussianFixedSigma_065_075 = y_new_err_SersicNOne_GaussianFixedSigma_065_075(x_new_SersicNOne_GaussianFixedSigma_065_075)

#From Popt --- I_e, R_e, N, BG, A = [1.02079931 1.44814248 1.10012308 0.0694152  7.24326639]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.14541021 0.1125185  0.13341293 0.00666248 0.13185231]

i_e_SersicNOne_GaussianFixedSigma_065_075 = 1.02079931
r_e_SersicNOne_GaussianFixedSigma_065_075 = 1.44814248

n_SersicNOne_GaussianFixedSigma_065_075 = 1.10012308

bg_SersicNOne_GaussianFixedSigma_065_075 = min(yy_new_SersicNOne_GaussianFixedSigma_065_075)

a_SersicNOne_GaussianFixedSigma_065_075 = 7.24326639

initial_SersicNOne_GaussianFixedSigma_065_075 = np.array([i_e_SersicNOne_GaussianFixedSigma_065_075, r_e_SersicNOne_GaussianFixedSigma_065_075, n_SersicNOne_GaussianFixedSigma_065_075, bg_SersicNOne_GaussianFixedSigma_065_075, a_SersicNOne_GaussianFixedSigma_065_075]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_065_075 = 0
I_e_u_SersicNOne_GaussianFixedSigma_065_075 = i_e_SersicNOne_GaussianFixedSigma_065_075 + (0.01*i_e_SersicNOne_GaussianFixedSigma_065_075)

R_e_l_SersicNOne_GaussianFixedSigma_065_075 = 0
R_e_u_SersicNOne_GaussianFixedSigma_065_075 = r_e_SersicNOne_GaussianFixedSigma_065_075 + (0.01*r_e_SersicNOne_GaussianFixedSigma_065_075)

a_l_SersicNOne_GaussianFixedSigma_065_075 = 0
a_u_SersicNOne_GaussianFixedSigma_065_075 = a_SersicNOne_GaussianFixedSigma_065_075 + (0.01*a_SersicNOne_GaussianFixedSigma_065_075)

n_l_SersicNOne_GaussianFixedSigma_065_075 = 1.0
n_u_SersicNOne_GaussianFixedSigma_065_075 = n_SersicNOne_GaussianFixedSigma_065_075 + (0.01*n_SersicNOne_GaussianFixedSigma_065_075)

bg_l_SersicNOne_GaussianFixedSigma_065_075 = bg_SersicNOne_GaussianFixedSigma_065_075 - (bg_SersicNOne_GaussianFixedSigma_065_075*0.001)
bg_u_SersicNOne_GaussianFixedSigma_065_075 = bg_SersicNOne_GaussianFixedSigma_065_075 + (bg_SersicNOne_GaussianFixedSigma_065_075*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_065_075 = [I_e_l_SersicNOne_GaussianFixedSigma_065_075, R_e_l_SersicNOne_GaussianFixedSigma_065_075, n_l_SersicNOne_GaussianFixedSigma_065_075, bg_l_SersicNOne_GaussianFixedSigma_065_075, a_l_SersicNOne_GaussianFixedSigma_065_075]
Bounds_Upper_SersicNOne_GaussianFixedSigma_065_075 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_065_075, 500]


Popt_SersicNOne_GaussianFixedSigma_065_075, Pcov_SersicNOne_GaussianFixedSigma_065_075 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_SersicNOne_GaussianFixedSigma_065_075, p0=[i_e_SersicNOne_GaussianFixedSigma_065_075, r_e_SersicNOne_GaussianFixedSigma_065_075, n_SersicNOne_GaussianFixedSigma_065_075, bg_SersicNOne_GaussianFixedSigma_065_075, a_SersicNOne_GaussianFixedSigma_065_075], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_065_075, Bounds_Upper_SersicNOne_GaussianFixedSigma_065_075), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_065_075 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_065_075))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_065_075, R_e_l_SersicNOne_GaussianFixedSigma_065_075, n_l_SersicNOne_GaussianFixedSigma_065_075, bg_l_SersicNOne_GaussianFixedSigma_065_075, a_l_SersicNOne_GaussianFixedSigma_065_075 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_065_075, R_e_l_SersicNOne_GaussianFixedSigma_065_075, n_l_SersicNOne_GaussianFixedSigma_065_075, bg_l_SersicNOne_GaussianFixedSigma_065_075, a_l_SersicNOne_GaussianFixedSigma_065_075)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_065_075, R_e_l_SersicNOne_GaussianFixedSigma_065_075, n_l_SersicNOne_GaussianFixedSigma_065_075, bg_l_SersicNOne_GaussianFixedSigma_065_075, a_l_SersicNOne_GaussianFixedSigma_065_075 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_065_075 < i_e_SersicNOne_GaussianFixedSigma_065_075 < I_e_u_SersicNOne_GaussianFixedSigma_065_075 and R_e_l_SersicNOne_GaussianFixedSigma_065_075 < r_e_SersicNOne_GaussianFixedSigma_065_075 < R_e_u_SersicNOne_GaussianFixedSigma_065_075 and n_l_SersicNOne_GaussianFixedSigma_065_075 < n_SersicNOne_GaussianFixedSigma_065_075 < n_u_SersicNOne_GaussianFixedSigma_065_075 and bg_l_SersicNOne_GaussianFixedSigma_065_075 < bg_SersicNOne_GaussianFixedSigma_065_075 < bg_u_SersicNOne_GaussianFixedSigma_065_075 and a_l_SersicNOne_GaussianFixedSigma_065_075 < a_SersicNOne_GaussianFixedSigma_065_075 < a_u_SersicNOne_GaussianFixedSigma_065_075):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_065_075 = [(I_e_l_SersicNOne_GaussianFixedSigma_065_075, I_e_u_SersicNOne_GaussianFixedSigma_065_075), (R_e_l_SersicNOne_GaussianFixedSigma_065_075, R_e_u_SersicNOne_GaussianFixedSigma_065_075), (n_l_SersicNOne_GaussianFixedSigma_065_075, n_u_SersicNOne_GaussianFixedSigma_065_075), (bg_l_SersicNOne_GaussianFixedSigma_065_075, bg_u_SersicNOne_GaussianFixedSigma_065_075), (a_l_SersicNOne_GaussianFixedSigma_065_075, a_u_SersicNOne_GaussianFixedSigma_065_075)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_065_075, args=(x_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_err_SersicNOne_GaussianFixedSigma_065_075), bounds=bounds_SersicNOne_GaussianFixedSigma_065_075)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_065_075 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_065_075, ndim_SersicNOne_GaussianFixedSigma_065_075 = pos_SersicNOne_GaussianFixedSigma_065_075.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_065_075 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_065_075, ndim_SersicNOne_GaussianFixedSigma_065_075, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_err_SersicNOne_GaussianFixedSigma_065_075))
sampler_SersicNOne_GaussianFixedSigma_065_075.run_mcmc(pos_SersicNOne_GaussianFixedSigma_065_075, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_065_075 = sampler_SersicNOne_GaussianFixedSigma_065_075.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_065_075 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_065_075):
    mcmc_SersicNOne_GaussianFixedSigma_065_075 = np.percentile(samples_SersicNOne_GaussianFixedSigma_065_075[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_065_075.append(mcmc_SersicNOne_GaussianFixedSigma_065_075[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_065_075 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_065_075):
     mcmc_SersicNOne_GaussianFixedSigma_065_075 = np.percentile(samples_SersicNOne_GaussianFixedSigma_065_075[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_065_075.append([mcmc_SersicNOne_GaussianFixedSigma_065_075[2]-mcmc_SersicNOne_GaussianFixedSigma_065_075[1],mcmc_SersicNOne_GaussianFixedSigma_065_075[1]-mcmc_SersicNOne_GaussianFixedSigma_065_075[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_065_075))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_065_075))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *best_fit_params_SersicNOne_GaussianFixedSigma_065_075[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *best_fit_params_SersicNOne_GaussianFixedSigma_065_075[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *best_fit_params_SersicNOne_GaussianFixedSigma_065_075[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_SersicNOne_GaussianFixedSigma_065_075, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_065_075, y_data_SersicNOne_GaussianFixedSigma_065_075, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_SersicNOne_GaussianFixedSigma_065_075 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *best_fit_params_SersicNOne_GaussianFixedSigma_065_075[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, (yy_new_SersicNOne_GaussianFixedSigma_065_075 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *best_fit_params_SersicNOne_GaussianFixedSigma_065_075[:5]))/yy_new_SersicNOne_GaussianFixedSigma_065_075, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, yy_new_SersicNOne_GaussianFixedSigma_065_075, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_065_075, y_data_SersicNOne_GaussianFixedSigma_065_075, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, (yy_new_SersicNOne_GaussianFixedSigma_065_075 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, (yy_new_SersicNOne_GaussianFixedSigma_065_075 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:5]))/yy_new_SersicNOne_GaussianFixedSigma_065_075, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.65 < z < 0.75")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_065_075[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_065_075[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_065_075[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_065_075[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_065_075[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_065_075[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_065_075[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_065_075[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_065_075[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_065_075[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_065_075[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_065_075[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_065_075[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_065_075[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_065_075[4][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_075_085 = Equal_Area_Radius_075_085_Array
x_new_SersicNOne_GaussianFixedSigma_075_085 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_075_085),max(x_data_SersicNOne_GaussianFixedSigma_075_085), 200)

y_data_SersicNOne_GaussianFixedSigma_075_085 = np.array(Average_Counts_075_085)
y_new_SersicNOne_GaussianFixedSigma_075_085 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_075_085, y_data_SersicNOne_GaussianFixedSigma_075_085)

yy_new_SersicNOne_GaussianFixedSigma_075_085 = y_new_SersicNOne_GaussianFixedSigma_075_085(x_new_SersicNOne_GaussianFixedSigma_075_085)


y_err_SersicNOne_GaussianFixedSigma_075_085 = np.array(Average_Counts_SD_075_085)
y_new_err_SersicNOne_GaussianFixedSigma_075_085 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_075_085, y_err_SersicNOne_GaussianFixedSigma_075_085)

yy_new_err_SersicNOne_GaussianFixedSigma_075_085 = y_new_err_SersicNOne_GaussianFixedSigma_075_085(x_new_SersicNOne_GaussianFixedSigma_075_085)

#From Popt --- I_e, R_e, N, BG, A = [0.88049664 1.45350008 1.12579168 0.05964874 6.66174597]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.14249895 0.12887902 0.15162717 0.00650038 0.12823589]

i_e_SersicNOne_GaussianFixedSigma_075_085 = 0.88049664
r_e_SersicNOne_GaussianFixedSigma_075_085 = 1.45350008

n_SersicNOne_GaussianFixedSigma_075_085 = 1.12579168

bg_SersicNOne_GaussianFixedSigma_075_085 = min(yy_new_SersicNOne_GaussianFixedSigma_075_085)

a_SersicNOne_GaussianFixedSigma_075_085 = 6.66174597

initial_SersicNOne_GaussianFixedSigma_075_085 = np.array([i_e_SersicNOne_GaussianFixedSigma_075_085, r_e_SersicNOne_GaussianFixedSigma_075_085, n_SersicNOne_GaussianFixedSigma_075_085, bg_SersicNOne_GaussianFixedSigma_075_085, a_SersicNOne_GaussianFixedSigma_075_085]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_075_085 = 0
I_e_u_SersicNOne_GaussianFixedSigma_075_085 = i_e_SersicNOne_GaussianFixedSigma_075_085 + (0.01*i_e_SersicNOne_GaussianFixedSigma_075_085)

R_e_l_SersicNOne_GaussianFixedSigma_075_085 = 0
R_e_u_SersicNOne_GaussianFixedSigma_075_085 = r_e_SersicNOne_GaussianFixedSigma_075_085 + (0.01*r_e_SersicNOne_GaussianFixedSigma_075_085)

a_l_SersicNOne_GaussianFixedSigma_075_085 = 0
a_u_SersicNOne_GaussianFixedSigma_075_085 = a_SersicNOne_GaussianFixedSigma_075_085 + (0.01*a_SersicNOne_GaussianFixedSigma_075_085)

n_l_SersicNOne_GaussianFixedSigma_075_085 = 1.0
n_u_SersicNOne_GaussianFixedSigma_075_085 = n_SersicNOne_GaussianFixedSigma_075_085 + (0.01*n_SersicNOne_GaussianFixedSigma_075_085)

bg_l_SersicNOne_GaussianFixedSigma_075_085 = bg_SersicNOne_GaussianFixedSigma_075_085 - (bg_SersicNOne_GaussianFixedSigma_075_085*0.001)
bg_u_SersicNOne_GaussianFixedSigma_075_085 = bg_SersicNOne_GaussianFixedSigma_075_085 + (bg_SersicNOne_GaussianFixedSigma_075_085*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_075_085 = [I_e_l_SersicNOne_GaussianFixedSigma_075_085, R_e_l_SersicNOne_GaussianFixedSigma_075_085, n_l_SersicNOne_GaussianFixedSigma_075_085, bg_l_SersicNOne_GaussianFixedSigma_075_085, a_l_SersicNOne_GaussianFixedSigma_075_085]
Bounds_Upper_SersicNOne_GaussianFixedSigma_075_085 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_075_085, 500]

Popt_SersicNOne_GaussianFixedSigma_075_085, Pcov_SersicNOne_GaussianFixedSigma_075_085 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_075_085, p0=[i_e_SersicNOne_GaussianFixedSigma_075_085, r_e_SersicNOne_GaussianFixedSigma_075_085, n_SersicNOne_GaussianFixedSigma_075_085, bg_SersicNOne_GaussianFixedSigma_075_085, a_SersicNOne_GaussianFixedSigma_075_085], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_075_085, Bounds_Upper_SersicNOne_GaussianFixedSigma_075_085), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_075_085 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_075_085))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_075_085, R_e_l_SersicNOne_GaussianFixedSigma_075_085, n_l_SersicNOne_GaussianFixedSigma_075_085, bg_l_SersicNOne_GaussianFixedSigma_075_085, a_l_SersicNOne_GaussianFixedSigma_075_085 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_075_085, R_e_l_SersicNOne_GaussianFixedSigma_075_085, n_l_SersicNOne_GaussianFixedSigma_075_085, bg_l_SersicNOne_GaussianFixedSigma_075_085, a_l_SersicNOne_GaussianFixedSigma_075_085)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_075_085, R_e_l_SersicNOne_GaussianFixedSigma_075_085, n_l_SersicNOne_GaussianFixedSigma_075_085, bg_l_SersicNOne_GaussianFixedSigma_075_085, a_l_SersicNOne_GaussianFixedSigma_075_085 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_075_085 < i_e_SersicNOne_GaussianFixedSigma_075_085 < I_e_u_SersicNOne_GaussianFixedSigma_075_085 and R_e_l_SersicNOne_GaussianFixedSigma_075_085 < r_e_SersicNOne_GaussianFixedSigma_075_085 < R_e_u_SersicNOne_GaussianFixedSigma_075_085 and n_l_SersicNOne_GaussianFixedSigma_075_085 < n_SersicNOne_GaussianFixedSigma_075_085 < n_u_SersicNOne_GaussianFixedSigma_075_085 and bg_l_SersicNOne_GaussianFixedSigma_075_085 < bg_SersicNOne_GaussianFixedSigma_075_085 < bg_u_SersicNOne_GaussianFixedSigma_075_085 and a_l_SersicNOne_GaussianFixedSigma_075_085 < a_SersicNOne_GaussianFixedSigma_075_085 < a_u_SersicNOne_GaussianFixedSigma_075_085):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_075_085 = [(I_e_l_SersicNOne_GaussianFixedSigma_075_085, I_e_u_SersicNOne_GaussianFixedSigma_075_085), (R_e_l_SersicNOne_GaussianFixedSigma_075_085, R_e_u_SersicNOne_GaussianFixedSigma_075_085), (n_l_SersicNOne_GaussianFixedSigma_075_085, n_u_SersicNOne_GaussianFixedSigma_075_085), (bg_l_SersicNOne_GaussianFixedSigma_075_085, bg_u_SersicNOne_GaussianFixedSigma_075_085), (a_l_SersicNOne_GaussianFixedSigma_075_085, a_u_SersicNOne_GaussianFixedSigma_075_085)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_075_085, args=(x_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_err_SersicNOne_GaussianFixedSigma_075_085), bounds=bounds_SersicNOne_GaussianFixedSigma_075_085)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_075_085 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_075_085, ndim_SersicNOne_GaussianFixedSigma_075_085 = pos_SersicNOne_GaussianFixedSigma_075_085.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_075_085 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_075_085, ndim_SersicNOne_GaussianFixedSigma_075_085, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_err_SersicNOne_GaussianFixedSigma_075_085))
sampler_SersicNOne_GaussianFixedSigma_075_085.run_mcmc(pos_SersicNOne_GaussianFixedSigma_075_085, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_075_085 = sampler_SersicNOne_GaussianFixedSigma_075_085.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_075_085 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_075_085):
    mcmc_SersicNOne_GaussianFixedSigma_075_085 = np.percentile(samples_SersicNOne_GaussianFixedSigma_075_085[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_075_085.append(mcmc_SersicNOne_GaussianFixedSigma_075_085[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_075_085 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_075_085):
     mcmc_SersicNOne_GaussianFixedSigma_075_085 = np.percentile(samples_SersicNOne_GaussianFixedSigma_075_085[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_075_085.append([mcmc_SersicNOne_GaussianFixedSigma_075_085[2]-mcmc_SersicNOne_GaussianFixedSigma_075_085[1],mcmc_SersicNOne_GaussianFixedSigma_075_085[1]-mcmc_SersicNOne_GaussianFixedSigma_075_085[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_075_085))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_075_085))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *best_fit_params_SersicNOne_GaussianFixedSigma_075_085[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *best_fit_params_SersicNOne_GaussianFixedSigma_075_085[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *best_fit_params_SersicNOne_GaussianFixedSigma_075_085[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_075_085, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_075_085, y_data_SersicNOne_GaussianFixedSigma_075_085, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_075_085 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *best_fit_params_SersicNOne_GaussianFixedSigma_075_085[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, (yy_new_SersicNOne_GaussianFixedSigma_075_085 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *best_fit_params_SersicNOne_GaussianFixedSigma_075_085[:5]))/yy_new_SersicNOne_GaussianFixedSigma_075_085, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_075_085, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_075_085, y_data_SersicNOne_GaussianFixedSigma_075_085, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, (yy_new_SersicNOne_GaussianFixedSigma_075_085 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, (yy_new_SersicNOne_GaussianFixedSigma_075_085 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:5]))/yy_new_SersicNOne_GaussianFixedSigma_075_085, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.75 < z < 0.85")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_075_085[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_075_085[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_075_085[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_075_085[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_075_085[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_075_085[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_075_085[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_075_085[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_075_085[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_075_085[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_075_085[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_075_085[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_075_085[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_075_085[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_075_085[4][1]))

In [ ]:
# 1. Generate or load data
x_data_SersicNOne_GaussianFixedSigma_085_096 = Equal_Area_Radius_085_096_Array
x_new_SersicNOne_GaussianFixedSigma_085_096 = np.linspace(min(x_data_SersicNOne_GaussianFixedSigma_085_096),max(x_data_SersicNOne_GaussianFixedSigma_085_096), 200)

y_data_SersicNOne_GaussianFixedSigma_085_096 = np.array(Average_Counts_085_096)
y_new_SersicNOne_GaussianFixedSigma_085_096 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_085_096, y_data_SersicNOne_GaussianFixedSigma_085_096)

yy_new_SersicNOne_GaussianFixedSigma_085_096 = y_new_SersicNOne_GaussianFixedSigma_085_096(x_new_SersicNOne_GaussianFixedSigma_085_096)


y_err_SersicNOne_GaussianFixedSigma_085_096 = np.array(Average_Counts_SD_085_096)
y_new_err_SersicNOne_GaussianFixedSigma_085_096 = CubicSpline(x_data_SersicNOne_GaussianFixedSigma_085_096, y_err_SersicNOne_GaussianFixedSigma_085_096)

yy_new_err_SersicNOne_GaussianFixedSigma_085_096 = y_new_err_SersicNOne_GaussianFixedSigma_085_096(x_new_SersicNOne_GaussianFixedSigma_085_096)

#From Popt --- I_e, R_e, N, BG, A = [0.98682814 1.45199664 1.         0.06312366 8.17421655]
#From Popt Standard Deviation --- I_e, R_e, N, BG, A = [0.20170503 0.16022182 0.1902808  0.00970623 0.19081079]

i_e_SersicNOne_GaussianFixedSigma_085_096 = 0.98682814
r_e_SersicNOne_GaussianFixedSigma_085_096 = 1.45199664

n_SersicNOne_GaussianFixedSigma_085_096 = 1.0

bg_SersicNOne_GaussianFixedSigma_085_096 = min(yy_new_SersicNOne_GaussianFixedSigma_085_096)

a_SersicNOne_GaussianFixedSigma_085_096 = 8.17421655

initial_SersicNOne_GaussianFixedSigma_085_096 = np.array([i_e_SersicNOne_GaussianFixedSigma_085_096, r_e_SersicNOne_GaussianFixedSigma_085_096, n_SersicNOne_GaussianFixedSigma_085_096, bg_SersicNOne_GaussianFixedSigma_085_096, a_SersicNOne_GaussianFixedSigma_085_096]) #I_e,R_e,A


I_e_l_SersicNOne_GaussianFixedSigma_085_096 = 0
I_e_u_SersicNOne_GaussianFixedSigma_085_096 = i_e_SersicNOne_GaussianFixedSigma_085_096 + (0.01*i_e_SersicNOne_GaussianFixedSigma_085_096)

R_e_l_SersicNOne_GaussianFixedSigma_085_096 = 0
R_e_u_SersicNOne_GaussianFixedSigma_085_096 = r_e_SersicNOne_GaussianFixedSigma_085_096 + (0.01*r_e_SersicNOne_GaussianFixedSigma_085_096)

a_l_SersicNOne_GaussianFixedSigma_085_096 = 0
a_u_SersicNOne_GaussianFixedSigma_085_096 = a_SersicNOne_GaussianFixedSigma_085_096 + (0.01*a_SersicNOne_GaussianFixedSigma_085_096)

n_l_SersicNOne_GaussianFixedSigma_085_096 = 1.0
n_u_SersicNOne_GaussianFixedSigma_085_096 = n_SersicNOne_GaussianFixedSigma_085_096 + (0.01*n_SersicNOne_GaussianFixedSigma_085_096)

bg_l_SersicNOne_GaussianFixedSigma_085_096 = bg_SersicNOne_GaussianFixedSigma_085_096 - (bg_SersicNOne_GaussianFixedSigma_085_096*0.001)
bg_u_SersicNOne_GaussianFixedSigma_085_096 = bg_SersicNOne_GaussianFixedSigma_085_096 + (bg_SersicNOne_GaussianFixedSigma_085_096*0.001)

Bounds_Lower_SersicNOne_GaussianFixedSigma_085_096 = [I_e_l_SersicNOne_GaussianFixedSigma_085_096, R_e_l_SersicNOne_GaussianFixedSigma_085_096, n_l_SersicNOne_GaussianFixedSigma_085_096, bg_l_SersicNOne_GaussianFixedSigma_085_096, a_l_SersicNOne_GaussianFixedSigma_085_096]
Bounds_Upper_SersicNOne_GaussianFixedSigma_085_096 = [10, 10, 500, bg_u_SersicNOne_GaussianFixedSigma_085_096, 500]


Popt_SersicNOne_GaussianFixedSigma_085_096, Pcov_SersicNOne_GaussianFixedSigma_085_096 = curve_fit(Total_Model_BG, x_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_SersicNOne_GaussianFixedSigma_085_096, p0=[i_e_SersicNOne_GaussianFixedSigma_085_096, r_e_SersicNOne_GaussianFixedSigma_085_096, n_SersicNOne_GaussianFixedSigma_085_096, bg_SersicNOne_GaussianFixedSigma_085_096, a_SersicNOne_GaussianFixedSigma_085_096], bounds=(Bounds_Lower_SersicNOne_GaussianFixedSigma_085_096, Bounds_Upper_SersicNOne_GaussianFixedSigma_085_096), max_nfev=10000, method='trf')

Perr_SersicNOne_GaussianFixedSigma_085_096 = np.sqrt(np.diag(Pcov_SersicNOne_GaussianFixedSigma_085_096))



# 2. Define the likelihood function ############################################################################################################################################
def log_likelihood(theta, x, y, yerr):
    I_e_l_SersicNOne_GaussianFixedSigma_085_096, R_e_l_SersicNOne_GaussianFixedSigma_085_096, n_l_SersicNOne_GaussianFixedSigma_085_096, bg_l_SersicNOne_GaussianFixedSigma_085_096, a_l_SersicNOne_GaussianFixedSigma_085_096 = theta
    model = Total_Model_BG(x, I_e_l_SersicNOne_GaussianFixedSigma_085_096, R_e_l_SersicNOne_GaussianFixedSigma_085_096, n_l_SersicNOne_GaussianFixedSigma_085_096, bg_l_SersicNOne_GaussianFixedSigma_085_096, a_l_SersicNOne_GaussianFixedSigma_085_096)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))


def log_prior(theta):
    I_e_l_SersicNOne_GaussianFixedSigma_085_096, R_e_l_SersicNOne_GaussianFixedSigma_085_096, n_l_SersicNOne_GaussianFixedSigma_085_096, bg_l_SersicNOne_GaussianFixedSigma_085_096, a_l_SersicNOne_GaussianFixedSigma_085_096 = theta
    if (I_e_l_SersicNOne_GaussianFixedSigma_085_096 < i_e_SersicNOne_GaussianFixedSigma_085_096 < I_e_u_SersicNOne_GaussianFixedSigma_085_096 and R_e_l_SersicNOne_GaussianFixedSigma_085_096 < r_e_SersicNOne_GaussianFixedSigma_085_096 < R_e_u_SersicNOne_GaussianFixedSigma_085_096 and n_l_SersicNOne_GaussianFixedSigma_085_096 < n_SersicNOne_GaussianFixedSigma_085_096 < n_u_SersicNOne_GaussianFixedSigma_085_096 and bg_l_SersicNOne_GaussianFixedSigma_085_096 < bg_SersicNOne_GaussianFixedSigma_085_096 < bg_u_SersicNOne_GaussianFixedSigma_085_096 and a_l_SersicNOne_GaussianFixedSigma_085_096 < a_SersicNOne_GaussianFixedSigma_085_096 < a_u_SersicNOne_GaussianFixedSigma_085_096):
        return 0.0
    return -np.inf

    
# 5. Define the probability function ############################################################################################################################################
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# 3. Optimization (finding a good starting point) ############################################################################################################################################

bounds_SersicNOne_GaussianFixedSigma_085_096 = [(I_e_l_SersicNOne_GaussianFixedSigma_085_096, I_e_u_SersicNOne_GaussianFixedSigma_085_096), (R_e_l_SersicNOne_GaussianFixedSigma_085_096, R_e_u_SersicNOne_GaussianFixedSigma_085_096), (n_l_SersicNOne_GaussianFixedSigma_085_096, n_u_SersicNOne_GaussianFixedSigma_085_096), (bg_l_SersicNOne_GaussianFixedSigma_085_096, bg_u_SersicNOne_GaussianFixedSigma_085_096), (a_l_SersicNOne_GaussianFixedSigma_085_096, a_u_SersicNOne_GaussianFixedSigma_085_096)] # Parameter bounds for optimization
sol = minimize(lambda *args: -log_likelihood(*args), initial_SersicNOne_GaussianFixedSigma_085_096, args=(x_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_err_SersicNOne_GaussianFixedSigma_085_096), bounds=bounds_SersicNOne_GaussianFixedSigma_085_096)

#soln = minimize(lambda *args: -log_likelihood(*args), initial, args=(x_data, y_data, y_err))
pos_SersicNOne_GaussianFixedSigma_085_096 = sol.x + 1e-4 * np.random.randn(32, len(sol.x))
nwalkers_SersicNOne_GaussianFixedSigma_085_096, ndim_SersicNOne_GaussianFixedSigma_085_096 = pos_SersicNOne_GaussianFixedSigma_085_096.shape


# 4. MCMC sampling ############################################################################################################################################
sampler_SersicNOne_GaussianFixedSigma_085_096 = emcee.EnsembleSampler(nwalkers_SersicNOne_GaussianFixedSigma_085_096, ndim_SersicNOne_GaussianFixedSigma_085_096, log_probability, args=(x_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_err_SersicNOne_GaussianFixedSigma_085_096))
sampler_SersicNOne_GaussianFixedSigma_085_096.run_mcmc(pos_SersicNOne_GaussianFixedSigma_085_096, 5000, progress=True)

# 5. Analyze results
samples_SersicNOne_GaussianFixedSigma_085_096 = sampler_SersicNOne_GaussianFixedSigma_085_096.get_chain(flat=True)

# 6. Best fit parameters ############################################################################################################################################
best_fit_params_SersicNOne_GaussianFixedSigma_085_096 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_085_096):
    mcmc_SersicNOne_GaussianFixedSigma_085_096 = np.percentile(samples_SersicNOne_GaussianFixedSigma_085_096[:, i], [68, 95, 99.7])
    best_fit_params_SersicNOne_GaussianFixedSigma_085_096.append(mcmc_SersicNOne_GaussianFixedSigma_085_096[1])

# 10. Error estimation ############################################################################################################################################
errors_SersicNOne_GaussianFixedSigma_085_096 = []
for i in range(ndim_SersicNOne_GaussianFixedSigma_085_096):
     mcmc_SersicNOne_GaussianFixedSigma_085_096 = np.percentile(samples_SersicNOne_GaussianFixedSigma_085_096[:, i], [68, 95, 99.7])
     errors_SersicNOne_GaussianFixedSigma_085_096.append([mcmc_SersicNOne_GaussianFixedSigma_085_096[2]-mcmc_SersicNOne_GaussianFixedSigma_085_096[1],mcmc_SersicNOne_GaussianFixedSigma_085_096[1]-mcmc_SersicNOne_GaussianFixedSigma_085_096[0]])
      
# 12. Plot results ############################################################################################################################################
# 12. Plot results ############################################################################################################################################

#fig = plt.figure(figsize = (20, 10),  facecolor = "white")
fig1, axes1 = plt.subplots(6, 1, sharex=True, figsize=(15, 20))

print("From Popt --- I_e, R_e, N, BG, A = " +str(Popt_SersicNOne_GaussianFixedSigma_085_096))
print("From Popt Standard Deviation --- I_e, R_e, N, BG, A = " +str(Perr_SersicNOne_GaussianFixedSigma_085_096))

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *best_fit_params_SersicNOne_GaussianFixedSigma_085_096[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *best_fit_params_SersicNOne_GaussianFixedSigma_085_096[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *best_fit_params_SersicNOne_GaussianFixedSigma_085_096[:4]), color = "green", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[0].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_SersicNOne_GaussianFixedSigma_085_096, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_085_096, y_data_SersicNOne_GaussianFixedSigma_085_096, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[0].legend()



axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_SersicNOne_GaussianFixedSigma_085_096 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *best_fit_params_SersicNOne_GaussianFixedSigma_085_096[:5]), c = "#A294F9", edgecolor="black") 




axes1[2].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, (yy_new_SersicNOne_GaussianFixedSigma_085_096 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *best_fit_params_SersicNOne_GaussianFixedSigma_085_096[:5]))/yy_new_SersicNOne_GaussianFixedSigma_085_096, c = "#AA60C8", edgecolor="black") 





axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:5]), color = "#CF3030", linewidth=4.0, label="fit", zorder=10)

axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[3:5]), color = "#1A3263", linewidth=4.0, label="Gauss fit", zorder=10)
axes1[3].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:4]), color = "#DB005B", linewidth=4.0, label="Sersic fit", zorder=10)

axes1[3].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, yy_new_SersicNOne_GaussianFixedSigma_085_096, color = "#FFE05D", label="smoothed data", s=100)#, linewidth=2.0, label="smoothed data")
axes1[3].step(x_data_SersicNOne_GaussianFixedSigma_085_096, y_data_SersicNOne_GaussianFixedSigma_085_096, color = "black", linewidth=3.0, where="mid", zorder=1, label="data")

axes1[3].legend()


axes1[4].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, (yy_new_SersicNOne_GaussianFixedSigma_085_096 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:5])), c = "#AA60C8", edgecolor="black") 

axes1[5].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, (yy_new_SersicNOne_GaussianFixedSigma_085_096 - Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:5]))/yy_new_SersicNOne_GaussianFixedSigma_085_096, c = "#AA60C8", edgecolor="black") 


axes1[0].set_title("Guassian Fixed Sigma + Sersic Min One N -- 0.85 < z < 0.35")

axes1[5].set_xlabel("Radius from center [arcsecond]")

axes1[0].set_ylabel("Average Counts")
axes1[1].set_ylabel("Residuals")
axes1[2].set_ylabel("Percent Error")
axes1[3].set_ylabel("Average Counts")
axes1[4].set_ylabel("Residuals")
axes1[5].set_ylabel("Percent Error")

ax2 = axes1[0].twinx()
ax2.set_yticks([])
ax2.set_ylabel("MCMC", rotation=360, labelpad=30)

ax3 = axes1[3].twinx()
ax3.set_yticks([])
ax3.set_ylabel("Curve_fit", rotation=360, labelpad=35)


axes1[0].grid()
axes1[1].grid()
axes1[2].grid()
axes1[3].grid()
axes1[4].grid()
axes1[5].grid()

# Show the plot
plt.show(block=False)


# Close the figure
#############################################################################################################################################
print("I_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_085_096[0]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_085_096[0][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_085_096[0][1]))
print("")
print("R_e = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_085_096[1]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_085_096[1][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_085_096[1][1]))
print("")
print("N = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_085_096[2]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_085_096[2][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_085_096[2][1]))
print("")
print("BG = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_085_096[3]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_085_096[3][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_085_096[3][1]))
print("")
print("A = " +str(best_fit_params_SersicNOne_GaussianFixedSigma_085_096[4]) + " +" +str(errors_SersicNOne_GaussianFixedSigma_085_096[4][0]) + " - " +str(errors_SersicNOne_GaussianFixedSigma_085_096[4][1]))

## <font color='#FF0B55' size=6 > Finding the area under the gaussian with fixed sigama (PSF/2.345) and sersic fit with an N fixed to be above one and finding the fraction of the two. Additionally, getting the galaxy component magnitude </font>

In [ ]:
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_025_035, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_025_035 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_025_035[3], Popt_SersicNOne_GaussianFixedSigma_025_035[4], Perr_SersicNOne_GaussianFixedSigma_025_035[3], Perr_SersicNOne_GaussianFixedSigma_025_035[4])
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_035_045, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_035_045 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_035_045[3], Popt_SersicNOne_GaussianFixedSigma_035_045[4], Perr_SersicNOne_GaussianFixedSigma_035_045[3], Perr_SersicNOne_GaussianFixedSigma_035_045[4])
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_045_055, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_045_055 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_045_055[3], Popt_SersicNOne_GaussianFixedSigma_045_055[4], Perr_SersicNOne_GaussianFixedSigma_045_055[3], Perr_SersicNOne_GaussianFixedSigma_045_055[4])
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_055_065, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_055_065 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_055_065[3], Popt_SersicNOne_GaussianFixedSigma_055_065[4], Perr_SersicNOne_GaussianFixedSigma_055_065[3], Perr_SersicNOne_GaussianFixedSigma_055_065[4])
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_065_075, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_065_075 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_065_075[3], Popt_SersicNOne_GaussianFixedSigma_065_075[4], Perr_SersicNOne_GaussianFixedSigma_065_075[3], Perr_SersicNOne_GaussianFixedSigma_065_075[4])
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_075_085, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_075_085 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_075_085[3], Popt_SersicNOne_GaussianFixedSigma_075_085[4], Perr_SersicNOne_GaussianFixedSigma_075_085[3], Perr_SersicNOne_GaussianFixedSigma_075_085[4])
Area_Under_Narrow_SersicNOne_GaussianFixedSigma_085_096, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_085_096 = Monte_Carlo_Gaussian_Fixed_Sigma_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_085_096[3], Popt_SersicNOne_GaussianFixedSigma_085_096[4], Perr_SersicNOne_GaussianFixedSigma_085_096[3], Perr_SersicNOne_GaussianFixedSigma_085_096[4])

Area_Under_Broad_SersicNOne_GaussianFixedSigma_025_035, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_025_035 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_025_035[0], Popt_SersicNOne_GaussianFixedSigma_025_035[1], Popt_SersicNOne_GaussianFixedSigma_025_035[2], Popt_SersicNOne_GaussianFixedSigma_025_035[3], Perr_SersicNOne_GaussianFixedSigma_025_035[0], Perr_SersicNOne_GaussianFixedSigma_025_035[1], Perr_SersicNOne_GaussianFixedSigma_025_035[2], Perr_SersicNOne_GaussianFixedSigma_025_035[3])
Area_Under_Broad_SersicNOne_GaussianFixedSigma_035_045, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_035_045 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_035_045[0], Popt_SersicNOne_GaussianFixedSigma_035_045[1], Popt_SersicNOne_GaussianFixedSigma_035_045[2], Popt_SersicNOne_GaussianFixedSigma_035_045[3], Perr_SersicNOne_GaussianFixedSigma_035_045[0], Perr_SersicNOne_GaussianFixedSigma_035_045[1], Perr_SersicNOne_GaussianFixedSigma_035_045[2], Perr_SersicNOne_GaussianFixedSigma_035_045[3])
Area_Under_Broad_SersicNOne_GaussianFixedSigma_045_055, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_045_055 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_045_055[0], Popt_SersicNOne_GaussianFixedSigma_045_055[1], Popt_SersicNOne_GaussianFixedSigma_045_055[2], Popt_SersicNOne_GaussianFixedSigma_045_055[3], Perr_SersicNOne_GaussianFixedSigma_045_055[0], Perr_SersicNOne_GaussianFixedSigma_045_055[1], Perr_SersicNOne_GaussianFixedSigma_045_055[2], Perr_SersicNOne_GaussianFixedSigma_045_055[3])
Area_Under_Broad_SersicNOne_GaussianFixedSigma_055_065, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_055_065 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_055_065[0], Popt_SersicNOne_GaussianFixedSigma_055_065[1], Popt_SersicNOne_GaussianFixedSigma_055_065[2], Popt_SersicNOne_GaussianFixedSigma_055_065[3], Perr_SersicNOne_GaussianFixedSigma_055_065[0], Perr_SersicNOne_GaussianFixedSigma_055_065[1], Perr_SersicNOne_GaussianFixedSigma_055_065[2], Perr_SersicNOne_GaussianFixedSigma_055_065[3])
Area_Under_Broad_SersicNOne_GaussianFixedSigma_065_075, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_065_075 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_065_075[0], Popt_SersicNOne_GaussianFixedSigma_065_075[1], Popt_SersicNOne_GaussianFixedSigma_065_075[2], Popt_SersicNOne_GaussianFixedSigma_065_075[3], Perr_SersicNOne_GaussianFixedSigma_065_075[0], Perr_SersicNOne_GaussianFixedSigma_065_075[1], Perr_SersicNOne_GaussianFixedSigma_065_075[2], Perr_SersicNOne_GaussianFixedSigma_065_075[3])
Area_Under_Broad_SersicNOne_GaussianFixedSigma_075_085, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_075_085 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_075_085[0], Popt_SersicNOne_GaussianFixedSigma_075_085[1], Popt_SersicNOne_GaussianFixedSigma_075_085[2], Popt_SersicNOne_GaussianFixedSigma_075_085[3], Perr_SersicNOne_GaussianFixedSigma_075_085[0], Perr_SersicNOne_GaussianFixedSigma_075_085[1], Perr_SersicNOne_GaussianFixedSigma_075_085[2], Perr_SersicNOne_GaussianFixedSigma_075_085[3])
Area_Under_Broad_SersicNOne_GaussianFixedSigma_085_096, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_085_096 = Monte_Carlo_Sersic_BG_Area(x_new_SersicNOne_GaussianFixedSigma_085_096, Popt_SersicNOne_GaussianFixedSigma_085_096[0], Popt_SersicNOne_GaussianFixedSigma_085_096[1], Popt_SersicNOne_GaussianFixedSigma_085_096[2], Popt_SersicNOne_GaussianFixedSigma_085_096[3], Perr_SersicNOne_GaussianFixedSigma_085_096[0], Perr_SersicNOne_GaussianFixedSigma_085_096[1], Perr_SersicNOne_GaussianFixedSigma_085_096[2], Perr_SersicNOne_GaussianFixedSigma_085_096[3])


In [ ]:
# Fractional mag for the OG fits (Fake errors)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_025_035, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_025_035 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_025_035, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_025_035, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_025_035, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_025_035)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_035_045, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_035_045 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_035_045, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_035_045, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_035_045, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_035_045)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_045_055, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_045_055 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_045_055, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_045_055, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_045_055, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_045_055)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_055_065, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_055_065 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_055_065, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_055_065, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_055_065, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_055_065)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_065_075, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_065_075 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_065_075, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_065_075, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_065_075, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_065_075)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_075_085, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_075_085 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_075_085, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_075_085, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_075_085, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_075_085)
Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_085_096, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_085_096 = fractional_area_uncertainty(Area_Under_Broad_SersicNOne_GaussianFixedSigma_085_096, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_085_096, Area_Under_Broad_SersicNOne_GaussianFixedSigma_SD_085_096, Area_Under_Narrow_SersicNOne_GaussianFixedSigma_SD_085_096)


In [ ]:
# The Magnitude of the galaxy component for each redshift
Gal_Mag_SersicNOne_GaussianFixedSigma_025_035, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_025_035 = gal_mag_uncertainty(Mag_Total_025_035, Mag_Total_SD_025_035, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_025_035, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_025_035)
Gal_Mag_SersicNOne_GaussianFixedSigma_035_045, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_035_045 = gal_mag_uncertainty(Mag_Total_035_045, Mag_Total_SD_035_045, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_035_045, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_035_045)
Gal_Mag_SersicNOne_GaussianFixedSigma_045_055, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_045_055 = gal_mag_uncertainty(Mag_Total_045_055, Mag_Total_SD_045_055, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_045_055, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_045_055)
Gal_Mag_SersicNOne_GaussianFixedSigma_055_065, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_055_065 = gal_mag_uncertainty(Mag_Total_055_065, Mag_Total_SD_055_065, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_055_065, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_055_065)
Gal_Mag_SersicNOne_GaussianFixedSigma_065_075, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_065_075 = gal_mag_uncertainty(Mag_Total_065_075, Mag_Total_SD_065_075, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_065_075, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_065_075)
Gal_Mag_SersicNOne_GaussianFixedSigma_075_085, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_075_085 = gal_mag_uncertainty(Mag_Total_075_085, Mag_Total_SD_075_085, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_075_085, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_075_085)
Gal_Mag_SersicNOne_GaussianFixedSigma_085_096, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_085_096 = gal_mag_uncertainty(Mag_Total_085_096, Mag_Total_SD_085_096, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_085_096, Fractional_Area_Broad_SersicNOne_GaussianFixedSigma_SD_085_096)


In [ ]:
Gal_Mag_SersicNOne_GaussianFixedSigma_Array = np.array([Gal_Mag_SersicNOne_GaussianFixedSigma_025_035, Gal_Mag_SersicNOne_GaussianFixedSigma_035_045, Gal_Mag_SersicNOne_GaussianFixedSigma_045_055, Gal_Mag_SersicNOne_GaussianFixedSigma_055_065, Gal_Mag_SersicNOne_GaussianFixedSigma_065_075, Gal_Mag_SersicNOne_GaussianFixedSigma_075_085, Gal_Mag_SersicNOne_GaussianFixedSigma_085_096])


In [ ]:
Gal_Mag_SersicNOne_GaussianFixedSigma_SD_Array = np.array([Gal_Mag_SersicNOne_GaussianFixedSigma_SD_025_035, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_035_045, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_045_055, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_055_065, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_065_075, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_075_085, Gal_Mag_SersicNOne_GaussianFixedSigma_SD_085_096])


## <font color='#FF0B55' size=6 > Makeing the Paper Ready Plots </font>

In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_025_035, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_025_035, 
              y_data_SersicNOne_GaussianFixedSigma_025_035, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, 
              yy_new_SersicNOne_GaussianFixedSigma_025_035, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_025_035,
    yy_new_SersicNOne_GaussianFixedSigma_025_035 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_025_035,
    yy_new_SersicNOne_GaussianFixedSigma_025_035 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_025_035,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_025_035, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.25–0.35", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_025_035, 
                 (yy_new_SersicNOne_GaussianFixedSigma_025_035 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_025_035, *Popt_SersicNOne_GaussianFixedSigma_025_035[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_025_035.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_025_035.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_035_045, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_035_045, 
              y_data_SersicNOne_GaussianFixedSigma_035_045, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, 
              yy_new_SersicNOne_GaussianFixedSigma_035_045, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_035_045,
    yy_new_SersicNOne_GaussianFixedSigma_035_045 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_035_045,
    yy_new_SersicNOne_GaussianFixedSigma_035_045 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_035_045,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_035_045, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.35–0.45", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_035_045, 
                 (yy_new_SersicNOne_GaussianFixedSigma_035_045 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_035_045, *Popt_SersicNOne_GaussianFixedSigma_035_045[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_035_045.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_035_045.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_045_055, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_045_055, 
              y_data_SersicNOne_GaussianFixedSigma_045_055, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, 
              yy_new_SersicNOne_GaussianFixedSigma_045_055, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_045_055,
    yy_new_SersicNOne_GaussianFixedSigma_045_055 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_045_055,
    yy_new_SersicNOne_GaussianFixedSigma_045_055 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_045_055,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_045_055, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.45–0.55", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_045_055, 
                 (yy_new_SersicNOne_GaussianFixedSigma_045_055 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_045_055, *Popt_SersicNOne_GaussianFixedSigma_045_055[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_045_055.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_045_055.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_055_065, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_055_065, 
              y_data_SersicNOne_GaussianFixedSigma_055_065, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, 
              yy_new_SersicNOne_GaussianFixedSigma_055_065, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_055_065,
    yy_new_SersicNOne_GaussianFixedSigma_055_065 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_055_065,
    yy_new_SersicNOne_GaussianFixedSigma_055_065 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_055_065,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_055_065, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.55–0.65", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_055_065, 
                 (yy_new_SersicNOne_GaussianFixedSigma_055_065 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_055_065, *Popt_SersicNOne_GaussianFixedSigma_055_065[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_055_065.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_055_065.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_065_075, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_065_075, 
              y_data_SersicNOne_GaussianFixedSigma_065_075, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, 
              yy_new_SersicNOne_GaussianFixedSigma_065_075, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_065_075,
    yy_new_SersicNOne_GaussianFixedSigma_065_075 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_065_075,
    yy_new_SersicNOne_GaussianFixedSigma_065_075 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_065_075,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_065_075, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.65–0.75", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_065_075, 
                 (yy_new_SersicNOne_GaussianFixedSigma_065_075 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_065_075, *Popt_SersicNOne_GaussianFixedSigma_065_075[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_065_075.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_065_075.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_075_085, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_075_085, 
              y_data_SersicNOne_GaussianFixedSigma_075_085, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, 
              yy_new_SersicNOne_GaussianFixedSigma_075_085, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_075_085,
    yy_new_SersicNOne_GaussianFixedSigma_075_085 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_075_085,
    yy_new_SersicNOne_GaussianFixedSigma_075_085 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_075_085,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_075_085, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.75–0.85", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_075_085, 
                 (yy_new_SersicNOne_GaussianFixedSigma_075_085 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_075_085, *Popt_SersicNOne_GaussianFixedSigma_075_085[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_075_085.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_075_085.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
# Define your figure and axes (adjusted to make the top panel taller than the bottom one)
fig_Paper_085_096, axes1 = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

# Plot 1 (Top Panel) - Making it taller

axes1[0].step(x_data_SersicNOne_GaussianFixedSigma_085_096, 
              y_data_SersicNOne_GaussianFixedSigma_085_096, 
              color="black", linewidth=4.0, where="mid", zorder=1, label="Observed Data", linestyle='-')  # Black for original data

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, 
              yy_new_SersicNOne_GaussianFixedSigma_085_096, 
              color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')  # Yellow for smoothed data (new line style)

axes1[0].fill_between(
    x_new_SersicNOne_GaussianFixedSigma_085_096,
    yy_new_SersicNOne_GaussianFixedSigma_085_096 - 3*yy_new_err_SersicNOne_GaussianFixedSigma_085_096,
    yy_new_SersicNOne_GaussianFixedSigma_085_096 + 3*yy_new_err_SersicNOne_GaussianFixedSigma_085_096,
    color='#FFC1DA',
    alpha=0.3
)

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, 
              Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:5]), 
              color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10, linestyle='-')  # Red for fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, 
              Gaussian_Component_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[3:5]), 
              color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')  # Navy Blue for Gauss fit

axes1[0].plot(x_new_SersicNOne_GaussianFixedSigma_085_096, 
              Sersic_Convolved_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:4]), 
              color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')  # Magenta for Sersic fit

axes1[0].text(0.02, 0.96, "Redshift: 0.85–0.96", transform=axes1[0].transAxes,
              fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))
        
axes1[0].legend(
    loc='upper right', 
    fontsize=12, 
    frameon=True, 
    fancybox=True, 
    shadow=True, 
    borderpad=1, 
    edgecolor='black', 
    facecolor='w', 
    handlelength=4.0,  # Increase the length of the legend handles (lines)
    columnspacing=2.0,  # Increase the space between columns
    handleheight=1.5  # Increase the height of the legend handles
)


# Make the axes thicker
for ax in axes1:
    ax.spines['top'].set_linewidth(2.5)     # Top border
    ax.spines['right'].set_linewidth(2.5)   # Right border
    ax.spines['left'].set_linewidth(2.5)    # Left border
    ax.spines['bottom'].set_linewidth(2.5)  # Bottom border

    # You can also adjust the tick marks if needed
    ax.tick_params(axis='both', which='major', width=2)  # Increase tick width
    ax.tick_params(axis='both', which='minor', width=1)  # Minor ticks



# Use Times New Roman for a more formal look in publications
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Set tick label size
axes1[0].tick_params(axis='both', which='major', labelsize=12)
axes1[0].tick_params(axis='both', which='minor', labelsize=10)

axes1[1].tick_params(axis='both', which='major', labelsize=12)
axes1[1].tick_params(axis='both', which='minor', labelsize=10)


axes1[0].set_title("Model Fit of AGN Data with Gaussian and Sersic Components", fontsize=14)

axes1[0].set_ylabel("Average Counts / Second", fontsize=14)

# Plot 2 (Bottom Panel)
axes1[1].scatter(x_new_SersicNOne_GaussianFixedSigma_085_096, 
                 (yy_new_SersicNOne_GaussianFixedSigma_085_096 - 
                  Total_Model_BG(x_new_SersicNOne_GaussianFixedSigma_085_096, *Popt_SersicNOne_GaussianFixedSigma_085_096[:5])), 
                 c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')  # Bright Green for residuals

# Adding a horizontal line at zero for reference
axes1[1].axhline(0, color="#000000", linestyle='--', linewidth=1.5)  # Black for horizontal line at zero

axes1[1].set_xlabel("Radius from center [arcsecond]", fontsize=14)
axes1[1].set_ylabel("Residuals", fontsize=14)

# Extend the upper y-limit of the top panel
ymin, ymax = axes1[0].get_ylim()
axes1[0].set_ylim(ymin, ymax * 1.07)  # Increase the upper limit by 20%

axes1[1].grid(True)


# Final adjustments
plt.tight_layout()
fig_Paper_085_096.savefig("AGN_Gaussian_Sersic_Fit_Paper_Ready_R_085_096.png", dpi=300, bbox_inches='tight')
plt.show(block=False)


In [ ]:
x_data_array = np.array([x_data_SersicNOne_GaussianFixedSigma_025_035, x_data_SersicNOne_GaussianFixedSigma_035_045, x_data_SersicNOne_GaussianFixedSigma_045_055, x_data_SersicNOne_GaussianFixedSigma_055_065, x_data_SersicNOne_GaussianFixedSigma_065_075,
                        x_data_SersicNOne_GaussianFixedSigma_075_085, x_data_SersicNOne_GaussianFixedSigma_085_096])

y_data_array = np.array([y_data_SersicNOne_GaussianFixedSigma_025_035, y_data_SersicNOne_GaussianFixedSigma_035_045, y_data_SersicNOne_GaussianFixedSigma_045_055, y_data_SersicNOne_GaussianFixedSigma_055_065, y_data_SersicNOne_GaussianFixedSigma_065_075,
                        y_data_SersicNOne_GaussianFixedSigma_075_085, y_data_SersicNOne_GaussianFixedSigma_085_096])

x_smoothed_array = np.array([x_new_SersicNOne_GaussianFixedSigma_025_035, x_new_SersicNOne_GaussianFixedSigma_035_045, x_new_SersicNOne_GaussianFixedSigma_045_055, x_new_SersicNOne_GaussianFixedSigma_055_065, x_new_SersicNOne_GaussianFixedSigma_065_075,
                        x_new_SersicNOne_GaussianFixedSigma_075_085, x_new_SersicNOne_GaussianFixedSigma_085_096])

y_smoothed_array = np.array([yy_new_SersicNOne_GaussianFixedSigma_025_035, yy_new_SersicNOne_GaussianFixedSigma_035_045, yy_new_SersicNOne_GaussianFixedSigma_045_055, yy_new_SersicNOne_GaussianFixedSigma_055_065, yy_new_SersicNOne_GaussianFixedSigma_065_075,
                        yy_new_SersicNOne_GaussianFixedSigma_075_085, yy_new_SersicNOne_GaussianFixedSigma_085_096])

y_smoothed_SD_array = np.array([yy_new_err_SersicNOne_GaussianFixedSigma_025_035, yy_new_err_SersicNOne_GaussianFixedSigma_035_045, yy_new_err_SersicNOne_GaussianFixedSigma_045_055, yy_new_err_SersicNOne_GaussianFixedSigma_055_065, yy_new_err_SersicNOne_GaussianFixedSigma_065_075,
                        yy_new_err_SersicNOne_GaussianFixedSigma_075_085, yy_new_err_SersicNOne_GaussianFixedSigma_085_096])

popt_array = np.array([Popt_SersicNOne_GaussianFixedSigma_025_035, Popt_SersicNOne_GaussianFixedSigma_035_045, Popt_SersicNOne_GaussianFixedSigma_045_055, Popt_SersicNOne_GaussianFixedSigma_055_065, Popt_SersicNOne_GaussianFixedSigma_065_075,
                        Popt_SersicNOne_GaussianFixedSigma_075_085, Popt_SersicNOne_GaussianFixedSigma_085_096])

redshift_labels = ["Redshift: 0.25–0.35", "Redshift: 0.35–0.45", "Redshift: 0.45–0.55", "Redshift: 0.55–0.65", "Redshift: 0.65–0.75", "Redshift: 0.75–0.85", "Redshift: 0.85–0.96"]

In [ ]:
# Settings
n_bins = len(x_data_array)
n_cols = int(np.ceil(np.sqrt(n_bins)))
n_rows = int(np.ceil(n_bins / n_cols))

# Increase figure size to enlarge individual panels
fig_Paper_Grid = plt.figure(figsize=(n_cols * 7.5, n_rows * 6.5))
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Loop over redshift bins
for i in range(n_bins):
    x_data = x_data_array[i]
    y_data = y_data_array[i]
    x_new = x_smoothed_array[i]
    y_smooth = y_smoothed_array[i]
    y_err = y_smoothed_SD_array[i]
    popt = popt_array[i]
    redshift_label = redshift_labels[i]

    # Create outer grid for rows and columns
    outer_gs = fig_Paper_Grid.add_gridspec(n_rows, n_cols, wspace=0.1, hspace=0.1)
    row, col = divmod(i, n_cols)

    # Create subgridspec for top and bottom panels (model and residuals)
    inner_gs = outer_gs[row, col].subgridspec(2, 1, height_ratios=[3, 1], hspace=0)

    ax_top = fig_Paper_Grid.add_subplot(inner_gs[0])
    ax_bottom = fig_Paper_Grid.add_subplot(inner_gs[1], sharex=ax_top)

    # --- Top Panel (Model) ---
    ax_top.step(x_data, y_data, color="black", linewidth=4.0, where="mid", zorder=1, linestyle='-', label="Observed Data")
    ax_top.plot(x_new, y_smooth, color="#309898", label="Smoothed Data", linewidth=8.0, zorder=5, linestyle=':')
    ax_top.fill_between(x_new, y_smooth - 3*y_err, y_smooth + 3*y_err, color='#FFC1DA', alpha=0.3)
    ax_top.plot(x_new, Total_Model_BG(x_new, *popt[:5]), color="#DB3EB1", linewidth=4.0, label="Combined Model Fit", zorder=10)
    ax_top.plot(x_new, Gaussian_Component_BG(x_new, *popt[3:5]), color="#1D5799", linewidth=4.0, label="Gauss Component", zorder=10, linestyle='--')
    ax_top.plot(x_new, Sersic_Convolved_Model_BG(x_new, *popt[:4]), color="#9467BD", linewidth=4.0, label="Sersic Component", zorder=10, linestyle='-.')

    ax_top.text(0.02, 0.96, redshift_label, transform=ax_top.transAxes,
                fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))

    # Only show legend on the first panel (top-left)
    if i == 0:
        ax_top.legend(loc='upper right', fontsize=12, frameon=True, fancybox=True, shadow=True,
                      borderpad=1, edgecolor='black', facecolor='w',
                      handlelength=4.0, columnspacing=2.0, handleheight=1.5)

    # Always define the labels, but they'll only be displayed based on position
    ax_top.set_ylabel("Avg Counts / Sec", fontsize=14)
    ymin, ymax = ax_top.get_ylim()
    ax_top.set_ylim(ymin, ymax * 1.07)

    # --- Bottom Panel (Residuals) ---
    residuals = y_smooth - Total_Model_BG(x_new, *popt[:5])
    ax_bottom.scatter(x_new, residuals, c="#009E73", edgecolor="black", s=100, label="Residuals", marker='x')
    ax_bottom.axhline(0, color="#000000", linestyle='--', linewidth=1.5)
    ax_bottom.set_xlabel("Radius from center [arcsecond]", fontsize=14)
    ax_bottom.set_ylabel("Residuals", fontsize=14)
    
    # Set specific y-limits for residuals
    if row == 0 and col == 0:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-2.5, 2.5)

    # Set specific y-limits for residuals
    if row == 0 and col == 1:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-1.5, 1.5)

    # Set specific y-limits for residuals
    if row == 0 and col == 2:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-1.25, 1.25)

    # Set specific y-limits for residuals
    if row == 1 and col == 0:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-1.25, 1.25)

    # Set specific y-limits for residuals
    if row == 1 and col == 1:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-1.25, 1.25)

    # Set specific y-limits for residuals
    if row == 1 and col == 2:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-1.25, 1.25)
        
    # Set specific y-limits for residuals
    if row == 2 and col == 0:
        # Set y-limit to ensure there are tick values above zero
        # Adjust these values as needed based on your data range
        ax_bottom.set_ylim(-1.25, 1.25)
    
    ax_bottom.grid(True)

    # --- Styling for both axes ---
    for ax in [ax_top, ax_bottom]:
        ax.spines['top'].set_linewidth(2.5)
        ax.spines['right'].set_linewidth(2.5)
        ax.spines['left'].set_linewidth(2.5)
        ax.spines['bottom'].set_linewidth(2.5)
        ax.tick_params(axis='both', which='major', width=2, labelsize=12)
        ax.tick_params(axis='both', which='minor', width=1, labelsize=10)
        ax.tick_params(axis='both', which='both', direction='in', length=6)
    
    # Only show y-axis labels on leftmost panels
    if col != 0:
        ax_top.set_ylabel("")
        ax_bottom.set_ylabel("")
    
    # Show x-axis labels on bottom row panels and specific panels in row 1, columns 1 and 2
    # Note: row and col are zero-indexed, so row=1 is second row, col=1 is second column, col=2 is third column
    if row != n_rows - 1 and not (row == 1 and (col == 1 or col == 2)):
        ax_bottom.set_xlabel("")
    
# Final layout adjustments
plt.tight_layout()
fig_Paper_Grid.savefig("AGN_Gaussian_Sersic_Grid_PaperReady_R.png", dpi=300, bbox_inches='tight')

# Save and show the figure
plt.show(block=False)


## <font color='#FF0B55' size=6 > Putting the outputs into files and closing the images </font>

In [ ]:
Table_start_a = ([(Gal_Mag_Two_Gaussian_Array[0])])
Table_start_b = ([(Gal_Mag_Two_Gaussian_SD_Array[0])])
Table_start_c = ([(Gal_Mag_Sersic_Gaussian_Array[0])])
Table_start_d = ([(Gal_Mag_Sersic_Gaussian_SD_Array[0])])
Table_start_e = ([(Gal_Mag_SersicFixedN_GaussianFixedSigma_Array[0])])
Table_start_f = ([(Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_Array[0])])
Table_start_g = ([(Gal_Mag_SersicNOne_GaussianFixedSigma_Array[0])])
Table_start_h = ([(Gal_Mag_SersicNOne_GaussianFixedSigma_SD_Array[0])])

Table_b = QTable([Table_start_a, Table_start_b, Table_start_c, Table_start_d, Table_start_e, Table_start_f, Table_start_g, Table_start_h], names=( "Gal_Mag_Two_Gaussian_Array","Gal_Mag_Two_Gaussian_SD_Array", "Gal_Mag_Sersic_Gaussian_Array", "Gal_Mag_Sersic_Gaussian_SD_Array", "Gal_Mag_SersicFixedN_GaussianFixedSigma_Array", "Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_Array", "Gal_Mag_SersicNOne_GaussianFixedSigma_Array", "Gal_Mag_SersicNOne_GaussianFixedSigma_SD_Array"))


In [ ]:
for i in np.arange(1,len(Gal_Mag_Two_Gaussian_Array),1):
    a = ([(Gal_Mag_Two_Gaussian_Array[i])])
    b = ([(Gal_Mag_Two_Gaussian_SD_Array[i])])
    c = ([(Gal_Mag_Sersic_Gaussian_Array[i])])
    d = ([(Gal_Mag_Sersic_Gaussian_SD_Array[i])])
    e = ([(Gal_Mag_SersicFixedN_GaussianFixedSigma_Array[i])])
    f = ([(Gal_Mag_SersicFixedN_GaussianFixedSigma_SD_Array[i])])
    g = ([(Gal_Mag_SersicNOne_GaussianFixedSigma_Array[i])])
    h = ([(Gal_Mag_SersicNOne_GaussianFixedSigma_SD_Array[i])])

    Table_b.add_row([a,b,c,d,e,f,g,h])
    

In [ ]:
Table_b.write("/home/jovyan/work/stampede3/AGN-Black-Hole-Research/Table_Gal_Mags_R.fits", overwrite = True)


In [ ]:
pdf_filename = "/home/jovyan/work/stampede3/AGN-Black-Hole-Research/Figures_For_Filter_R.pdf"
with PdfPages(pdf_filename) as pdf:
    figures = [plt.figure(n) for n in plt.get_fignums()]
    print(plt.get_fignums())
    for fig in figures:
        pdf.savefig(fig)
    plt.close('all')
print(f"All figures saved to {pdf_filename}")
